<a href="https://colab.research.google.com/github/AIManifest/deforum-stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $ {\large \textsf{AI Manifest's Deforum Stable Diffusion v0.7 Custom Notebook}}$

**I DO NOT CLAIM ANY OFFICIAL CODE IN THIS NOTEBOOK**

THIS NOTEBOOK IS PURELY FOR MY FUN AND ALL CREDITS ARE DUE
TO THEIR RESPECTIVE OWNERS. I SIMPLY MADE A FUN INTERFACE TO 
INTERACT WITH FOR STABLE DIFFUSION, UPSCALING, INTERPOLATION, AND MODEL MERGING.

[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). Notebook by [deforum](https://discord.gg/upmXXsrwZc)

[Quick Guide](https://docs.google.com/document/d/1RrQv7FntzOuLg4ohjRZPVL7iptIyBhwwbcEYEW2OfcI/edit?usp=sharing) to Deforum v0.7

$\color{red} {\textsf {FOR CONTROLNET, YOU ONLY NEED TO RUN THE ENVIRONMENT SETUP AND CONTROLNET CELL}}$

$\color{red} {\textsf {DO NOT RUN THE PATH SETUP CELL OR TEXTUAL INVERSION CELL WITH CONTROLNET}}$

In [ ]:
#@title $  {\large \textsf{Environment Setup / NVIDIA GPU -- **Mandatory** Run to Setup Everything for this Notebook}}$
print("\033[0;34m")
mount_drive_for_colab = True #@param {type:"boolean"}
if mount_drive_for_colab:
  from google.colab import drive
  drive.mount("/content/drive/")
from IPython import display
import random
import nltk
from nltk.corpus import wordnet
nltk.download('all')
display.clear_output(wait=True)
import subprocess, os, sys, gc, time
from tqdm.notebook import tqdm
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"\033[0;34m{sub_p_res[:-1]}")
!pip install -qq -U kora
def setup_environment():
    start_time = time.time()
    print_subprocess = False
    use_xformers_for_colab = True
    try:
        ipy = get_ipython()
    except:
        ipy = 'could not get_ipython'
    if 'google.colab' in str(ipy):
        print("\033[0;34m..setting up environment")

        # weird hack
        import torch
        
        all_process = [
            ['pip', 'install', 'facexlib>=0.2.5', 'gfpgan>=1.3.5', 'basicsr>=1.4.2', 'sk-video>=1.1.10', 'omegaconf', 'einops==0.4.1', 'pytorch-lightning==1.7.7', 'torchmetrics', 'transformers', 'safetensors', 'kornia'],
            ['pip', 'install', '-qq', '--user', 'ffmpeg-python'],
            ['git', 'clone', 'https://github.com/AIManifest/deforum-stable-diffusion.git'],
            ['git', 'clone', 'https://github.com/AIManifest/Practical-RIFE.git'],
            ['git', 'clone', 'https://github.com/AIManifest/Real-ESRGAN.git'],
            ['git', 'clone', 'https://github.com/AIManifest/ESRGAN.git'],
            ['git', 'clone', 'https://github.com/lllyasviel/ControlNet.git', 'deforum-stable-diffusion/ControlNet'],
            ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq','scikit-learn','torchsde','open-clip-torch','numpngw', 'imageio-ffmpeg'],
        ]
        for process in tqdm(all_process):
            running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
            if print_subprocess:
                print(running)
        with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
            f.write('')
        sys.path.extend([
            'deforum-stable-diffusion/',
            'deforum-stable-diffusion/src',
            'deforum-stable-diffusion/ControlNet'
        ])
        if use_xformers_for_colab:

            print("\033[0;34m..installing triton and xformers")

            all_process = [['pip', 'install', 'triton==2.0.0.dev20221202', 'xformers==0.0.16']]
            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)
    else:
        sys.path.extend([
            'src'
        ])
    end_time = time.time()
    print(f"\033[0;34m..environment set up in {end_time-start_time:.0f} seconds")
    return

setup_environment()
with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
  f.write('')
sys.path.extend([
  'deforum-stable-diffusion/',
  'deforum-stable-diffusion/src',
  'deforum-stable-diffusion/ControlNet'
])
import torch
import cv2
import random
import clip
import yaml
from IPython import display
from ipywidgets import widgets
from IPython import display
import glob
import os
import re
import shutil
import torch
import tqdm
import safetensors.torch
import requests
from types import SimpleNamespace
from helpers.save_images import get_output_folder
from helpers.settings import load_args
from helpers.render import render_animation, render_input_video, render_image_batch, render_interpolation
from helpers.model_load import make_linear_decode, load_model, get_model_output_paths
from helpers.aesthetics import load_aesthetics_model
# !pip install -qq rich
# from rich import pretty
# pretty.install()

In [ ]:
#@title $ {\large \textsf{Path Setup}}$
print("\033[0;34m")
start_time = time.time()

#@markdown **Download CivitAI Models Here, if you use this, the `filename` parameter will override the `model_checkpoint` parameter.**
download_civit_model = False #@param{type:"boolean"}
if download_civit_model:
    model_URL = "https://civitai.com/api/download/models/4222" #@param {type:"string"}
    folder_path = "/content/drive/MyDrive/AII/hypernetworks" #@param {type:"string"}
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    #Name for the model
    filename = "classic_statue_marble.pt" #@param{type:"string"}

    # Download the file and save it to the folder
    response = requests.get(model_URL)
    with open(os.path.join(folder_path, filename), "wb") as f:
        f.write(response.content)

    print(f"File {filename} downloaded and saved to {folder_path}")

#@markdown **Deforum Path Setup**
def Root():
    models_path = "models" #@param {type:"string"}
    configs_path = "configs" #@param {type:"string"}
    output_path = "outputs" #@param {type:"string"}
    mount_google_drive = True #@param {type:"boolean"}
    models_path_gdrive = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion" #@param {type:"string"}
    output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}

    #@markdown **Model Setup**
    map_location = "cuda" #@param ["cpu", "cuda"]
    model_config = "custom" #@param ["custom","v2-inference.yaml","v2-inference-v.yaml","v1-inference.yaml"]
    model_checkpoint =  "custom" #@param ["custom","v2-1_768-ema-pruned.ckpt","v2-1_512-ema-pruned.ckpt","768-v-ema.ckpt","512-base-ema.ckpt","Protogen_V2.2.ckpt","v1-5-pruned.ckpt","v1-5-pruned-emaonly.ckpt","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","wd-v1-3-float16.ckpt"]
    custom_config_path = "/content/deforum-stable-diffusion/ControlNet/models/cldm_v15.yaml" #@param {type:"string"}
    custom_checkpoint_path = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/control_sd15_depth.pth" #@param {type:"string"}
    control_net_model = True #@param{type:"boolean"}
    #@markdown $ \color{red} {\textsf{Leave These Blank if you Don't Need to Use Them, }}$
    #@markdown 
    #@markdown $ \color{red} {\textsf{But Generally, These Directories Should be Where All of }}$
    #@markdown 
    #@markdown $ \color{red} {\textsf{Your Embeddings, HyperNetworks, and Parent Directories are Respectively}}$
    embeddings_dir = "/content/drive/MyDrive/sd/stable-diffusion-webui/textual_inversion/2023-02-13/xxpjccrajararaxx/embeddings" #@param{type:'string'}
    hypernetwork_dir = "/content/drive/MyDrive/AI/hypernetworks" #@param{type:'string'}
    data_dir = "/content/drive/MyDrive/sd/stable-diffusion-webui" #@param{type:'string'}
    #@markdown **Don't use yet, still working on it**
    use_xformers = True #@param{type:'boolean'}
    use_sub_quad_attention = False #@param{type:'boolean'}
    use_split_attention_v1 = False #@param{type:'boolean'}
    use_split_cross_attention_forward_invokeAI = False #@param{type:'boolean'}
    use_cross_attention_attnblock_forward = False #@param{type:'boolean'}

    variables = locals()
    with open("Deforum.yaml", "w") as file:
      yaml.dump(variables, file)
    return variables

root = Root()
root = SimpleNamespace(**root)

if download_civit_model:
    root.model_checkpoint = f"{folder_path}/{filename}"

root.models_path, root.output_path = get_model_output_paths(root)
root.model, root.device = load_model(root, load_on_run_all=True, check_sha256=True)

end_time = time.time()
time_elapsed = end_time - start_time

print(f"..Setup Paths in {time_elapsed} seconds..")

In [ ]:
#@title $ {\large \textsf{Load Textual Inversion!}}$

#@markdown $\color{red} {\textsf {Run This Cell Only if you Intend to Load Textual Inversion Files}}$
#@markdown $\color{red} {\textsf {Otherwise, no need to run this cell.}}$
model = root.model
device = root.device
root.embeddings_dir = "/content/drive/MyDrive/embeddings_ai"
root.hypernetwork_dir = "/content/drive/MyDrive/AI/hypernetworks"
# cpu = torch.device("cpu")
# device = device_interrogate = device_gfpgan = device_esrgan = device_codeformer = None
dtype = torch.float32
dtype_vae = torch.float32
dtype_unet = torch.float32
unet_needs_upcast = True
precision = "full"
import os
import sys
import traceback
import inspect
from collections import namedtuple

import torch
import tqdm
import html
import datetime
import csv
import safetensors.torch

import numpy as np
from PIL import Image, PngImagePlugin
from torch.utils.tensorboard import SummaryWriter
root.model_checkpoint = root.custom_checkpoint_path
checkpoint_name = os.path.splitext(os.path.basename(root.model_checkpoint))
print("\033[92m")
import hashlib
# print("..checking sha256")
# with open(root.custom_checkpoint_path, "rb") as f:
#   bytes = f.read() 
#   ckpt_hash = hashlib.sha256(bytes).hexdigest()
  

#learn_rate_scheduler

import tqdm

def autocast(disable=False):
    
    if disable:
        return contextlib.nullcontext()

    if dtype == torch.float32 or precision == "full":
        return contextlib.nullcontext()

    return torch.autocast("cuda")
    
class LearnScheduleIterator:
    def __init__(self, learn_rate, max_steps, cur_step=0):
        """
        specify learn_rate as "0.001:100, 0.00001:1000, 1e-5:10000" to have lr of 0.001 until step 100, 0.00001 until 1000, and 1e-5 until 10000
        """
        print(learn_rate)
        pairs = learn_rate.split(',')
        self.rates = []
        self.it = 0
        self.maxit = 0
        try:
            for i, pair in enumerate(pairs):
                if not pair.strip():
                    continue
                tmp = pair.split(':')
                if len(tmp) == 2:
                    step = int(tmp[1])
                    if step > cur_step:
                        self.rates.append((float(tmp[0]), min(step, max_steps)))
                        self.maxit += 1
                        if step > max_steps:
                            return
                    elif step == -1:
                        self.rates.append((float(tmp[0]), max_steps))
                        self.maxit += 1
                        return
                else:
                    self.rates.append((float(tmp[0]), max_steps))
                    self.maxit += 1
                    return
            assert self.rates
        except (ValueError, AssertionError):
            raise Exception('Invalid learning rate schedule. It should be a number or, for example, like "0.001:100, 0.00001:1000, 1e-5:10000" to have lr of 0.001 until step 100, 0.00001 until 1000, and 1e-5 until 10000.')


    def __iter__(self):
        return self

    def __next__(self):
        if self.it < self.maxit:
            self.it += 1
            return self.rates[self.it - 1]
        else:
            raise StopIteration


class LearnRateScheduler:
    def __init__(self, learn_rate, max_steps, cur_step=0, verbose=True):
        self.schedules = LearnScheduleIterator(learn_rate, max_steps, cur_step)
        (self.learn_rate,  self.end_step) = next(self.schedules)
        self.verbose = verbose

        if self.verbose:
            print(f'Training at rate of {self.learn_rate} until step {self.end_step}')

        self.finished = False

    def step(self, step_number):
        if step_number < self.end_step:
            return False

        try:
            (self.learn_rate, self.end_step) = next(self.schedules)
        except StopIteration:
            self.finished = True
            return False
        return True

    def apply(self, optimizer, step_number):
        if not self.step(step_number):
            return

        if self.verbose:
            tqdm.tqdm.write(f'Training at rate of {self.learn_rate} until step {self.end_step}')

        for pg in optimizer.param_groups:
            pg['lr'] = self.learn_rate

#dataset
import os
import numpy as np
import PIL
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision import transforms
from collections import defaultdict
from random import shuffle, choices

import random
import tqdm
import re

from ldm.modules.distributions.distributions import DiagonalGaussianDistribution

re_numbers_at_start = re.compile(r"^[-\d]+\s*")


class DatasetEntry:
    def __init__(self, filename=None, filename_text=None, latent_dist=None, latent_sample=None, cond=None, cond_text=None, pixel_values=None):
        self.filename = filename
        self.filename_text = filename_text
        self.latent_dist = latent_dist
        self.latent_sample = latent_sample
        self.cond = cond
        self.cond_text = cond_text
        self.pixel_values = pixel_values

dataset_filename_join_string= " "
dataset_filename_word_regex = ""
class PersonalizedBase(Dataset):
    def __init__(self, data_root, width, height, repeats, flip_p=0.5, placeholder_token="*", model=None, cond_model=None, device=None, template_file=None, include_cond=False, batch_size=1, gradient_step=1, shuffle_tags=False, tag_drop_out=0, latent_sampling_method='once', varsize=False):
        re_word = re.compile(dataset_filename_word_regex) if len(dataset_filename_word_regex) > 0 else None

        self.placeholder_token = placeholder_token

        self.flip = transforms.RandomHorizontalFlip(p=flip_p)

        self.dataset = []

        with open(template_file, "r") as file:
            lines = [x.strip() for x in file.readlines()]

        self.lines = lines

        assert data_root, 'dataset directory not specified'
        assert os.path.isdir(data_root), "Dataset directory doesn't exist"
        assert os.listdir(data_root), "Dataset directory is empty"

        self.image_paths = [os.path.join(data_root, file_path) for file_path in os.listdir(data_root)]

        self.shuffle_tags = shuffle_tags
        self.tag_drop_out = tag_drop_out
        groups = defaultdict(list)

        print("Preparing dataset...")
        for path in tqdm.tqdm(self.image_paths):
            # if shared.state.interrupted:
            #     raise Exception("interrupted")
            try:
                image = Image.open(path).convert('RGB')
                if not varsize:
                    image = image.resize((width, height), PIL.Image.BICUBIC)
            except Exception:
                continue

            text_filename = os.path.splitext(path)[0] + ".txt"
            filename = os.path.basename(path)

            if os.path.exists(text_filename):
                with open(text_filename, "r", encoding="utf8") as file:
                    filename_text = file.read()
            else:
                filename_text = os.path.splitext(filename)[0]
                filename_text = re.sub(re_numbers_at_start, '', filename_text)
                if re_word:
                    tokens = re_word.findall(filename_text)
                    filename_text = (dataset_filename_join_string or "").join(tokens)

            npimage = np.array(image).astype(np.uint8)
            npimage = (npimage / 127.5 - 1.0).astype(np.float32)

            torchdata = torch.from_numpy(npimage).permute(2, 0, 1).to(device=device, dtype=torch.float32)
            latent_sample = None

            with torch.autocast("cuda"):
                latent_dist = model.encode_first_stage(torchdata.unsqueeze(dim=0))

            if latent_sampling_method == "once" or (latent_sampling_method == "deterministic" and not isinstance(latent_dist, DiagonalGaussianDistribution)):
                latent_sample = model.get_first_stage_encoding(latent_dist).squeeze().to("cuda")
                latent_sampling_method = "once"
                entry = DatasetEntry(filename=path, filename_text=filename_text, latent_sample=latent_sample)
            elif latent_sampling_method == "deterministic":
                # Works only for DiagonalGaussianDistribution
                latent_dist.std = 0
                latent_sample = model.get_first_stage_encoding(latent_dist).squeeze().to("cuda")
                entry = DatasetEntry(filename=path, filename_text=filename_text, latent_sample=latent_sample)
            elif latent_sampling_method == "random":
                entry = DatasetEntry(filename=path, filename_text=filename_text, latent_dist=latent_dist)

            if not (self.tag_drop_out != 0 or self.shuffle_tags):
                entry.cond_text = self.create_text(filename_text)

            if include_cond and not (self.tag_drop_out != 0 or self.shuffle_tags):
                with autocast():
                    entry.cond = cond_model([entry.cond_text]).to("cuda").squeeze(0)
            groups[image.size].append(len(self.dataset))
            self.dataset.append(entry)
            del torchdata
            del latent_dist
            del latent_sample

        self.length = len(self.dataset)
        self.groups = list(groups.values())
        assert self.length > 0, "No images have been found in the dataset."
        self.batch_size = min(batch_size, self.length)
        self.gradient_step = min(gradient_step, self.length // self.batch_size)
        self.latent_sampling_method = latent_sampling_method

        if len(groups) > 1:
            print("Buckets:")
            for (w, h), ids in sorted(groups.items(), key=lambda x: x[0]):
                print(f"  {w}x{h}: {len(ids)}")
            print()

    def create_text(self, filename_text):
        text = random.choice(self.lines)
        tags = filename_text.split(',')
        if self.tag_drop_out != 0:
            tags = [t for t in tags if random.random() > self.tag_drop_out]
        if self.shuffle_tags:
            random.shuffle(tags)
        text = text.replace("[filewords]", ','.join(tags))
        text = text.replace("[name]", self.placeholder_token)
        return text

    def __len__(self):
        return self.length

    def __getitem__(self, i):
        entry = self.dataset[i]
        if self.tag_drop_out != 0 or self.shuffle_tags:
            entry.cond_text = self.create_text(entry.filename_text)
        if self.latent_sampling_method == "random":
            entry.latent_sample = model.get_first_stage_encoding(entry.latent_dist).to("cuda")
        return entry


class GroupedBatchSampler(Sampler):
    def __init__(self, data_source: PersonalizedBase, batch_size: int):
        super().__init__(data_source)

        n = len(data_source)
        self.groups = data_source.groups
        self.len = n_batch = n // batch_size
        expected = [len(g) / n * n_batch * batch_size for g in data_source.groups]
        self.base = [int(e) // batch_size for e in expected]
        self.n_rand_batches = nrb = n_batch - sum(self.base)
        self.probs = [e%batch_size/nrb/batch_size if nrb>0 else 0 for e in expected]
        self.batch_size = batch_size

    def __len__(self):
        return self.len

    def __iter__(self):
        b = self.batch_size

        for g in self.groups:
            shuffle(g)

        batches = []
        for g in self.groups:
            batches.extend(g[i*b:(i+1)*b] for i in range(len(g) // b))
        for _ in range(self.n_rand_batches):
            rand_group = choices(self.groups, self.probs)[0]
            batches.append(choices(rand_group, k=b))

        shuffle(batches)

        yield from batches


class PersonalizedDataLoader(DataLoader):
    def __init__(self, dataset, latent_sampling_method="once", batch_size=1, pin_memory=False):
        super(PersonalizedDataLoader, self).__init__(dataset, batch_sampler=GroupedBatchSampler(dataset, batch_size), pin_memory=pin_memory)
        if latent_sampling_method == "random":
            self.collate_fn = collate_wrapper_random
        else:
            self.collate_fn = collate_wrapper


class BatchLoader:
    def __init__(self, data):
        self.cond_text = [entry.cond_text for entry in data]
        self.cond = [entry.cond for entry in data]
        self.latent_sample = torch.stack([entry.latent_sample for entry in data]).squeeze(1)
        #self.emb_index = [entry.emb_index for entry in data]
        #print(self.latent_sample.device)

    def pin_memory(self):
        self.latent_sample = self.latent_sample.pin_memory()
        return self

def collate_wrapper(batch):
    return BatchLoader(batch)

class BatchLoaderRandom(BatchLoader):
    def __init__(self, data):
        super().__init__(data)

    def pin_memory(self):
        return self

def collate_wrapper_random(batch):
    return BatchLoaderRandom(batch)

TextualInversionTemplate = namedtuple("TextualInversionTemplate", ["name", "path"])
textual_inversion_templates = {}

class Embedding:
    def __init__(self, vec, name, step=None):
        self.vec = vec
        self.name = name
        self.step = step
        self.shape = None
        self.vectors = 0
        self.cached_checksum = None
        self.sd_checkpoint = None
        self.sd_checkpoint_name = None
        self.optimizer_state_dict = None
        self.filename = None

    def save(self, filename):
        embedding_data = {
            "string_to_token": {"*": 265},
            "string_to_param": {"*": self.vec},
            "name": self.name,
            "step": self.step,
            "sd_checkpoint": self.sd_checkpoint,
            "sd_checkpoint_name": self.sd_checkpoint_name,
        }

        torch.save(embedding_data, filename)

        save_optimizer_state=True
        if save_optimizer_state and self.optimizer_state_dict is not None:
            optimizer_saved_dict = {
                'hash': self.checksum(),
                'optimizer_state_dict': self.optimizer_state_dict,
            }
            torch.save(optimizer_saved_dict, filename + '.optim')

    def checksum(self):
        if self.cached_checksum is not None:
            return self.cached_checksum

        def const_hash(a):
            r = 0
            for v in a:
                r = (r * 281 ^ int(v) * 997) & 0xFFFFFFFF
            return r

        self.cached_checksum = f'{const_hash(self.vec.reshape(-1) * 100) & 0xffff:04x}'
        return self.cached_checksum


class DirWithTextualInversionEmbeddings:
    def __init__(self, path):
        self.path = path
        self.mtime = None

    def has_changed(self):
        if not os.path.isdir(self.path):
            return False

        mt = os.path.getmtime(self.path)
        if self.mtime is None or mt > self.mtime:
            return True

    def update(self):
        if not os.path.isdir(self.path):
            return

        self.mtime = os.path.getmtime(self.path)


class EmbeddingDatabase:
    def __init__(self):
        self.ids_lookup = {}
        self.word_embeddings = {}
        self.skipped_embeddings = {}
        self.expected_shape = -1
        self.embedding_dirs = {}

    def add_embedding_dir(self, path):
        self.embedding_dirs[path] = DirWithTextualInversionEmbeddings(path)

    def clear_embedding_dirs(self):
        self.embedding_dirs.clear()

    def register_embedding(self, embedding, model):
        self.word_embeddings[embedding.name] = embedding

        ids = model.cond_stage_model.tokenize([embedding.name])[0]

        first_id = ids[0]
        if first_id not in self.ids_lookup:
            self.ids_lookup[first_id] = []

        self.ids_lookup[first_id] = sorted(self.ids_lookup[first_id] + [(ids, embedding)], key=lambda x: len(x[0]), reverse=True)

        return embedding

    def get_expected_shape(self):
        vec = root.model.cond_stage_model.encode_embedding_init_text(",", 1)
        return vec.shape[1]

    def load_from_file(self, path, filename):
        name, ext = os.path.splitext(filename)
        ext = ext.upper()

        if ext in ['.PNG', '.WEBP', '.JXL', '.AVIF']:
            _, second_ext = os.path.splitext(name)
            if second_ext.upper() == '.PREVIEW':
                return

            embed_image = Image.open(path)
            if hasattr(embed_image, 'text') and 'sd-ti-embedding' in embed_image.text:
                data = embedding_from_b64(embed_image.text['sd-ti-embedding'])
                name = data.get('name', name)
            else:
                data = extract_image_data_embed(embed_image)
                name = data.get('name', name)
        elif ext in ['.BIN', '.PT']:
            data = torch.load(path, map_location=root.map_location)
        elif ext in ['.SAFETENSORS']:
            data = safetensors.torch.load_file(path, device=root.map_location)
        else:
            return

        # textual inversion embeddings
        if 'string_to_param' in data:
            param_dict = data['string_to_param']
            if hasattr(param_dict, '_parameters'):
                param_dict = getattr(param_dict, '_parameters')  # fix for torch 1.12.1 loading saved file from torch 1.11
            assert len(param_dict) == 1, 'embedding file has multiple terms in it'
            emb = next(iter(param_dict.items()))[1]
        # diffuser concepts
        elif type(data) == dict and type(next(iter(data.values()))) == torch.Tensor:
            assert len(data.keys()) == 1, 'embedding file has multiple terms in it'

            emb = next(iter(data.values()))
            if len(emb.shape) == 1:
                emb = emb.unsqueeze(0)
        else:
            raise Exception(f"Couldn't identify {filename} as neither textual inversion embedding nor diffuser concept.")

        vec = emb.detach().to(root.device, dtype=torch.float32)
        embedding = Embedding(vec, name)
        embedding.step = data.get('step', None)
        embedding.sd_checkpoint = data.get('sd_checkpoint', None)
        embedding.sd_checkpoint_name = data.get('sd_checkpoint_name', None)
        embedding.vectors = vec.shape[0]
        embedding.shape = vec.shape[-1]
        embedding.filename = path

        if self.expected_shape == -1 or self.expected_shape == embedding.shape:
            self.register_embedding(embedding, root.model)
        else:
            self.skipped_embeddings[name] = embedding

    def load_from_dir(self, embdir):
        if not os.path.isdir(embdir.path):
            return

        for root, dirs, fns in os.walk(embdir.path):
            for fn in fns:
                try:
                    fullfn = os.path.join(root, fn)

                    if os.stat(fullfn).st_size == 0:
                        continue

                    self.load_from_file(fullfn, fn)
                except Exception:
                    print(f"Error loading embedding {fn}:", file=sys.stderr)
                    print(traceback.format_exc(), file=sys.stderr)
                    continue

    def load_textual_inversion_embeddings(self, force_reload=False):
        if not force_reload:
            need_reload = False
            for path, embdir in self.embedding_dirs.items():
                if embdir.has_changed():
                    need_reload = True
                    break

            if not need_reload:
                return

        self.ids_lookup.clear()
        self.word_embeddings.clear()
        self.skipped_embeddings.clear()
        self.expected_shape = self.get_expected_shape()

        for path, embdir in self.embedding_dirs.items():
            self.load_from_dir(embdir)
            embdir.update()

        print(f"Textual inversion embeddings loaded({len(self.word_embeddings)}): {', '.join(self.word_embeddings.keys())}")
        if len(self.skipped_embeddings) > 0:
            print(f"Textual inversion embeddings skipped({len(self.skipped_embeddings)}): {', '.join(self.skipped_embeddings.keys())}")

    def find_embedding_at_position(self, tokens, offset):
        token = tokens[offset]
        possible_matches = self.ids_lookup.get(token, None)

        if possible_matches is None:
            return None, None

        for ids, embedding in possible_matches:
            if tokens[offset:offset + len(ids)] == ids:
                return embedding, len(ids)

        return None, None

#train embeddings
def autocast(disable=False):
    
    if disable:
        return contextlib.nullcontext()

    if dtype == torch.float32 or precision == "full":
        return contextlib.nullcontext()

    return torch.autocast("cuda")

##markdown $ {\textsf{These Params are FOR TEXTUAL INVERSION TRAINING ONLY!}}$

def create_embedding(name, num_vectors_per_token, overwrite_old, init_text='*'):
    cond_model = model.cond_stage_model

    with autocast():
        cond_model([""])  # will send cond model to GPU if lowvram/medvram is active

    #cond_model expects at least some text, so we provide '*' as backup.
    embedded = cond_model.encode_embedding_init_text(init_text or '*', num_vectors_per_token)
    vec = torch.zeros((num_vectors_per_token, embedded.shape[1]), device=root.device)

    #Only copy if we provided an init_text, otherwise keep vectors as zeros
    if init_text:
        for i in range(num_vectors_per_token):
            vec[i] = embedded[i * int(embedded.shape[0]) // num_vectors_per_token]

    # Remove illegal characters from name.
    name = "".join( x for x in name if (x.isalnum() or x in "._- "))
    fn = os.path.join(root.embeddings_dir, f"{name}.pt")
    if not overwrite_old:
        assert not os.path.exists(fn), f"file {fn} already exists"

    embedding = Embedding(vec, name)
    embedding.step = 0
    embedding.save(fn)

    return fn

training_write_csv_every=0
def write_loss(log_directory, filename, step, epoch_len, values):
    if training_write_csv_every == 0:
        return

    if step % training_write_csv_every != 0:
        return
    write_csv_header = False if os.path.exists(os.path.join(log_directory, filename)) else True

    with open(os.path.join(log_directory, filename), "a+", newline='') as fout:
        csv_writer = csv.DictWriter(fout, fieldnames=["step", "epoch", "epoch_step", *(values.keys())])

        if write_csv_header:
            csv_writer.writeheader()

        epoch = (step - 1) // epoch_len
        epoch_step = (step - 1) % epoch_len

        csv_writer.writerow({
            "step": step,
            "epoch": epoch,
            "epoch_step": epoch_step,
            **values,
        })

def tensorboard_setup(log_directory):
    os.makedirs(os.path.join(log_directory, "tensorboard"), exist_ok=True)
    training_tensorboard_flush_every=120
    return SummaryWriter(
            log_dir=os.path.join(log_directory, "tensorboard"),
            flush_secs=training_tensorboard_flush_every)

def tensorboard_add(tensorboard_writer, loss, global_step, step, learn_rate, epoch_num):
    tensorboard_add_scaler(tensorboard_writer, "Loss/train", loss, global_step)
    tensorboard_add_scaler(tensorboard_writer, f"Loss/train/epoch-{epoch_num}", loss, step)
    tensorboard_add_scaler(tensorboard_writer, "Learn rate/train", learn_rate, global_step)
    tensorboard_add_scaler(tensorboard_writer, f"Learn rate/train/epoch-{epoch_num}", learn_rate, step)

def tensorboard_add_scaler(tensorboard_writer, tag, value, step):
    tensorboard_writer.add_scalar(tag=tag, 
        scalar_value=value, global_step=step)

def tensorboard_add_image(tensorboard_writer, tag, pil_image, step):
    # Convert a pil image to a torch tensor
    img_tensor = torch.as_tensor(np.array(pil_image, copy=True))
    img_tensor = img_tensor.view(pil_image.size[1], pil_image.size[0], 
        len(pil_image.getbands()))
    img_tensor = img_tensor.permute((2, 0, 1))
                
    tensorboard_writer.add_image(tag, img_tensor, global_step=step)

def validate_train_inputs(model_name, learn_rate, batch_size, gradient_step, data_root, template_file, template_filename, steps, save_model_every, create_image_every, log_directory, name="embedding"):
    assert model_name, f"{name} not selected"
    assert learn_rate, "Learning rate is empty or 0"
    assert isinstance(batch_size, int), "Batch size must be integer"
    assert batch_size > 0, "Batch size must be positive"
    assert isinstance(gradient_step, int), "Gradient accumulation step must be integer"
    assert gradient_step > 0, "Gradient accumulation step must be positive"
    assert data_root, "Dataset directory is empty"
    assert os.path.isdir(data_root), "Dataset directory doesn't exist"
    assert os.listdir(data_root), "Dataset directory is empty"
    assert template_filename, "Prompt template file not selected"
    assert template_file, f"Prompt template file {template_filename} not found"
    assert os.path.isfile(template_file), f"Prompt template file {template_filename} doesn't exist"
    assert steps, "Max steps is empty or 0"
    assert isinstance(steps, int), "Max steps must be integer"
    assert steps > 0, "Max steps must be positive"
    assert isinstance(save_model_every, int), "Save {name} must be integer"
    assert save_model_every >= 0, "Save {name} must be positive or 0"
    assert isinstance(create_image_every, int), "Create image must be integer"
    assert create_image_every >= 0, "Create image must be positive or 0"
    if save_model_every or create_image_every:
        assert log_directory, "Log directory is empty"

import os
def train_embedding(id_task, embedding_name, learn_rate, batch_size, gradient_step, data_root, log_directory, training_width, training_height, varsize, steps, clip_grad_mode, clip_grad_value, shuffle_tags, tag_drop_out, latent_sampling_method, create_image_every, save_embedding_every, template_filename, save_image_with_stored_embedding, preview_from_txt2img, preview_prompt, preview_negative_prompt, preview_steps, preview_sampler_index, preview_cfg_scale, preview_seed, preview_width, preview_height):
    save_embedding_every = save_embedding_every or 0
    create_image_every = create_image_every or 0
    template_file = template_filename
    validate_train_inputs(embedding_name, learn_rate, batch_size, gradient_step, data_root, template_file, template_filename, steps, save_embedding_every, create_image_every, log_directory, name="embedding")
    template_file = template_filename


    job = "train-embedding"
    textinfo = "Initializing textual inversion training..."
    job_count = steps
    import os
    import datetime
    filename = os.path.join(root.embeddings_dir, f'{embedding_name}.pt')

    log_directory = os.path.join(log_directory, datetime.datetime.now().strftime("%Y-%m-%d"), embedding_name)
    unload_models_when_training=False
    unload = unload_models_when_training

    if save_embedding_every > 0:
        embedding_dir = os.path.join(log_directory, "embeddings")
        os.makedirs(embedding_dir, exist_ok=True)
    else:
        embedding_dir = None

    if create_image_every > 0:
        images_dir = os.path.join(log_directory, "images")
        os.makedirs(images_dir, exist_ok=True)
    else:
        images_dir = None

    if create_image_every > 0 and save_image_with_stored_embedding:
        images_embeds_dir = os.path.join(log_directory, "image_embeddings")
        os.makedirs(images_embeds_dir, exist_ok=True)
    else:
        images_embeds_dir = None

    hijack = model_hijack

    embedding = hijack.embedding_db.word_embeddings[embedding_name]
    checkpoint = root.model

    initial_step = embedding.step or 0
    if initial_step >= steps:
        extinfo = "Model has already been trained beyond specified max steps"
        return embedding, filename
    
    scheduler = LearnRateScheduler(learn_rate, steps, initial_step)
    clip_grad = torch.nn.utils.clip_grad_value_ if clip_grad_mode == "value" else \
        torch.nn.utils.clip_grad_norm_ if clip_grad_mode == "norm" else \
        None
    if clip_grad:
        clip_grad_sched = LearnRateScheduler(clip_grad_value, steps, initial_step, verbose=False)
    # dataset loading may take a while, so input validations and early returns should be done before this
    textinfo = f"Preparing dataset from {html.escape(data_root)}..."
    parallel_processing_allowed=False
    old_parallel_processing_allowed = parallel_processing_allowed
    
    training_enable_tensorboard=True
    if training_enable_tensorboard:
        tensorboard_writer = tensorboard_setup(log_directory)

    pin_memory = False

    ds = PersonalizedBase(data_root=data_root, width=training_width, height=training_height, repeats=1, placeholder_token=embedding_name, model=model, cond_model=model.cond_stage_model, device=root.device, template_file=template_file, batch_size=batch_size, gradient_step=gradient_step, shuffle_tags=shuffle_tags, tag_drop_out=tag_drop_out, latent_sampling_method=latent_sampling_method, varsize=varsize)

    import datetime
    import json
    import os

    saved_params_shared = {"model_name", "model_hash", "initial_step", "num_of_dataset_images", "learn_rate", "batch_size", "clip_grad_mode", "clip_grad_value", "gradient_step", "data_root", "log_directory", "training_width", "training_height", "steps", "create_image_every", "template_file", "gradient_step", "latent_sampling_method"}
    saved_params_ti = {"embedding_name", "num_vectors_per_token", "save_embedding_every", "save_image_with_stored_embedding"}
    saved_params_hypernet = {"hypernetwork_name", "layer_structure", "activation_func", "weight_init", "add_layer_norm", "use_dropout", "save_hypernetwork_every"}
    saved_params_all = saved_params_shared | saved_params_ti | saved_params_hypernet
    saved_params_previews = {"preview_prompt", "preview_negative_prompt", "preview_steps", "preview_sampler_index", "preview_cfg_scale", "preview_seed", "preview_width", "preview_height"}


    def save_settings_to_file(log_directory, all_params):
        now = datetime.datetime.now()
        params = {"datetime": now.strftime("%Y-%m-%d %H:%M:%S")}

        keys = saved_params_all
        if all_params.get('preview_from_txt2img'):
          keys = keys | saved_params_previews

        params.update({k: v for k, v in all_params.items() if k in keys})

        filename = f'settings-{now.strftime("%Y-%m-%d-%H-%M-%S")}.json'
        with open(os.path.join(log_directory, filename), "w") as file:
            json.dump(params, file, indent=4)

    save_training_settings_to_txt=True
    if save_training_settings_to_txt:
        save_settings_to_file(log_directory, {**dict(model_name=root.model_checkpoint, model_hash=ckpt_hash, num_of_dataset_images=len(ds), num_vectors_per_token=len(embedding.vec)), **locals()})

    latent_sampling_method = ds.latent_sampling_method

    dl = PersonalizedDataLoader(ds, latent_sampling_method=latent_sampling_method, batch_size=ds.batch_size, pin_memory=pin_memory)

    if unload:
        parallel_processing_allowed = False
        model.first_stage_model.to(root.map_location)

    embedding.vec.requires_grad = True
    optimizer = torch.optim.AdamW([embedding.vec], lr=scheduler.learn_rate, weight_decay=0.0)
    save_optimizer_state=True
    if save_optimizer_state:
        optimizer_state_dict = None
        if os.path.exists(filename + '.optim'):
            optimizer_saved_dict = torch.load(filename + '.optim', map_location='cuda')
            if embedding.checksum() == optimizer_saved_dict.get('hash', None):
                optimizer_state_dict = optimizer_saved_dict.get('optimizer_state_dict', None)
    
        if optimizer_state_dict is not None:
            optimizer.load_state_dict(optimizer_state_dict)
            print("Loaded existing optimizer from checkpoint")
        else:
            print("No saved optimizer exists in checkpoint")

    scaler = torch.cuda.amp.GradScaler()

    batch_size = ds.batch_size
    gradient_step = ds.gradient_step
    # n steps = batch_size * gradient_step * n image processed
    steps_per_epoch = len(ds) // batch_size // gradient_step
    max_steps_per_epoch = len(ds) // batch_size - (len(ds) // batch_size) % gradient_step
    loss_step = 0
    _loss_step = 0 #internal

    last_saved_file = "<none>"
    last_saved_image = "<none>"
    forced_filename = "<none>"
    embedding_yet_to_be_embedded = False

    # is_training_inpainting_model = root.model.conditioning_key in {'hybrid', 'concat'}
    img_c = None

   ## hijack_checkpoint

    from torch.utils.checkpoint import checkpoint

    import ldm.modules.attention
    import ldm.modules.diffusionmodules.openaimodel


    def BasicTransformerBlock_forward(self, x, context=None):
        return checkpoint(self._forward, x, context)


    def AttentionBlock_forward(self, x):
        return checkpoint(self._forward, x)


    def ResBlock_forward(self, x, emb):
        return checkpoint(self._forward, x, emb)


    stored = []


    def hijack_checkpoint_add():
        if len(stored) != 0:
            return

        stored.extend([
            ldm.modules.attention.BasicTransformerBlock.forward,
            ldm.modules.diffusionmodules.openaimodel.ResBlock.forward,
            ldm.modules.diffusionmodules.openaimodel.AttentionBlock.forward
        ])

        ldm.modules.attention.BasicTransformerBlock.forward = BasicTransformerBlock_forward
        ldm.modules.diffusionmodules.openaimodel.ResBlock.forward = ResBlock_forward
        ldm.modules.diffusionmodules.openaimodel.AttentionBlock.forward = AttentionBlock_forward


    def hijack_checkpoint_remove():
        if len(stored) == 0:
            return

        ldm.modules.attention.BasicTransformerBlock.forward = stored[0]
        ldm.modules.diffusionmodules.openaimodel.ResBlock.forward = stored[1]
        ldm.modules.diffusionmodules.openaimodel.AttentionBlock.forward = stored[2]

        stored.clear()

    pbar = tqdm.tqdm(total=steps - initial_step)
    try:
        hijack_checkpoint_add()

        for i in range((steps-initial_step) * gradient_step):
            # if scheduler.finished:
            #     break
            # if interrupted:
            #     break
            for j, batch in enumerate(dl):
                # works as a drop_last=True for gradient accumulation
                if j == max_steps_per_epoch:
                    break
                scheduler.apply(optimizer, embedding.step)
                # if scheduler.finished:
                #     break
                # if shared.state.interrupted:
                #     break
                if clip_grad:
                    clip_grad_sched.step(embedding.step)
                with autocast():
                    x = batch.latent_sample.to(torch.float32, non_blocking=pin_memory)
                    c = model.cond_stage_model(batch.cond_text).float()
                    is_training_inpainting_model=False
                    if is_training_inpainting_model:
                        if img_c is None:
                            img_c = txt2img_image_conditioning(shared.sd_model, c, training_width, training_height)

                        cond = {"c_concat": [img_c], "c_crossattn": [c]}
                    else:
                        cond = c
                    cond = cond.to("cuda")
                    loss = model(x, cond)[0] / gradient_step
                    del x

                    _loss_step += loss.item()
                scaler.scale(loss).backward()

                # go back until we reach gradient accumulation steps
                if (j + 1) % gradient_step != 0:
                    continue
                
                if clip_grad:
                    clip_grad(embedding.vec, clip_grad_sched.learn_rate)

                scaler.step(optimizer)
                scaler.update()
                embedding.step += 1
                pbar.update()
                optimizer.zero_grad(set_to_none=True)
                loss_step = _loss_step
                _loss_step = 0

                steps_done = embedding.step + 1

                epoch_num = embedding.step // steps_per_epoch
                epoch_step = embedding.step % steps_per_epoch

                description = f"Training textual inversion [Epoch {epoch_num}: {epoch_step+1}/{steps_per_epoch}] loss: {loss_step:.7f}"
                pbar.set_description(description)
                if embedding_dir is not None and steps_done % save_embedding_every == 0:
                    # Before saving, change name to match current checkpoint.
                    embedding_name_every = f'{embedding_name}-{steps_done}'
                    last_saved_file = os.path.join(embedding_dir, f'{embedding_name_every}.pt')
                    save_embedding(embedding, optimizer, checkpoint, embedding_name_every, last_saved_file, remove_cached_checksum=True)
                    embedding_yet_to_be_embedded = True

                write_loss(log_directory, "textual_inversion_loss.csv", embedding.step, steps_per_epoch, {
                    "loss": f"{loss_step:.7f}",
                    "learn_rate": scheduler.learn_rate
                })

                training_tensorboard_save_images=False

                if images_dir is not None and steps_done % create_image_every == 0:
                    forced_filename = f'{embedding_name}-{steps_done}'
                    last_saved_image = os.path.join(images_dir, forced_filename)

                    model.first_stage_model.to(root.device)

                    # p = processing.StableDiffusionProcessingTxt2Img(
                    #     sd_model=shared.sd_model,
                    #     do_not_save_grid=True,
                    #     do_not_save_samples=True,
                    #     do_not_reload_embeddings=True,
                    # )

                    if preview_from_txt2img:
                        p.prompt = preview_prompt
                        p.negative_prompt = preview_negative_prompt
                        p.steps = preview_steps
                        p.sampler_name = sd_samplers.samplers[preview_sampler_index].name
                        p.cfg_scale = preview_cfg_scale
                        p.seed = preview_seed
                        p.width = preview_width
                        p.height = preview_height
                    else:
                        prompt = batch.cond_text[0]
                        steps = 20
                        width = training_width
                        height = training_height

                    preview_text = prompt

                    # processed = processing.process_images(p)
                    # image = processed.images[0] if len(processed.images) > 0 else None

                    # def assign_current_image(self, image):
                    #     self.current_image = image
                    #     self.id_live_preview += 1
                    # if unload:
                    #     model.first_stage_model.to("cuda")

                    # if image is not None:
                    #     assign_current_image(image)

                    #     last_saved_image, last_text_info = images.save_image(image, images_dir, "", seed, prompt, samples_format="png", processed.infotexts[0], p=p, forced_filename=forced_filename, save_to_dirs=False)
                    #     last_saved_image += f", prompt: {preview_text}"

                    #     if training_enable_tensorboard and training_tensorboard_save_images:
                    #         tensorboard_add_image(tensorboard_writer, f"Validation at epoch {epoch_num}", image, embedding.step)

                    # if save_image_with_stored_embedding and os.path.exists(last_saved_file) and embedding_yet_to_be_embedded:

                    #     last_saved_image_chunks = os.path.join(images_embeds_dir, f'{embedding_name}-{steps_done}.png')

                    #     info = PngImagePlugin.PngInfo()
                    #     data = torch.load(last_saved_file)
                    #     info.add_text("sd-ti-embedding", embedding_to_b64(data))

                    #     title = "<{}>".format(data.get('name', '???'))

                    #     try:
                    #         vectorSize = list(data['string_to_param'].values())[0].shape[0]
                    #     except Exception as e:
                    #         vectorSize = '?'

                    #     checkpoint = root.model_checkpoint
                    #     footer_left = checkpoint.model_name
                    #     footer_mid = '[{}]'.format(checkpoint.shorthash)
                    #     footer_right = '{}v {}s'.format(vectorSize, steps_done)

                    #     captioned_image = caption_image_overlay(image, title, footer_left, footer_mid, footer_right)
                    #     captioned_image = insert_image_data_embed(captioned_image, data)

                    #     captioned_image.save(last_saved_image_chunks, "PNG", pnginfo=info)
                    #     embedding_yet_to_be_embedded = False

                    # last_saved_image, last_text_info = images.save_image(image, images_dir, "", p.seed, p.prompt, shared.opts.samples_format, processed.infotexts[0], p=p, forced_filename=forced_filename, save_to_dirs=False)
                    # last_saved_image += f", prompt: {preview_text}"

                job_no = embedding.step

                textinfo = f"""
# <p>
# Loss: {loss_step:.7f}<br/>
# Step: {steps_done}<br/>
# Last prompt: {html.escape(batch.cond_text[0])}<br/>
# Last saved embedding: {html.escape(last_saved_file)}<br/>
# Last saved image: {html.escape(last_saved_image)}<br/>
# </p>
# """
        filename = os.path.join(root.embeddings_dir, f'{embedding_name}.pt')
        save_embedding(embedding, optimizer, checkpoint, embedding_name, filename, remove_cached_checksum=True)
    except Exception:
        print(traceback.format_exc(), file=sys.stderr)
        pass
    finally:
        pbar.leave = False
        pbar.close()
        model.first_stage_model.to(root.device)
        parallel_processing_allowed = old_parallel_processing_allowed
        hijack_checkpoint_remove()

    return embedding, filename


def save_embedding(embedding, optimizer, checkpoint, embedding_name, filename, remove_cached_checksum=True):
    old_embedding_name = embedding.name
    old_sd_checkpoint = embedding.sd_checkpoint if hasattr(embedding, "sd_checkpoint") else None
    old_sd_checkpoint_name = embedding.sd_checkpoint_name if hasattr(embedding, "sd_checkpoint_name") else None
    old_cached_checksum = embedding.cached_checksum if hasattr(embedding, "cached_checksum") else None
    try:
        embedding.sd_checkpoint = checkpoint.shorthash
        embedding.sd_checkpoint_name = checkpoint.model_name
        if remove_cached_checksum:
            embedding.cached_checksum = None
        embedding.name = embedding_name
        embedding.optimizer_state_dict = optimizer.state_dict()
        embedding.save(filename)
    except:
        embedding.sd_checkpoint = old_sd_checkpoint
        embedding.sd_checkpoint_name = old_sd_checkpoint_name
        embedding.name = old_embedding_name
        embedding.cached_checksum = old_cached_checksum
        raise

#cond_func
import importlib

class CondFunc:
    def __new__(cls, orig_func, sub_func, cond_func):
        self = super(CondFunc, cls).__new__(cls)
        if isinstance(orig_func, str):
            func_path = orig_func.split('.')
            for i in range(len(func_path)-1, -1, -1):
                try:
                    resolved_obj = importlib.import_module('.'.join(func_path[:i]))
                    break
                except ImportError:
                    pass
            for attr_name in func_path[i:-1]:
                resolved_obj = getattr(resolved_obj, attr_name)
            orig_func = getattr(resolved_obj, func_path[-1])
            setattr(resolved_obj, func_path[-1], lambda *args, **kwargs: self(*args, **kwargs))
        self.__init__(orig_func, sub_func, cond_func)
        return lambda *args, **kwargs: self(*args, **kwargs)
    def __init__(self, orig_func, sub_func, cond_func):
        self.__orig_func = orig_func
        self.__sub_func = sub_func
        self.__cond_func = cond_func
    def __call__(self, *args, **kwargs):
        if not self.__cond_func or self.__cond_func(self.__orig_func, *args, **kwargs):
            return self.__sub_func(self.__orig_func, *args, **kwargs)
        else:
            return self.__orig_func(*args, **kwargs)

#errors
import sys
import traceback


def print_error_explanation(message):
    lines = message.strip().split("\n")
    max_len = max([len(x) for x in lines])

    print('=' * max_len, file=sys.stderr)
    for line in lines:
        print(line, file=sys.stderr)
    print('=' * max_len, file=sys.stderr)


def display(e: Exception, task):
    print(f"{task or 'error'}: {type(e).__name__}", file=sys.stderr)
    print(traceback.format_exc(), file=sys.stderr)

    message = str(e)
    if "copying a param with shape torch.Size([640, 1024]) from checkpoint, the shape in current model is torch.Size([640, 768])" in message:
        print_error_explanation("""
The most likely cause of this is you are trying to load Stable Diffusion 2.0 model without specifying its config file.
See https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features#stable-diffusion-20 for how to solve this.
        """)


already_displayed = {}


def display_once(e: Exception, task):
    if task in already_displayed:
        return

    display(e, task)

    already_displayed[task] = 1


def run(code, task):
    try:
        code()
    except Exception as e:
        display(task, e)

#sub_quad_attn
from functools import partial
import torch
from torch import Tensor
from torch.utils.checkpoint import checkpoint
import math
from typing import Optional, NamedTuple, List

def without_autocast(disable=False):
    return torch.autocast("cuda", enabled=False) if torch.is_autocast_enabled() and not disable else contextlib.nullcontext()

def narrow_trunc(
    input: Tensor,
    dim: int,
    start: int,
    length: int
) -> Tensor:
    return torch.narrow(input, dim, start, length if input.shape[dim] >= start + length else input.shape[dim] - start)


class AttnChunk(NamedTuple):
    exp_values: Tensor
    exp_weights_sum: Tensor
    max_score: Tensor


class SummarizeChunk:
    @staticmethod
    def __call__(
        query: Tensor,
        key: Tensor,
        value: Tensor,
    ) -> AttnChunk: ...


class ComputeQueryChunkAttn:
    @staticmethod
    def __call__(
        query: Tensor,
        key: Tensor,
        value: Tensor,
    ) -> Tensor: ...


def _summarize_chunk(
    query: Tensor,
    key: Tensor,
    value: Tensor,
    scale: float,
) -> AttnChunk:
    attn_weights = torch.baddbmm(
        torch.empty(1, 1, 1, device=query.device, dtype=query.dtype),
        query,
        key.transpose(1,2),
        alpha=scale,
        beta=0,
    )
    max_score, _ = torch.max(attn_weights, -1, keepdim=True)
    max_score = max_score.detach()
    exp_weights = torch.exp(attn_weights - max_score)
    exp_values = torch.bmm(exp_weights, value) if query.device.type == 'mps' else torch.bmm(exp_weights, value.to(exp_weights.dtype)).to(value.dtype)
    max_score = max_score.squeeze(-1)
    return AttnChunk(exp_values, exp_weights.sum(dim=-1), max_score)


def _query_chunk_attention(
    query: Tensor,
    key: Tensor,
    value: Tensor,
    summarize_chunk: SummarizeChunk,
    kv_chunk_size: int,
) -> Tensor:
    batch_x_heads, k_tokens, k_channels_per_head = key.shape
    _, _, v_channels_per_head = value.shape

    def chunk_scanner(chunk_idx: int) -> AttnChunk:
        key_chunk = narrow_trunc(
            key,
            1,
            chunk_idx,
            kv_chunk_size
        )
        value_chunk = narrow_trunc(
            value,
            1,
            chunk_idx,
            kv_chunk_size
        )
        return summarize_chunk(query, key_chunk, value_chunk)

    chunks: List[AttnChunk] = [
        chunk_scanner(chunk) for chunk in torch.arange(0, k_tokens, kv_chunk_size)
    ]
    acc_chunk = AttnChunk(*map(torch.stack, zip(*chunks)))
    chunk_values, chunk_weights, chunk_max = acc_chunk

    global_max, _ = torch.max(chunk_max, 0, keepdim=True)
    max_diffs = torch.exp(chunk_max - global_max)
    chunk_values *= torch.unsqueeze(max_diffs, -1)
    chunk_weights *= max_diffs

    all_values = chunk_values.sum(dim=0)
    all_weights = torch.unsqueeze(chunk_weights, -1).sum(dim=0)
    return all_values / all_weights


# TODO: refactor CrossAttention#get_attention_scores to share code with this
def _get_attention_scores_no_kv_chunking(
    query: Tensor,
    key: Tensor,
    value: Tensor,
    scale: float,
) -> Tensor:
    attn_scores = torch.baddbmm(
        torch.empty(1, 1, 1, device=query.device, dtype=query.dtype),
        query,
        key.transpose(1,2),
        alpha=scale,
        beta=0,
    )
    attn_probs = attn_scores.softmax(dim=-1)
    del attn_scores
    hidden_states_slice = torch.bmm(attn_probs, value) if query.device.type == 'mps' else torch.bmm(attn_probs, value.to(attn_probs.dtype)).to(value.dtype)
    return hidden_states_slice

#sub_quadratic_attention
class ScannedChunk(NamedTuple):
    chunk_idx: int
    attn_chunk: AttnChunk

def efficient_dot_product_attention(
    query: Tensor,
    key: Tensor,
    value: Tensor,
    query_chunk_size=1024,
    kv_chunk_size: Optional[int] = None,
    kv_chunk_size_min: Optional[int] = None,
    use_checkpoint=True,
):
    """Computes efficient dot-product attention given query, key, and value.
      This is efficient version of attention presented in
      https://arxiv.org/abs/2112.05682v2 which comes with O(sqrt(n)) memory requirements.
      Args:
        query: queries for calculating attention with shape of
          `[batch * num_heads, tokens, channels_per_head]`.
        key: keys for calculating attention with shape of
          `[batch * num_heads, tokens, channels_per_head]`.
        value: values to be used in attention with shape of
          `[batch * num_heads, tokens, channels_per_head]`.
        query_chunk_size: int: query chunks size
        kv_chunk_size: Optional[int]: key/value chunks size. if None: defaults to sqrt(key_tokens)
        kv_chunk_size_min: Optional[int]: key/value minimum chunk size. only considered when kv_chunk_size is None. changes `sqrt(key_tokens)` into `max(sqrt(key_tokens), kv_chunk_size_min)`, to ensure our chunk sizes don't get too small (smaller chunks = more chunks = less concurrent work done).
        use_checkpoint: bool: whether to use checkpointing (recommended True for training, False for inference)
      Returns:
        Output of shape `[batch * num_heads, query_tokens, channels_per_head]`.
      """
    batch_x_heads, q_tokens, q_channels_per_head = query.shape
    _, k_tokens, _ = key.shape
    scale = q_channels_per_head ** -0.5

    kv_chunk_size = min(kv_chunk_size or int(math.sqrt(k_tokens)), k_tokens)
    if kv_chunk_size_min is not None:
        kv_chunk_size = max(kv_chunk_size, kv_chunk_size_min)

    def get_query_chunk(chunk_idx: int) -> Tensor:
        return narrow_trunc(
            query,
            1,
            chunk_idx,
            min(query_chunk_size, q_tokens)
        )
    
    summarize_chunk: SummarizeChunk = partial(_summarize_chunk, scale=scale)
    summarize_chunk: SummarizeChunk = partial(checkpoint, summarize_chunk) if use_checkpoint else summarize_chunk
    compute_query_chunk_attn: ComputeQueryChunkAttn = partial(
        _get_attention_scores_no_kv_chunking,
        scale=scale
    ) if k_tokens <= kv_chunk_size else (
        # fast-path for when there's just 1 key-value chunk per query chunk (this is just sliced attention btw)
        partial(
            _query_chunk_attention,
            kv_chunk_size=kv_chunk_size,
            summarize_chunk=summarize_chunk,
        )
    )

    if q_tokens <= query_chunk_size:
        # fast-path for when there's just 1 query chunk
        return compute_query_chunk_attn(
            query=query,
            key=key,
            value=value,
        )
    
    # TODO: maybe we should use torch.empty_like(query) to allocate storage in-advance,
    # and pass slices to be mutated, instead of torch.cat()ing the returned slices
    res = torch.cat([
        compute_query_chunk_attn(
            query=get_query_chunk(i * query_chunk_size),
            key=key,
            value=value,
        ) for i in range(math.ceil(q_tokens / query_chunk_size))
    ], dim=1)
    return res

#cache
import hashlib
import json
import os.path

import filelock

cache_filename = os.path.join(root.data_dir, "cache.json")
cache_data = None


def dump_cache():
    with filelock.FileLock(cache_filename+".lock"):
        with open(cache_filename, "w", encoding="utf8") as file:
            json.dump(cache_data, file, indent=4)


def cache(subsection):
    global cache_data

    if cache_data is None:
        with filelock.FileLock(cache_filename+".lock"):
            if not os.path.isfile(cache_filename):
                cache_data = {}
            else:
                with open(cache_filename, "r", encoding="utf8") as file:
                    cache_data = json.load(file)

    s = cache_data.get(subsection, {})
    cache_data[subsection] = s

    return s


def calculate_sha256(filename):
    hash_sha256 = hashlib.sha256()
    blksize = 1024 * 1024

    with open(filename, "rb") as f:
        for chunk in iter(lambda: f.read(blksize), b""):
            hash_sha256.update(chunk)

    return hash_sha256.hexdigest()


def sha256_from_cache(filename, title):
    hashes = cache("hashes")
    ondisk_mtime = os.path.getmtime(filename)

    if title not in hashes:
        return None

    cached_sha256 = hashes[title].get("sha256", None)
    cached_mtime = hashes[title].get("mtime", 0)

    if ondisk_mtime > cached_mtime or cached_sha256 is None:
        return None

    return cached_sha256


def sha256(filename, title):
    hashes = cache("hashes")

    sha256_value = sha256_from_cache(filename, title)
    if sha256_value is not None:
        return sha256_value

    print(f"Calculating sha256 for {filename}: ", end='')
    sha256_value = calculate_sha256(filename)
    print(f"{sha256_value}")

    hashes[title] = {
        "mtime": os.path.getmtime(filename),
        "sha256": sha256_value,
    }

    dump_cache()

    return sha256_value

#hypernetwork
loaded_hypernetworks=[]
import csv
import datetime
import glob
import html
import os
import sys
import traceback
import inspect

import torch
import tqdm
from einops import rearrange, repeat
from ldm.util import default
from torch import einsum
from torch.nn.init import normal_, xavier_normal_, xavier_uniform_, kaiming_normal_, kaiming_uniform_, zeros_

from collections import defaultdict, deque
from statistics import stdev, mean


optimizer_dict = {optim_name : cls_obj for optim_name, cls_obj in inspect.getmembers(torch.optim, inspect.isclass) if optim_name != "Optimizer"}

class HypernetworkModule(torch.nn.Module):
    activation_dict = {
        "linear": torch.nn.Identity,
        "relu": torch.nn.ReLU,
        "leakyrelu": torch.nn.LeakyReLU,
        "elu": torch.nn.ELU,
        "swish": torch.nn.Hardswish,
        "tanh": torch.nn.Tanh,
        "sigmoid": torch.nn.Sigmoid,
    }
    activation_dict.update({cls_name.lower(): cls_obj for cls_name, cls_obj in inspect.getmembers(torch.nn.modules.activation) if inspect.isclass(cls_obj) and cls_obj.__module__ == 'torch.nn.modules.activation'})

    def __init__(self, dim, state_dict=None, layer_structure=None, activation_func=None, weight_init='Normal',
                 add_layer_norm=False, activate_output=False, dropout_structure=None):
        super().__init__()

        self.multiplier = 1.0

        assert layer_structure is not None, "layer_structure must not be None"
        assert layer_structure[0] == 1, "Multiplier Sequence should start with size 1!"
        assert layer_structure[-1] == 1, "Multiplier Sequence should end with size 1!"

        linears = []
        for i in range(len(layer_structure) - 1):

            # Add a fully-connected layer
            linears.append(torch.nn.Linear(int(dim * layer_structure[i]), int(dim * layer_structure[i+1])))

            # Add an activation func except last layer
            if activation_func == "linear" or activation_func is None or (i >= len(layer_structure) - 2 and not activate_output):
                pass
            elif activation_func in self.activation_dict:
                linears.append(self.activation_dict[activation_func]())
            else:
                raise RuntimeError(f'hypernetwork uses an unsupported activation function: {activation_func}')

            # Add layer normalization
            if add_layer_norm:
                linears.append(torch.nn.LayerNorm(int(dim * layer_structure[i+1])))

            # Everything should be now parsed into dropout structure, and applied here.
            # Since we only have dropouts after layers, dropout structure should start with 0 and end with 0.
            if dropout_structure is not None and dropout_structure[i+1] > 0:
                assert 0 < dropout_structure[i+1] < 1, "Dropout probability should be 0 or float between 0 and 1!"
                linears.append(torch.nn.Dropout(p=dropout_structure[i+1]))
            # Code explanation : [1, 2, 1] -> dropout is missing when last_layer_dropout is false. [1, 2, 2, 1] -> [0, 0.3, 0, 0], when its True, [0, 0.3, 0.3, 0].

        self.linear = torch.nn.Sequential(*linears)

        if state_dict is not None:
            self.fix_old_state_dict(state_dict)
            self.load_state_dict(state_dict)
        else:
            for layer in self.linear:
                if type(layer) == torch.nn.Linear or type(layer) == torch.nn.LayerNorm:
                    w, b = layer.weight.data, layer.bias.data
                    if weight_init == "Normal" or type(layer) == torch.nn.LayerNorm:
                        normal_(w, mean=0.0, std=0.01)
                        normal_(b, mean=0.0, std=0)
                    elif weight_init == 'XavierUniform':
                        xavier_uniform_(w)
                        zeros_(b)
                    elif weight_init == 'XavierNormal':
                        xavier_normal_(w)
                        zeros_(b)
                    elif weight_init == 'KaimingUniform':
                        kaiming_uniform_(w, nonlinearity='leaky_relu' if 'leakyrelu' == activation_func else 'relu')
                        zeros_(b)
                    elif weight_init == 'KaimingNormal':
                        kaiming_normal_(w, nonlinearity='leaky_relu' if 'leakyrelu' == activation_func else 'relu')
                        zeros_(b)
                    else:
                        raise KeyError(f"Key {weight_init} is not defined as initialization!")
        self.to(root.device)

    def fix_old_state_dict(self, state_dict):
        changes = {
            'linear1.bias': 'linear.0.bias',
            'linear1.weight': 'linear.0.weight',
            'linear2.bias': 'linear.1.bias',
            'linear2.weight': 'linear.1.weight',
        }

        for fr, to in changes.items():
            x = state_dict.get(fr, None)
            if x is None:
                continue

            del state_dict[fr]
            state_dict[to] = x

    def forward(self, x):
        return x + self.linear(x) * (self.multiplier if not self.training else 1)

    def trainables(self):
        layer_structure = []
        for layer in self.linear:
            if type(layer) == torch.nn.Linear or type(layer) == torch.nn.LayerNorm:
                layer_structure += [layer.weight, layer.bias]
        return layer_structure


#param layer_structure : sequence used for length, use_dropout : controlling boolean, last_layer_dropout : for compatibility check.
def parse_dropout_structure(layer_structure, use_dropout, last_layer_dropout):
    if layer_structure is None:
        layer_structure = [1, 2, 1]
    if not use_dropout:
        return [0] * len(layer_structure)
    dropout_values = [0]
    dropout_values.extend([0.3] * (len(layer_structure) - 3))
    if last_layer_dropout:
        dropout_values.append(0.3)
    else:
        dropout_values.append(0)
    dropout_values.append(0)
    return dropout_values

def shorthash(self):
        sha256 = sha256(self.filename, f'hypernet/{self.name}')

        return sha256[0:10]

class Hypernetwork:
    filename = None
    name = None

    def __init__(self, name=None, enable_sizes=None, layer_structure=None, activation_func=None, weight_init=None, add_layer_norm=False, use_dropout=False, activate_output=False, **kwargs):
        self.filename = None
        self.name = name
        self.layers = {}
        self.step = 0
        self.sd_checkpoint = None
        self.sd_checkpoint_name = None
        self.layer_structure = layer_structure
        self.activation_func = activation_func
        self.weight_init = weight_init
        self.add_layer_norm = add_layer_norm
        self.use_dropout = use_dropout
        self.activate_output = activate_output
        self.last_layer_dropout = kwargs.get('last_layer_dropout', True)
        self.dropout_structure = kwargs.get('dropout_structure', None)
        if self.dropout_structure is None:
            self.dropout_structure = parse_dropout_structure(self.layer_structure, self.use_dropout, self.last_layer_dropout)
        self.optimizer_name = None
        self.optimizer_state_dict = None
        self.optional_info = None

        for size in enable_sizes or []:
            self.layers[size] = (
                HypernetworkModule(size, None, self.layer_structure, self.activation_func, self.weight_init,
                                   self.add_layer_norm, self.activate_output, dropout_structure=self.dropout_structure),
                HypernetworkModule(size, None, self.layer_structure, self.activation_func, self.weight_init,
                                   self.add_layer_norm, self.activate_output, dropout_structure=self.dropout_structure),
            )
        self.eval()

    def weights(self):
        res = []
        for k, layers in self.layers.items():
            for layer in layers:
                res += layer.parameters()
        return res

    def train(self, mode=True):
        for k, layers in self.layers.items():
            for layer in layers:
                layer.train(mode=mode)
                for param in layer.parameters():
                    param.requires_grad = mode

    def to(self, device):
        for k, layers in self.layers.items():
            for layer in layers:
                layer.to(device)

        return self

    def set_multiplier(self, multiplier):
        for k, layers in self.layers.items():
            for layer in layers:
                layer.multiplier = multiplier

        return self

    def eval(self):
        for k, layers in self.layers.items():
            for layer in layers:
                layer.eval()
                for param in layer.parameters():
                    param.requires_grad = False

    def save(self, filename):
        state_dict = {}
        optimizer_saved_dict = {}

        for k, v in self.layers.items():
            state_dict[k] = (v[0].state_dict(), v[1].state_dict())

        state_dict['step'] = self.step
        state_dict['name'] = self.name
        state_dict['layer_structure'] = self.layer_structure
        state_dict['activation_func'] = self.activation_func
        state_dict['is_layer_norm'] = self.add_layer_norm
        state_dict['weight_initialization'] = self.weight_init
        state_dict['sd_checkpoint'] = self.sd_checkpoint
        state_dict['sd_checkpoint_name'] = self.sd_checkpoint_name
        state_dict['activate_output'] = self.activate_output
        state_dict['use_dropout'] = self.use_dropout
        state_dict['dropout_structure'] = self.dropout_structure
        state_dict['last_layer_dropout'] = (self.dropout_structure[-2] != 0) if self.dropout_structure is not None else self.last_layer_dropout
        state_dict['optional_info'] = self.optional_info if self.optional_info else None

        if self.optimizer_name is not None:
            optimizer_saved_dict['optimizer_name'] = self.optimizer_name

        torch.save(state_dict, filename)
        save_optimizer_state=True
        if save_optimizer_state and self.optimizer_state_dict:
            optimizer_saved_dict['hash'] = self.shorthash()
            optimizer_saved_dict['optimizer_state_dict'] = self.optimizer_state_dict
            torch.save(optimizer_saved_dict, filename + '.optim')
    
    def sha256(filename, title):
        hashes = cache("hashes")

        sha256_value = sha256_from_cache(filename, title)
        if sha256_value is not None:
            return sha256_value

        print(f"Calculating sha256 for {filename}: ", end='')
        sha256_value = calculate_sha256(filename)
        print(f"{sha256_value}")

        hashes[title] = {
            "mtime": os.path.getmtime(filename),
            "sha256": sha256_value,
        }

        dump_cache()

        return sha256_value
    def load(self, filename):
        self.filename = filename
        if self.name is None:
            self.name = os.path.splitext(os.path.basename(filename))[0]

        state_dict = torch.load(filename, map_location='cuda')

        self.layer_structure = state_dict.get('layer_structure', [1, 2, 1])
        self.optional_info = state_dict.get('optional_info', None)
        self.activation_func = state_dict.get('activation_func', None)
        self.weight_init = state_dict.get('weight_initialization', 'Normal')
        self.add_layer_norm = state_dict.get('is_layer_norm', False)
        self.dropout_structure = state_dict.get('dropout_structure', None)
        self.use_dropout = True if self.dropout_structure is not None and any(self.dropout_structure) else state_dict.get('use_dropout', False)
        self.activate_output = state_dict.get('activate_output', True)
        self.last_layer_dropout = state_dict.get('last_layer_dropout', False)
        # Dropout structure should have same length as layer structure, Every digits should be in [0,1), and last digit must be 0.
        if self.dropout_structure is None:
            self.dropout_structure = parse_dropout_structure(self.layer_structure, self.use_dropout, self.last_layer_dropout)

        print_hypernet_extra=True
        if print_hypernet_extra:
            if self.optional_info is not None:
                print(f"  INFO:\n {self.optional_info}\n")

            print(f"  Layer structure: {self.layer_structure}")
            print(f"  Activation function: {self.activation_func}")
            print(f"  Weight initialization: {self.weight_init}")
            print(f"  Layer norm: {self.add_layer_norm}")
            print(f"  Dropout usage: {self.use_dropout}" )
            print(f"  Activate last layer: {self.activate_output}")
            print(f"  Dropout structure: {self.dropout_structure}")

        optimizer_saved_dict = torch.load(self.filename + '.optim', map_location='cuda') if os.path.exists(self.filename + '.optim') else {}

        if self.shorthash() == optimizer_saved_dict.get('hash', None):
            self.optimizer_state_dict = optimizer_saved_dict.get('optimizer_state_dict', None)
        else:
            self.optimizer_state_dict = None
        if self.optimizer_state_dict:
            self.optimizer_name = optimizer_saved_dict.get('optimizer_name', 'AdamW')
            if print_hypernet_extra:
                print("Loaded existing optimizer from checkpoint")
                print(f"Optimizer name is {self.optimizer_name}")
        else:
            self.optimizer_name = "AdamW"
            if print_hypernet_extra:
                print("No saved optimizer exists in checkpoint")

        for size, sd in state_dict.items():
            if type(size) == int:
                self.layers[size] = (
                    HypernetworkModule(size, sd[0], self.layer_structure, self.activation_func, self.weight_init,
                                       self.add_layer_norm, self.activate_output, self.dropout_structure),
                    HypernetworkModule(size, sd[1], self.layer_structure, self.activation_func, self.weight_init,
                                       self.add_layer_norm, self.activate_output, self.dropout_structure),
                )

        self.name = state_dict.get('name', self.name)
        self.step = state_dict.get('step', 0)
        self.sd_checkpoint = state_dict.get('sd_checkpoint', None)
        self.sd_checkpoint_name = state_dict.get('sd_checkpoint_name', None)
        self.eval()
    
    import hashlib

    def shorthash(self):
        sha256 = hashlib.sha256()
        sha256.update(f'{self.filename}hypernet/{self.name}'.encode())
        print(f"sha256= ", sha256.hexdigest()[0:10])
        return sha256.hexdigest()[0:10]


def list_hypernetworks(path):
    res = {}
    for filename in sorted(glob.iglob(os.path.join(path, '**/*.pt'), recursive=True)):
        name = os.path.splitext(os.path.basename(filename))[0]
        # Prevent a hypothetical "None.pt" from being listed.
        if name != "None":
            res[name] = filename
    return res


def load_hypernetwork(name):
    path = hypernetworks.get(name, None)

    if path is None:
        return None

    hypernetwork = Hypernetwork()

    try:
        hypernetwork.load(path)
    except Exception:
        print(f"Error loading hypernetwork {path}", file=sys.stderr)
        print(traceback.format_exc(), file=sys.stderr)
        return None

    return hypernetwork


def load_hypernetworks(names, multipliers=None):
    already_loaded = {}

    for hypernetwork in loaded_hypernetworks:
        if hypernetwork.name in names:
            already_loaded[hypernetwork.name] = hypernetwork

    loaded_hypernetworks.clear()

    for i, name in enumerate(names):
        hypernetwork = already_loaded.get(name, None)
        if hypernetwork is None:
            hypernetwork = load_hypernetwork(name)

        if hypernetwork is None:
            continue

        hypernetwork.set_multiplier(multipliers[i] if multipliers else 1.0)
        loaded_hypernetworks.append(hypernetwork)
    print(f"Hypernetworks Loaded", names)

def find_closest_hypernetwork_name(search: str):
    if not search:
        return None
    search = search.lower()
    applicable = [name for name in hypernetworks if search in name.lower()]
    if not applicable:
        return None
    applicable = sorted(applicable, key=lambda name: len(name))
    return applicable[0]


def apply_single_hypernetwork(hypernetwork, context_k, context_v, layer=None):
    hypernetwork_layers = (hypernetwork.layers if hypernetwork is not None else {}).get(context_k.shape[2], None)

    if hypernetwork_layers is None:
        return context_k, context_v

    if layer is not None:
        layer.hyper_k = hypernetwork_layers[0]
        layer.hyper_v = hypernetwork_layers[1]

    context_k = hypernetwork_layers[0](context_k)
    context_v = hypernetwork_layers[1](context_v)
    return context_k, context_v


def apply_hypernetworks(hypernetworks, context, layer=None):
    context_k = context
    context_v = context
    for hypernetwork in hypernetworks:
        context_k, context_v = apply_single_hypernetwork(hypernetwork, context_k, context_v, layer)

    return context_k, context_v


def attention_CrossAttention_forward(self, x, context=None, mask=None):
    h = self.heads

    q = self.to_q(x)
    context = default(context, x)

    context_k, context_v = apply_hypernetworks(loaded_hypernetworks, context, self)
    k = self.to_k(context_k)
    v = self.to_v(context_v)

    q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=h), (q, k, v))

    sim = einsum('b i d, b j d -> b i j', q, k) * self.scale

    if mask is not None:
        mask = rearrange(mask, 'b ... -> b (...)')
        max_neg_value = -torch.finfo(sim.dtype).max
        mask = repeat(mask, 'b j -> (b h) () j', h=h)
        sim.masked_fill_(~mask, max_neg_value)

    # attention, what we cannot get enough of
    attn = sim.softmax(dim=-1)

    out = einsum('b i j, b j d -> b i d', attn, v)
    out = rearrange(out, '(b h) n d -> b n (h d)', h=h)
    return self.to_out(out)


def stack_conds(conds):
    if len(conds) == 1:
        return torch.stack(conds)

    # same as in reconstruct_multicond_batch
    token_count = max([x.shape[0] for x in conds])
    for i in range(len(conds)):
        if conds[i].shape[0] != token_count:
            last_vector = conds[i][-1:]
            last_vector_repeated = last_vector.repeat([token_count - conds[i].shape[0], 1])
            conds[i] = torch.vstack([conds[i], last_vector_repeated])

    return torch.stack(conds)


def statistics(data):
    if len(data) < 2:
        std = 0
    else:
        std = stdev(data)
    total_information = f"loss:{mean(data):.3f}" + u"\u00B1" + f"({std/ (len(data) ** 0.5):.3f})"
    recent_data = data[-32:]
    if len(recent_data) < 2:
        std = 0
    else:
        std = stdev(recent_data)
    recent_information = f"recent 32 loss:{mean(recent_data):.3f}" + u"\u00B1" + f"({std / (len(recent_data) ** 0.5):.3f})"
    return total_information, recent_information


def report_statistics(loss_info:dict):
    keys = sorted(loss_info.keys(), key=lambda x: sum(loss_info[x]) / len(loss_info[x]))
    for key in keys:
        try:
            print("Loss statistics for file " + key)
            info, recent = statistics(list(loss_info[key]))
            print(info)
            print(recent)
        except Exception as e:
            print(e)

def create_hypernetwork(name, enable_sizes, overwrite_old, layer_structure=None, activation_func=None, weight_init=None, add_layer_norm=False, use_dropout=False, dropout_structure=None):
    # Remove illegal characters from name.
    name = "".join( x for x in name if (x.isalnum() or x in "._- "))
    assert name, "Name cannot be empty!"

    fn = os.path.join(root.hypernetwork_dir, f"{name}.pt")
    if not overwrite_old:
        assert not os.path.exists(fn), f"file {fn} already exists"

    if type(layer_structure) == str:
        layer_structure = [float(x.strip()) for x in layer_structure.split(",")]

    if use_dropout and dropout_structure and type(dropout_structure) == str:
        dropout_structure = [float(x.strip()) for x in dropout_structure.split(",")]
    else:
        dropout_structure = [0] * len(layer_structure)

    hypernet = Hypernetwork(
        name=name,
        enable_sizes=[int(x) for x in enable_sizes],
        layer_structure=layer_structure,
        activation_func=activation_func,
        weight_init=weight_init,
        add_layer_norm=add_layer_norm,
        use_dropout=use_dropout,
        dropout_structure=dropout_structure
    )
    hypernet.save(fn)

    reload_hypernetworks()

#apply_optimizations
hypernetwork=Hypernetwork()
import math
import sys
import traceback
import psutil
import contextlib
import torch
from torch import einsum

from ldm.util import default
from einops import rearrange

if root.use_xformers:
    try:
        import xformers.ops
        xformers_available = True
    except Exception:
        print("Cannot import xformers", file=sys.stderr)
        print(traceback.format_exc(), file=sys.stderr)


def get_available_vram():
    if root.device == 'cuda':
        stats = torch.cuda.memory_stats(root.device)
        mem_active = stats['active_bytes.all.current']
        mem_reserved = stats['reserved_bytes.all.current']
        mem_free_cuda, _ = torch.cuda.mem_get_info(torch.cuda.current_device())
        mem_free_torch = mem_reserved - mem_active
        mem_free_total = mem_free_cuda + mem_free_torch
        return mem_free_total
    else:
        return psutil.virtual_memory().available

# see https://github.com/basujindal/stable-diffusion/pull/117 for discussion
def split_cross_attention_forward_v1(self, x, context=None, mask=None):
    h = self.heads

    q_in = self.to_q(x)
    context = default(context, x)

    context_k, context_v = hypernetwork.apply_hypernetworks(loaded_hypernetworks, context)
    k_in = self.to_k(context_k)
    v_in = self.to_v(context_v)
    del context, context_k, context_v, x

    q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=h), (q_in, k_in, v_in))
    del q_in, k_in, v_in

    dtype = q.dtype
    upcast_attn=True
    if upcast_attn:
        q, k, v = q.float(), k.float(), v.float()
    
    upcast_attn=True
    with without_autocast(disable=not upcast_attn):
        r1 = torch.zeros(q.shape[0], q.shape[1], v.shape[2], device=q.device, dtype=q.dtype)
        for i in range(0, q.shape[0], 2):
            end = i + 2
            s1 = einsum('b i d, b j d -> b i j', q[i:end], k[i:end])
            s1 *= self.scale
    
            s2 = s1.softmax(dim=-1)
            del s1
    
            r1[i:end] = einsum('b i j, b j d -> b i d', s2, v[i:end])
            del s2
        del q, k, v

    r1 = r1.to(dtype)

    r2 = rearrange(r1, '(b h) n d -> b n (h d)', h=h)
    del r1

    return self.to_out(r2)


# taken from https://github.com/Doggettx/stable-diffusion and modified
def split_cross_attention_forward(self, x, context=None, mask=None):
    h = self.heads

    q_in = self.to_q(x)
    context = default(context, x)

    context_k, context_v = hypernetwork.apply_hypernetworks(loaded_hypernetworks, context)
    k_in = self.to_k(context_k)
    v_in = self.to_v(context_v)

    dtype = q_in.dtype
    upcast_attn=True
    if upcast_attn:
        q_in, k_in, v_in = q_in.float(), k_in.float(), v_in if v_in.device.type == 'mps' else v_in.float()
    upcast_attn=True
    with without_autocast(disable=not upcast_attn):
        k_in = k_in * self.scale
    
        del context, x
    
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=h), (q_in, k_in, v_in))
        del q_in, k_in, v_in
    
        r1 = torch.zeros(q.shape[0], q.shape[1], v.shape[2], device=q.device, dtype=q.dtype)
    
        mem_free_total = get_available_vram()
    
        gb = 1024 ** 3
        tensor_size = q.shape[0] * q.shape[1] * k.shape[1] * q.element_size()
        modifier = 3 if q.element_size() == 2 else 2.5
        mem_required = tensor_size * modifier
        steps = 1
    
        if mem_required > mem_free_total:
            steps = 2 ** (math.ceil(math.log(mem_required / mem_free_total, 2)))
            # print(f"Expected tensor size:{tensor_size/gb:0.1f}GB, cuda free:{mem_free_cuda/gb:0.1f}GB "
            #       f"torch free:{mem_free_torch/gb:0.1f} total:{mem_free_total/gb:0.1f} steps:{steps}")
    
        if steps > 64:
            max_res = math.floor(math.sqrt(math.sqrt(mem_free_total / 2.5)) / 8) * 64
            raise RuntimeError(f'Not enough memory, use lower resolution (max approx. {max_res}x{max_res}). '
                               f'Need: {mem_required / 64 / gb:0.1f}GB free, Have:{mem_free_total / gb:0.1f}GB free')
    
        slice_size = q.shape[1] // steps if (q.shape[1] % steps) == 0 else q.shape[1]
        for i in range(0, q.shape[1], slice_size):
            end = i + slice_size
            s1 = einsum('b i d, b j d -> b i j', q[:, i:end], k)
    
            s2 = s1.softmax(dim=-1, dtype=q.dtype)
            del s1
    
            r1[:, i:end] = einsum('b i j, b j d -> b i d', s2, v)
            del s2
    
        del q, k, v

    r1 = r1.to(dtype)

    r2 = rearrange(r1, '(b h) n d -> b n (h d)', h=h)
    del r1

    return self.to_out(r2)


# -- Taken from https://github.com/invoke-ai/InvokeAI and modified --
mem_total_gb = psutil.virtual_memory().total // (1 << 30)

def einsum_op_compvis(q, k, v):
    s = einsum('b i d, b j d -> b i j', q, k)
    s = s.softmax(dim=-1, dtype=s.dtype)
    return einsum('b i j, b j d -> b i d', s, v)

def einsum_op_slice_0(q, k, v, slice_size):
    r = torch.zeros(q.shape[0], q.shape[1], v.shape[2], device=q.device, dtype=q.dtype)
    for i in range(0, q.shape[0], slice_size):
        end = i + slice_size
        r[i:end] = einsum_op_compvis(q[i:end], k[i:end], v[i:end])
    return r

def einsum_op_slice_1(q, k, v, slice_size):
    r = torch.zeros(q.shape[0], q.shape[1], v.shape[2], device=q.device, dtype=q.dtype)
    for i in range(0, q.shape[1], slice_size):
        end = i + slice_size
        r[:, i:end] = einsum_op_compvis(q[:, i:end], k, v)
    return r

def einsum_op_mps_v1(q, k, v):
    if q.shape[0] * q.shape[1] <= 2**16: # (512x512) max q.shape[1]: 4096
        return einsum_op_compvis(q, k, v)
    else:
        slice_size = math.floor(2**30 / (q.shape[0] * q.shape[1]))
        if slice_size % 4096 == 0:
            slice_size -= 1
        return einsum_op_slice_1(q, k, v, slice_size)

def einsum_op_mps_v2(q, k, v):
    if mem_total_gb > 8 and q.shape[0] * q.shape[1] <= 2**16:
        return einsum_op_compvis(q, k, v)
    else:
        return einsum_op_slice_0(q, k, v, 1)

def einsum_op_tensor_mem(q, k, v, max_tensor_mb):
    size_mb = q.shape[0] * q.shape[1] * k.shape[1] * q.element_size() // (1 << 20)
    if size_mb <= max_tensor_mb:
        return einsum_op_compvis(q, k, v)
    div = 1 << int((size_mb - 1) / max_tensor_mb).bit_length()
    if div <= q.shape[0]:
        return einsum_op_slice_0(q, k, v, q.shape[0] // div)
    return einsum_op_slice_1(q, k, v, max(q.shape[1] // div, 1))

def einsum_op_cuda(q, k, v):
    stats = torch.cuda.memory_stats(q.device)
    mem_active = stats['active_bytes.all.current']
    mem_reserved = stats['reserved_bytes.all.current']
    mem_free_cuda, _ = torch.cuda.mem_get_info(q.device)
    mem_free_torch = mem_reserved - mem_active
    mem_free_total = mem_free_cuda + mem_free_torch
    # Divide factor of safety as there's copying and fragmentation
    return einsum_op_tensor_mem(q, k, v, mem_free_total / 3.3 / (1 << 20))

def einsum_op(q, k, v):
    if q.device.type == 'cuda':
        return einsum_op_cuda(q, k, v)

    if q.device.type == 'mps':
        if mem_total_gb >= 32 and q.shape[0] % 32 != 0 and q.shape[0] * q.shape[1] < 2**18:
            return einsum_op_mps_v1(q, k, v)
        return einsum_op_mps_v2(q, k, v)

    # Smaller slices are faster due to L2/L3/SLC caches.
    # Tested on i7 with 8MB L3 cache.
    return einsum_op_tensor_mem(q, k, v, 32)

def split_cross_attention_forward_invokeAI(self, x, context=None, mask=None):
    h = self.heads

    q = self.to_q(x)
    context = default(context, x)

    context_k, context_v = hypernetwork.apply_hypernetworks(loaded_hypernetworks, context)
    k = self.to_k(context_k)
    v = self.to_v(context_v)
    del context, context_k, context_v, x

    dtype = q.dtype
    upcast_attn=True
    if upcast_attn:
        q, k, v = q.float(), k.float(), v if v.device.type == 'mps' else v.float()

    with without_autocast(disable=not upcast_attn):
        k = k * self.scale
    
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=h), (q, k, v))
        r = einsum_op(q, k, v)
    r = r.to(dtype)
    return self.to_out(rearrange(r, '(b h) n d -> b n (h d)', h=h))

# -- End of code from https://github.com/invoke-ai/InvokeAI --


# Based on Birch-san's modified implementation of sub-quadratic attention from https://github.com/Birch-san/diffusers/pull/1
# The sub_quad_attention_forward function is under the MIT License listed under Memory Efficient Attention in the Licenses section of the web UI interface
def sub_quad_attention_forward(self, x, context=None, mask=None):
    assert mask is None, "attention-mask not currently implemented for SubQuadraticCrossAttnProcessor."

    h = self.heads

    q = self.to_q(x)
    context = default(context, x)

    context_k, context_v = hypernetwork.apply_hypernetworks(loaded_hypernetworks, context)
    k = self.to_k(context_k)
    v = self.to_v(context_v)
    del context, context_k, context_v, x

    q = q.unflatten(-1, (h, -1)).transpose(1,2).flatten(end_dim=1)
    k = k.unflatten(-1, (h, -1)).transpose(1,2).flatten(end_dim=1)
    v = v.unflatten(-1, (h, -1)).transpose(1,2).flatten(end_dim=1)

    dtype = q.dtype
    upcast_attn=True
    if upcast_attn:
        q, k = q.float(), k.float()
    sub_quad_q_chunk_size=1024
    x = sub_quad_attention(q, k, v, q_chunk_size=sub_quad_q_chunk_size, kv_chunk_size=sub_quad_kv_chunk_size, chunk_threshold=shared.cmd_opts.sub_quad_chunk_threshold, use_checkpoint=self.training)

    x = x.to(dtype)

    x = x.unflatten(0, (-1, h)).transpose(1,2).flatten(start_dim=2)

    out_proj, dropout = self.to_out
    x = out_proj(x)
    x = dropout(x)

    return x

def sub_quad_attention(q, k, v, q_chunk_size=1024, kv_chunk_size=None, kv_chunk_size_min=None, chunk_threshold=None, use_checkpoint=True):
    bytes_per_token = torch.finfo(q.dtype).bits//8
    batch_x_heads, q_tokens, _ = q.shape
    _, k_tokens, _ = k.shape
    qk_matmul_size_bytes = batch_x_heads * bytes_per_token * q_tokens * k_tokens

    if chunk_threshold is None:
        chunk_threshold_bytes = int(get_available_vram() * 0.9) if q.device.type == 'mps' else int(get_available_vram() * 0.7)
    elif chunk_threshold == 0:
        chunk_threshold_bytes = None
    else:
        chunk_threshold_bytes = int(0.01 * chunk_threshold * get_available_vram())

    if kv_chunk_size_min is None and chunk_threshold_bytes is not None:
        kv_chunk_size_min = chunk_threshold_bytes // (batch_x_heads * bytes_per_token * (k.shape[2] + v.shape[2]))
    elif kv_chunk_size_min == 0:
        kv_chunk_size_min = None

    if chunk_threshold_bytes is not None and qk_matmul_size_bytes <= chunk_threshold_bytes:
        # the big matmul fits into our memory limit; do everything in 1 chunk,
        # i.e. send it down the unchunked fast-path
        query_chunk_size = q_tokens
        kv_chunk_size = k_tokens

    with without_autocast(disable=q.dtype == v.dtype):
        return efficient_dot_product_attention(
            q,
            k,
            v,
            query_chunk_size=q_chunk_size,
            kv_chunk_size=kv_chunk_size,
            kv_chunk_size_min = kv_chunk_size_min,
            use_checkpoint=use_checkpoint,
        )

xformers_flash_attention=True
def get_xformers_flash_attention_op(q, k, v):
    if not xformers_flash_attention:
        return None

    try:
        flash_attention_op = xformers.ops.MemoryEfficientAttentionFlashAttentionOp
        fw, bw = flash_attention_op
        if fw.supports(xformers.ops.fmha.Inputs(query=q, key=k, value=v, attn_bias=None)):
            return flash_attention_op
    except Exception as e:
        display_once(e, "enabling flash attention")

    return None


def xformers_attention_forward(self, x, context=None, mask=None):
    h = self.heads
    q_in = self.to_q(x)
    context = default(context, x)

    context_k, context_v = hypernetwork.apply_hypernetworks(loaded_hypernetworks, context)
    k_in = self.to_k(context_k)
    v_in = self.to_v(context_v)

    q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b n h d', h=h), (q_in, k_in, v_in))
    del q_in, k_in, v_in

    dtype = q.dtype
    upcast_attn=True
    if upcast_attn:
        q, k = q.float(), k.float()

    out = xformers.ops.memory_efficient_attention(q, k, v, attn_bias=None, op=get_xformers_flash_attention_op(q, k, v))

    out = out.to(dtype)

    out = rearrange(out, 'b n h d -> b n (h d)', h=h)
    return self.to_out(out)

def cross_attention_attnblock_forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q1 = self.q(h_)
        k1 = self.k(h_)
        v = self.v(h_)

        # compute attention
        b, c, h, w = q1.shape

        q2 = q1.reshape(b, c, h*w)
        del q1

        q = q2.permute(0, 2, 1)   # b,hw,c
        del q2

        k = k1.reshape(b, c, h*w) # b,c,hw
        del k1

        h_ = torch.zeros_like(k, device=q.device)

        mem_free_total = get_available_vram()

        tensor_size = q.shape[0] * q.shape[1] * k.shape[2] * q.element_size()
        mem_required = tensor_size * 2.5
        steps = 1

        if mem_required > mem_free_total:
            steps = 2**(math.ceil(math.log(mem_required / mem_free_total, 2)))

        slice_size = q.shape[1] // steps if (q.shape[1] % steps) == 0 else q.shape[1]
        for i in range(0, q.shape[1], slice_size):
            end = i + slice_size

            w1 = torch.bmm(q[:, i:end], k)     # b,hw,hw    w[b,i,j]=sum_c q[b,i,c]k[b,c,j]
            w2 = w1 * (int(c)**(-0.5))
            del w1
            w3 = torch.nn.functional.softmax(w2, dim=2, dtype=q.dtype)
            del w2

            # attend to values
            v1 = v.reshape(b, c, h*w)
            w4 = w3.permute(0, 2, 1)   # b,hw,hw (first hw of k, second of q)
            del w3

            h_[:, :, i:end] = torch.bmm(v1, w4)     # b, c,hw (hw of q) h_[b,c,j] = sum_i v[b,c,i] w_[b,i,j]
            del v1, w4

        h2 = h_.reshape(b, c, h, w)
        del h_

        h3 = self.proj_out(h2)
        del h2

        h3 += x

        return h3
    
def xformers_attnblock_forward(self, x):
    try:
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)
        b, c, h, w = q.shape
        q, k, v = map(lambda t: rearrange(t, 'b c h w -> b (h w) c'), (q, k, v))
        dtype = q.dtype
        upcast_attn=True
        if upcast_attn:
            q, k = q.float(), k.float()
        q = q.contiguous()
        k = k.contiguous()
        v = v.contiguous()
        out = xformers.ops.memory_efficient_attention(q, k, v, op=get_xformers_flash_attention_op(q, k, v))
        out = out.to(dtype)
        out = rearrange(out, 'b (h w) c -> b c h w', h=h)
        out = self.proj_out(out)
        return x + out
    except NotImplementedError:
        return cross_attention_attnblock_forward(self, x)

def sub_quad_attnblock_forward(self, x):
    h_ = x
    h_ = self.norm(h_)
    q = self.q(h_)
    k = self.k(h_)
    v = self.v(h_)
    b, c, h, w = q.shape
    q, k, v = map(lambda t: rearrange(t, 'b c h w -> b (h w) c'), (q, k, v))
    q = q.contiguous()
    k = k.contiguous()
    v = v.contiguous()
    sub_quad_q_chunk_size=1024
    sub_quad_kv_chunk_size=None
    sub_quad_chunk_threshold=None
    out = sub_quad_attention(q, k, v, q_chunk_size=sub_quad_q_chunk_size, kv_chunk_size=sub_quad_kv_chunk_size, chunk_threshold=sub_quad_chunk_threshold, use_checkpoint=self.training)
    out = rearrange(out, 'b (h w) c -> b c h w', h=h)
    out = self.proj_out(out)
    return x + out

#sd_hijack_unet
import torch
from packaging import version

def autocast(disable=False):
    
    if disable:
        return contextlib.nullcontext()

    if dtype == torch.float32 or precision == "full":
        return contextlib.nullcontext()

    return torch.autocast("cuda")

class TorchHijackForUnet:
    """
    This is torch, but with cat that resizes tensors to appropriate dimensions if they do not match;
    this makes it possible to create pictures with dimensions that are multiples of 8 rather than 64
    """

    def __getattr__(self, item):
        if item == 'cat':
            return self.cat

        if hasattr(torch, item):
            return getattr(torch, item)

        raise AttributeError("'{}' object has no attribute '{}'".format(type(self).__name__, item))

    def cat(self, tensors, *args, **kwargs):
        if len(tensors) == 2:
            a, b = tensors
            if a.shape[-2:] != b.shape[-2:]:
                a = torch.nn.functional.interpolate(a, b.shape[-2:], mode="nearest")

            tensors = (a, b)

        return torch.cat(tensors, *args, **kwargs)


th = TorchHijackForUnet()


# Below are monkey patches to enable upcasting a float16 UNet for float32 sampling
def apply_model(orig_func, self, x_noisy, t, cond, **kwargs):

    if isinstance(cond, dict):
        for y in cond.keys():
            cond[y] = [x.to(dtype_unet) if isinstance(x, torch.Tensor) else x for x in cond[y]]

    with autocast():
        return orig_func(self, x_noisy.to(dtype_unet), t.to(dtype_unet), cond, **kwargs).float()
unet_needs_upcast=True
class GELUHijack(torch.nn.GELU, torch.nn.Module):
    def __init__(self, *args, **kwargs):
        torch.nn.GELU.__init__(self, *args, **kwargs)
    def forward(self, x):
        if unet_needs_upcast:
            return torch.nn.GELU.forward(self.float(), x.float()).to(dtype_unet)
        else:
            return torch.nn.GELU.forward(self, x)

ddpm_edit_hijack = None
def hijack_ddpm_edit():
    global ddpm_edit_hijack
    if not ddpm_edit_hijack:
        CondFunc('modules.models.diffusion.ddpm_edit.LatentDiffusion.decode_first_stage', first_stage_sub, first_stage_cond)
        CondFunc('modules.models.diffusion.ddpm_edit.LatentDiffusion.encode_first_stage', first_stage_sub, first_stage_cond)
        ddpm_edit_hijack = CondFunc('modules.models.diffusion.ddpm_edit.LatentDiffusion.apply_model', apply_model, unet_needs_upcast)

unet_needs_upcast = lambda *args, **kwargs: unet_needs_upcast
CondFunc('ldm.models.diffusion.ddpm.LatentDiffusion.apply_model', apply_model, unet_needs_upcast)
CondFunc('ldm.modules.diffusionmodules.openaimodel.timestep_embedding', lambda orig_func, timesteps, *args, **kwargs: orig_func(timesteps, *args, **kwargs).to(torch.float32 if timesteps.dtype == torch.int64 else dtype_unet), unet_needs_upcast)
if version.parse(torch.__version__) <= version.parse("1.13.1"):
    CondFunc('ldm.modules.diffusionmodules.util.GroupNorm32.forward', lambda orig_func, self, *args, **kwargs: orig_func(self.float(), *args, **kwargs), unet_needs_upcast)
    CondFunc('ldm.modules.attention.GEGLU.forward', lambda orig_func, self, x: orig_func(self.float(), x.float()).to(dtype_unet), unet_needs_upcast)
    CondFunc('open_clip.transformer.ResidualAttentionBlock.__init__', lambda orig_func, *args, **kwargs: kwargs.update({'act_layer': GELUHijack}) and False or orig_func(*args, **kwargs), lambda _, *args, **kwargs: kwargs.get('act_layer') is None or kwargs['act_layer'] == torch.nn.GELU)

first_stage_cond = lambda _, self, *args, **kwargs: unet_needs_upcast and self.model.diffusion_model.dtype == torch.float16
first_stage_sub = lambda orig_func, self, x, **kwargs: orig_func(self, x.to(dtype_vae), **kwargs)
CondFunc('ldm.models.diffusion.ddpm.LatentDiffusion.decode_first_stage', first_stage_sub, first_stage_cond)
CondFunc('ldm.models.diffusion.ddpm.LatentDiffusion.encode_first_stage', first_stage_sub, first_stage_cond)
CondFunc('ldm.models.diffusion.ddpm.LatentDiffusion.get_first_stage_encoding', lambda orig_func, *args, **kwargs: orig_func(*args, **kwargs).float(), first_stage_cond)

#@title sd_hijack
import torch
from torch.nn.functional import silu
from types import MethodType
import ldm.modules.attention
import ldm.modules.diffusionmodules.model
import ldm.modules.diffusionmodules.openaimodel
import ldm.models.diffusion.ddim
import ldm.models.diffusion.plms
import ldm.modules.encoders.modules

attention_CrossAttention_forward = ldm.modules.attention.CrossAttention.forward
diffusionmodules_model_nonlinearity = ldm.modules.diffusionmodules.model.nonlinearity
diffusionmodules_model_AttnBlock_forward = ldm.modules.diffusionmodules.model.AttnBlock.forward

# new memory efficient cross attention blocks do not support hypernets and we already
# have memory efficient cross attention anyway, so this disables SD2.0's memory efficient cross attention
ldm.modules.attention.MemoryEfficientCrossAttention = ldm.modules.attention.CrossAttention
ldm.modules.attention.BasicTransformerBlock.ATTENTION_MODES["softmax-xformers"] = ldm.modules.attention.CrossAttention

# silence new console spam from SD2
ldm.modules.attention.print = lambda *args: None
ldm.modules.diffusionmodules.model.print = lambda *args: None
th = TorchHijackForUnet()
def apply_optimizations():
    undo_optimizations()

    ldm.modules.diffusionmodules.model.nonlinearity = silu
    ldm.modules.diffusionmodules.openaimodel.th = th
    
    optimization_method = None

    if root.use_xformers and torch.version.cuda and (6, 0) <= torch.cuda.get_device_capability(root.device) <= (9, 0):
        print("Applying xformers cross attention optimization.")
        ldm.modules.attention.CrossAttention.forward = xformers_attention_forward
        ldm.modules.diffusionmodules.model.AttnBlock.forward = xformers_attnblock_forward
        optimization_method = 'xformers'
    elif root.use_sub_quad_attention and not root.use_xformers:
        print("Applying sub-quadratic cross attention optimization.")
        ldm.modules.attention.CrossAttention.forward = sub_quad_attention_forward
        ldm.modules.diffusionmodules.model.AttnBlock.forward = sub_quad_attnblock_forward
        optimization_method = 'sub-quadratic'
    elif root.use_split_attention_v1 and not root.use_sub_quad_attention and not root.use_xformers:
        print("Applying v1 cross attention optimization.")
        ldm.modules.attention.CrossAttention.forward = split_cross_attention_forward_v1
        optimization_method = 'V1'
    elif not disable_opt_split_attention and (root.use_split_cross_attention_forward_invokeAI or not opt_split_attention and not torch.cuda.is_available()):
        print("Applying cross attention optimization (InvokeAI).")
        ldm.modules.attention.CrossAttention.forward = split_cross_attention_forward_invokeAI
        optimization_method = 'InvokeAI'
    elif not disable_opt_split_attention and (opt_split_attention or torch.cuda.is_available()):
        print("Applying cross attention optimization (Doggettx).")
        ldm.modules.attention.CrossAttention.forward = split_cross_attention_forward
        ldm.modules.diffusionmodules.model.AttnBlock.forward = cross_attention_attnblock_forward
        optimization_method = 'Doggettx'

    return optimization_method


def undo_optimizations():
    ldm.modules.attention.CrossAttention.forward = attention_CrossAttention_forward
    ldm.modules.diffusionmodules.model.nonlinearity = diffusionmodules_model_nonlinearity
    ldm.modules.diffusionmodules.model.AttnBlock.forward = diffusionmodules_model_AttnBlock_forward


def fix_checkpoint():
    """checkpoints are now added and removed in embedding/hypernet code, since torch doesn't want
    checkpoints to be added when not training (there's a warning)"""

    pass

def weighted_loss(model, pred, target, mean=True):
    #Calculate the weight normally, but ignore the mean
    loss = model._old_get_loss(pred, target, mean=False)
    
    #Check if we have weights available
    weight = getattr(model, '_custom_loss_weight', None)
    if weight is not None:
        loss *= weight
    
    #Return the loss, as mean if specified
    return loss.mean() if mean else loss

def weighted_forward(model, x, c, w, *args, **kwargs):
    try:
        #Temporarily append weights to a place accessible during loss calc
        model._custom_loss_weight = w
        
        #Replace 'get_loss' with a weight-aware one. Otherwise we need to reimplement 'forward' completely
        #Keep 'get_loss', but don't overwrite the previous old_get_loss if it's already set
        if not hasattr(model, '_old_get_loss'):
            model._old_get_loss = model.get_loss
        model.get_loss = MethodType(weighted_loss, model)

        #Run the standard forward function, but with the patched 'get_loss'
        return model.forward(x, c, *args, **kwargs)
    finally:
        try:
            #Delete temporary weights if appended
            del model._custom_loss_weight
        except AttributeError as e:
            pass
            
        #If we have an old loss function, reset the loss function to the original one
        if hasattr(model, '_old_get_loss'):
            model.get_loss = model._old_get_loss
            del model._old_get_loss

def apply_weighted_forward(model):
    #Add new function 'weighted_forward' that can be called to calc weighted loss
    model.weighted_forward = MethodType(weighted_forward, model)

def undo_weighted_forward(model):
    try:
        del model.weighted_forward
    except AttributeError as e:
        pass

from transformers import BertPreTrainedModel,BertModel,BertConfig
import torch.nn as nn
import torch
from transformers.models.xlm_roberta.configuration_xlm_roberta import XLMRobertaConfig
from transformers import XLMRobertaModel,XLMRobertaTokenizer
from typing import Optional

class BertSeriesConfig(BertConfig):
    def __init__(self, vocab_size=30522, hidden_size=768, num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072, hidden_act="gelu", hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1, max_position_embeddings=512, type_vocab_size=2, initializer_range=0.02, layer_norm_eps=1e-12, pad_token_id=0, position_embedding_type="absolute", use_cache=True, classifier_dropout=None,project_dim=512, pooler_fn="average",learn_encoder=False,model_type='bert',**kwargs):

        super().__init__(vocab_size, hidden_size, num_hidden_layers, num_attention_heads, intermediate_size, hidden_act, hidden_dropout_prob, attention_probs_dropout_prob, max_position_embeddings, type_vocab_size, initializer_range, layer_norm_eps, pad_token_id, position_embedding_type, use_cache, classifier_dropout, **kwargs)
        self.project_dim = project_dim
        self.pooler_fn = pooler_fn
        self.learn_encoder = learn_encoder

class RobertaSeriesConfig(XLMRobertaConfig):
    def __init__(self, pad_token_id=1, bos_token_id=0, eos_token_id=2,project_dim=512,pooler_fn='cls',learn_encoder=False, **kwargs):
        super().__init__(pad_token_id=pad_token_id, bos_token_id=bos_token_id, eos_token_id=eos_token_id, **kwargs)
        self.project_dim = project_dim
        self.pooler_fn = pooler_fn
        self.learn_encoder = learn_encoder


class BertSeriesModelWithTransformation(BertPreTrainedModel):

    _keys_to_ignore_on_load_unexpected = [r"pooler"]
    _keys_to_ignore_on_load_missing = [r"position_ids", r"predictions.decoder.bias"]
    config_class = BertSeriesConfig

    def __init__(self, config=None, **kargs):
        # modify initialization for autoloading 
        if config is None:
            config = XLMRobertaConfig()
            config.attention_probs_dropout_prob= 0.1
            config.bos_token_id=0
            config.eos_token_id=2
            config.hidden_act='gelu'
            config.hidden_dropout_prob=0.1
            config.hidden_size=1024
            config.initializer_range=0.02
            config.intermediate_size=4096
            config.layer_norm_eps=1e-05
            config.max_position_embeddings=514

            config.num_attention_heads=16
            config.num_hidden_layers=24
            config.output_past=True
            config.pad_token_id=1
            config.position_embedding_type= "absolute"

            config.type_vocab_size= 1
            config.use_cache=True
            config.vocab_size= 250002
            config.project_dim = 768
            config.learn_encoder = False
        super().__init__(config)
        self.roberta = XLMRobertaModel(config)
        self.transformation = nn.Linear(config.hidden_size,config.project_dim)
        self.pre_LN=nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')
        self.pooler = lambda x: x[:,0]
        self.post_init()

    def encode(self,c):
        device = next(self.parameters()).device
        text = self.tokenizer(c,
                        truncation=True,
                        max_length=77,
                        return_length=False,
                        return_overflowing_tokens=False,
                        padding="max_length",
                        return_tensors="pt")
        text["input_ids"] = torch.tensor(text["input_ids"]).to(device)
        text["attention_mask"] = torch.tensor(
            text['attention_mask']).to(device)
        features = self(**text)
        return features['projection_state'] 

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
    ) :
        r"""
        """

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict


        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=True,
            return_dict=return_dict,
        )

        # last module outputs
        sequence_output = outputs[0]


        # project every module
        sequence_output_ln = self.pre_LN(sequence_output)

        # pooler
        pooler_output = self.pooler(sequence_output_ln)
        pooler_output = self.transformation(pooler_output)
        projection_state = self.transformation(outputs.last_hidden_state)

        return {
            'pooler_output':pooler_output,
            'last_hidden_state':outputs.last_hidden_state,
            'hidden_states':outputs.hidden_states,
            'attentions':outputs.attentions,
            'projection_state':projection_state,
            'sequence_out': sequence_output
        }


class RobertaSeriesModelWithTransformation(BertSeriesModelWithTransformation):
    base_model_prefix = 'roberta'
    config_class= RobertaSeriesConfig

class StableDiffusionModelHijack:
    fixes = None
    comments = []
    layers = None
    circular_enabled = False
    clip = None
    optimization_method = None

    embedding_db = EmbeddingDatabase()

    def __init__(self):
        self.embedding_db.add_embedding_dir(root.embeddings_dir)

    def hijack(self, m):

        # if type(m.cond_stage_model) == BertSeriesModelWithTransformation:
        #     model_embeddings = m.cond_stage_model.roberta.embeddings
        #     model_embeddings.token_embedding = EmbeddingsWithFixes(model_embeddings.word_embeddings, self)
        #     m.cond_stage_model = FrozenXLMREmbedderWithCustomWords(m.cond_stage_model, self)

        if type(m.cond_stage_model) == ldm.modules.encoders.modules.FrozenCLIPEmbedder:
            model_embeddings = m.cond_stage_model.transformer.text_model.embeddings
            model_embeddings.token_embedding = EmbeddingsWithFixes(model_embeddings.token_embedding, self)
            m.cond_stage_model = FrozenCLIPEmbedderWithCustomWords(m.cond_stage_model, self)

        elif type(m.cond_stage_model) == ldm.modules.encoders.modules.FrozenOpenCLIPEmbedder:
            m.cond_stage_model.model.token_embedding = EmbeddingsWithFixes(m.cond_stage_model.model.token_embedding, self)
            m.cond_stage_model = FrozenOpenCLIPEmbedderWithCustomWords(m.cond_stage_model, self)

        apply_weighted_forward(m)
        if m.cond_stage_key == "edit":
            hijack_ddpm_edit()

        optimization_method = apply_optimizations()

        self.clip = m.cond_stage_model

        def flatten(el):
            flattened = [flatten(children) for children in el.children()]
            res = [el]
            for c in flattened:
                res += c
            return res

        self.layers = flatten(m)

    def undo_hijack(self, m):
        
        if type(m.cond_stage_model) == FrozenCLIPEmbedderWithCustomWords:
            m.cond_stage_model = m.cond_stage_model.wrapped

            model_embeddings = m.cond_stage_model.transformer.text_model.embeddings
            if type(model_embeddings.token_embedding) == EmbeddingsWithFixes:
                model_embeddings.token_embedding = model_embeddings.token_embedding.wrapped
        elif type(m.cond_stage_model) == FrozenOpenCLIPEmbedderWithCustomWords:
            m.cond_stage_model.wrapped.model.token_embedding = m.cond_stage_model.wrapped.model.token_embedding.wrapped
            m.cond_stage_model = m.cond_stage_model.wrapped

        self.apply_circular(False)
        self.layers = None
        self.clip = None

    def apply_circular(self, enable):
        if self.circular_enabled == enable:
            return

        self.circular_enabled = enable

        for layer in [layer for layer in self.layers if type(layer) == torch.nn.Conv2d]:
            layer.padding_mode = 'circular' if enable else 'zeros'

    def clear_comments(self):
        self.comments = []

    def get_prompt_lengths(self, text):
        _, token_count = self.clip.process_texts([text])

        return token_count, self.clip.get_target_prompt_token_count(token_count)

def cond_cast_unet(input):
    return input.to(dtype_unet) if unet_needs_upcast else input

class EmbeddingsWithFixes(torch.nn.Module):
    def __init__(self, wrapped, embeddings):
        super().__init__()
        self.wrapped = wrapped
        self.embeddings = embeddings

    def forward(self, input_ids):
        batch_fixes = self.embeddings.fixes
        self.embeddings.fixes = None

        inputs_embeds = self.wrapped(input_ids)

        if batch_fixes is None or len(batch_fixes) == 0 or max([len(x) for x in batch_fixes]) == 0:
            return inputs_embeds

        vecs = []
        for fixes, tensor in zip(batch_fixes, inputs_embeds):
            for offset, embedding in fixes:
                emb = cond_cast_unet(embedding.vec)
                emb_len = min(tensor.shape[0] - offset - 1, emb.shape[0])
                tensor = torch.cat([tensor[0:offset + 1], emb[0:emb_len], tensor[offset + 1 + emb_len:]])

            vecs.append(tensor)

        return torch.stack(vecs)


def add_circular_option_to_conv_2d():
    conv2d_constructor = torch.nn.Conv2d.__init__

    def conv2d_constructor_circular(self, *args, **kwargs):
        return conv2d_constructor(self, *args, padding_mode='circular', **kwargs)

    torch.nn.Conv2d.__init__ = conv2d_constructor_circular


model_hijack = StableDiffusionModelHijack()


def register_buffer(self, name, attr):
    """
    Fix register buffer bug for Mac OS.
    """

    if type(attr) == torch.Tensor:
        if attr.device != root.device:
            attr = attr.to(device=root.device, dtype=(torch.float32 if root.device.type == 'mps' else None))

    setattr(self, name, attr)


ldm.models.diffusion.ddim.DDIMSampler.register_buffer = register_buffer
ldm.models.diffusion.plms.PLMSSampler.register_buffer = register_buffer

#@title hijack_checkpoint

from torch.utils.checkpoint import checkpoint

import ldm.modules.attention
import ldm.modules.diffusionmodules.openaimodel


def BasicTransformerBlock_forward(self, x, context=None):
    return checkpoint(self._forward, x, context)


def AttentionBlock_forward(self, x):
    return checkpoint(self._forward, x)


def ResBlock_forward(self, x, emb):
    return checkpoint(self._forward, x, emb)


stored = []


def hijack_checkpoint_add():
    if len(stored) != 0:
        return

    stored.extend([
        ldm.modules.attention.BasicTransformerBlock.forward,
        ldm.modules.diffusionmodules.openaimodel.ResBlock.forward,
        ldm.modules.diffusionmodules.openaimodel.AttentionBlock.forward
    ])

    ldm.modules.attention.BasicTransformerBlock.forward = BasicTransformerBlock_forward
    ldm.modules.diffusionmodules.openaimodel.ResBlock.forward = ResBlock_forward
    ldm.modules.diffusionmodules.openaimodel.AttentionBlock.forward = AttentionBlock_forward


def hijack_checkpoint_remove():
    if len(stored) == 0:
        return

    ldm.modules.attention.BasicTransformerBlock.forward = stored[0]
    ldm.modules.diffusionmodules.openaimodel.ResBlock.forward = stored[1]
    ldm.modules.diffusionmodules.openaimodel.AttentionBlock.forward = stored[2]

    stored.clear()


#@title hijack_clip

import math
from collections import namedtuple

import torch

class PromptChunk:
    """
    This object contains token ids, weight (multipliers:1.4) and textual inversion embedding info for a chunk of prompt.
    If a prompt is short, it is represented by one PromptChunk, otherwise, multiple are necessary.
    Each PromptChunk contains an exact amount of tokens - 77, which includes one for start and end token,
    so just 75 tokens from prompt.
    """

    def __init__(self):
        self.tokens = []
        self.multipliers = []
        self.fixes = []


PromptChunkFix = namedtuple('PromptChunkFix', ['offset', 'embedding'])
"""An object of this type is a marker showing that textual inversion embedding's vectors have to placed at offset in the prompt
chunk. Thos objects are found in PromptChunk.fixes and, are placed into FrozenCLIPEmbedderWithCustomWordsBase.hijack.fixes, and finally
are applied by sd_hijack.EmbeddingsWithFixes's forward function."""


class FrozenCLIPEmbedderWithCustomWordsBase(torch.nn.Module):
    """A pytorch module that is a wrapper for FrozenCLIPEmbedder module. it enhances FrozenCLIPEmbedder, making it possible to
    have unlimited prompt length and assign weights to tokens in prompt.
    """

    def __init__(self, wrapped, hijack):
        super().__init__()

        self.wrapped = wrapped
        """Original FrozenCLIPEmbedder module; can also be FrozenOpenCLIPEmbedder or xlmr.BertSeriesModelWithTransformation,
        depending on model."""

        self.hijack: StableDiffusionModelHijack = hijack
        self.chunk_length = 75

    def empty_chunk(self):
        """creates an empty PromptChunk and returns it"""

        chunk = PromptChunk()
        chunk.tokens = [self.id_start] + [self.id_end] * (self.chunk_length + 1)
        chunk.multipliers = [1.0] * (self.chunk_length + 2)
        return chunk

    def get_target_prompt_token_count(self, token_count):
        """returns the maximum number of tokens a prompt of a known length can have before it requires one more PromptChunk to be represented"""

        return math.ceil(max(token_count, 1) / self.chunk_length) * self.chunk_length

    def tokenize(self, texts):
        """Converts a batch of texts into a batch of token ids"""

        raise NotImplementedError

    def encode_with_transformers(self, tokens):
        """
        converts a batch of token ids (in python lists) into a single tensor with numeric respresentation of those tokens;
        All python lists with tokens are assumed to have same length, usually 77.
        if input is a list with B elements and each element has T tokens, expected output shape is (B, T, C), where C depends on
        model - can be 768 and 1024.
        Among other things, this call will read self.hijack.fixes, apply it to its inputs, and clear it (setting it to None).
        """

        raise NotImplementedError

    def encode_embedding_init_text(self, init_text, nvpt):
        """Converts text into a tensor with this text's tokens' embeddings. Note that those are embeddings before they are passed through
        transformers. nvpt is used as a maximum length in tokens. If text produces less teokens than nvpt, only this many is returned."""

        raise NotImplementedError

    def tokenize_line(self, line):
        """
        this transforms a single prompt into a list of PromptChunk objects - as many as needed to
        represent the prompt.
        Returns the list and the total number of tokens in the prompt.
        """


        parsed = [[line, 1.0]]

        tokenized = self.tokenize([text for text, _ in parsed])

        chunks = []
        chunk = PromptChunk()
        token_count = 0
        last_comma = -1

        def next_chunk(is_last=False):
            """puts current chunk into the list of results and produces the next one - empty;
            if is_last is true, tokens <end-of-text> tokens at the end won't add to token_count"""
            nonlocal token_count
            nonlocal last_comma
            nonlocal chunk

            if is_last:
                token_count += len(chunk.tokens)
            else:
                token_count += self.chunk_length

            to_add = self.chunk_length - len(chunk.tokens)
            if to_add > 0:
                chunk.tokens += [self.id_end] * to_add
                chunk.multipliers += [1.0] * to_add

            chunk.tokens = [self.id_start] + chunk.tokens + [self.id_end]
            chunk.multipliers = [1.0] + chunk.multipliers + [1.0]

            last_comma = -1
            chunks.append(chunk)
            chunk = PromptChunk()

        for tokens, (text, weight) in zip(tokenized, parsed):
            if text == 'BREAK' and weight == -1:
                next_chunk()
                continue

            position = 0
            while position < len(tokens):
                token = tokens[position]

                if token == self.comma_token:
                    last_comma = len(chunk.tokens)
                # this is when we are at the end of alloted 75 tokens for the current chunk, and the current token is not a comma. opts.comma_padding_backtrack
                # is a setting that specifies that if there is a comma nearby, the text after the comma should be moved out of this chunk and into the next.
                # comma_padding_backtrack=74    
                # elif comma_padding_backtrack != 0 and len(chunk.tokens) == self.chunk_length and last_comma != -1 and len(chunk.tokens) - last_comma <= comma_padding_backtrack:
                #     break_location = last_comma + 1

                #     reloc_tokens = chunk.tokens[break_location:]
                #     reloc_mults = chunk.multipliers[break_location:]

                #     chunk.tokens = chunk.tokens[:break_location]
                #     chunk.multipliers = chunk.multipliers[:break_location]

                #     next_chunk()
                #     chunk.tokens = reloc_tokens
                #     chunk.multipliers = reloc_mults

                if len(chunk.tokens) == self.chunk_length:
                    next_chunk()

                embedding, embedding_length_in_tokens = self.hijack.embedding_db.find_embedding_at_position(tokens, position)
                if embedding is None:
                    chunk.tokens.append(token)
                    chunk.multipliers.append(weight)
                    position += 1
                    continue

                emb_len = int(embedding.vec.shape[0])
                if len(chunk.tokens) + emb_len > self.chunk_length:
                    next_chunk()

                chunk.fixes.append(PromptChunkFix(len(chunk.tokens), embedding))

                chunk.tokens += [0] * emb_len
                chunk.multipliers += [weight] * emb_len
                position += embedding_length_in_tokens

        if len(chunk.tokens) > 0 or len(chunks) == 0:
            next_chunk(is_last=True)

        return chunks, token_count

    def process_texts(self, texts):
        """
        Accepts a list of texts and calls tokenize_line() on each, with cache. Returns the list of results and maximum
        length, in tokens, of all texts.
        """

        token_count = 0

        cache = {}
        batch_chunks = []
        for line in texts:
            if line in cache:
                chunks = cache[line]
            else:
                chunks, current_token_count = self.tokenize_line(line)
                token_count = max(current_token_count, token_count)

                cache[line] = chunks

            batch_chunks.append(chunks)

        return batch_chunks, token_count

    def forward(self, texts):
        """
        Accepts an array of texts; Passes texts through transformers network to create a tensor with numerical representation of those texts.
        Returns a tensor with shape of (B, T, C), where B is length of the array; T is length, in tokens, of texts (including padding) - T will
        be a multiple of 77; and C is dimensionality of each token - for SD1 it's 768, and for SD2 it's 1024.
        An example shape returned by this function can be: (2, 77, 768).
        Webui usually sends just one text at a time through this function - the only time when texts is an array with more than one elemenet
        is when you do prompt editing: "a picture of a [cat:dog:0.4] eating ice cream"
        """

        # if opts.use_old_emphasis_implementation:
        #     import modules.sd_hijack_clip_old
        #     return modules.sd_hijack_clip_old.forward_old(self, texts)

        batch_chunks, token_count = self.process_texts(texts)

        used_embeddings = {}
        chunk_count = max([len(x) for x in batch_chunks])

        zs = []
        for i in range(chunk_count):
            batch_chunk = [chunks[i] if i < len(chunks) else self.empty_chunk() for chunks in batch_chunks]

            tokens = [x.tokens for x in batch_chunk]
            multipliers = [x.multipliers for x in batch_chunk]
            self.hijack.fixes = [x.fixes for x in batch_chunk]

            for fixes in self.hijack.fixes:
                for position, embedding in fixes:
                    used_embeddings[embedding.name] = embedding

            z = self.process_tokens(tokens, multipliers).float()
            zs.append(z)

        if len(used_embeddings) > 0:
            embeddings_list = ", ".join([f'{name} [{embedding.checksum()}]' for name, embedding in used_embeddings.items()])
            self.hijack.comments.append(f"Used embeddings: {embeddings_list}")

        return torch.hstack(zs)

    def process_tokens(self, remade_batch_tokens, batch_multipliers):
        """
        sends one single prompt chunk to be encoded by transformers neural network.
        remade_batch_tokens is a batch of tokens - a list, where every element is a list of tokens; usually
        there are exactly 77 tokens in the list. batch_multipliers is the same but for multipliers instead of tokens.
        Multipliers are used to give more or less weight to the outputs of transformers network. Each multiplier
        corresponds to one token.
        """
        tokens = torch.asarray(remade_batch_tokens).to(root.device)

        # this is for SD2: SD1 uses the same token for padding and end of text, while SD2 uses different ones.
        if self.id_end != self.id_pad:
            for batch_pos in range(len(remade_batch_tokens)):
                index = remade_batch_tokens[batch_pos].index(self.id_end)
                tokens[batch_pos, index+1:tokens.shape[1]] = self.id_pad

        z = self.encode_with_transformers(tokens).float()

        # restoring original mean is likely not correct, but it seems to work well to prevent artifacts that happen otherwise
        batch_multipliers = torch.asarray(batch_multipliers).to(root.device)
        original_mean = z.mean()
        z = z * batch_multipliers.reshape(batch_multipliers.shape + (1,)).expand(z.shape)
        new_mean = z.mean()
        z = z * (original_mean / new_mean)

        return z


class FrozenCLIPEmbedderWithCustomWords(FrozenCLIPEmbedderWithCustomWordsBase):
    def __init__(self, wrapped, hijack):
        super().__init__(wrapped, hijack)
        self.tokenizer = wrapped.tokenizer

        vocab = self.tokenizer.get_vocab()

        self.comma_token = vocab.get(',</w>', None)

        self.token_mults = {}
        tokens_with_parens = [(k, v) for k, v in vocab.items() if '(' in k or ')' in k or '[' in k or ']' in k]
        for text, ident in tokens_with_parens:
            mult = 1.0
            for c in text:
                if c == '[':
                    mult /= 1.1
                if c == ']':
                    mult *= 1.1
                if c == '(':
                    mult *= 1.1
                if c == ')':
                    mult /= 1.1

            if mult != 1.0:
                self.token_mults[ident] = mult

        self.id_start = self.wrapped.tokenizer.bos_token_id
        self.id_end = self.wrapped.tokenizer.eos_token_id
        self.id_pad = self.id_end

    def tokenize(self, texts):
        tokenized = self.wrapped.tokenizer(texts, truncation=False, add_special_tokens=False)["input_ids"]

        return tokenized

    def encode_with_transformers(self, tokens):
        CLIP_stop_at_last_layers=1
        outputs = self.wrapped.transformer(input_ids=tokens, output_hidden_states=CLIP_stop_at_last_layers)

        z = outputs.last_hidden_state

        return z

    def encode_embedding_init_text(self, init_text, nvpt):
        embedding_layer = self.wrapped.transformer.text_model.embeddings
        ids = self.wrapped.tokenizer(init_text, max_length=nvpt, return_tensors="pt", add_special_tokens=False)["input_ids"]
        embedded = embedding_layer.token_embedding.wrapped(ids.to(embedding_layer.token_embedding.wrapped.weight.device)).squeeze(0)

        return embedded

#@title hijack_clip_old
# from drive.MyDrive import sd_hijack_clip
def process_text_old(self: FrozenCLIPEmbedderWithCustomWordsBase, texts):
    id_start = self.id_start
    id_end = self.id_end
    maxlen = self.wrapped.max_length  # you get to stay at 77
    used_custom_terms = []
    remade_batch_tokens = []
    hijack_comments = []
    hijack_fixes = []
    token_count = 0

    cache = {}
    batch_tokens = self.tokenize(texts)
    batch_multipliers = []
    for tokens in batch_tokens:
        tuple_tokens = tuple(tokens)

        if tuple_tokens in cache:
            remade_tokens, fixes, multipliers = cache[tuple_tokens]
        else:
            fixes = []
            remade_tokens = []
            multipliers = []
            mult = 1.0

            i = 0
            while i < len(tokens):
                token = tokens[i]

                embedding, embedding_length_in_tokens = self.hijack.embedding_db.find_embedding_at_position(tokens, i)

                mult_change = self.token_mults.get(token)
                if mult_change is not None:
                    mult *= mult_change
                    i += 1
                elif embedding is None:
                    remade_tokens.append(token)
                    multipliers.append(mult)
                    i += 1
                else:
                    emb_len = int(embedding.vec.shape[0])
                    fixes.append((len(remade_tokens), embedding))
                    remade_tokens += [0] * emb_len
                    multipliers += [mult] * emb_len
                    used_custom_terms.append((embedding.name, embedding.checksum()))
                    i += embedding_length_in_tokens

            if len(remade_tokens) > maxlen - 2:
                vocab = {v: k for k, v in self.wrapped.tokenizer.get_vocab().items()}
                ovf = remade_tokens[maxlen - 2:]
                overflowing_words = [vocab.get(int(x), "") for x in ovf]
                overflowing_text = self.wrapped.tokenizer.convert_tokens_to_string(''.join(overflowing_words))
                hijack_comments.append(f"Warning: too many input tokens; some ({len(overflowing_words)}) have been truncated:\n{overflowing_text}\n")

            token_count = len(remade_tokens)
            remade_tokens = remade_tokens + [id_end] * (maxlen - 2 - len(remade_tokens))
            remade_tokens = [id_start] + remade_tokens[0:maxlen - 2] + [id_end]
            cache[tuple_tokens] = (remade_tokens, fixes, multipliers)

        multipliers = multipliers + [1.0] * (maxlen - 2 - len(multipliers))
        multipliers = [1.0] + multipliers[0:maxlen - 2] + [1.0]

        remade_batch_tokens.append(remade_tokens)
        hijack_fixes.append(fixes)
        batch_multipliers.append(multipliers)
    return batch_multipliers, remade_batch_tokens, used_custom_terms, hijack_comments, hijack_fixes, token_count


def forward_old(self: FrozenCLIPEmbedderWithCustomWordsBase, texts):
    batch_multipliers, remade_batch_tokens, used_custom_terms, hijack_comments, hijack_fixes, token_count = process_text_old(self, texts)

    self.hijack.comments += hijack_comments

    if len(used_custom_terms) > 0:
        self.hijack.comments.append("Used embeddings: " + ", ".join([f'{word} [{checksum}]' for word, checksum in used_custom_terms]))

    self.hijack.fixes = hijack_fixes
    return self.process_tokens(remade_batch_tokens, batch_multipliers)

#@title hijack_open_clip

import open_clip.tokenizer
import torch

tokenizer = open_clip.tokenizer._tokenizer


class FrozenOpenCLIPEmbedderWithCustomWords(FrozenCLIPEmbedderWithCustomWordsBase):
    def __init__(self, wrapped, hijack):
        super().__init__(wrapped, hijack)

        self.comma_token = [v for k, v in tokenizer.encoder.items() if k == ',</w>'][0]
        self.id_start = tokenizer.encoder["<start_of_text>"]
        self.id_end = tokenizer.encoder["<end_of_text>"]
        self.id_pad = 0

    def tokenize(self, texts):

        tokenized = [tokenizer.encode(text) for text in texts]

        return tokenized

    def encode_with_transformers(self, tokens):
        # set self.wrapped.layer_idx here according to opts.CLIP_stop_at_last_layers
        z = self.wrapped.encode_with_transformer(tokens)

        return z

    def encode_embedding_init_text(self, init_text, nvpt):
        ids = tokenizer.encode(init_text)
        ids = torch.asarray([ids], device=root.map_location, dtype=torch.int)
        embedded = self.wrapped.model.token_embedding.wrapped(ids).squeeze(0)

        return embedded

import open_clip.tokenizer
import torch

class FrozenXLMREmbedderWithCustomWords(FrozenCLIPEmbedderWithCustomWords):
    def __init__(self, wrapped, hijack):
        super().__init__(wrapped, hijack)

        self.id_start = wrapped.config.bos_token_id
        self.id_end = wrapped.config.eos_token_id
        self.id_pad = wrapped.config.pad_token_id

        self.comma_token = self.tokenizer.get_vocab().get(',', None)  # alt diffusion doesn't have </w> bits for comma

    def encode_with_transformers(self, tokens):
        # there's no CLIP Skip here because all hidden layers have size of 1024 and the last one uses a
        # trained layer to transform those 1024 into 768 for unet; so you can't choose which transformer
        # layer to work with - you have to use the last

        attention_mask = (tokens != self.id_pad).to(device=tokens.device, dtype=torch.int64)
        features = self.wrapped(input_ids=tokens, attention_mask=attention_mask)
        z = features['projection_state']

        return z

    def encode_embedding_init_text(self, init_text, nvpt):
        embedding_layer = self.wrapped.roberta.embeddings
        ids = self.wrapped.tokenizer(init_text, max_length=nvpt, return_tensors="pt", add_special_tokens=False)["input_ids"]
        embedded = embedding_layer.token_embedding.wrapped(ids.to(root.device)).squeeze(0)

        return embedded

def reload_hypernetworks():
    
    global hypernetworks

    hypernetworks = list_hypernetworks(root.hypernetwork_dir)

model_hijack = StableDiffusionModelHijack()
embedding_db = EmbeddingDatabase()
hypernetwork = Hypernetwork()
model_hijack.hijack(model)
model_hijack.embedding_db.load_textual_inversion_embeddings(force_reload=True)
reload_hypernetworks()
names = []
names_list1 = list_hypernetworks(root.hypernetwork_dir)
names_list = tuple(list_hypernetworks(root.hypernetwork_dir).keys())
names.append(names_list)
multipliers=3.0
load_hypernetworks(names, multipliers)
for name in names_list:
    load_hypernetwork(name)
apply_optimizations()

In [ ]:
#@title **ControlNet Inference**
import helpers.controlnet
from helpers.controlnet import load_controlnet, generate_control
from cldm.ddim_hacked import DDIMSampler
from annotator.hed import HEDdetector
from annotator.canny import CannyDetector
from annotator.midas import MidasDetector

def Control():
    load_controlnet_opt = True #@param{type: "boolean"}
    #@markdown **KEEP ENABLED**
    controlnet_model_type = "apply_depth" #@param["apply_hed", "apply_canny", "apply_depth"]
    #@markdown **SELECT WHICH CONTROL NET INFERENCE TO RUN**
    if controlnet_model_type == "apply_canny":
        controlnet_apply_type = CannyDetector()
    elif controlnet_model_type == "apply_hed":
        controlnet_apply_type = HEDdetector()
    else:
        controlnet_apply_type = MidasDetector()
    controlnet_config_path = "/content/deforum-stable-diffusion/ControlNet/models/cldm_v15.yaml" #@param{type: "string"}
    controlnet_model_path = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/control_sd15_depth.pth" #@param{type: "string"}
    #@markdown **YOU MUST DOWNLOAD THE MODELS FIRST, THEN INPUT THE PATH TO THE MODEL**
    IN_dir = "/content/drive/MyDrive/canny2" #@param{type: "string"}
    #@markdown **INPUT DIRECTORY, WHACK NAME, WILL CHANGE LATER**
    if not os.path.exists(IN_dir):
        os.makedirs(IN_dir)
    IN_batch = "canny" #@param{type: "string"}
    #@markdown **INPUT THE NAME YOU WISH YOUR BATCH TO BE**
    OUT_dir = "/content/drive/MyDrive/canny_out2" #@param{type: "string"}
    #@markdown **INPUT THE FOLDER YOU WISH TO SAVE YOUR IMAGEES TO**
    generate_frames = True #@param{type: "boolean"}
    video_path = "/content/drive/MyDrive/Looping.MP4" #@param {type: "string"}
    #@markdown **INIT VIDEO, `generate_frames` and `video_path` WILL SPLIT THE INIT VIDEO INTO `IN_dir` AND WILL USE THE FRAMES FOR GENERATION.**
    img_idx = 9 #@param{type: "number"}
    #@markdown **SET `img_idx` TO THE NUMBER YOU WANT TO RESUME FROM**
    # max_frames = len(os.listdir(IN_dir))
    resume_control = False #@param{type: "boolean"}
    #@markdown **ENABLES RESUME FROM `img_idx`**
    prompt = "winterpunk" #@param{type: "string"}
    a_prompt = "4k"#@param{type: "string"}
    #@markdown **ADDED PROMPT**
    n_prompt = "low resolution" #@param{type: "string"}
    #@markdown **NEGATIVE PROMPT**
    num_samples = 1 #@param{type: "number"}
    image_resolution = 512 #@param{type: "number"}
    detect_resolution = 512 #@param{type: "number"}
    ddim_steps = 25 #@param{type: "number"}
    guess_mode = False #@param{type: "boolean"}
    strength = 1 #@param {type:"slider", min:0, max:1, step:0.01}
    scale = 9 #@param{type: "number"}
    seed = 212985896 #@param{type: "number"}
    eta = 0 #@param{type: "number"}
    canny_low_threshold = 100 #@param {type:"slider", min:0, max:255, step:1}
    canny_high_threshold = 200 #@param {type:"slider", min:0, max:255, step:1}
    render_video_every = 30 #@param{type: "number"}
    if load_controlnet_opt:
        model = load_controlnet(controlnet_config_path, controlnet_model_path)
    model = model
    ddim_sampler = DDIMSampler(model)
    save_memory = False
    return locals()

control = Control()
control = SimpleNamespace(**control)

# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

generate_control(control)

In [4]:
#@title $ {\large \textsf{Animation Settings}}$
def DeforumAnimArgs():

    #@markdown ####**Animation:**
    animation_mode = '3D' #@param ['None', '2D', '3D', 'Video Input', 'Interpolation'] {type:'string'}
    max_frames = 15510 #@param {type:"number"}
    border = 'wrap' #@param ['wrap', 'replicate'] {type:'string'}

    #@markdown $\color{red} {\textsf {Experimental! Produces Very Nice Movements.. use rotation_3d_w}}$
    use_quaternion_rotation = False #@param {type:"boolean"}

    #@markdown ####**Motion Parameters:**
    angle = "0:(0)"#@param {type:"string"}
    zoom = "0:(1.02+0.02*sin(2*3.14*t/300))"#@param {type:"string"}
    translation_x = "0: (0), 5: (-3*(sin(3.141*t/300)))"#@param {type:"string"}
    translation_y = "0:(3+0.02*sin(2*3.14*t/300)-3)"#@param {type:"string"}
    translation_z = "0: (4.00), 1: (4.00), 2: (4.00), 3: (4.00), 4: (4.00), 5: (4.00), 6: (4.00), 7: (4.00), 8: (4.00), 9: (4.00), 10: (4.00), 11: (4.00), 12: (4.00), 13: (4.00), 14: (4.00), 15: (4.00), 16: (4.00), 17: (4.00), 18: (4.00), 19: (4.00), 20: (4.00), 21: (4.00), 22: (4.00), 23: (4.00), 24: (4.00), 25: (4.00), 26: (4.00), 27: (4.00), 28: (4.00), 29: (4.00), 30: (4.00), 31: (4.00), 32: (4.00), 33: (4.00), 34: (4.00), 35: (4.00), 36: (4.00), 37: (4.00), 38: (4.00), 39: (4.00), 40: (4.00), 41: (4.00), 42: (4.00), 43: (4.00), 44: (4.00), 45: (4.00), 46: (4.00), 47: (4.00), 48: (4.00), 49: (4.00), 50: (4.00), 51: (4.00), 52: (4.00), 53: (4.00), 54: (4.00), 55: (4.00), 56: (4.00), 57: (4.00), 58: (4.00), 59: (4.00), 60: (4.00), 61: (4.00), 62: (4.00), 63: (4.00), 64: (4.00), 65: (4.00), 66: (4.00), 67: (4.00), 68: (4.00), 69: (4.00), 70: (4.00), 71: (4.00), 72: (4.00), 73: (4.00), 74: (4.00), 75: (4.00), 76: (4.00), 77: (4.00), 78: (4.00), 79: (4.00), 80: (4.00), 81: (4.00), 82: (4.00), 83: (4.00), 84: (4.00), 85: (4.00), 86: (4.00), 87: (4.00), 88: (4.00), 89: (4.00), 90: (4.00), 91: (4.00), 92: (4.00), 93: (4.00), 94: (4.00), 95: (4.00), 96: (4.00), 97: (4.00), 98: (4.00), 99: (4.00), 100: (4.00), 101: (4.00), 102: (4.00), 103: (4.00), 104: (4.00), 105: (4.00), 106: (4.00), 107: (4.00), 108: (4.00), 109: (4.00), 110: (4.00), 111: (4.00), 112: (4.00), 113: (4.00), 114: (4.00), 115: (4.00), 116: (4.00), 117: (4.00), 118: (4.00), 119: (4.00), 120: (4.00), 121: (4.00), 122: (4.00), 123: (4.00), 124: (4.00), 125: (4.00), 126: (4.00), 127: (4.00), 128: (4.00), 129: (4.00), 130: (4.00), 131: (4.00), 132: (4.00), 133: (4.00), 134: (4.00), 135: (4.00), 136: (4.00), 137: (4.00), 138: (4.00), 139: (4.00), 140: (4.00), 141: (4.00), 142: (4.00), 143: (4.00), 144: (4.00), 145: (4.00), 146: (4.00), 147: (4.00), 148: (4.00), 149: (4.00), 150: (4.00), 151: (4.00), 152: (4.00), 153: (4.00), 154: (4.00), 155: (4.00), 156: (4.00), 157: (4.00), 158: (4.00), 159: (4.00), 160: (4.00), 161: (4.00), 162: (4.00), 163: (4.00), 164: (4.00), 165: (4.00), 166: (4.00), 167: (4.00), 168: (4.00), 169: (4.00), 170: (4.00), 171: (4.00), 172: (4.00), 173: (4.00), 174: (4.00), 175: (4.00), 176: (4.00), 177: (4.00), 178: (4.00), 179: (4.00), 180: (4.00), 181: (4.00), 182: (4.00), 183: (4.00), 184: (4.00), 185: (4.00), 186: (4.00), 187: (4.00), 188: (4.00), 189: (4.00), 190: (4.00), 191: (4.00), 192: (4.00), 193: (4.00), 194: (4.00), 195: (4.00), 196: (4.00), 197: (4.00), 198: (4.00), 199: (4.00), 200: (4.00), 201: (4.00), 202: (4.00), 203: (4.00), 204: (4.00), 205: (4.00), 206: (4.00), 207: (4.00), 208: (4.00), 209: (4.00), 210: (4.00), 211: (4.00), 212: (4.00), 213: (4.00), 214: (4.00), 215: (4.00), 216: (4.00), 217: (4.00), 218: (4.00), 219: (4.00), 220: (4.00), 221: (4.00), 222: (4.00), 223: (4.00), 224: (4.00), 225: (4.00), 226: (4.00), 227: (4.00), 228: (4.00), 229: (4.00), 230: (4.00), 231: (4.00), 232: (4.00), 233: (4.00), 234: (4.00), 235: (4.00), 236: (4.00), 237: (4.00), 238: (4.00), 239: (4.00), 240: (4.00), 241: (4.00), 242: (4.00), 243: (4.00), 244: (4.00), 245: (4.00), 246: (4.00), 247: (4.00), 248: (4.00), 249: (4.00), 250: (4.00), 251: (4.00), 252: (4.00), 253: (4.00), 254: (4.00), 255: (4.00), 256: (4.00), 257: (4.00), 258: (4.00), 259: (4.00), 260: (4.00), 261: (4.00), 262: (4.00), 263: (4.00), 264: (4.00), 265: (4.00), 266: (4.00), 267: (4.00), 268: (4.00), 269: (4.00), 270: (4.00), 271: (17.17), 272: (17.01), 273: (4.00), 274: (4.00), 275: (4.00), 276: (4.00), 277: (4.00), 278: (4.00), 279: (4.00), 280: (4.00), 281: (4.00), 282: (4.00), 283: (4.00), 284: (4.00), 285: (17.23), 286: (4.00), 287: (4.00), 288: (4.00), 289: (4.00), 290: (4.00), 291: (4.00), 292: (4.00), 293: (4.00), 294: (17.01), 295: (17.01), 296: (4.00), 297: (4.00), 298: (4.00), 299: (4.00), 300: (4.00), 301: (4.00), 302: (4.00), 303: (4.00), 304: (4.00), 305: (4.00), 306: (4.00), 307: (4.00), 308: (17.01), 309: (4.00), 310: (4.00), 311: (4.00), 312: (4.00), 313: (4.00), 314: (4.00), 315: (4.00), 316: (4.00), 317: (17.28), 318: (17.01), 319: (4.00), 320: (4.00), 321: (4.00), 322: (4.00), 323: (4.00), 324: (17.01), 325: (4.00), 326: (4.00), 327: (4.00), 328: (4.00), 329: (4.00), 330: (4.00), 331: (4.00), 332: (4.00), 333: (4.00), 334: (4.00), 335: (4.00), 336: (4.00), 337: (4.00), 338: (4.00), 339: (4.00), 340: (17.01), 341: (4.00), 342: (4.00), 343: (4.00), 344: (4.00), 345: (4.00), 346: (4.00), 347: (4.00), 348: (4.00), 349: (4.00), 350: (4.00), 351: (4.00), 352: (4.00), 353: (4.00), 354: (17.01), 355: (4.00), 356: (4.00), 357: (4.00), 358: (4.00), 359: (4.00), 360: (4.00), 361: (4.00), 362: (4.00), 363: (17.06), 364: (4.00), 365: (17.01), 366: (4.00), 367: (4.00), 368: (4.00), 369: (4.00), 370: (4.00), 371: (4.00), 372: (4.00), 373: (4.00), 374: (4.00), 375: (4.00), 376: (4.00), 377: (17.05), 378: (17.01), 379: (4.00), 380: (4.00), 381: (4.00), 382: (4.00), 383: (4.00), 384: (4.00), 385: (4.00), 386: (4.00), 387: (17.16), 388: (4.00), 389: (4.00), 390: (4.00), 391: (4.00), 392: (4.00), 393: (4.00), 394: (4.00), 395: (4.00), 396: (4.00), 397: (4.00), 398: (4.00), 399: (4.00), 400: (17.03), 401: (17.02), 402: (4.00), 403: (4.00), 404: (4.00), 405: (4.00), 406: (4.00), 407: (4.00), 408: (4.00), 409: (17.07), 410: (17.01), 411: (4.00), 412: (4.00), 413: (4.00), 414: (4.00), 415: (4.00), 416: (4.00), 417: (4.00), 418: (4.00), 419: (4.00), 420: (4.00), 421: (4.00), 422: (4.00), 423: (4.00), 424: (4.00), 425: (4.00), 426: (4.00), 427: (4.00), 428: (4.00), 429: (4.00), 430: (4.00), 431: (4.00), 432: (4.00), 433: (4.00), 434: (4.00), 435: (4.00), 436: (4.00), 437: (4.00), 438: (4.00), 439: (4.00), 440: (4.00), 441: (4.00), 442: (4.00), 443: (4.00), 444: (4.00), 445: (4.00), 446: (4.00), 447: (4.00), 448: (4.00), 449: (4.00), 450: (4.00), 451: (4.00), 452: (4.00), 453: (4.00), 454: (4.00), 455: (4.00), 456: (17.06), 457: (4.00), 458: (4.00), 459: (4.00), 460: (4.00), 461: (4.00), 462: (4.00), 463: (4.00), 464: (4.00), 465: (4.00), 466: (4.00), 467: (4.00), 468: (4.00), 469: (17.01), 470: (17.02), 471: (4.00), 472: (4.00), 473: (4.00), 474: (4.00), 475: (4.00), 476: (4.00), 477: (4.00), 478: (4.00), 479: (17.27), 480: (17.01), 481: (4.00), 482: (4.00), 483: (4.00), 484: (4.00), 485: (4.00), 486: (4.00), 487: (4.00), 488: (4.00), 489: (4.00), 490: (4.00), 491: (4.00), 492: (17.06), 493: (4.00), 494: (4.00), 495: (4.00), 496: (4.00), 497: (4.00), 498: (4.00), 499: (4.00), 500: (4.00), 501: (4.00), 502: (17.09), 503: (17.01), 504: (4.00), 505: (4.00), 506: (4.00), 507: (4.00), 508: (4.00), 509: (4.00), 510: (4.00), 511: (4.00), 512: (4.00), 513: (4.00), 514: (4.00), 515: (4.00), 516: (17.23), 517: (4.00), 518: (4.00), 519: (4.00), 520: (4.00), 521: (4.00), 522: (4.00), 523: (17.01), 524: (4.00), 525: (17.04), 526: (17.01), 527: (4.00), 528: (4.00), 529: (4.00), 530: (4.00), 531: (4.00), 532: (4.00), 533: (4.00), 534: (4.00), 535: (4.00), 536: (4.00), 537: (4.00), 538: (4.00), 539: (17.01), 540: (4.00), 541: (4.00), 542: (4.00), 543: (4.00), 544: (4.00), 545: (4.00), 546: (4.00), 547: (4.00), 548: (17.01), 549: (17.08), 550: (4.00), 551: (4.00), 552: (4.00), 553: (4.00), 554: (4.00), 555: (4.00), 556: (4.00), 557: (4.00), 558: (4.00), 559: (4.00), 560: (4.00), 561: (4.00), 562: (4.00), 563: (4.00), 564: (4.00), 565: (4.00), 566: (4.00), 567: (4.00), 568: (4.00), 569: (4.00), 570: (4.00), 571: (4.00), 572: (17.01), 573: (4.00), 574: (4.00), 575: (4.00), 576: (4.00), 577: (4.00), 578: (4.00), 579: (4.00), 580: (4.00), 581: (4.00), 582: (4.00), 583: (4.00), 584: (4.00), 585: (17.01), 586: (17.01), 587: (4.00), 588: (4.00), 589: (4.00), 590: (4.00), 591: (4.00), 592: (4.00), 593: (4.00), 594: (17.03), 595: (17.03), 596: (17.01), 597: (17.01), 598: (4.00), 599: (4.00), 600: (17.01), 601: (4.00), 602: (17.01), 603: (4.00), 604: (4.00), 605: (4.00), 606: (4.00), 607: (4.00), 608: (4.00), 609: (17.01), 610: (4.00), 611: (4.00), 612: (4.00), 613: (4.00), 614: (4.00), 615: (4.00), 616: (4.00), 617: (4.00), 618: (4.00), 619: (4.00), 620: (4.00), 621: (4.00), 622: (4.00), 623: (4.00), 624: (4.00), 625: (4.00), 626: (4.00), 627: (4.00), 628: (4.00), 629: (4.00), 630: (4.00), 631: (4.00), 632: (4.00), 633: (4.00), 634: (4.00), 635: (4.00), 636: (4.00), 637: (4.00), 638: (4.00), 639: (4.00), 640: (4.00), 641: (17.31), 642: (17.05), 643: (17.01), 644: (17.01), 645: (17.01), 646: (17.01), 647: (17.01), 648: (17.02), 649: (17.03), 650: (17.02), 651: (4.00), 652: (17.01), 653: (4.00), 654: (17.03), 655: (17.15), 656: (17.04), 657: (17.02), 658: (17.01), 659: (17.01), 660: (4.00), 661: (4.00), 662: (17.01), 663: (4.00), 664: (17.02), 665: (17.02), 666: (17.01), 667: (4.00), 668: (4.00), 669: (17.01), 670: (4.00), 671: (4.00), 672: (4.00), 673: (4.00), 674: (17.01), 675: (4.00), 676: (17.01), 677: (4.00), 678: (17.02), 679: (17.01), 680: (4.00), 681: (4.00), 682: (4.00), 683: (4.00), 684: (4.00), 685: (4.00), 686: (4.00), 687: (17.05), 688: (17.02), 689: (4.00), 690: (4.00), 691: (4.00), 692: (4.00), 693: (4.00), 694: (4.00), 695: (4.00), 696: (4.00), 697: (4.00), 698: (4.00), 699: (4.00), 700: (4.00), 701: (17.14), 702: (17.01), 703: (4.00), 704: (17.01), 705: (4.00), 706: (4.00), 707: (4.00), 708: (4.00), 709: (4.00), 710: (4.00), 711: (4.00), 712: (4.00), 713: (4.00), 714: (4.00), 715: (4.00), 716: (4.00), 717: (4.00), 718: (4.00), 719: (17.01), 720: (4.00), 721: (4.00), 722: (4.00), 723: (4.00), 724: (17.05), 725: (4.00), 726: (4.00), 727: (4.00), 728: (4.00), 729: (4.00), 730: (4.00), 731: (4.00), 732: (4.00), 733: (17.16), 734: (17.03), 735: (17.01), 736: (4.00), 737: (4.00), 738: (4.00), 739: (4.00), 740: (4.00), 741: (4.00), 742: (4.00), 743: (4.00), 744: (4.00), 745: (4.00), 746: (4.00), 747: (17.04), 748: (17.01), 749: (4.00), 750: (4.00), 751: (4.00), 752: (4.00), 753: (4.00), 754: (4.00), 755: (4.00), 756: (4.00), 757: (17.01), 758: (4.00), 759: (4.00), 760: (4.00), 761: (4.00), 762: (4.00), 763: (4.00), 764: (4.00), 765: (17.04), 766: (4.00), 767: (17.01), 768: (17.01), 769: (17.01), 770: (17.03), 771: (17.03), 772: (17.22), 773: (17.08), 774: (4.00), 775: (17.01), 776: (17.01), 777: (4.00), 778: (4.00), 779: (17.12), 780: (17.06), 781: (4.00), 782: (4.00), 783: (4.00), 784: (4.00), 785: (4.00), 786: (4.00), 787: (4.00), 788: (4.00), 789: (4.00), 790: (4.00), 791: (4.00), 792: (4.00), 793: (4.00), 794: (17.11), 795: (4.00), 796: (4.00), 797: (4.00), 798: (4.00), 799: (4.00), 800: (4.00), 801: (4.00), 802: (4.00), 803: (17.02), 804: (17.01), 805: (4.00), 806: (4.00), 807: (4.00), 808: (4.00), 809: (4.00), 810: (4.00), 811: (4.00), 812: (4.00), 813: (4.00), 814: (4.00), 815: (4.00), 816: (4.00), 817: (17.02), 818: (17.03), 819: (17.02), 820: (17.10), 821: (17.01), 822: (4.00), 823: (4.00), 824: (4.00), 825: (4.00), 826: (17.24), 827: (17.13), 828: (17.02), 829: (17.01), 830: (17.01), 831: (17.01), 832: (17.01), 833: (17.01), 834: (17.01), 835: (4.00), 836: (17.01), 837: (4.00), 838: (17.01), 839: (4.00), 840: (17.13), 841: (17.01), 842: (17.01), 843: (17.01), 844: (17.01), 845: (4.00), 846: (4.00), 847: (4.00), 848: (4.00), 849: (4.00), 850: (17.02), 851: (17.02), 852: (4.00), 853: (4.00), 854: (4.00), 855: (17.01), 856: (4.00), 857: (4.00), 858: (4.00), 859: (4.00), 860: (4.00), 861: (4.00), 862: (4.00), 863: (17.01), 864: (17.01), 865: (4.00), 866: (4.00), 867: (4.00), 868: (4.00), 869: (4.00), 870: (4.00), 871: (4.00), 872: (4.00), 873: (17.17), 874: (17.01), 875: (4.00), 876: (4.00), 877: (4.00), 878: (4.00), 879: (4.00), 880: (4.00), 881: (4.00), 882: (4.00), 883: (4.00), 884: (4.00), 885: (4.00), 886: (4.00), 887: (17.08), 888: (17.01), 889: (4.00), 890: (4.00), 891: (4.00), 892: (4.00), 893: (4.00), 894: (4.00), 895: (4.00), 896: (4.00), 897: (17.01), 898: (4.00), 899: (4.00), 900: (4.00), 901: (4.00), 902: (4.00), 903: (4.00), 904: (4.00), 905: (4.00), 906: (4.00), 907: (4.00), 908: (4.00), 909: (4.00), 910: (17.01), 911: (4.00), 912: (17.01), 913: (4.00), 914: (17.01), 915: (17.01), 916: (17.01), 917: (4.00), 918: (4.00), 919: (17.16), 920: (17.01), 921: (4.00), 922: (4.00), 923: (4.00), 924: (4.00), 925: (4.00), 926: (4.00), 927: (4.00), 928: (4.00), 929: (4.00), 930: (4.00), 931: (4.00), 932: (4.00), 933: (17.04), 934: (17.01), 935: (4.00), 936: (4.00), 937: (4.00), 938: (4.00), 939: (4.00), 940: (4.00), 941: (4.00), 942: (4.00), 943: (4.00), 944: (4.00), 945: (4.00), 946: (4.00), 947: (4.00), 948: (4.00), 949: (4.00), 950: (4.00), 951: (4.00), 952: (4.00), 953: (4.00), 954: (4.00), 955: (4.00), 956: (4.00), 957: (4.00), 958: (4.00), 959: (4.00), 960: (4.00), 961: (4.00), 962: (4.00), 963: (4.00), 964: (4.00), 965: (4.00), 966: (17.04), 967: (17.01), 968: (4.00), 969: (4.00), 970: (4.00), 971: (4.00), 972: (4.00), 973: (4.00), 974: (4.00), 975: (4.00), 976: (4.00), 977: (4.00), 978: (4.00), 979: (4.00), 980: (17.07), 981: (4.00), 982: (4.00), 983: (4.00), 984: (4.00), 985: (4.00), 986: (4.00), 987: (4.00), 988: (4.00), 989: (17.01), 990: (4.00), 991: (4.00), 992: (4.00), 993: (4.00), 994: (4.00), 995: (4.00), 996: (4.00), 997: (4.00), 998: (4.00), 999: (4.00), 1000: (4.00), 1001: (4.00), 1002: (4.00), 1003: (17.01), 1004: (4.00), 1005: (4.00), 1006: (4.00), 1007: (4.00), 1008: (4.00), 1009: (4.00), 1010: (4.00), 1011: (4.00), 1012: (4.00), 1013: (4.00), 1014: (4.00), 1015: (4.00), 1016: (4.00), 1017: (4.00), 1018: (4.00), 1019: (4.00), 1020: (4.00), 1021: (4.00), 1022: (4.00), 1023: (4.00), 1024: (4.00), 1025: (4.00), 1026: (4.00), 1027: (4.00), 1028: (4.00), 1029: (4.00), 1030: (4.00), 1031: (4.00), 1032: (4.00), 1033: (4.00), 1034: (4.00), 1035: (4.00), 1036: (4.00), 1037: (4.00), 1038: (4.00), 1039: (4.00), 1040: (4.00), 1041: (4.00), 1042: (4.00), 1043: (4.00), 1044: (4.00), 1045: (4.00), 1046: (4.00), 1047: (4.00), 1048: (4.00), 1049: (4.00), 1050: (4.00), 1051: (4.00), 1052: (4.00), 1053: (4.00), 1054: (4.00), 1055: (4.00), 1056: (4.00), 1057: (4.00), 1058: (4.00), 1059: (4.00), 1060: (4.00), 1061: (4.00), 1062: (4.00), 1063: (4.00), 1064: (4.00), 1065: (4.00), 1066: (4.00), 1067: (4.00), 1068: (4.00), 1069: (4.00), 1070: (4.00), 1071: (4.00), 1072: (4.00), 1073: (4.00), 1074: (4.00), 1075: (4.00), 1076: (4.00), 1077: (4.00), 1078: (4.00), 1079: (4.00), 1080: (4.00), 1081: (4.00), 1082: (4.00), 1083: (4.00), 1084: (4.00), 1085: (4.00), 1086: (4.00), 1087: (4.00), 1088: (4.00), 1089: (4.00), 1090: (4.00), 1091: (4.00), 1092: (4.00), 1093: (4.00), 1094: (4.00), 1095: (4.00), 1096: (4.00), 1097: (4.00), 1098: (4.00), 1099: (4.00), 1100: (4.00), 1101: (4.00), 1102: (4.00), 1103: (4.00), 1104: (4.00), 1105: (4.00), 1106: (4.00), 1107: (4.00), 1108: (4.00), 1109: (4.00), 1110: (4.00), 1111: (4.00), 1112: (4.00), 1113: (4.00), 1114: (4.00), 1115: (4.00), 1116: (4.00), 1117: (4.00), 1118: (4.00), 1119: (4.00), 1120: (4.00), 1121: (4.00), 1122: (4.00), 1123: (4.00), 1124: (4.00), 1125: (4.00), 1126: (4.00), 1127: (4.00), 1128: (4.00), 1129: (4.00), 1130: (4.00), 1131: (4.00), 1132: (4.00), 1133: (4.00), 1134: (4.00), 1135: (4.00), 1136: (4.00), 1137: (4.00), 1138: (4.00), 1139: (4.00), 1140: (4.00), 1141: (4.00), 1142: (4.00), 1143: (4.00), 1144: (4.00), 1145: (4.00), 1146: (4.00), 1147: (4.00), 1148: (4.00), 1149: (4.00), 1150: (4.00), 1151: (4.00), 1152: (4.00), 1153: (4.00), 1154: (4.00), 1155: (4.00), 1156: (4.00), 1157: (4.00), 1158: (4.00), 1159: (4.00), 1160: (4.00), 1161: (4.00), 1162: (4.00), 1163: (4.00), 1164: (4.00), 1165: (4.00), 1166: (4.00), 1167: (4.00), 1168: (4.00), 1169: (4.00), 1170: (4.00), 1171: (4.00), 1172: (4.00), 1173: (4.00), 1174: (4.00), 1175: (4.00), 1176: (4.00), 1177: (4.00), 1178: (4.00), 1179: (4.00), 1180: (4.00), 1181: (4.00), 1182: (4.00), 1183: (4.00), 1184: (4.00), 1185: (4.00), 1186: (4.00), 1187: (4.00), 1188: (4.00), 1189: (4.00), 1190: (4.00), 1191: (4.00), 1192: (4.00), 1193: (4.00), 1194: (4.00), 1195: (4.00), 1196: (4.00), 1197: (4.00), 1198: (4.00), 1199: (4.00), 1200: (4.00), 1201: (4.00), 1202: (4.00), 1203: (4.00), 1204: (4.00), 1205: (4.00), 1206: (4.00), 1207: (4.00), 1208: (4.00), 1209: (4.00), 1210: (4.00), 1211: (4.00), 1212: (4.00), 1213: (4.00), 1214: (4.00), 1215: (4.00), 1216: (4.00), 1217: (4.00), 1218: (4.00), 1219: (4.00), 1220: (4.00), 1221: (4.00), 1222: (4.00), 1223: (4.00), 1224: (4.00), 1225: (4.00), 1226: (4.00), 1227: (4.00), 1228: (4.00), 1229: (4.00), 1230: (4.00), 1231: (4.00), 1232: (4.00), 1233: (4.00), 1234: (4.00), 1235: (4.00), 1236: (4.00), 1237: (4.00), 1238: (4.00), 1239: (4.00), 1240: (4.00), 1241: (4.00), 1242: (4.00), 1243: (4.00), 1244: (4.00), 1245: (4.00), 1246: (4.00), 1247: (4.00), 1248: (4.00), 1249: (4.00), 1250: (4.00), 1251: (4.00), 1252: (4.00), 1253: (4.00), 1254: (4.00), 1255: (4.00), 1256: (4.00), 1257: (4.00), 1258: (4.00), 1259: (4.00), 1260: (4.00), 1261: (4.00), 1262: (4.00), 1263: (4.00), 1264: (4.00), 1265: (4.00), 1266: (4.00), 1267: (4.00), 1268: (4.00), 1269: (4.00), 1270: (4.00), 1271: (4.00), 1272: (4.00), 1273: (4.00), 1274: (4.00), 1275: (4.00), 1276: (4.00), 1277: (4.00), 1278: (4.00), 1279: (4.00), 1280: (4.00), 1281: (4.00), 1282: (4.00), 1283: (4.00), 1284: (4.00), 1285: (4.00), 1286: (4.00), 1287: (4.00), 1288: (4.00), 1289: (4.00), 1290: (4.00), 1291: (4.00), 1292: (4.00), 1293: (4.00), 1294: (4.00), 1295: (4.00), 1296: (4.00), 1297: (4.00), 1298: (4.00), 1299: (4.00), 1300: (4.00), 1301: (4.00), 1302: (4.00), 1303: (4.00), 1304: (4.00), 1305: (4.00), 1306: (4.00), 1307: (4.00), 1308: (4.00), 1309: (4.00), 1310: (4.00), 1311: (4.00), 1312: (4.00), 1313: (4.00), 1314: (4.00), 1315: (4.00), 1316: (4.00), 1317: (4.00), 1318: (4.00), 1319: (4.00), 1320: (4.00), 1321: (4.00), 1322: (4.00), 1323: (4.00), 1324: (4.00), 1325: (4.00), 1326: (4.00), 1327: (4.00), 1328: (4.00), 1329: (4.00), 1330: (4.00), 1331: (4.00), 1332: (4.00), 1333: (4.00), 1334: (4.00), 1335: (4.00), 1336: (4.00), 1337: (4.00), 1338: (4.00), 1339: (4.00), 1340: (4.00), 1341: (4.00), 1342: (4.00), 1343: (4.00), 1344: (4.00), 1345: (4.00), 1346: (4.00), 1347: (4.00), 1348: (4.00), 1349: (4.00), 1350: (4.00), 1351: (4.00), 1352: (4.00), 1353: (4.00), 1354: (4.00), 1355: (4.00), 1356: (4.00), 1357: (4.00), 1358: (4.00), 1359: (4.00), 1360: (4.00), 1361: (4.00), 1362: (4.00), 1363: (4.00), 1364: (4.00), 1365: (4.00), 1366: (4.00), 1367: (4.00), 1368: (4.00), 1369: (4.00), 1370: (4.00), 1371: (4.00), 1372: (4.00), 1373: (4.00), 1374: (4.00), 1375: (4.00), 1376: (4.00), 1377: (4.00), 1378: (4.00), 1379: (4.00), 1380: (4.00), 1381: (4.00), 1382: (4.00), 1383: (4.00), 1384: (4.00), 1385: (4.00), 1386: (4.00), 1387: (4.00), 1388: (4.00), 1389: (4.00), 1390: (4.00), 1391: (4.00), 1392: (4.00), 1393: (4.00), 1394: (4.00), 1395: (4.00), 1396: (4.00), 1397: (4.00), 1398: (4.00), 1399: (4.00), 1400: (4.00), 1401: (4.00), 1402: (4.00), 1403: (4.00), 1404: (4.00), 1405: (4.00), 1406: (4.00), 1407: (4.00), 1408: (4.00), 1409: (4.00), 1410: (4.00), 1411: (4.00), 1412: (4.00), 1413: (4.00), 1414: (4.00), 1415: (4.00), 1416: (4.00), 1417: (4.00), 1418: (4.00), 1419: (4.00), 1420: (4.00), 1421: (4.00), 1422: (4.00), 1423: (4.00), 1424: (4.00), 1425: (4.00), 1426: (4.00), 1427: (4.00), 1428: (4.00), 1429: (4.00), 1430: (4.00), 1431: (4.00), 1432: (4.00), 1433: (4.00), 1434: (4.00), 1435: (4.00), 1436: (4.00), 1437: (4.00), 1438: (4.00), 1439: (4.00), 1440: (4.00), 1441: (4.00), 1442: (4.00), 1443: (4.00), 1444: (4.00), 1445: (4.00), 1446: (4.00), 1447: (4.00), 1448: (4.00), 1449: (4.00), 1450: (4.00), 1451: (4.00), 1452: (4.00), 1453: (4.00), 1454: (4.00), 1455: (4.00), 1456: (4.00), 1457: (4.00), 1458: (4.00), 1459: (4.00), 1460: (4.00), 1461: (4.00), 1462: (4.00), 1463: (4.00), 1464: (4.00), 1465: (4.00), 1466: (4.00), 1467: (4.00), 1468: (4.00), 1469: (4.00), 1470: (4.00), 1471: (4.00), 1472: (4.00), 1473: (4.00), 1474: (4.00), 1475: (4.00), 1476: (4.00), 1477: (4.00), 1478: (4.00), 1479: (4.00), 1480: (4.00), 1481: (4.00), 1482: (4.00), 1483: (4.00), 1484: (4.00), 1485: (4.00), 1486: (4.00), 1487: (4.00), 1488: (4.00), 1489: (4.00), 1490: (4.00), 1491: (4.00), 1492: (4.00), 1493: (4.00), 1494: (4.00), 1495: (4.00), 1496: (4.00), 1497: (4.00), 1498: (4.00), 1499: (4.00), 1500: (4.00), 1501: (4.00), 1502: (4.00), 1503: (4.00), 1504: (4.00), 1505: (4.00), 1506: (4.00), 1507: (4.00), 1508: (4.00), 1509: (4.00), 1510: (4.00), 1511: (4.00), 1512: (4.00), 1513: (4.00), 1514: (4.00), 1515: (4.00), 1516: (4.00), 1517: (4.00), 1518: (4.00), 1519: (4.00), 1520: (4.00), 1521: (4.00), 1522: (4.00), 1523: (4.00), 1524: (4.00), 1525: (4.00), 1526: (4.00), 1527: (4.00), 1528: (4.00), 1529: (4.00), 1530: (4.00), 1531: (4.00), 1532: (4.00), 1533: (4.00), 1534: (4.00), 1535: (4.00), 1536: (4.00), 1537: (4.00), 1538: (4.00), 1539: (4.00), 1540: (4.00), 1541: (4.00), 1542: (4.00), 1543: (4.00), 1544: (4.00), 1545: (4.00), 1546: (4.00), 1547: (4.00), 1548: (4.00), 1549: (4.00), 1550: (4.00), 1551: (4.00), 1552: (4.00), 1553: (4.00), 1554: (4.00), 1555: (4.00), 1556: (4.00), 1557: (4.00), 1558: (4.00), 1559: (4.00), 1560: (4.00), 1561: (4.00), 1562: (4.00), 1563: (4.00), 1564: (4.00), 1565: (4.00), 1566: (4.00), 1567: (4.00), 1568: (4.00), 1569: (4.00), 1570: (4.00), 1571: (4.00), 1572: (4.00), 1573: (4.00), 1574: (4.00), 1575: (4.00), 1576: (4.00), 1577: (4.00), 1578: (4.00), 1579: (4.00), 1580: (4.00), 1581: (4.00), 1582: (4.00), 1583: (4.00), 1584: (4.00), 1585: (4.00), 1586: (4.00), 1587: (4.00), 1588: (4.00), 1589: (4.00), 1590: (4.00), 1591: (4.00), 1592: (4.00), 1593: (4.00), 1594: (4.00), 1595: (4.00), 1596: (4.00), 1597: (4.00), 1598: (4.00), 1599: (4.00), 1600: (4.00), 1601: (4.00), 1602: (4.00), 1603: (4.00), 1604: (4.00), 1605: (4.00), 1606: (4.00), 1607: (4.00), 1608: (4.00), 1609: (4.00), 1610: (4.00), 1611: (4.00), 1612: (4.00), 1613: (4.00), 1614: (4.00), 1615: (4.00), 1616: (4.00), 1617: (4.00), 1618: (4.00), 1619: (4.00), 1620: (4.00), 1621: (4.00), 1622: (4.00), 1623: (4.00), 1624: (4.00), 1625: (4.00), 1626: (4.00), 1627: (4.00), 1628: (4.00), 1629: (4.00), 1630: (4.00), 1631: (4.00), 1632: (4.00), 1633: (4.00), 1634: (4.00), 1635: (4.00), 1636: (4.00), 1637: (4.00), 1638: (4.00), 1639: (4.00), 1640: (4.00), 1641: (4.00), 1642: (4.00), 1643: (4.00), 1644: (4.00), 1645: (4.00), 1646: (4.00), 1647: (4.00), 1648: (4.00), 1649: (4.00), 1650: (4.00), 1651: (4.00), 1652: (4.00), 1653: (4.00), 1654: (4.00), 1655: (4.00), 1656: (4.00), 1657: (4.00), 1658: (4.00), 1659: (4.00), 1660: (4.00), 1661: (4.00), 1662: (4.00), 1663: (4.00), 1664: (4.00), 1665: (4.00), 1666: (4.00), 1667: (4.00), 1668: (4.00), 1669: (4.00), 1670: (4.00), 1671: (4.00), 1672: (4.00), 1673: (4.00), 1674: (4.00), 1675: (4.00), 1676: (4.00), 1677: (4.00), 1678: (4.00), 1679: (4.00), 1680: (4.00), 1681: (4.00), 1682: (4.00), 1683: (4.00), 1684: (4.00), 1685: (4.00), 1686: (4.00), 1687: (4.00), 1688: (4.00), 1689: (4.00), 1690: (4.00), 1691: (4.00), 1692: (4.00), 1693: (4.00), 1694: (4.00), 1695: (4.00), 1696: (4.00), 1697: (4.00), 1698: (4.00), 1699: (4.00), 1700: (4.00), 1701: (4.00), 1702: (4.00), 1703: (4.00), 1704: (4.00), 1705: (4.00), 1706: (4.00), 1707: (4.00), 1708: (4.00), 1709: (4.00), 1710: (4.00), 1711: (4.00), 1712: (4.00), 1713: (4.00), 1714: (4.00), 1715: (4.00), 1716: (4.00), 1717: (4.00), 1718: (4.00), 1719: (4.00), 1720: (4.00), 1721: (4.00), 1722: (4.00), 1723: (4.00), 1724: (4.00), 1725: (4.00), 1726: (4.00), 1727: (4.00), 1728: (4.00), 1729: (4.00), 1730: (4.00), 1731: (4.00), 1732: (4.00), 1733: (4.00), 1734: (4.00), 1735: (4.00), 1736: (4.00), 1737: (4.00), 1738: (4.00), 1739: (4.00), 1740: (4.00), 1741: (4.00), 1742: (4.00), 1743: (4.00), 1744: (4.00), 1745: (4.00), 1746: (4.00), 1747: (4.00), 1748: (4.00), 1749: (4.00), 1750: (4.00), 1751: (4.00), 1752: (4.00), 1753: (4.00), 1754: (4.00), 1755: (4.00), 1756: (4.00), 1757: (4.00), 1758: (4.00), 1759: (4.00), 1760: (4.00), 1761: (4.00), 1762: (4.00), 1763: (4.00), 1764: (4.00), 1765: (4.00), 1766: (4.00), 1767: (4.00), 1768: (4.00), 1769: (4.00), 1770: (4.00), 1771: (4.00), 1772: (4.00), 1773: (4.00), 1774: (4.00), 1775: (4.00), 1776: (4.00), 1777: (4.00), 1778: (4.00), 1779: (4.00), 1780: (4.00), 1781: (4.00), 1782: (4.00), 1783: (4.00), 1784: (4.00), 1785: (4.00), 1786: (4.00), 1787: (4.00), 1788: (4.00), 1789: (4.00), 1790: (4.00), 1791: (4.00), 1792: (4.00), 1793: (4.00), 1794: (4.00), 1795: (4.00), 1796: (4.00), 1797: (4.00), 1798: (4.00), 1799: (4.00), 1800: (4.00), 1801: (4.00), 1802: (4.00), 1803: (4.00), 1804: (4.00), 1805: (4.00), 1806: (4.00), 1807: (4.00), 1808: (4.00), 1809: (4.00), 1810: (4.00), 1811: (4.00), 1812: (4.00), 1813: (4.00), 1814: (4.00), 1815: (4.00), 1816: (4.00), 1817: (4.00), 1818: (4.00), 1819: (4.00), 1820: (4.00), 1821: (4.00), 1822: (4.00), 1823: (4.00), 1824: (4.00), 1825: (4.00), 1826: (4.00), 1827: (4.00), 1828: (4.00), 1829: (4.00), 1830: (4.00), 1831: (4.00), 1832: (4.00), 1833: (4.00), 1834: (4.00), 1835: (4.00), 1836: (4.00), 1837: (4.00), 1838: (4.00), 1839: (4.00), 1840: (4.00), 1841: (4.00), 1842: (4.00), 1843: (4.00), 1844: (4.00), 1845: (4.00), 1846: (4.00), 1847: (4.00), 1848: (4.00), 1849: (4.00), 1850: (4.00), 1851: (4.00), 1852: (4.00), 1853: (4.00), 1854: (4.00), 1855: (4.00), 1856: (4.00), 1857: (4.00), 1858: (4.00), 1859: (4.00), 1860: (4.00), 1861: (4.00), 1862: (4.00), 1863: (4.00), 1864: (4.00), 1865: (4.00), 1866: (4.00), 1867: (4.00), 1868: (4.00), 1869: (4.00), 1870: (4.00), 1871: (4.00), 1872: (4.00), 1873: (4.00), 1874: (4.00), 1875: (4.00), 1876: (4.00), 1877: (4.00), 1878: (4.00), 1879: (4.00), 1880: (4.00), 1881: (4.00), 1882: (4.00), 1883: (4.00), 1884: (4.00), 1885: (4.00), 1886: (4.00), 1887: (4.00), 1888: (4.00), 1889: (4.00), 1890: (4.00), 1891: (4.00), 1892: (17.01), 1893: (17.02), 1894: (4.00), 1895: (4.00), 1896: (4.00), 1897: (4.00), 1898: (4.00), 1899: (4.00), 1900: (4.00), 1901: (4.00), 1902: (4.00), 1903: (4.00), 1904: (4.00), 1905: (4.00), 1906: (4.00), 1907: (4.00), 1908: (4.00), 1909: (4.00), 1910: (4.00), 1911: (4.00), 1912: (4.00), 1913: (4.00), 1914: (4.00), 1915: (4.00), 1916: (4.00), 1917: (4.00), 1918: (4.00), 1919: (4.00), 1920: (4.00), 1921: (4.00), 1922: (4.00), 1923: (4.00), 1924: (4.00), 1925: (4.00), 1926: (4.00), 1927: (4.00), 1928: (4.00), 1929: (4.00), 1930: (4.00), 1931: (4.00), 1932: (4.00), 1933: (17.01), 1934: (17.02), 1935: (17.01), 1936: (4.00), 1937: (4.00), 1938: (17.03), 1939: (17.02), 1940: (17.01), 1941: (4.00), 1942: (4.00), 1943: (4.00), 1944: (4.00), 1945: (4.00), 1946: (4.00), 1947: (4.00), 1948: (17.01), 1949: (17.02), 1950: (17.01), 1951: (17.02), 1952: (4.00), 1953: (4.00), 1954: (4.00), 1955: (17.01), 1956: (17.01), 1957: (4.00), 1958: (4.00), 1959: (4.00), 1960: (4.00), 1961: (4.00), 1962: (4.00), 1963: (4.00), 1964: (4.00), 1965: (4.00), 1966: (4.00), 1967: (4.00), 1968: (4.00), 1969: (4.00), 1970: (4.00), 1971: (4.00), 1972: (4.00), 1973: (4.00), 1974: (4.00), 1975: (4.00), 1976: (4.00), 1977: (4.00), 1978: (4.00), 1979: (17.01), 1980: (17.02), 1981: (17.01), 1982: (4.00), 1983: (4.00), 1984: (4.00), 1985: (4.00), 1986: (4.00), 1987: (4.00), 1988: (4.00), 1989: (4.00), 1990: (4.00), 1991: (4.00), 1992: (4.00), 1993: (4.00), 1994: (4.00), 1995: (4.00), 1996: (4.00), 1997: (4.00), 1998: (4.00), 1999: (4.00), 2000: (4.00), 2001: (4.00), 2002: (4.00), 2003: (17.01), 2004: (4.00), 2005: (4.00), 2006: (4.00), 2007: (4.00), 2008: (17.01), 2009: (4.00), 2010: (4.00), 2011: (4.00), 2012: (4.00), 2013: (4.00), 2014: (4.00), 2015: (4.00), 2016: (4.00), 2017: (4.00), 2018: (4.00), 2019: (4.00), 2020: (4.00), 2021: (4.00), 2022: (4.00), 2023: (4.00), 2024: (4.00), 2025: (4.00), 2026: (4.00), 2027: (4.00), 2028: (4.00), 2029: (4.00), 2030: (4.00), 2031: (17.01), 2032: (17.02), 2033: (17.01), 2034: (17.01), 2035: (4.00), 2036: (4.00), 2037: (4.00), 2038: (4.00), 2039: (4.00), 2040: (4.00), 2041: (4.00), 2042: (4.00), 2043: (17.01), 2044: (17.01), 2045: (4.00), 2046: (4.00), 2047: (4.00), 2048: (4.00), 2049: (4.00), 2050: (4.00), 2051: (4.00), 2052: (4.00), 2053: (4.00), 2054: (4.00), 2055: (4.00), 2056: (4.00), 2057: (4.00), 2058: (4.00), 2059: (4.00), 2060: (4.00), 2061: (4.00), 2062: (4.00), 2063: (4.00), 2064: (4.00), 2065: (4.00), 2066: (4.00), 2067: (4.00), 2068: (4.00), 2069: (4.00), 2070: (4.00), 2071: (4.00), 2072: (4.00), 2073: (4.00), 2074: (4.00), 2075: (4.00), 2076: (4.00), 2077: (4.00), 2078: (4.00), 2079: (4.00), 2080: (4.00), 2081: (4.00), 2082: (4.00), 2083: (4.00), 2084: (4.00), 2085: (4.00), 2086: (4.00), 2087: (4.00), 2088: (4.00), 2089: (4.00), 2090: (4.00), 2091: (4.00), 2092: (4.00), 2093: (4.00), 2094: (4.00), 2095: (4.00), 2096: (4.00), 2097: (4.00), 2098: (4.00), 2099: (4.00), 2100: (4.00), 2101: (4.00), 2102: (4.00), 2103: (4.00), 2104: (4.00), 2105: (4.00), 2106: (4.00), 2107: (4.00), 2108: (4.00), 2109: (4.00), 2110: (4.00), 2111: (4.00), 2112: (4.00), 2113: (4.00), 2114: (4.00), 2115: (4.00), 2116: (4.00), 2117: (4.00), 2118: (4.00), 2119: (4.00), 2120: (17.02), 2121: (17.01), 2122: (4.00), 2123: (4.00), 2124: (17.01), 2125: (17.01), 2126: (4.00), 2127: (4.00), 2128: (4.00), 2129: (4.00), 2130: (4.00), 2131: (4.00), 2132: (4.00), 2133: (4.00), 2134: (4.00), 2135: (4.00), 2136: (4.00), 2137: (17.02), 2138: (17.01), 2139: (4.00), 2140: (4.00), 2141: (4.00), 2142: (4.00), 2143: (4.00), 2144: (4.00), 2145: (4.00), 2146: (4.00), 2147: (4.00), 2148: (4.00), 2149: (4.00), 2150: (4.00), 2151: (4.00), 2152: (4.00), 2153: (4.00), 2154: (4.00), 2155: (4.00), 2156: (4.00), 2157: (4.00), 2158: (4.00), 2159: (4.00), 2160: (4.00), 2161: (4.00), 2162: (4.00), 2163: (4.00), 2164: (4.00), 2165: (4.00), 2166: (4.00), 2167: (4.00), 2168: (4.00), 2169: (4.00), 2170: (4.00), 2171: (17.01), 2172: (4.00), 2173: (4.00), 2174: (4.00), 2175: (4.00), 2176: (4.00), 2177: (4.00), 2178: (4.00), 2179: (4.00), 2180: (4.00), 2181: (4.00), 2182: (4.00), 2183: (4.00), 2184: (4.00), 2185: (4.00), 2186: (4.00), 2187: (4.00), 2188: (4.00), 2189: (4.00), 2190: (4.00), 2191: (4.00), 2192: (4.00), 2193: (4.00), 2194: (4.00), 2195: (4.00), 2196: (4.00), 2197: (4.00), 2198: (4.00), 2199: (4.00), 2200: (4.00), 2201: (4.00), 2202: (4.00), 2203: (4.00), 2204: (4.00), 2205: (4.00), 2206: (4.00), 2207: (4.00), 2208: (4.00), 2209: (4.00), 2210: (4.00), 2211: (4.00), 2212: (4.00), 2213: (4.00), 2214: (17.01), 2215: (17.02), 2216: (17.01), 2217: (4.00), 2218: (4.00), 2219: (4.00), 2220: (4.00), 2221: (4.00), 2222: (4.00), 2223: (4.00), 2224: (4.00), 2225: (4.00), 2226: (4.00), 2227: (4.00), 2228: (4.00), 2229: (4.00), 2230: (17.02), 2231: (17.04), 2232: (17.02), 2233: (4.00), 2234: (17.01), 2235: (4.00), 2236: (4.00), 2237: (17.15), 2238: (17.01), 2239: (4.00), 2240: (4.00), 2241: (4.00), 2242: (4.00), 2243: (4.00), 2244: (4.00), 2245: (4.00), 2246: (4.00), 2247: (4.00), 2248: (4.00), 2249: (4.00), 2250: (4.00), 2251: (4.00), 2252: (4.00), 2253: (4.00), 2254: (4.00), 2255: (4.00), 2256: (4.00), 2257: (4.00), 2258: (17.11), 2259: (4.00), 2260: (4.00), 2261: (4.00), 2262: (4.00), 2263: (4.00), 2264: (4.00), 2265: (4.00), 2266: (4.00), 2267: (4.00), 2268: (4.00), 2269: (4.00), 2270: (4.00), 2271: (4.00), 2272: (4.00), 2273: (4.00), 2274: (4.00), 2275: (4.00), 2276: (4.00), 2277: (4.00), 2278: (17.14), 2279: (17.01), 2280: (4.00), 2281: (4.00), 2282: (4.00), 2283: (4.00), 2284: (4.00), 2285: (4.00), 2286: (4.00), 2287: (4.00), 2288: (4.00), 2289: (17.04), 2290: (4.00), 2291: (4.00), 2292: (4.00), 2293: (4.00), 2294: (4.00), 2295: (4.00), 2296: (4.00), 2297: (4.00), 2298: (17.03), 2299: (17.03), 2300: (4.00), 2301: (4.00), 2302: (4.00), 2303: (4.00), 2304: (4.00), 2305: (4.00), 2306: (4.00), 2307: (4.00), 2308: (17.05), 2309: (4.00), 2310: (4.00), 2311: (4.00), 2312: (4.00), 2313: (4.00), 2314: (4.00), 2315: (4.00), 2316: (4.00), 2317: (17.01), 2318: (17.03), 2319: (4.00), 2320: (4.00), 2321: (4.00), 2322: (4.00), 2323: (4.00), 2324: (4.00), 2325: (17.07), 2326: (4.00), 2327: (4.00), 2328: (4.00), 2329: (4.00), 2330: (4.00), 2331: (17.03), 2332: (4.00), 2333: (4.00), 2334: (4.00), 2335: (4.00), 2336: (4.00), 2337: (17.02), 2338: (17.01), 2339: (4.00), 2340: (4.00), 2341: (4.00), 2342: (4.00), 2343: (17.01), 2344: (4.00), 2345: (4.00), 2346: (4.00), 2347: (4.00), 2348: (4.00), 2349: (17.04), 2350: (4.00), 2351: (4.00), 2352: (4.00), 2353: (4.00), 2354: (4.00), 2355: (4.00), 2356: (17.08), 2357: (17.01), 2358: (4.00), 2359: (4.00), 2360: (4.00), 2361: (17.11), 2362: (4.00), 2363: (4.00), 2364: (4.00), 2365: (4.00), 2366: (17.08), 2367: (4.00), 2368: (4.00), 2369: (4.00), 2370: (4.00), 2371: (4.00), 2372: (4.00), 2373: (4.00), 2374: (4.00), 2375: (4.00), 2376: (17.12), 2377: (4.00), 2378: (4.00), 2379: (4.00), 2380: (4.00), 2381: (17.03), 2382: (4.00), 2383: (4.00), 2384: (4.00), 2385: (17.02), 2386: (17.02), 2387: (4.00), 2388: (4.00), 2389: (4.00), 2390: (17.01), 2391: (4.00), 2392: (4.00), 2393: (4.00), 2394: (4.00), 2395: (17.08), 2396: (17.02), 2397: (17.01), 2398: (17.07), 2399: (17.01), 2400: (17.01), 2401: (4.00), 2402: (17.01), 2403: (17.11), 2404: (17.01), 2405: (17.04), 2406: (17.01), 2407: (17.01), 2408: (17.15), 2409: (17.01), 2410: (4.00), 2411: (4.00), 2412: (17.09), 2413: (4.00), 2414: (4.00), 2415: (17.11), 2416: (17.01), 2417: (17.02), 2418: (4.00), 2419: (17.01), 2420: (17.02), 2421: (17.02), 2422: (17.01), 2423: (4.00), 2424: (4.00), 2425: (17.02), 2426: (17.05), 2427: (4.00), 2428: (4.00), 2429: (4.00), 2430: (17.01), 2431: (17.05), 2432: (17.01), 2433: (17.01), 2434: (4.00), 2435: (4.00), 2436: (4.00), 2437: (4.00), 2438: (4.00), 2439: (4.00), 2440: (4.00), 2441: (4.00), 2442: (4.00), 2443: (4.00), 2444: (4.00), 2445: (4.00), 2446: (4.00), 2447: (4.00), 2448: (4.00), 2449: (4.00), 2450: (4.00), 2451: (4.00), 2452: (4.00), 2453: (4.00), 2454: (4.00), 2455: (4.00), 2456: (4.00), 2457: (4.00), 2458: (4.00), 2459: (4.00), 2460: (4.00), 2461: (4.00), 2462: (4.00), 2463: (4.00), 2464: (4.00), 2465: (4.00), 2466: (4.00), 2467: (4.00), 2468: (4.00), 2469: (4.00), 2470: (4.00), 2471: (4.00), 2472: (4.00), 2473: (4.00), 2474: (4.00), 2475: (4.00), 2476: (4.00), 2477: (4.00), 2478: (4.00), 2479: (4.00), 2480: (4.00), 2481: (4.00), 2482: (4.00), 2483: (4.00), 2484: (4.00), 2485: (4.00), 2486: (4.00), 2487: (4.00), 2488: (4.00), 2489: (4.00), 2490: (4.00), 2491: (4.00), 2492: (4.00), 2493: (4.00), 2494: (4.00), 2495: (4.00), 2496: (4.00), 2497: (4.00), 2498: (4.00), 2499: (4.00), 2500: (4.00), 2501: (4.00), 2502: (4.00), 2503: (4.00), 2504: (4.00), 2505: (4.00), 2506: (4.00), 2507: (4.00), 2508: (4.00), 2509: (4.00), 2510: (4.00), 2511: (4.00), 2512: (4.00), 2513: (4.00), 2514: (4.00), 2515: (4.00), 2516: (4.00), 2517: (4.00), 2518: (4.00), 2519: (4.00), 2520: (4.00), 2521: (4.00), 2522: (4.00), 2523: (4.00), 2524: (4.00), 2525: (4.00), 2526: (4.00), 2527: (4.00), 2528: (4.00), 2529: (4.00), 2530: (4.00), 2531: (4.00), 2532: (4.00), 2533: (4.00), 2534: (4.00), 2535: (4.00), 2536: (4.00), 2537: (4.00), 2538: (4.00), 2539: (4.00), 2540: (4.00), 2541: (4.00), 2542: (4.00), 2543: (4.00), 2544: (4.00), 2545: (4.00), 2546: (4.00), 2547: (4.00), 2548: (4.00), 2549: (4.00), 2550: (4.00), 2551: (4.00), 2552: (4.00), 2553: (4.00), 2554: (4.00), 2555: (4.00), 2556: (4.00), 2557: (4.00), 2558: (4.00), 2559: (4.00), 2560: (4.00), 2561: (4.00), 2562: (4.00), 2563: (4.00), 2564: (4.00), 2565: (4.00), 2566: (4.00), 2567: (4.00), 2568: (4.00), 2569: (4.00), 2570: (4.00), 2571: (4.00), 2572: (4.00), 2573: (4.00), 2574: (4.00), 2575: (4.00), 2576: (4.00), 2577: (4.00), 2578: (4.00), 2579: (4.00), 2580: (4.00), 2581: (4.00), 2582: (4.00), 2583: (4.00), 2584: (4.00), 2585: (4.00), 2586: (4.00), 2587: (4.00), 2588: (4.00), 2589: (4.00), 2590: (4.00), 2591: (4.00), 2592: (4.00), 2593: (4.00), 2594: (4.00), 2595: (4.00), 2596: (4.00), 2597: (4.00), 2598: (4.00), 2599: (4.00), 2600: (4.00), 2601: (4.00), 2602: (4.00), 2603: (4.00), 2604: (4.00), 2605: (4.00), 2606: (4.00), 2607: (4.00), 2608: (4.00), 2609: (4.00), 2610: (4.00), 2611: (4.00), 2612: (4.00), 2613: (4.00), 2614: (4.00), 2615: (4.00), 2616: (4.00), 2617: (4.00), 2618: (4.00), 2619: (4.00), 2620: (4.00), 2621: (4.00), 2622: (4.00), 2623: (4.00), 2624: (4.00), 2625: (4.00), 2626: (4.00), 2627: (4.00), 2628: (4.00), 2629: (4.00), 2630: (4.00), 2631: (4.00), 2632: (4.00), 2633: (4.00), 2634: (4.00), 2635: (4.00), 2636: (4.00), 2637: (4.00), 2638: (4.00), 2639: (4.00), 2640: (4.00), 2641: (4.00), 2642: (4.00), 2643: (4.00), 2644: (4.00), 2645: (4.00), 2646: (4.00), 2647: (4.00), 2648: (4.00), 2649: (4.00), 2650: (4.00), 2651: (4.00), 2652: (4.00), 2653: (4.00), 2654: (4.00), 2655: (4.00), 2656: (4.00), 2657: (4.00), 2658: (4.00), 2659: (4.00), 2660: (4.00), 2661: (4.00), 2662: (4.00), 2663: (4.00), 2664: (4.00), 2665: (4.00), 2666: (4.00), 2667: (4.00), 2668: (4.00), 2669: (4.00), 2670: (4.00), 2671: (4.00), 2672: (4.00), 2673: (4.00), 2674: (4.00), 2675: (4.00), 2676: (4.00), 2677: (4.00), 2678: (4.00), 2679: (4.00), 2680: (4.00), 2681: (4.00), 2682: (4.00), 2683: (4.00), 2684: (4.00), 2685: (4.00), 2686: (4.00), 2687: (4.00), 2688: (4.00), 2689: (4.00), 2690: (4.00), 2691: (4.00), 2692: (4.00), 2693: (4.00), 2694: (4.00), 2695: (4.00), 2696: (4.00), 2697: (4.00), 2698: (4.00), 2699: (4.00), 2700: (4.00), 2701: (4.00), 2702: (4.00), 2703: (4.00), 2704: (4.00), 2705: (4.00), 2706: (4.00), 2707: (4.00), 2708: (4.00), 2709: (4.00), 2710: (4.00), 2711: (4.00), 2712: (4.00), 2713: (4.00), 2714: (4.00), 2715: (4.00), 2716: (4.00), 2717: (4.00), 2718: (4.00), 2719: (4.00), 2720: (4.00), 2721: (4.00), 2722: (4.00), 2723: (4.00), 2724: (4.00), 2725: (4.00), 2726: (4.00), 2727: (4.00), 2728: (4.00), 2729: (4.00), 2730: (4.00), 2731: (4.00), 2732: (4.00), 2733: (4.00), 2734: (4.00), 2735: (4.00), 2736: (4.00), 2737: (4.00), 2738: (4.00), 2739: (4.00), 2740: (4.00), 2741: (4.00), 2742: (4.00), 2743: (4.00), 2744: (4.00), 2745: (4.00), 2746: (4.00), 2747: (4.00), 2748: (4.00), 2749: (4.00), 2750: (4.00), 2751: (4.00), 2752: (4.00), 2753: (4.00), 2754: (4.00), 2755: (4.00), 2756: (4.00), 2757: (4.00), 2758: (4.00), 2759: (4.00), 2760: (4.00), 2761: (4.00), 2762: (4.00), 2763: (4.00), 2764: (4.00), 2765: (4.00), 2766: (4.00), 2767: (4.00), 2768: (4.00), 2769: (4.00), 2770: (4.00), 2771: (4.00), 2772: (4.00), 2773: (4.00), 2774: (4.00), 2775: (4.00), 2776: (4.00), 2777: (4.00), 2778: (4.00), 2779: (4.00), 2780: (4.00), 2781: (4.00), 2782: (4.00), 2783: (4.00), 2784: (4.00), 2785: (4.00), 2786: (4.00), 2787: (4.00), 2788: (4.00), 2789: (4.00), 2790: (4.00), 2791: (4.00), 2792: (4.00), 2793: (4.00), 2794: (4.00), 2795: (4.00), 2796: (4.00), 2797: (4.00), 2798: (4.00), 2799: (4.00), 2800: (4.00), 2801: (4.00), 2802: (4.00), 2803: (4.00), 2804: (4.00), 2805: (4.00), 2806: (4.00), 2807: (4.00), 2808: (4.00), 2809: (4.00), 2810: (4.00), 2811: (4.00), 2812: (4.00), 2813: (4.00), 2814: (4.00), 2815: (4.00), 2816: (4.00), 2817: (4.00), 2818: (4.00), 2819: (4.00), 2820: (4.00), 2821: (4.00), 2822: (4.00), 2823: (4.00), 2824: (4.00), 2825: (4.00), 2826: (4.00), 2827: (4.00), 2828: (4.00), 2829: (4.00), 2830: (4.00), 2831: (4.00), 2832: (4.00), 2833: (4.00), 2834: (4.00), 2835: (4.00), 2836: (4.00), 2837: (4.00), 2838: (4.00), 2839: (4.00), 2840: (4.00), 2841: (4.00), 2842: (4.00), 2843: (4.00), 2844: (4.00), 2845: (4.00), 2846: (4.00), 2847: (4.00), 2848: (4.00), 2849: (4.00), 2850: (4.00), 2851: (4.00), 2852: (4.00), 2853: (4.00), 2854: (4.00), 2855: (4.00), 2856: (4.00), 2857: (4.00), 2858: (4.00), 2859: (4.00), 2860: (4.00), 2861: (4.00), 2862: (4.00), 2863: (4.00), 2864: (4.00), 2865: (4.00), 2866: (4.00), 2867: (4.00), 2868: (4.00), 2869: (4.00), 2870: (4.00), 2871: (4.00), 2872: (4.00), 2873: (4.00), 2874: (4.00), 2875: (4.00), 2876: (4.00), 2877: (4.00), 2878: (4.00), 2879: (4.00), 2880: (4.00), 2881: (4.00), 2882: (4.00), 2883: (4.00), 2884: (4.00), 2885: (4.00), 2886: (4.00), 2887: (4.00), 2888: (4.00), 2889: (4.00), 2890: (4.00), 2891: (4.00), 2892: (4.00), 2893: (4.00), 2894: (4.00), 2895: (4.00), 2896: (4.00), 2897: (4.00), 2898: (4.00), 2899: (4.00), 2900: (4.00), 2901: (4.00), 2902: (4.00), 2903: (4.00), 2904: (4.00), 2905: (4.00), 2906: (4.00), 2907: (4.00), 2908: (4.00), 2909: (4.00), 2910: (4.00), 2911: (4.00), 2912: (4.00), 2913: (4.00), 2914: (4.00), 2915: (4.00), 2916: (4.00), 2917: (4.00), 2918: (4.00), 2919: (4.00), 2920: (4.00), 2921: (4.00), 2922: (4.00), 2923: (4.00), 2924: (4.00), 2925: (4.00), 2926: (4.00), 2927: (4.00), 2928: (4.00), 2929: (4.00), 2930: (4.00), 2931: (4.00), 2932: (4.00), 2933: (4.00), 2934: (4.00), 2935: (4.00), 2936: (4.00), 2937: (4.00), 2938: (4.00), 2939: (4.00), 2940: (4.00), 2941: (4.00), 2942: (4.00), 2943: (4.00), 2944: (4.00), 2945: (4.00), 2946: (4.00), 2947: (4.00), 2948: (4.00), 2949: (4.00), 2950: (4.00), 2951: (4.00), 2952: (4.00), 2953: (4.00), 2954: (4.00), 2955: (4.00), 2956: (4.00), 2957: (4.00), 2958: (4.00), 2959: (4.00), 2960: (4.00), 2961: (4.00), 2962: (4.00), 2963: (4.00), 2964: (4.00), 2965: (4.00), 2966: (4.00), 2967: (4.00), 2968: (4.00), 2969: (4.00), 2970: (4.00), 2971: (4.00), 2972: (4.00), 2973: (4.00), 2974: (4.00), 2975: (4.00), 2976: (4.00), 2977: (4.00), 2978: (4.00), 2979: (4.00), 2980: (4.00), 2981: (4.00), 2982: (4.00), 2983: (4.00), 2984: (4.00), 2985: (4.00), 2986: (4.00), 2987: (4.00), 2988: (4.00), 2989: (4.00), 2990: (4.00), 2991: (4.00), 2992: (4.00), 2993: (4.00), 2994: (4.00), 2995: (4.00), 2996: (4.00), 2997: (4.00), 2998: (4.00), 2999: (4.00), 3000: (4.00), 3001: (4.00), 3002: (4.00), 3003: (4.00), 3004: (4.00), 3005: (4.00), 3006: (4.00), 3007: (4.00), 3008: (4.00), 3009: (4.00), 3010: (4.00), 3011: (4.00), 3012: (4.00), 3013: (4.00), 3014: (4.00), 3015: (4.00), 3016: (4.00), 3017: (4.00), 3018: (4.00), 3019: (4.00), 3020: (4.00), 3021: (4.00), 3022: (4.00), 3023: (4.00), 3024: (4.00), 3025: (4.00), 3026: (4.00), 3027: (4.00), 3028: (4.00), 3029: (4.00), 3030: (4.00), 3031: (4.00), 3032: (4.00), 3033: (4.00), 3034: (4.00), 3035: (4.00), 3036: (4.00), 3037: (4.00), 3038: (4.00), 3039: (4.00), 3040: (4.00), 3041: (4.00), 3042: (4.00), 3043: (4.00), 3044: (4.00), 3045: (4.00), 3046: (4.00), 3047: (4.00), 3048: (4.00), 3049: (4.00), 3050: (4.00), 3051: (4.00), 3052: (4.00), 3053: (4.00), 3054: (4.00), 3055: (4.00), 3056: (4.00), 3057: (4.00), 3058: (4.00), 3059: (4.00), 3060: (4.00), 3061: (4.00), 3062: (4.00), 3063: (4.00), 3064: (4.00), 3065: (4.00), 3066: (4.00), 3067: (4.00), 3068: (4.00), 3069: (4.00), 3070: (4.00), 3071: (4.00), 3072: (4.00), 3073: (4.00), 3074: (4.00), 3075: (4.00), 3076: (4.00), 3077: (4.00), 3078: (4.00), 3079: (4.00), 3080: (4.00), 3081: (4.00), 3082: (4.00), 3083: (4.00), 3084: (4.00), 3085: (4.00), 3086: (4.00), 3087: (4.00), 3088: (4.00), 3089: (4.00), 3090: (4.00), 3091: (4.00), 3092: (4.00), 3093: (4.00), 3094: (4.00), 3095: (4.00), 3096: (4.00), 3097: (4.00), 3098: (4.00), 3099: (4.00), 3100: (4.00), 3101: (4.00), 3102: (4.00), 3103: (4.00), 3104: (4.00), 3105: (4.00), 3106: (4.00), 3107: (4.00), 3108: (4.00), 3109: (4.00), 3110: (4.00), 3111: (4.00), 3112: (4.00), 3113: (4.00), 3114: (4.00), 3115: (4.00), 3116: (4.00), 3117: (4.00), 3118: (4.00), 3119: (4.00), 3120: (4.00), 3121: (4.00), 3122: (4.00), 3123: (4.00), 3124: (4.00), 3125: (4.00), 3126: (4.00), 3127: (4.00), 3128: (4.00), 3129: (4.00), 3130: (4.00), 3131: (4.00), 3132: (4.00), 3133: (4.00), 3134: (4.00), 3135: (4.00), 3136: (4.00), 3137: (4.00), 3138: (4.00), 3139: (4.00), 3140: (4.00), 3141: (4.00), 3142: (4.00), 3143: (4.00), 3144: (4.00), 3145: (4.00), 3146: (4.00), 3147: (4.00), 3148: (4.00), 3149: (4.00), 3150: (4.00), 3151: (4.00), 3152: (4.00), 3153: (4.00), 3154: (4.00), 3155: (4.00), 3156: (4.00), 3157: (4.00), 3158: (4.00), 3159: (4.00), 3160: (4.00), 3161: (4.00), 3162: (4.00), 3163: (4.00), 3164: (4.00), 3165: (4.00), 3166: (4.00), 3167: (4.00), 3168: (4.00), 3169: (4.00), 3170: (4.00), 3171: (4.00), 3172: (4.00), 3173: (4.00), 3174: (4.00), 3175: (4.00), 3176: (4.00), 3177: (4.00), 3178: (4.00), 3179: (4.00), 3180: (4.00), 3181: (4.00), 3182: (4.00), 3183: (4.00), 3184: (4.00), 3185: (4.00), 3186: (4.00), 3187: (4.00), 3188: (4.00), 3189: (17.10), 3190: (4.00), 3191: (4.00), 3192: (4.00), 3193: (4.00), 3194: (4.00), 3195: (4.00), 3196: (4.00), 3197: (4.00), 3198: (4.00), 3199: (4.00), 3200: (4.00), 3201: (4.00), 3202: (4.00), 3203: (4.00), 3204: (4.00), 3205: (4.00), 3206: (4.00), 3207: (4.00), 3208: (4.00), 3209: (4.00), 3210: (4.00), 3211: (4.00), 3212: (4.00), 3213: (4.00), 3214: (4.00), 3215: (4.00), 3216: (4.00), 3217: (4.00), 3218: (4.00), 3219: (4.00), 3220: (4.00), 3221: (4.00), 3222: (4.00), 3223: (17.01), 3224: (17.01), 3225: (4.00), 3226: (4.00), 3227: (4.00), 3228: (4.00), 3229: (4.00), 3230: (4.00), 3231: (4.00), 3232: (4.00), 3233: (4.00), 3234: (4.00), 3235: (4.00), 3236: (4.00), 3237: (17.01), 3238: (4.00), 3239: (4.00), 3240: (4.00), 3241: (4.00), 3242: (4.00), 3243: (4.00), 3244: (4.00), 3245: (4.00), 3246: (17.04), 3247: (4.00), 3248: (4.00), 3249: (4.00), 3250: (4.00), 3251: (4.00), 3252: (4.00), 3253: (4.00), 3254: (4.00), 3255: (4.00), 3256: (4.00), 3257: (4.00), 3258: (4.00), 3259: (4.00), 3260: (4.00), 3261: (17.01), 3262: (4.00), 3263: (4.00), 3264: (4.00), 3265: (4.00), 3266: (4.00), 3267: (4.00), 3268: (4.00), 3269: (4.00), 3270: (4.00), 3271: (4.00), 3272: (4.00), 3273: (4.00), 3274: (4.00), 3275: (17.01), 3276: (4.00), 3277: (4.00), 3278: (4.00), 3279: (4.00), 3280: (4.00), 3281: (4.00), 3282: (4.00), 3283: (4.00), 3284: (17.06), 3285: (4.00), 3286: (4.00), 3287: (4.00), 3288: (4.00), 3289: (4.00), 3290: (4.00), 3291: (4.00), 3292: (4.00), 3293: (4.00), 3294: (4.00), 3295: (4.00), 3296: (4.00), 3297: (4.00), 3298: (4.00), 3299: (4.00), 3300: (4.00), 3301: (4.00), 3302: (4.00), 3303: (17.06), 3304: (4.00), 3305: (4.00), 3306: (4.00), 3307: (4.00), 3308: (4.00), 3309: (4.00), 3310: (4.00), 3311: (4.00), 3312: (4.00), 3313: (4.00), 3314: (4.00), 3315: (4.00), 3316: (4.00), 3317: (17.14), 3318: (4.00), 3319: (4.00), 3320: (4.00), 3321: (4.00), 3322: (4.00), 3323: (4.00), 3324: (4.00), 3325: (4.00), 3326: (4.00), 3327: (4.00), 3328: (4.00), 3329: (4.00), 3330: (17.07), 3331: (4.00), 3332: (4.00), 3333: (4.00), 3334: (4.00), 3335: (4.00), 3336: (4.00), 3337: (4.00), 3338: (4.00), 3339: (4.00), 3340: (17.01), 3341: (4.00), 3342: (4.00), 3343: (4.00), 3344: (4.00), 3345: (4.00), 3346: (4.00), 3347: (4.00), 3348: (4.00), 3349: (4.00), 3350: (4.00), 3351: (4.00), 3352: (4.00), 3353: (4.00), 3354: (17.03), 3355: (4.00), 3356: (4.00), 3357: (4.00), 3358: (4.00), 3359: (4.00), 3360: (4.00), 3361: (4.00), 3362: (4.00), 3363: (4.00), 3364: (4.00), 3365: (4.00), 3366: (4.00), 3367: (4.00), 3368: (4.00), 3369: (4.00), 3370: (4.00), 3371: (4.00), 3372: (4.00), 3373: (4.00), 3374: (4.00), 3375: (4.00), 3376: (4.00), 3377: (4.00), 3378: (17.03), 3379: (4.00), 3380: (4.00), 3381: (4.00), 3382: (4.00), 3383: (4.00), 3384: (4.00), 3385: (4.00), 3386: (4.00), 3387: (4.00), 3388: (4.00), 3389: (4.00), 3390: (4.00), 3391: (4.00), 3392: (4.00), 3393: (4.00), 3394: (4.00), 3395: (4.00), 3396: (17.02), 3397: (4.00), 3398: (4.00), 3399: (4.00), 3400: (4.00), 3401: (4.00), 3402: (4.00), 3403: (4.00), 3404: (4.00), 3405: (4.00), 3406: (4.00), 3407: (4.00), 3408: (4.00), 3409: (4.00), 3410: (17.01), 3411: (4.00), 3412: (4.00), 3413: (4.00), 3414: (4.00), 3415: (4.00), 3416: (4.00), 3417: (4.00), 3418: (4.00), 3419: (4.00), 3420: (4.00), 3421: (4.00), 3422: (4.00), 3423: (4.00), 3424: (17.01), 3425: (4.00), 3426: (4.00), 3427: (4.00), 3428: (4.00), 3429: (4.00), 3430: (4.00), 3431: (4.00), 3432: (4.00), 3433: (17.02), 3434: (17.01), 3435: (4.00), 3436: (4.00), 3437: (4.00), 3438: (4.00), 3439: (4.00), 3440: (4.00), 3441: (4.00), 3442: (4.00), 3443: (4.00), 3444: (4.00), 3445: (4.00), 3446: (4.00), 3447: (4.00), 3448: (4.00), 3449: (4.00), 3450: (4.00), 3451: (4.00), 3452: (4.00), 3453: (4.00), 3454: (4.00), 3455: (4.00), 3456: (4.00), 3457: (4.00), 3458: (4.00), 3459: (4.00), 3460: (4.00), 3461: (17.01), 3462: (4.00), 3463: (4.00), 3464: (4.00), 3465: (4.00), 3466: (4.00), 3467: (4.00), 3468: (4.00), 3469: (4.00), 3470: (4.00), 3471: (17.08), 3472: (4.00), 3473: (4.00), 3474: (4.00), 3475: (4.00), 3476: (4.00), 3477: (4.00), 3478: (4.00), 3479: (4.00), 3480: (4.00), 3481: (4.00), 3482: (4.00), 3483: (4.00), 3484: (4.00), 3485: (4.00), 3486: (4.00), 3487: (4.00), 3488: (4.00), 3489: (17.07), 3490: (4.00), 3491: (4.00), 3492: (4.00), 3493: (4.00), 3494: (4.00), 3495: (4.00), 3496: (4.00), 3497: (4.00), 3498: (4.00), 3499: (17.01), 3500: (4.00), 3501: (4.00), 3502: (4.00), 3503: (17.04), 3504: (4.00), 3505: (4.00), 3506: (4.00), 3507: (4.00), 3508: (4.00), 3509: (4.00), 3510: (4.00), 3511: (4.00), 3512: (4.00), 3513: (4.00), 3514: (4.00), 3515: (4.00), 3516: (4.00), 3517: (17.01), 3518: (4.00), 3519: (4.00), 3520: (4.00), 3521: (4.00), 3522: (4.00), 3523: (4.00), 3524: (4.00), 3525: (4.00), 3526: (17.11), 3527: (4.00), 3528: (4.00), 3529: (4.00), 3530: (4.00), 3531: (4.00), 3532: (4.00), 3533: (4.00), 3534: (4.00), 3535: (4.00), 3536: (4.00), 3537: (4.00), 3538: (4.00), 3539: (4.00), 3540: (17.01), 3541: (4.00), 3542: (4.00), 3543: (4.00), 3544: (4.00), 3545: (4.00), 3546: (4.00), 3547: (4.00), 3548: (4.00), 3549: (4.00), 3550: (4.00), 3551: (4.00), 3552: (4.00), 3553: (4.00), 3554: (4.00), 3555: (4.00), 3556: (4.00), 3557: (4.00), 3558: (4.00), 3559: (4.00), 3560: (4.00), 3561: (4.00), 3562: (4.00), 3563: (4.00), 3564: (17.02), 3565: (4.00), 3566: (4.00), 3567: (4.00), 3568: (4.00), 3569: (4.00), 3570: (4.00), 3571: (4.00), 3572: (4.00), 3573: (4.00), 3574: (4.00), 3575: (4.00), 3576: (4.00), 3577: (4.00), 3578: (4.00), 3579: (4.00), 3580: (4.00), 3581: (4.00), 3582: (17.04), 3583: (4.00), 3584: (4.00), 3585: (4.00), 3586: (4.00), 3587: (4.00), 3588: (4.00), 3589: (4.00), 3590: (4.00), 3591: (4.00), 3592: (17.01), 3593: (4.00), 3594: (4.00), 3595: (4.00), 3596: (17.05), 3597: (4.00), 3598: (4.00), 3599: (4.00), 3600: (4.00), 3601: (4.00), 3602: (4.00), 3603: (4.00), 3604: (4.00), 3605: (4.00), 3606: (4.00), 3607: (4.00), 3608: (4.00), 3609: (4.00), 3610: (17.02), 3611: (4.00), 3612: (4.00), 3613: (4.00), 3614: (4.00), 3615: (4.00), 3616: (4.00), 3617: (4.00), 3618: (4.00), 3619: (17.08), 3620: (4.00), 3621: (4.00), 3622: (4.00), 3623: (4.00), 3624: (4.00), 3625: (4.00), 3626: (4.00), 3627: (4.00), 3628: (4.00), 3629: (4.00), 3630: (4.00), 3631: (4.00), 3632: (4.00), 3633: (4.00), 3634: (17.02), 3635: (4.00), 3636: (4.00), 3637: (4.00), 3638: (4.00), 3639: (4.00), 3640: (4.00), 3641: (4.00), 3642: (4.00), 3643: (4.00), 3644: (4.00), 3645: (4.00), 3646: (4.00), 3647: (4.00), 3648: (17.03), 3649: (4.00), 3650: (4.00), 3651: (4.00), 3652: (4.00), 3653: (4.00), 3654: (4.00), 3655: (4.00), 3656: (4.00), 3657: (17.05), 3658: (4.00), 3659: (4.00), 3660: (4.00), 3661: (4.00), 3662: (4.00), 3663: (4.00), 3664: (4.00), 3665: (4.00), 3666: (4.00), 3667: (4.00), 3668: (4.00), 3669: (4.00), 3670: (4.00), 3671: (4.00), 3672: (4.00), 3673: (4.00), 3674: (4.00), 3675: (4.00), 3676: (17.06), 3677: (4.00), 3678: (4.00), 3679: (4.00), 3680: (4.00), 3681: (4.00), 3682: (4.00), 3683: (4.00), 3684: (4.00), 3685: (4.00), 3686: (4.00), 3687: (4.00), 3688: (4.00), 3689: (17.01), 3690: (17.02), 3691: (4.00), 3692: (4.00), 3693: (4.00), 3694: (4.00), 3695: (4.00), 3696: (4.00), 3697: (4.00), 3698: (4.00), 3699: (4.00), 3700: (4.00), 3701: (4.00), 3702: (4.00), 3703: (4.00), 3704: (4.00), 3705: (4.00), 3706: (4.00), 3707: (4.00), 3708: (4.00), 3709: (4.00), 3710: (4.00), 3711: (4.00), 3712: (4.00), 3713: (17.04), 3714: (4.00), 3715: (4.00), 3716: (4.00), 3717: (4.00), 3718: (4.00), 3719: (4.00), 3720: (4.00), 3721: (4.00), 3722: (4.00), 3723: (4.00), 3724: (4.00), 3725: (4.00), 3726: (4.00), 3727: (17.02), 3728: (4.00), 3729: (4.00), 3730: (4.00), 3731: (4.00), 3732: (17.01), 3733: (4.00), 3734: (4.00), 3735: (4.00), 3736: (4.00), 3737: (4.00), 3738: (4.00), 3739: (4.00), 3740: (4.00), 3741: (4.00), 3742: (4.00), 3743: (4.00), 3744: (4.00), 3745: (4.00), 3746: (4.00), 3747: (4.00), 3748: (4.00), 3749: (4.00), 3750: (4.00), 3751: (17.04), 3752: (4.00), 3753: (4.00), 3754: (4.00), 3755: (4.00), 3756: (4.00), 3757: (4.00), 3758: (4.00), 3759: (4.00), 3760: (4.00), 3761: (4.00), 3762: (4.00), 3763: (4.00), 3764: (4.00), 3765: (4.00), 3766: (4.00), 3767: (4.00), 3768: (4.00), 3769: (17.04), 3770: (4.00), 3771: (4.00), 3772: (4.00), 3773: (4.00), 3774: (4.00), 3775: (4.00), 3776: (4.00), 3777: (4.00), 3778: (4.00), 3779: (4.00), 3780: (4.00), 3781: (4.00), 3782: (4.00), 3783: (17.05), 3784: (4.00), 3785: (4.00), 3786: (4.00), 3787: (4.00), 3788: (4.00), 3789: (4.00), 3790: (4.00), 3791: (4.00), 3792: (4.00), 3793: (4.00), 3794: (4.00), 3795: (4.00), 3796: (17.02), 3797: (17.01), 3798: (4.00), 3799: (4.00), 3800: (4.00), 3801: (4.00), 3802: (4.00), 3803: (4.00), 3804: (4.00), 3805: (4.00), 3806: (17.02), 3807: (4.00), 3808: (4.00), 3809: (4.00), 3810: (4.00), 3811: (4.00), 3812: (4.00), 3813: (4.00), 3814: (4.00), 3815: (4.00), 3816: (4.00), 3817: (4.00), 3818: (4.00), 3819: (4.00), 3820: (4.00), 3821: (4.00), 3822: (4.00), 3823: (4.00), 3824: (4.00), 3825: (4.00), 3826: (4.00), 3827: (4.00), 3828: (4.00), 3829: (4.00), 3830: (4.00), 3831: (4.00), 3832: (4.00), 3833: (4.00), 3834: (17.02), 3835: (17.01), 3836: (4.00), 3837: (4.00), 3838: (4.00), 3839: (4.00), 3840: (4.00), 3841: (4.00), 3842: (4.00), 3843: (4.00), 3844: (17.04), 3845: (4.00), 3846: (4.00), 3847: (4.00), 3848: (4.00), 3849: (4.00), 3850: (4.00), 3851: (4.00), 3852: (4.00), 3853: (4.00), 3854: (4.00), 3855: (4.00), 3856: (4.00), 3857: (4.00), 3858: (4.00), 3859: (4.00), 3860: (4.00), 3861: (4.00), 3862: (4.00), 3863: (17.03), 3864: (4.00), 3865: (4.00), 3866: (4.00), 3867: (4.00), 3868: (4.00), 3869: (4.00), 3870: (4.00), 3871: (4.00), 3872: (4.00), 3873: (4.00), 3874: (4.00), 3875: (4.00), 3876: (17.04), 3877: (4.00), 3878: (4.00), 3879: (4.00), 3880: (4.00), 3881: (4.00), 3882: (4.00), 3883: (4.00), 3884: (4.00), 3885: (4.00), 3886: (4.00), 3887: (4.00), 3888: (4.00), 3889: (4.00), 3890: (17.01), 3891: (4.00), 3892: (4.00), 3893: (4.00), 3894: (4.00), 3895: (4.00), 3896: (4.00), 3897: (4.00), 3898: (4.00), 3899: (17.01), 3900: (4.00), 3901: (4.00), 3902: (4.00), 3903: (4.00), 3904: (4.00), 3905: (4.00), 3906: (4.00), 3907: (4.00), 3908: (4.00), 3909: (4.00), 3910: (4.00), 3911: (4.00), 3912: (4.00), 3913: (4.00), 3914: (17.03), 3915: (4.00), 3916: (4.00), 3917: (4.00), 3918: (4.00), 3919: (4.00), 3920: (4.00), 3921: (4.00), 3922: (4.00), 3923: (4.00), 3924: (4.00), 3925: (4.00), 3926: (4.00), 3927: (17.06), 3928: (4.00), 3929: (4.00), 3930: (4.00), 3931: (4.00), 3932: (4.00), 3933: (4.00), 3934: (4.00), 3935: (4.00), 3936: (4.00), 3937: (17.18), 3938: (17.01), 3939: (4.00), 3940: (4.00), 3941: (4.00), 3942: (4.00), 3943: (4.00), 3944: (4.00), 3945: (4.00), 3946: (4.00), 3947: (4.00), 3948: (4.00), 3949: (4.00), 3950: (4.00), 3951: (17.04), 3952: (4.00), 3953: (17.03), 3954: (4.00), 3955: (17.02), 3956: (4.00), 3957: (4.00), 3958: (4.00), 3959: (17.03), 3960: (17.04), 3961: (17.02), 3962: (4.00), 3963: (17.02), 3964: (17.01), 3965: (17.01), 3966: (4.00), 3967: (4.00), 3968: (4.00), 3969: (4.00), 3970: (17.01), 3971: (4.00), 3972: (17.01), 3973: (17.04), 3974: (17.03), 3975: (17.01), 3976: (4.00), 3977: (17.03), 3978: (17.01), 3979: (4.00), 3980: (4.00), 3981: (4.00), 3982: (4.00), 3983: (4.00), 3984: (4.00), 3985: (4.00), 3986: (4.00), 3987: (4.00), 3988: (17.01), 3989: (4.00), 3990: (4.00), 3991: (17.02), 3992: (17.09), 3993: (17.01), 3994: (17.02), 3995: (4.00), 3996: (4.00), 3997: (4.00), 3998: (4.00), 3999: (4.00), 4000: (4.00), 4001: (17.01), 4002: (4.00), 4003: (4.00), 4004: (4.00), 4005: (4.00), 4006: (4.00), 4007: (4.00), 4008: (4.00), 4009: (4.00), 4010: (17.01), 4011: (4.00), 4012: (4.00), 4013: (4.00), 4014: (4.00), 4015: (4.00), 4016: (4.00), 4017: (4.00), 4018: (4.00), 4019: (17.04), 4020: (4.00), 4021: (17.01), 4022: (4.00), 4023: (17.01), 4024: (4.00), 4025: (4.00), 4026: (4.00), 4027: (4.00), 4028: (17.06), 4029: (17.02), 4030: (17.01), 4031: (17.01), 4032: (17.01), 4033: (4.00), 4034: (4.00), 4035: (4.00), 4036: (4.00), 4037: (4.00), 4038: (4.00), 4039: (17.01), 4040: (17.01), 4041: (4.00), 4042: (17.02), 4043: (17.02), 4044: (17.01), 4045: (17.01), 4046: (17.01), 4047: (4.00), 4048: (4.00), 4049: (4.00), 4050: (4.00), 4051: (17.24), 4052: (17.02), 4053: (17.01), 4054: (17.01), 4055: (4.00), 4056: (4.00), 4057: (4.00), 4058: (4.00), 4059: (4.00), 4060: (17.01), 4061: (4.00), 4062: (4.00), 4063: (4.00), 4064: (17.01), 4065: (17.06), 4066: (4.00), 4067: (17.01), 4068: (17.01), 4069: (17.01), 4070: (4.00), 4071: (4.00), 4072: (4.00), 4073: (4.00), 4074: (4.00), 4075: (4.00), 4076: (4.00), 4077: (4.00), 4078: (4.00), 4079: (4.00), 4080: (4.00), 4081: (17.01), 4082: (17.03), 4083: (17.19), 4084: (17.11), 4085: (17.09), 4086: (17.30), 4087: (17.16), 4088: (17.02), 4089: (17.01), 4090: (4.00), 4091: (17.03), 4092: (17.14), 4093: (17.22), 4094: (17.07), 4095: (17.01), 4096: (4.00), 4097: (4.00), 4098: (17.01), 4099: (17.02), 4100: (17.02), 4101: (17.01), 4102: (4.00), 4103: (4.00), 4104: (4.00), 4105: (4.00), 4106: (4.00), 4107: (4.00), 4108: (4.00), 4109: (4.00), 4110: (4.00), 4111: (4.00), 4112: (4.00), 4113: (4.00), 4114: (17.01), 4115: (4.00), 4116: (4.00), 4117: (4.00), 4118: (4.00), 4119: (17.18), 4120: (17.31), 4121: (17.02), 4122: (17.03), 4123: (17.06), 4124: (4.00), 4125: (4.00), 4126: (4.00), 4127: (4.00), 4128: (4.00), 4129: (17.01), 4130: (4.00), 4131: (4.00), 4132: (4.00), 4133: (4.00), 4134: (4.00), 4135: (17.03), 4136: (17.01), 4137: (17.02), 4138: (4.00), 4139: (4.00), 4140: (4.00), 4141: (4.00), 4142: (17.22), 4143: (17.05), 4144: (17.01), 4145: (17.02), 4146: (17.03), 4147: (4.00), 4148: (4.00), 4149: (4.00), 4150: (4.00), 4151: (17.01), 4152: (17.01), 4153: (4.00), 4154: (17.01), 4155: (4.00), 4156: (17.04), 4157: (17.01), 4158: (4.00), 4159: (17.02), 4160: (4.00), 4161: (4.00), 4162: (4.00), 4163: (4.00), 4164: (4.00), 4165: (4.00), 4166: (4.00), 4167: (4.00), 4168: (4.00), 4169: (4.00), 4170: (17.01), 4171: (17.01), 4172: (4.00), 4173: (4.00), 4174: (4.00), 4175: (4.00), 4176: (4.00), 4177: (4.00), 4178: (4.00), 4179: (4.00), 4180: (4.00), 4181: (4.00), 4182: (4.00), 4183: (17.02), 4184: (17.01), 4185: (4.00), 4186: (4.00), 4187: (4.00), 4188: (4.00), 4189: (4.00), 4190: (4.00), 4191: (4.00), 4192: (4.00), 4193: (4.00), 4194: (4.00), 4195: (4.00), 4196: (17.03), 4197: (17.04), 4198: (4.00), 4199: (17.03), 4200: (17.01), 4201: (17.04), 4202: (4.00), 4203: (17.02), 4204: (4.00), 4205: (17.03), 4206: (4.00), 4207: (4.00), 4208: (4.00), 4209: (4.00), 4210: (17.38), 4211: (17.01), 4212: (4.00), 4213: (17.01), 4214: (17.01), 4215: (4.00), 4216: (4.00), 4217: (4.00), 4218: (4.00), 4219: (4.00), 4220: (4.00), 4221: (4.00), 4222: (4.00), 4223: (4.00), 4224: (17.01), 4225: (4.00), 4226: (4.00), 4227: (4.00), 4228: (17.01), 4229: (4.00), 4230: (4.00), 4231: (4.00), 4232: (4.00), 4233: (17.35), 4234: (17.01), 4235: (17.01), 4236: (17.01), 4237: (17.01), 4238: (4.00), 4239: (4.00), 4240: (4.00), 4241: (4.00), 4242: (4.00), 4243: (4.00), 4244: (4.00), 4245: (4.00), 4246: (17.01), 4247: (17.04), 4248: (17.01), 4249: (17.01), 4250: (17.01), 4251: (4.00), 4252: (4.00), 4253: (4.00), 4254: (4.00), 4255: (4.00), 4256: (4.00), 4257: (4.00), 4258: (4.00), 4259: (4.00), 4260: (4.00), 4261: (4.00), 4262: (4.00), 4263: (4.00), 4264: (4.00), 4265: (17.04), 4266: (17.22), 4267: (17.01), 4268: (17.02), 4269: (17.33), 4270: (17.04), 4271: (17.03), 4272: (4.00), 4273: (17.03), 4274: (17.10), 4275: (17.03), 4276: (17.02), 4277: (17.04), 4278: (17.03), 4279: (4.00), 4280: (4.00), 4281: (4.00), 4282: (4.00), 4283: (17.16), 4284: (4.00), 4285: (17.01), 4286: (4.00), 4287: (17.02), 4288: (17.02), 4289: (17.01), 4290: (4.00), 4291: (17.02), 4292: (17.01), 4293: (4.00), 4294: (17.02), 4295: (17.01), 4296: (4.00), 4297: (4.00), 4298: (4.00), 4299: (4.00), 4300: (4.00), 4301: (17.19), 4302: (4.00), 4303: (4.00), 4304: (4.00), 4305: (4.00), 4306: (4.00), 4307: (4.00), 4308: (4.00), 4309: (4.00), 4310: (4.00), 4311: (4.00), 4312: (4.00), 4313: (4.00), 4314: (4.00), 4315: (4.00), 4316: (4.00), 4317: (4.00), 4318: (4.00), 4319: (4.00), 4320: (4.00), 4321: (4.00), 4322: (4.00), 4323: (17.01), 4324: (17.06), 4325: (4.00), 4326: (4.00), 4327: (17.02), 4328: (4.00), 4329: (4.00), 4330: (17.01), 4331: (4.00), 4332: (4.00), 4333: (17.03), 4334: (17.01), 4335: (17.02), 4336: (17.01), 4337: (17.04), 4338: (17.01), 4339: (17.07), 4340: (4.00), 4341: (17.02), 4342: (4.00), 4343: (4.00), 4344: (4.00), 4345: (4.00), 4346: (4.00), 4347: (4.00), 4348: (17.01), 4349: (4.00), 4350: (17.01), 4351: (4.00), 4352: (17.01), 4353: (17.01), 4354: (4.00), 4355: (17.02), 4356: (17.10), 4357: (17.02), 4358: (17.01), 4359: (17.05), 4360: (17.01), 4361: (17.01), 4362: (4.00), 4363: (4.00), 4364: (17.02), 4365: (17.21), 4366: (17.01), 4367: (17.02), 4368: (17.01), 4369: (17.01), 4370: (17.01), 4371: (4.00), 4372: (4.00), 4373: (4.00), 4374: (4.00), 4375: (4.00), 4376: (4.00), 4377: (4.00), 4378: (17.01), 4379: (17.03), 4380: (17.01), 4381: (17.03), 4382: (4.00), 4383: (17.10), 4384: (17.05), 4385: (4.00), 4386: (17.03), 4387: (4.00), 4388: (4.00), 4389: (4.00), 4390: (17.01), 4391: (17.01), 4392: (17.09), 4393: (17.01), 4394: (4.00), 4395: (17.01), 4396: (17.01), 4397: (4.00), 4398: (4.00), 4399: (4.00), 4400: (4.00), 4401: (4.00), 4402: (4.00), 4403: (4.00), 4404: (4.00), 4405: (4.00), 4406: (4.00), 4407: (4.00), 4408: (4.00), 4409: (4.00), 4410: (4.00), 4411: (4.00), 4412: (4.00), 4413: (4.00), 4414: (4.00), 4415: (17.03), 4416: (4.00), 4417: (4.00), 4418: (4.00), 4419: (4.00), 4420: (4.00), 4421: (4.00), 4422: (4.00), 4423: (4.00), 4424: (4.00), 4425: (4.00), 4426: (4.00), 4427: (4.00), 4428: (17.07), 4429: (17.07), 4430: (4.00), 4431: (17.01), 4432: (17.02), 4433: (17.01), 4434: (4.00), 4435: (4.00), 4436: (4.00), 4437: (4.00), 4438: (17.01), 4439: (17.01), 4440: (17.01), 4441: (17.01), 4442: (4.00), 4443: (4.00), 4444: (4.00), 4445: (4.00), 4446: (17.02), 4447: (17.03), 4448: (17.01), 4449: (4.00), 4450: (4.00), 4451: (17.01), 4452: (4.00), 4453: (4.00), 4454: (4.00), 4455: (17.11), 4456: (17.04), 4457: (17.01), 4458: (4.00), 4459: (17.06), 4460: (17.03), 4461: (4.00), 4462: (4.00), 4463: (4.00), 4464: (4.00), 4465: (17.03), 4466: (4.00), 4467: (4.00), 4468: (4.00), 4469: (17.03), 4470: (17.05), 4471: (4.00), 4472: (4.00), 4473: (17.04), 4474: (17.05), 4475: (4.00), 4476: (4.00), 4477: (4.00), 4478: (4.00), 4479: (4.00), 4480: (4.00), 4481: (4.00), 4482: (17.01), 4483: (17.28), 4484: (17.06), 4485: (17.02), 4486: (17.10), 4487: (4.00), 4488: (17.02), 4489: (17.01), 4490: (17.01), 4491: (17.01), 4492: (17.01), 4493: (17.01), 4494: (4.00), 4495: (4.00), 4496: (17.04), 4497: (17.20), 4498: (17.01), 4499: (17.02), 4500: (17.01), 4501: (17.01), 4502: (4.00), 4503: (4.00), 4504: (4.00), 4505: (4.00), 4506: (17.13), 4507: (17.01), 4508: (17.01), 4509: (17.05), 4510: (4.00), 4511: (17.01), 4512: (17.01), 4513: (17.01), 4514: (17.02), 4515: (4.00), 4516: (17.03), 4517: (4.00), 4518: (17.01), 4519: (17.03), 4520: (17.01), 4521: (17.01), 4522: (4.00), 4523: (17.03), 4524: (4.00), 4525: (4.00), 4526: (17.01), 4527: (4.00), 4528: (4.00), 4529: (4.00), 4530: (4.00), 4531: (17.01), 4532: (4.00), 4533: (4.00), 4534: (17.02), 4535: (4.00), 4536: (17.01), 4537: (17.01), 4538: (17.05), 4539: (17.02), 4540: (17.01), 4541: (17.06), 4542: (4.00), 4543: (4.00), 4544: (4.00), 4545: (4.00), 4546: (17.09), 4547: (17.06), 4548: (4.00), 4549: (4.00), 4550: (4.00), 4551: (4.00), 4552: (4.00), 4553: (4.00), 4554: (4.00), 4555: (4.00), 4556: (4.00), 4557: (4.00), 4558: (4.00), 4559: (4.00), 4560: (4.00), 4561: (4.00), 4562: (4.00), 4563: (4.00), 4564: (4.00), 4565: (4.00), 4566: (4.00), 4567: (4.00), 4568: (4.00), 4569: (17.01), 4570: (4.00), 4571: (4.00), 4572: (4.00), 4573: (4.00), 4574: (17.00), 4575: (17.01), 4576: (4.00), 4577: (4.00), 4578: (4.00), 4579: (4.00), 4580: (4.00), 4581: (4.00), 4582: (4.00), 4583: (4.00), 4584: (4.00), 4585: (4.00), 4586: (4.00), 4587: (4.00), 4588: (17.02), 4589: (4.00), 4590: (17.02), 4591: (17.01), 4592: (17.01), 4593: (4.00), 4594: (4.00), 4595: (4.00), 4596: (4.00), 4597: (17.19), 4598: (17.02), 4599: (17.01), 4600: (17.01), 4601: (17.02), 4602: (4.00), 4603: (4.00), 4604: (17.01), 4605: (4.00), 4606: (17.01), 4607: (17.02), 4608: (17.02), 4609: (17.01), 4610: (4.00), 4611: (17.01), 4612: (17.02), 4613: (17.01), 4614: (4.00), 4615: (4.00), 4616: (4.00), 4617: (4.00), 4618: (4.00), 4619: (4.00), 4620: (4.00), 4621: (4.00), 4622: (4.00), 4623: (4.00), 4624: (17.03), 4625: (4.00), 4626: (4.00), 4627: (4.00), 4628: (17.01), 4629: (17.23), 4630: (17.08), 4631: (17.11), 4632: (17.26), 4633: (17.03), 4634: (4.00), 4635: (4.00), 4636: (4.00), 4637: (17.01), 4638: (17.04), 4639: (4.00), 4640: (4.00), 4641: (4.00), 4642: (17.01), 4643: (4.00), 4644: (4.00), 4645: (4.00), 4646: (4.00), 4647: (4.00), 4648: (4.00), 4649: (4.00), 4650: (4.00), 4651: (4.00), 4652: (17.03), 4653: (4.00), 4654: (17.01), 4655: (17.03), 4656: (17.03), 4657: (4.00), 4658: (17.04), 4659: (17.01), 4660: (4.00), 4661: (4.00), 4662: (4.00), 4663: (4.00), 4664: (4.00), 4665: (4.00), 4666: (4.00), 4667: (4.00), 4668: (4.00), 4669: (4.00), 4670: (4.00), 4671: (4.00), 4672: (4.00), 4673: (4.00), 4674: (4.00), 4675: (4.00), 4676: (4.00), 4677: (4.00), 4678: (4.00), 4679: (4.00), 4680: (4.00), 4681: (4.00), 4682: (4.00), 4683: (4.00), 4684: (4.00), 4685: (4.00), 4686: (4.00), 4687: (4.00), 4688: (4.00), 4689: (4.00), 4690: (4.00), 4691: (4.00), 4692: (4.00), 4693: (4.00), 4694: (4.00), 4695: (4.00), 4696: (4.00), 4697: (4.00), 4698: (4.00), 4699: (4.00), 4700: (4.00), 4701: (4.00), 4702: (4.00), 4703: (4.00), 4704: (4.00), 4705: (4.00), 4706: (4.00), 4707: (4.00), 4708: (4.00), 4709: (4.00), 4710: (4.00), 4711: (4.00), 4712: (4.00), 4713: (4.00), 4714: (4.00), 4715: (4.00), 4716: (4.00), 4717: (4.00), 4718: (4.00), 4719: (4.00), 4720: (4.00), 4721: (4.00), 4722: (4.00), 4723: (4.00), 4724: (4.00), 4725: (4.00), 4726: (4.00), 4727: (4.00), 4728: (4.00), 4729: (4.00), 4730: (4.00), 4731: (4.00), 4732: (4.00), 4733: (4.00), 4734: (4.00), 4735: (4.00), 4736: (4.00), 4737: (4.00), 4738: (4.00), 4739: (4.00), 4740: (4.00), 4741: (4.00), 4742: (4.00), 4743: (4.00), 4744: (4.00), 4745: (4.00), 4746: (4.00), 4747: (4.00), 4748: (4.00), 4749: (4.00), 4750: (4.00), 4751: (4.00), 4752: (4.00), 4753: (4.00), 4754: (4.00), 4755: (4.00), 4756: (4.00), 4757: (4.00), 4758: (4.00), 4759: (4.00), 4760: (4.00), 4761: (4.00), 4762: (4.00), 4763: (4.00), 4764: (4.00), 4765: (4.00), 4766: (4.00), 4767: (4.00), 4768: (4.00), 4769: (4.00), 4770: (4.00), 4771: (4.00), 4772: (4.00), 4773: (4.00), 4774: (4.00), 4775: (4.00), 4776: (4.00), 4777: (4.00), 4778: (4.00), 4779: (4.00), 4780: (4.00), 4781: (4.00), 4782: (4.00), 4783: (4.00), 4784: (4.00), 4785: (4.00), 4786: (4.00), 4787: (4.00), 4788: (4.00), 4789: (4.00), 4790: (4.00), 4791: (4.00), 4792: (4.00), 4793: (4.00), 4794: (4.00), 4795: (4.00), 4796: (4.00), 4797: (4.00), 4798: (4.00), 4799: (4.00), 4800: (4.00), 4801: (4.00), 4802: (4.00), 4803: (4.00), 4804: (4.00), 4805: (4.00), 4806: (4.00), 4807: (4.00), 4808: (4.00), 4809: (4.00), 4810: (4.00), 4811: (4.00), 4812: (4.00), 4813: (4.00), 4814: (4.00), 4815: (4.00), 4816: (4.00), 4817: (4.00), 4818: (4.00), 4819: (4.00), 4820: (4.00), 4821: (4.00), 4822: (4.00), 4823: (4.00), 4824: (4.00), 4825: (4.00), 4826: (4.00), 4827: (4.00), 4828: (4.00), 4829: (4.00), 4830: (4.00), 4831: (4.00), 4832: (4.00), 4833: (4.00), 4834: (4.00), 4835: (4.00), 4836: (4.00), 4837: (4.00), 4838: (4.00), 4839: (4.00), 4840: (4.00), 4841: (4.00), 4842: (4.00), 4843: (4.00), 4844: (4.00), 4845: (4.00), 4846: (4.00), 4847: (4.00), 4848: (4.00), 4849: (4.00), 4850: (4.00), 4851: (17.25), 4852: (17.03), 4853: (17.01), 4854: (4.00), 4855: (4.00), 4856: (4.00), 4857: (4.00), 4858: (4.00), 4859: (4.00), 4860: (4.00), 4861: (17.02), 4862: (17.07), 4863: (17.02), 4864: (17.01), 4865: (4.00), 4866: (4.00), 4867: (4.00), 4868: (4.00), 4869: (17.01), 4870: (4.00), 4871: (17.02), 4872: (17.01), 4873: (17.01), 4874: (4.00), 4875: (4.00), 4876: (4.00), 4877: (4.00), 4878: (17.01), 4879: (17.01), 4880: (4.00), 4881: (4.00), 4882: (4.00), 4883: (4.00), 4884: (4.00), 4885: (4.00), 4886: (4.00), 4887: (4.00), 4888: (17.04), 4889: (17.01), 4890: (4.00), 4891: (4.00), 4892: (4.00), 4893: (4.00), 4894: (4.00), 4895: (4.00), 4896: (4.00), 4897: (4.00), 4898: (17.01), 4899: (4.00), 4900: (17.01), 4901: (4.00), 4902: (4.00), 4903: (4.00), 4904: (4.00), 4905: (17.01), 4906: (17.10), 4907: (4.00), 4908: (4.00), 4909: (4.00), 4910: (4.00), 4911: (4.00), 4912: (4.00), 4913: (4.00), 4914: (4.00), 4915: (17.01), 4916: (17.01), 4917: (4.00), 4918: (17.01), 4919: (4.00), 4920: (4.00), 4921: (17.04), 4922: (4.00), 4923: (17.01), 4924: (4.00), 4925: (4.00), 4926: (4.00), 4927: (4.00), 4928: (4.00), 4929: (4.00), 4930: (4.00), 4931: (4.00), 4932: (17.03), 4933: (4.00), 4934: (4.00), 4935: (4.00), 4936: (4.00), 4937: (4.00), 4938: (4.00), 4939: (4.00), 4940: (4.00), 4941: (4.00), 4942: (17.04), 4943: (4.00), 4944: (4.00), 4945: (4.00), 4946: (4.00), 4947: (4.00), 4948: (4.00), 4949: (4.00), 4950: (4.00), 4951: (4.00), 4952: (4.00), 4953: (4.00), 4954: (4.00), 4955: (4.00), 4956: (4.00), 4957: (4.00), 4958: (4.00), 4959: (17.01), 4960: (17.07), 4961: (4.00), 4962: (17.02), 4963: (17.01), 4964: (17.01), 4965: (4.00), 4966: (4.00), 4967: (4.00), 4968: (4.00), 4969: (17.01), 4970: (17.01), 4971: (4.00), 4972: (4.00), 4973: (4.00), 4974: (4.00), 4975: (4.00), 4976: (4.00), 4977: (4.00), 4978: (4.00), 4979: (4.00), 4980: (4.00), 4981: (4.00), 4982: (4.00), 4983: (4.00), 4984: (4.00), 4985: (4.00), 4986: (4.00), 4987: (4.00), 4988: (4.00), 4989: (4.00), 4990: (4.00), 4991: (4.00), 4992: (4.00), 4993: (4.00), 4994: (4.00), 4995: (4.00), 4996: (17.05), 4997: (4.00), 4998: (17.01), 4999: (17.01), 5000: (17.02), 5001: (17.01), 5002: (4.00), 5003: (4.00), 5004: (4.00), 5005: (17.01), 5006: (17.03), 5007: (17.01), 5008: (4.00), 5009: (17.01), 5010: (17.03), 5011: (4.00), 5012: (17.06), 5013: (17.02), 5014: (17.01), 5015: (4.00), 5016: (4.00), 5017: (17.04), 5018: (17.02), 5019: (4.00), 5020: (4.00), 5021: (17.01), 5022: (4.00), 5023: (17.07), 5024: (4.00), 5025: (4.00), 5026: (4.00), 5027: (4.00), 5028: (4.00), 5029: (4.00), 5030: (4.00), 5031: (4.00), 5032: (17.04), 5033: (17.06), 5034: (17.01), 5035: (4.00), 5036: (4.00), 5037: (4.00), 5038: (4.00), 5039: (17.01), 5040: (17.02), 5041: (4.00), 5042: (4.00), 5043: (4.00), 5044: (17.01), 5045: (4.00), 5046: (4.00), 5047: (4.00), 5048: (4.00), 5049: (4.00), 5050: (4.00), 5051: (4.00), 5052: (4.00), 5053: (4.00), 5054: (4.00), 5055: (4.00), 5056: (4.00), 5057: (4.00), 5058: (4.00), 5059: (4.00), 5060: (17.02), 5061: (4.00), 5062: (17.01), 5063: (4.00), 5064: (4.00), 5065: (4.00), 5066: (4.00), 5067: (4.00), 5068: (4.00), 5069: (4.00), 5070: (4.00), 5071: (4.00), 5072: (4.00), 5073: (4.00), 5074: (4.00), 5075: (4.00), 5076: (4.00), 5077: (4.00), 5078: (4.00), 5079: (17.03), 5080: (17.01), 5081: (17.02), 5082: (4.00), 5083: (4.00), 5084: (4.00), 5085: (4.00), 5086: (17.01), 5087: (17.11), 5088: (4.00), 5089: (4.00), 5090: (4.00), 5091: (4.00), 5092: (4.00), 5093: (4.00), 5094: (17.02), 5095: (17.01), 5096: (4.00), 5097: (4.00), 5098: (4.00), 5099: (4.00), 5100: (17.01), 5101: (4.00), 5102: (17.01), 5103: (17.02), 5104: (4.00), 5105: (4.00), 5106: (4.00), 5107: (4.00), 5108: (4.00), 5109: (4.00), 5110: (4.00), 5111: (4.00), 5112: (4.00), 5113: (4.00), 5114: (17.01), 5115: (4.00), 5116: (4.00), 5117: (4.00), 5118: (17.01), 5119: (4.00), 5120: (4.00), 5121: (4.00), 5122: (4.00), 5123: (17.04), 5124: (4.00), 5125: (17.01), 5126: (4.00), 5127: (4.00), 5128: (4.00), 5129: (17.02), 5130: (4.00), 5131: (4.00), 5132: (4.00), 5133: (4.00), 5134: (4.00), 5135: (4.00), 5136: (4.00), 5137: (4.00), 5138: (4.00), 5139: (4.00), 5140: (4.00), 5141: (4.00), 5142: (4.00), 5143: (4.00), 5144: (4.00), 5145: (4.00), 5146: (4.00), 5147: (4.00), 5148: (4.00), 5149: (4.00), 5150: (4.00), 5151: (4.00), 5152: (4.00), 5153: (4.00), 5154: (4.00), 5155: (4.00), 5156: (4.00), 5157: (4.00), 5158: (4.00), 5159: (17.01), 5160: (4.00), 5161: (4.00), 5162: (4.00), 5163: (4.00), 5164: (4.00), 5165: (4.00), 5166: (4.00), 5167: (4.00), 5168: (4.00), 5169: (4.00), 5170: (17.02), 5171: (4.00), 5172: (17.01), 5173: (4.00), 5174: (4.00), 5175: (4.00), 5176: (4.00), 5177: (17.01), 5178: (17.03), 5179: (4.00), 5180: (4.00), 5181: (17.01), 5182: (4.00), 5183: (4.00), 5184: (4.00), 5185: (4.00), 5186: (4.00), 5187: (4.00), 5188: (4.00), 5189: (4.00), 5190: (4.00), 5191: (17.03), 5192: (17.01), 5193: (4.00), 5194: (17.01), 5195: (17.01), 5196: (4.00), 5197: (4.00), 5198: (17.01), 5199: (17.03), 5200: (4.00), 5201: (4.00), 5202: (4.00), 5203: (4.00), 5204: (17.01), 5205: (4.00), 5206: (4.00), 5207: (4.00), 5208: (4.00), 5209: (4.00), 5210: (4.00), 5211: (4.00), 5212: (4.00), 5213: (17.06), 5214: (17.01), 5215: (17.01), 5216: (4.00), 5217: (4.00), 5218: (4.00), 5219: (4.00), 5220: (4.00), 5221: (4.00), 5222: (4.00), 5223: (17.01), 5224: (17.01), 5225: (17.01), 5226: (17.01), 5227: (17.01), 5228: (4.00), 5229: (17.04), 5230: (4.00), 5231: (4.00), 5232: (4.00), 5233: (4.00), 5234: (4.00), 5235: (4.00), 5236: (4.00), 5237: (4.00), 5238: (4.00), 5239: (4.00), 5240: (4.00), 5241: (4.00), 5242: (4.00), 5243: (4.00), 5244: (4.00), 5245: (4.00), 5246: (4.00), 5247: (4.00), 5248: (4.00), 5249: (4.00), 5250: (4.00), 5251: (4.00), 5252: (4.00), 5253: (4.00), 5254: (17.04), 5255: (4.00), 5256: (4.00), 5257: (4.00), 5258: (4.00), 5259: (4.00), 5260: (4.00), 5261: (4.00), 5262: (4.00), 5263: (4.00), 5264: (4.00), 5265: (4.00), 5266: (4.00), 5267: (17.01), 5268: (4.00), 5269: (4.00), 5270: (17.01), 5271: (17.02), 5272: (17.01), 5273: (4.00), 5274: (4.00), 5275: (4.00), 5276: (4.00), 5277: (17.01), 5278: (4.00), 5279: (4.00), 5280: (4.00), 5281: (4.00), 5282: (4.00), 5283: (17.01), 5284: (17.02), 5285: (4.00), 5286: (4.00), 5287: (4.00), 5288: (4.00), 5289: (4.00), 5290: (4.00), 5291: (4.00), 5292: (4.00), 5293: (4.00), 5294: (17.01), 5295: (4.00), 5296: (4.00), 5297: (4.00), 5298: (4.00), 5299: (4.00), 5300: (4.00), 5301: (4.00), 5302: (4.00), 5303: (17.08), 5304: (17.01), 5305: (4.00), 5306: (4.00), 5307: (4.00), 5308: (4.00), 5309: (4.00), 5310: (4.00), 5311: (4.00), 5312: (4.00), 5313: (17.01), 5314: (17.01), 5315: (17.07), 5316: (17.01), 5317: (17.03), 5318: (17.02), 5319: (4.00), 5320: (17.01), 5321: (4.00), 5322: (4.00), 5323: (4.00), 5324: (17.01), 5325: (4.00), 5326: (4.00), 5327: (4.00), 5328: (4.00), 5329: (4.00), 5330: (4.00), 5331: (4.00), 5332: (4.00), 5333: (4.00), 5334: (4.00), 5335: (4.00), 5336: (4.00), 5337: (4.00), 5338: (4.00), 5339: (4.00), 5340: (4.00), 5341: (4.00), 5342: (4.00), 5343: (17.03), 5344: (17.01), 5345: (4.00), 5346: (4.00), 5347: (4.00), 5348: (4.00), 5349: (4.00), 5350: (4.00), 5351: (4.00), 5352: (4.00), 5353: (4.00), 5354: (4.00), 5355: (4.00), 5356: (4.00), 5357: (17.01), 5358: (4.00), 5359: (4.00), 5360: (4.00), 5361: (4.00), 5362: (4.00), 5363: (4.00), 5364: (4.00), 5365: (17.01), 5366: (4.00), 5367: (4.00), 5368: (4.00), 5369: (17.04), 5370: (4.00), 5371: (4.00), 5372: (17.02), 5373: (17.02), 5374: (17.02), 5375: (17.01), 5376: (4.00), 5377: (4.00), 5378: (17.01), 5379: (17.01), 5380: (17.03), 5381: (17.02), 5382: (4.00), 5383: (17.02), 5384: (17.15), 5385: (4.00), 5386: (4.00), 5387: (4.00), 5388: (4.00), 5389: (4.00), 5390: (4.00), 5391: (4.00), 5392: (4.00), 5393: (17.48), 5394: (17.06), 5395: (4.00), 5396: (4.00), 5397: (4.00), 5398: (4.00), 5399: (4.00), 5400: (4.00), 5401: (4.00), 5402: (4.00), 5403: (4.00), 5404: (4.00), 5405: (4.00), 5406: (4.00), 5407: (4.00), 5408: (4.00), 5409: (4.00), 5410: (4.00), 5411: (4.00), 5412: (4.00), 5413: (17.01), 5414: (17.01), 5415: (4.00), 5416: (4.00), 5417: (4.00), 5418: (4.00), 5419: (4.00), 5420: (17.01), 5421: (4.00), 5422: (4.00), 5423: (4.00), 5424: (4.00), 5425: (4.00), 5426: (4.00), 5427: (4.00), 5428: (4.00), 5429: (4.00), 5430: (4.00), 5431: (4.00), 5432: (4.00), 5433: (4.00), 5434: (4.00), 5435: (4.00), 5436: (4.00), 5437: (4.00), 5438: (17.02), 5439: (17.03), 5440: (17.01), 5441: (17.07), 5442: (17.01), 5443: (17.03), 5444: (4.00), 5445: (4.00), 5446: (4.00), 5447: (4.00), 5448: (17.02), 5449: (17.01), 5450: (4.00), 5451: (4.00), 5452: (4.00), 5453: (4.00), 5454: (4.00), 5455: (4.00), 5456: (4.00), 5457: (4.00), 5458: (4.00), 5459: (4.00), 5460: (17.01), 5461: (17.09), 5462: (17.01), 5463: (17.01), 5464: (17.06), 5465: (4.00), 5466: (17.01), 5467: (4.00), 5468: (4.00), 5469: (4.00), 5470: (4.00), 5471: (4.00), 5472: (4.00), 5473: (4.00), 5474: (4.00), 5475: (17.35), 5476: (17.04), 5477: (17.18), 5478: (17.09), 5479: (4.00), 5480: (17.09), 5481: (17.09), 5482: (17.05), 5483: (17.13), 5484: (17.60), 5485: (17.16), 5486: (17.14), 5487: (17.56), 5488: (17.04), 5489: (17.07), 5490: (17.01), 5491: (4.00), 5492: (17.01), 5493: (17.09), 5494: (17.19), 5495: (17.03), 5496: (17.22), 5497: (17.03), 5498: (17.05), 5499: (17.01), 5500: (17.01), 5501: (17.01), 5502: (17.02), 5503: (17.03), 5504: (17.04), 5505: (17.06), 5506: (4.00), 5507: (4.00), 5508: (4.00), 5509: (4.00), 5510: (17.01), 5511: (17.01), 5512: (17.01), 5513: (4.00), 5514: (17.01), 5515: (4.00), 5516: (4.00), 5517: (4.00), 5518: (4.00), 5519: (4.00), 5520: (4.00), 5521: (4.00), 5522: (17.01), 5523: (4.00), 5524: (17.01), 5525: (4.00), 5526: (17.01), 5527: (17.02), 5528: (17.01), 5529: (4.00), 5530: (4.00), 5531: (4.00), 5532: (4.00), 5533: (4.00), 5534: (4.00), 5535: (4.00), 5536: (4.00), 5537: (4.00), 5538: (17.02), 5539: (17.02), 5540: (17.11), 5541: (17.04), 5542: (17.13), 5543: (17.03), 5544: (17.02), 5545: (17.09), 5546: (17.01), 5547: (17.27), 5548: (17.02), 5549: (17.07), 5550: (17.01), 5551: (17.08), 5552: (17.01), 5553: (17.09), 5554: (17.12), 5555: (17.03), 5556: (17.02), 5557: (4.00), 5558: (4.00), 5559: (17.01), 5560: (17.01), 5561: (4.00), 5562: (4.00), 5563: (4.00), 5564: (17.09), 5565: (17.02), 5566: (4.00), 5567: (4.00), 5568: (4.00), 5569: (4.00), 5570: (4.00), 5571: (4.00), 5572: (4.00), 5573: (17.11), 5574: (17.01), 5575: (4.00), 5576: (17.01), 5577: (17.04), 5578: (17.01), 5579: (4.00), 5580: (4.00), 5581: (4.00), 5582: (4.00), 5583: (4.00), 5584: (17.02), 5585: (17.04), 5586: (17.01), 5587: (17.01), 5588: (4.00), 5589: (4.00), 5590: (4.00), 5591: (4.00), 5592: (17.04), 5593: (17.01), 5594: (17.03), 5595: (17.01), 5596: (17.01), 5597: (4.00), 5598: (4.00), 5599: (4.00), 5600: (4.00), 5601: (4.00), 5602: (4.00), 5603: (4.00), 5604: (4.00), 5605: (4.00), 5606: (4.00), 5607: (4.00), 5608: (4.00), 5609: (4.00), 5610: (4.00), 5611: (4.00), 5612: (4.00), 5613: (17.01), 5614: (17.01), 5615: (4.00), 5616: (17.01), 5617: (17.02), 5618: (4.00), 5619: (4.00), 5620: (17.03), 5621: (17.02), 5622: (4.00), 5623: (4.00), 5624: (4.00), 5625: (4.00), 5626: (4.00), 5627: (17.01), 5628: (4.00), 5629: (4.00), 5630: (4.00), 5631: (4.00), 5632: (4.00), 5633: (4.00), 5634: (4.00), 5635: (4.00), 5636: (4.00), 5637: (4.00), 5638: (17.03), 5639: (17.08), 5640: (17.07), 5641: (4.00), 5642: (17.03), 5643: (17.01), 5644: (4.00), 5645: (4.00), 5646: (4.00), 5647: (4.00), 5648: (4.00), 5649: (17.01), 5650: (4.00), 5651: (4.00), 5652: (4.00), 5653: (4.00), 5654: (17.01), 5655: (4.00), 5656: (17.01), 5657: (17.02), 5658: (4.00), 5659: (4.00), 5660: (4.00), 5661: (4.00), 5662: (4.00), 5663: (17.03), 5664: (4.00), 5665: (4.00), 5666: (4.00), 5667: (4.00), 5668: (4.00), 5669: (4.00), 5670: (4.00), 5671: (4.00), 5672: (4.00), 5673: (4.00), 5674: (17.01), 5675: (4.00), 5676: (4.00), 5677: (4.00), 5678: (17.01), 5679: (4.00), 5680: (4.00), 5681: (4.00), 5682: (4.00), 5683: (4.00), 5684: (4.00), 5685: (4.00), 5686: (4.00), 5687: (4.00), 5688: (4.00), 5689: (4.00), 5690: (17.01), 5691: (17.01), 5692: (4.00), 5693: (4.00), 5694: (4.00), 5695: (4.00), 5696: (4.00), 5697: (4.00), 5698: (4.00), 5699: (4.00), 5700: (4.00), 5701: (4.00), 5702: (4.00), 5703: (4.00), 5704: (4.00), 5705: (4.00), 5706: (4.00), 5707: (4.00), 5708: (4.00), 5709: (17.02), 5710: (17.08), 5711: (17.01), 5712: (17.02), 5713: (17.01), 5714: (4.00), 5715: (4.00), 5716: (17.03), 5717: (17.02), 5718: (4.00), 5719: (17.07), 5720: (17.02), 5721: (17.10), 5722: (17.04), 5723: (17.01), 5724: (17.02), 5725: (4.00), 5726: (17.02), 5727: (17.05), 5728: (17.04), 5729: (17.01), 5730: (17.01), 5731: (17.01), 5732: (17.13), 5733: (17.01), 5734: (17.01), 5735: (17.01), 5736: (17.01), 5737: (17.01), 5738: (17.03), 5739: (17.02), 5740: (4.00), 5741: (4.00), 5742: (17.01), 5743: (17.20), 5744: (17.01), 5745: (17.01), 5746: (4.00), 5747: (17.01), 5748: (4.00), 5749: (4.00), 5750: (4.00), 5751: (4.00), 5752: (17.08), 5753: (17.01), 5754: (4.00), 5755: (4.00), 5756: (4.00), 5757: (4.00), 5758: (4.00), 5759: (4.00), 5760: (4.00), 5761: (17.01), 5762: (17.02), 5763: (17.02), 5764: (17.01), 5765: (17.01), 5766: (4.00), 5767: (17.03), 5768: (4.00), 5769: (4.00), 5770: (4.00), 5771: (4.00), 5772: (17.01), 5773: (17.01), 5774: (4.00), 5775: (4.00), 5776: (4.00), 5777: (4.00), 5778: (4.00), 5779: (4.00), 5780: (4.00), 5781: (4.00), 5782: (4.00), 5783: (4.00), 5784: (4.00), 5785: (4.00), 5786: (4.00), 5787: (4.00), 5788: (4.00), 5789: (4.00), 5790: (4.00), 5791: (4.00), 5792: (4.00), 5793: (4.00), 5794: (4.00), 5795: (4.00), 5796: (4.00), 5797: (4.00), 5798: (17.02), 5799: (17.04), 5800: (17.01), 5801: (4.00), 5802: (4.00), 5803: (4.00), 5804: (4.00), 5805: (17.04), 5806: (17.01), 5807: (4.00), 5808: (4.00), 5809: (4.00), 5810: (4.00), 5811: (4.00), 5812: (4.00), 5813: (4.00), 5814: (4.00), 5815: (4.00), 5816: (4.00), 5817: (4.00), 5818: (4.00), 5819: (17.03), 5820: (17.02), 5821: (17.02), 5822: (4.00), 5823: (17.03), 5824: (4.00), 5825: (4.00), 5826: (4.00), 5827: (17.01), 5828: (17.01), 5829: (17.01), 5830: (4.00), 5831: (17.02), 5832: (17.05), 5833: (4.00), 5834: (4.00), 5835: (4.00), 5836: (17.02), 5837: (4.00), 5838: (4.00), 5839: (4.00), 5840: (4.00), 5841: (17.05), 5842: (17.01), 5843: (4.00), 5844: (4.00), 5845: (4.00), 5846: (4.00), 5847: (4.00), 5848: (4.00), 5849: (4.00), 5850: (4.00), 5851: (4.00), 5852: (4.00), 5853: (4.00), 5854: (4.00), 5855: (4.00), 5856: (4.00), 5857: (4.00), 5858: (4.00), 5859: (4.00), 5860: (4.00), 5861: (4.00), 5862: (4.00), 5863: (4.00), 5864: (4.00), 5865: (4.00), 5866: (4.00), 5867: (4.00), 5868: (17.01), 5869: (4.00), 5870: (4.00), 5871: (4.00), 5872: (4.00), 5873: (4.00), 5874: (4.00), 5875: (4.00), 5876: (4.00), 5877: (4.00), 5878: (17.01), 5879: (17.02), 5880: (17.01), 5881: (17.02), 5882: (17.01), 5883: (17.01), 5884: (4.00), 5885: (17.02), 5886: (17.09), 5887: (17.02), 5888: (17.02), 5889: (17.04), 5890: (17.01), 5891: (4.00), 5892: (4.00), 5893: (4.00), 5894: (17.01), 5895: (4.00), 5896: (4.00), 5897: (4.00), 5898: (4.00), 5899: (4.00), 5900: (4.00), 5901: (4.00), 5902: (4.00), 5903: (4.00), 5904: (4.00), 5905: (17.02), 5906: (17.32), 5907: (17.03), 5908: (17.02), 5909: (17.02), 5910: (17.02), 5911: (4.00), 5912: (17.01), 5913: (4.00), 5914: (4.00), 5915: (4.00), 5916: (4.00), 5917: (4.00), 5918: (4.00), 5919: (4.00), 5920: (4.00), 5921: (17.08), 5922: (17.01), 5923: (4.00), 5924: (4.00), 5925: (17.01), 5926: (17.04), 5927: (4.00), 5928: (17.02), 5929: (17.02), 5930: (17.23), 5931: (17.02), 5932: (4.00), 5933: (17.01), 5934: (17.05), 5935: (17.02), 5936: (4.00), 5937: (4.00), 5938: (4.00), 5939: (17.01), 5940: (4.00), 5941: (4.00), 5942: (4.00), 5943: (4.00), 5944: (4.00), 5945: (4.00), 5946: (4.00), 5947: (4.00), 5948: (4.00), 5949: (4.00), 5950: (4.00), 5951: (4.00), 5952: (4.00), 5953: (4.00), 5954: (4.00), 5955: (4.00), 5956: (4.00), 5957: (17.01), 5958: (4.00), 5959: (17.02), 5960: (17.04), 5961: (17.01), 5962: (4.00), 5963: (4.00), 5964: (17.01), 5965: (4.00), 5966: (17.01), 5967: (17.11), 5968: (17.01), 5969: (4.00), 5970: (4.00), 5971: (17.05), 5972: (17.02), 5973: (4.00), 5974: (17.01), 5975: (4.00), 5976: (4.00), 5977: (17.02), 5978: (4.00), 5979: (4.00), 5980: (17.02), 5981: (17.01), 5982: (17.05), 5983: (17.03), 5984: (17.01), 5985: (4.00), 5986: (4.00), 5987: (4.00), 5988: (4.00), 5989: (17.01), 5990: (4.00), 5991: (4.00), 5992: (4.00), 5993: (17.02), 5994: (17.01), 5995: (4.00), 5996: (17.01), 5997: (4.00), 5998: (17.01), 5999: (4.00), 6000: (4.00), 6001: (4.00), 6002: (17.02), 6003: (17.07), 6004: (17.02), 6005: (4.00), 6006: (4.00), 6007: (4.00), 6008: (4.00), 6009: (4.00), 6010: (4.00), 6011: (4.00), 6012: (4.00), 6013: (4.00), 6014: (4.00), 6015: (4.00), 6016: (4.00), 6017: (17.01), 6018: (17.01), 6019: (4.00), 6020: (4.00), 6021: (17.01), 6022: (4.00), 6023: (4.00), 6024: (4.00), 6025: (4.00), 6026: (4.00), 6027: (4.00), 6028: (4.00), 6029: (4.00), 6030: (17.02), 6031: (17.01), 6032: (17.01), 6033: (4.00), 6034: (4.00), 6035: (4.00), 6036: (4.00), 6037: (4.00), 6038: (4.00), 6039: (17.01), 6040: (4.00), 6041: (4.00), 6042: (4.00), 6043: (4.00), 6044: (4.00), 6045: (17.01), 6046: (17.01), 6047: (4.00), 6048: (4.00), 6049: (4.00), 6050: (4.00), 6051: (4.00), 6052: (4.00), 6053: (4.00), 6054: (4.00), 6055: (4.00), 6056: (4.00), 6057: (17.01), 6058: (17.01), 6059: (17.02), 6060: (17.01), 6061: (17.01), 6062: (4.00), 6063: (4.00), 6064: (4.00), 6065: (4.00), 6066: (4.00), 6067: (17.03), 6068: (17.01), 6069: (17.01), 6070: (4.00), 6071: (4.00), 6072: (4.00), 6073: (17.02), 6074: (17.01), 6075: (17.05), 6076: (17.02), 6077: (17.02), 6078: (17.02), 6079: (4.00), 6080: (17.17), 6081: (17.01), 6082: (17.04), 6083: (17.01), 6084: (17.02), 6085: (17.01), 6086: (4.00), 6087: (17.01), 6088: (4.00), 6089: (4.00), 6090: (17.01), 6091: (4.00), 6092: (4.00), 6093: (4.00), 6094: (17.03), 6095: (17.05), 6096: (17.19), 6097: (17.01), 6098: (17.09), 6099: (17.03), 6100: (17.01), 6101: (4.00), 6102: (17.01), 6103: (17.01), 6104: (17.03), 6105: (4.00), 6106: (4.00), 6107: (4.00), 6108: (4.00), 6109: (17.01), 6110: (4.00), 6111: (17.01), 6112: (17.01), 6113: (17.02), 6114: (17.02), 6115: (17.01), 6116: (17.05), 6117: (4.00), 6118: (17.01), 6119: (4.00), 6120: (17.01), 6121: (17.02), 6122: (17.01), 6123: (17.02), 6124: (17.01), 6125: (17.01), 6126: (4.00), 6127: (4.00), 6128: (4.00), 6129: (4.00), 6130: (4.00), 6131: (4.00), 6132: (17.01), 6133: (17.01), 6134: (4.00), 6135: (17.02), 6136: (17.07), 6137: (17.01), 6138: (17.01), 6139: (17.04), 6140: (17.02), 6141: (4.00), 6142: (4.00), 6143: (4.00), 6144: (4.00), 6145: (4.00), 6146: (4.00), 6147: (4.00), 6148: (17.01), 6149: (17.22), 6150: (17.01), 6151: (4.00), 6152: (17.01), 6153: (4.00), 6154: (17.01), 6155: (17.01), 6156: (4.00), 6157: (4.00), 6158: (4.00), 6159: (4.00), 6160: (4.00), 6161: (4.00), 6162: (4.00), 6163: (17.02), 6164: (17.01), 6165: (17.04), 6166: (17.01), 6167: (4.00), 6168: (4.00), 6169: (4.00), 6170: (4.00), 6171: (17.03), 6172: (17.39), 6173: (17.01), 6174: (17.03), 6175: (17.01), 6176: (17.01), 6177: (17.02), 6178: (17.02), 6179: (4.00), 6180: (17.06), 6181: (4.00), 6182: (17.06), 6183: (17.01), 6184: (17.02), 6185: (17.28), 6186: (17.01), 6187: (4.00), 6188: (17.01), 6189: (4.00), 6190: (4.00), 6191: (4.00), 6192: (4.00), 6193: (4.00), 6194: (4.00), 6195: (4.00), 6196: (4.00), 6197: (4.00), 6198: (4.00), 6199: (17.01), 6200: (4.00), 6201: (4.00), 6202: (17.01), 6203: (17.13), 6204: (17.02), 6205: (17.05), 6206: (17.02), 6207: (17.01), 6208: (17.02), 6209: (4.00), 6210: (4.00), 6211: (4.00), 6212: (17.03), 6213: (17.10), 6214: (17.02), 6215: (17.06), 6216: (4.00), 6217: (17.02), 6218: (17.01), 6219: (4.00), 6220: (4.00), 6221: (4.00), 6222: (17.01), 6223: (4.00), 6224: (4.00), 6225: (4.00), 6226: (17.01), 6227: (17.03), 6228: (17.02), 6229: (4.00), 6230: (17.01), 6231: (17.07), 6232: (17.03), 6233: (17.01), 6234: (17.04), 6235: (4.00), 6236: (17.01), 6237: (4.00), 6238: (4.00), 6239: (4.00), 6240: (17.89), 6241: (17.02), 6242: (4.00), 6243: (17.05), 6244: (4.00), 6245: (17.02), 6246: (4.00), 6247: (4.00), 6248: (17.01), 6249: (17.01), 6250: (17.04), 6251: (17.01), 6252: (4.00), 6253: (4.00), 6254: (17.06), 6255: (17.01), 6256: (17.01), 6257: (17.01), 6258: (17.01), 6259: (4.00), 6260: (17.01), 6261: (4.00), 6262: (4.00), 6263: (17.19), 6264: (17.02), 6265: (17.03), 6266: (17.05), 6267: (17.01), 6268: (17.08), 6269: (4.00), 6270: (4.00), 6271: (17.01), 6272: (17.06), 6273: (17.01), 6274: (4.00), 6275: (4.00), 6276: (17.15), 6277: (17.03), 6278: (4.00), 6279: (17.01), 6280: (17.02), 6281: (4.00), 6282: (17.01), 6283: (4.00), 6284: (4.00), 6285: (4.00), 6286: (4.00), 6287: (4.00), 6288: (4.00), 6289: (4.00), 6290: (17.01), 6291: (17.01), 6292: (4.00), 6293: (4.00), 6294: (4.00), 6295: (17.01), 6296: (17.01), 6297: (4.00), 6298: (4.00), 6299: (4.00), 6300: (4.00), 6301: (4.00), 6302: (4.00), 6303: (17.01), 6304: (17.52), 6305: (17.06), 6306: (17.02), 6307: (17.04), 6308: (17.01), 6309: (4.00), 6310: (4.00), 6311: (17.02), 6312: (17.02), 6313: (17.01), 6314: (17.01), 6315: (4.00), 6316: (17.01), 6317: (17.01), 6318: (17.02), 6319: (17.01), 6320: (17.01), 6321: (17.01), 6322: (17.06), 6323: (17.05), 6324: (17.03), 6325: (17.18), 6326: (17.02), 6327: (17.01), 6328: (17.01), 6329: (17.01), 6330: (4.00), 6331: (17.18), 6332: (17.01), 6333: (17.01), 6334: (17.01), 6335: (4.00), 6336: (4.00), 6337: (4.00), 6338: (4.00), 6339: (4.00), 6340: (17.01), 6341: (17.02), 6342: (17.01), 6343: (17.02), 6344: (17.01), 6345: (17.13), 6346: (17.03), 6347: (17.01), 6348: (17.01), 6349: (17.02), 6350: (17.03), 6351: (17.04), 6352: (17.03), 6353: (17.01), 6354: (17.32), 6355: (17.03), 6356: (17.01), 6357: (17.02), 6358: (17.01), 6359: (17.02), 6360: (17.01), 6361: (17.02), 6362: (17.01), 6363: (17.03), 6364: (4.00), 6365: (17.01), 6366: (17.01), 6367: (17.07), 6368: (17.02), 6369: (17.03), 6370: (17.01), 6371: (17.02), 6372: (17.01), 6373: (17.01), 6374: (17.01), 6375: (17.01), 6376: (17.02), 6377: (17.01), 6378: (4.00), 6379: (4.00), 6380: (4.00), 6381: (4.00), 6382: (17.01), 6383: (17.01), 6384: (4.00), 6385: (17.01), 6386: (4.00), 6387: (4.00), 6388: (17.04), 6389: (17.01), 6390: (17.03), 6391: (17.02), 6392: (17.01), 6393: (4.00), 6394: (4.00), 6395: (17.06), 6396: (17.02), 6397: (17.05), 6398: (17.01), 6399: (17.03), 6400: (4.00), 6401: (4.00), 6402: (4.00), 6403: (4.00), 6404: (17.03), 6405: (17.01), 6406: (4.00), 6407: (4.00), 6408: (17.01), 6409: (17.01), 6410: (4.00), 6411: (17.01), 6412: (4.00), 6413: (17.04), 6414: (17.02), 6415: (17.01), 6416: (17.01), 6417: (17.08), 6418: (4.00), 6419: (4.00), 6420: (4.00), 6421: (4.00), 6422: (17.04), 6423: (17.01), 6424: (4.00), 6425: (4.00), 6426: (4.00), 6427: (17.01), 6428: (17.02), 6429: (17.06), 6430: (17.07), 6431: (4.00), 6432: (17.01), 6433: (17.01), 6434: (17.02), 6435: (4.00), 6436: (17.03), 6437: (17.04), 6438: (4.00), 6439: (17.02), 6440: (17.03), 6441: (17.12), 6442: (17.05), 6443: (17.01), 6444: (4.00), 6445: (17.02), 6446: (17.02), 6447: (17.01), 6448: (17.02), 6449: (17.02), 6450: (17.01), 6451: (4.00), 6452: (4.00), 6453: (4.00), 6454: (17.04), 6455: (17.01), 6456: (17.05), 6457: (17.01), 6458: (17.03), 6459: (17.26), 6460: (17.02), 6461: (4.00), 6462: (17.01), 6463: (17.02), 6464: (4.00), 6465: (4.00), 6466: (17.01), 6467: (4.00), 6468: (17.01), 6469: (17.03), 6470: (17.01), 6471: (17.01), 6472: (4.00), 6473: (4.00), 6474: (4.00), 6475: (4.00), 6476: (4.00), 6477: (4.00), 6478: (4.00), 6479: (17.01), 6480: (4.00), 6481: (17.01), 6482: (17.01), 6483: (4.00), 6484: (17.01), 6485: (17.01), 6486: (17.11), 6487: (17.01), 6488: (17.01), 6489: (4.00), 6490: (17.02), 6491: (4.00), 6492: (4.00), 6493: (4.00), 6494: (4.00), 6495: (17.01), 6496: (4.00), 6497: (4.00), 6498: (4.00), 6499: (4.00), 6500: (4.00), 6501: (4.00), 6502: (4.00), 6503: (4.00), 6504: (4.00), 6505: (4.00), 6506: (4.00), 6507: (4.00), 6508: (4.00), 6509: (17.02), 6510: (4.00), 6511: (17.01), 6512: (17.01), 6513: (17.01), 6514: (17.06), 6515: (4.00), 6516: (4.00), 6517: (17.01), 6518: (17.01), 6519: (4.00), 6520: (4.00), 6521: (4.00), 6522: (4.00), 6523: (4.00), 6524: (4.00), 6525: (4.00), 6526: (4.00), 6527: (4.00), 6528: (17.01), 6529: (4.00), 6530: (17.01), 6531: (17.01), 6532: (4.00), 6533: (4.00), 6534: (4.00), 6535: (4.00), 6536: (4.00), 6537: (17.09), 6538: (17.01), 6539: (4.00), 6540: (4.00), 6541: (17.03), 6542: (4.00), 6543: (4.00), 6544: (4.00), 6545: (4.00), 6546: (4.00), 6547: (17.01), 6548: (4.00), 6549: (4.00), 6550: (17.04), 6551: (4.00), 6552: (4.00), 6553: (17.01), 6554: (17.01), 6555: (17.01), 6556: (4.00), 6557: (4.00), 6558: (4.00), 6559: (4.00), 6560: (4.00), 6561: (4.00), 6562: (4.00), 6563: (4.00), 6564: (17.03), 6565: (17.01), 6566: (4.00), 6567: (4.00), 6568: (4.00), 6569: (17.01), 6570: (4.00), 6571: (17.02), 6572: (4.00), 6573: (17.02), 6574: (4.00), 6575: (17.01), 6576: (4.00), 6577: (17.01), 6578: (17.07), 6579: (17.03), 6580: (4.00), 6581: (17.03), 6582: (4.00), 6583: (17.01), 6584: (17.01), 6585: (17.01), 6586: (17.02), 6587: (17.03), 6588: (17.01), 6589: (17.01), 6590: (17.01), 6591: (4.00), 6592: (4.00), 6593: (4.00), 6594: (4.00), 6595: (4.00), 6596: (4.00), 6597: (17.01), 6598: (17.16), 6599: (4.00), 6600: (17.05), 6601: (17.01), 6602: (4.00), 6603: (4.00), 6604: (4.00), 6605: (17.26), 6606: (17.02), 6607: (17.02), 6608: (17.01), 6609: (17.02), 6610: (17.01), 6611: (17.01), 6612: (17.02), 6613: (17.02), 6614: (4.00), 6615: (17.01), 6616: (17.01), 6617: (4.00), 6618: (17.01), 6619: (17.07), 6620: (17.04), 6621: (17.02), 6622: (17.08), 6623: (4.00), 6624: (17.02), 6625: (17.01), 6626: (17.02), 6627: (17.01), 6628: (17.18), 6629: (17.04), 6630: (17.03), 6631: (17.01), 6632: (17.02), 6633: (17.02), 6634: (4.00), 6635: (4.00), 6636: (4.00), 6637: (17.01), 6638: (17.01), 6639: (17.05), 6640: (4.00), 6641: (17.18), 6642: (17.05), 6643: (17.01), 6644: (17.01), 6645: (17.01), 6646: (17.02), 6647: (17.01), 6648: (4.00), 6649: (17.01), 6650: (17.01), 6651: (17.01), 6652: (17.01), 6653: (4.00), 6654: (4.00), 6655: (17.02), 6656: (17.01), 6657: (17.01), 6658: (4.00), 6659: (4.00), 6660: (17.01), 6661: (4.00), 6662: (17.05), 6663: (17.01), 6664: (17.04), 6665: (4.00), 6666: (17.01), 6667: (4.00), 6668: (17.01), 6669: (17.20), 6670: (4.00), 6671: (17.01), 6672: (17.01), 6673: (4.00), 6674: (4.00), 6675: (4.00), 6676: (4.00), 6677: (4.00), 6678: (4.00), 6679: (17.01), 6680: (4.00), 6681: (4.00), 6682: (4.00), 6683: (4.00), 6684: (4.00), 6685: (4.00), 6686: (4.00), 6687: (17.03), 6688: (4.00), 6689: (17.02), 6690: (17.02), 6691: (17.01), 6692: (17.01), 6693: (4.00), 6694: (4.00), 6695: (4.00), 6696: (17.23), 6697: (4.00), 6698: (4.00), 6699: (4.00), 6700: (4.00), 6701: (4.00), 6702: (4.00), 6703: (4.00), 6704: (4.00), 6705: (4.00), 6706: (4.00), 6707: (4.00), 6708: (4.00), 6709: (4.00), 6710: (17.15), 6711: (4.00), 6712: (17.01), 6713: (4.00), 6714: (17.02), 6715: (17.01), 6716: (4.00), 6717: (17.01), 6718: (17.07), 6719: (17.01), 6720: (17.01), 6721: (17.01), 6722: (17.01), 6723: (17.01), 6724: (4.00), 6725: (4.00), 6726: (4.00), 6727: (17.06), 6728: (17.01), 6729: (17.01), 6730: (17.02), 6731: (17.58), 6732: (17.03), 6733: (17.01), 6734: (17.03), 6735: (4.00), 6736: (17.06), 6737: (4.00), 6738: (4.00), 6739: (17.01), 6740: (17.03), 6741: (17.02), 6742: (17.01), 6743: (17.03), 6744: (17.05), 6745: (17.02), 6746: (4.00), 6747: (17.19), 6748: (17.01), 6749: (4.00), 6750: (17.07), 6751: (17.01), 6752: (4.00), 6753: (17.04), 6754: (17.03), 6755: (17.01), 6756: (17.13), 6757: (17.04), 6758: (17.01), 6759: (4.00), 6760: (4.00), 6761: (17.05), 6762: (17.04), 6763: (17.21), 6764: (17.01), 6765: (17.30), 6766: (17.02), 6767: (4.00), 6768: (17.05), 6769: (17.01), 6770: (4.00), 6771: (17.02), 6772: (4.00), 6773: (4.00), 6774: (4.00), 6775: (4.00), 6776: (4.00), 6777: (17.06), 6778: (17.02), 6779: (17.01), 6780: (4.00), 6781: (17.02), 6782: (4.00), 6783: (4.00), 6784: (4.00), 6785: (17.01), 6786: (17.02), 6787: (17.41), 6788: (17.01), 6789: (17.02), 6790: (17.01), 6791: (17.01), 6792: (17.01), 6793: (4.00), 6794: (4.00), 6795: (4.00), 6796: (4.00), 6797: (4.00), 6798: (17.01), 6799: (17.01), 6800: (17.02), 6801: (17.08), 6802: (4.00), 6803: (4.00), 6804: (4.00), 6805: (4.00), 6806: (17.01), 6807: (4.00), 6808: (4.00), 6809: (17.02), 6810: (17.07), 6811: (17.01), 6812: (4.00), 6813: (17.01), 6814: (17.01), 6815: (4.00), 6816: (4.00), 6817: (4.00), 6818: (17.02), 6819: (17.04), 6820: (17.02), 6821: (17.06), 6822: (17.05), 6823: (17.31), 6824: (17.02), 6825: (17.06), 6826: (17.04), 6827: (17.05), 6828: (4.00), 6829: (4.00), 6830: (17.05), 6831: (17.01), 6832: (17.02), 6833: (17.01), 6834: (17.08), 6835: (17.01), 6836: (17.04), 6837: (4.00), 6838: (17.01), 6839: (17.13), 6840: (4.00), 6841: (17.06), 6842: (17.05), 6843: (4.00), 6844: (4.00), 6845: (17.02), 6846: (17.02), 6847: (4.00), 6848: (4.00), 6849: (4.00), 6850: (17.02), 6851: (17.02), 6852: (4.00), 6853: (4.00), 6854: (17.02), 6855: (17.01), 6856: (17.01), 6857: (4.00), 6858: (4.00), 6859: (17.45), 6860: (17.03), 6861: (17.02), 6862: (17.12), 6863: (17.05), 6864: (4.00), 6865: (4.00), 6866: (4.00), 6867: (4.00), 6868: (17.06), 6869: (17.04), 6870: (17.01), 6871: (4.00), 6872: (17.06), 6873: (17.02), 6874: (4.00), 6875: (4.00), 6876: (4.00), 6877: (17.01), 6878: (17.13), 6879: (17.02), 6880: (17.01), 6881: (17.01), 6882: (17.01), 6883: (17.01), 6884: (17.01), 6885: (17.02), 6886: (4.00), 6887: (4.00), 6888: (17.06), 6889: (17.01), 6890: (17.01), 6891: (17.02), 6892: (17.11), 6893: (4.00), 6894: (17.01), 6895: (4.00), 6896: (17.01), 6897: (17.01), 6898: (17.01), 6899: (17.01), 6900: (17.07), 6901: (17.06), 6902: (4.00), 6903: (4.00), 6904: (17.01), 6905: (4.00), 6906: (4.00), 6907: (4.00), 6908: (4.00), 6909: (17.01), 6910: (4.00), 6911: (4.00), 6912: (4.00), 6913: (4.00), 6914: (17.02), 6915: (17.02), 6916: (17.02), 6917: (4.00), 6918: (4.00), 6919: (4.00), 6920: (4.00), 6921: (4.00), 6922: (4.00), 6923: (4.00), 6924: (17.01), 6925: (17.01), 6926: (4.00), 6927: (17.08), 6928: (17.01), 6929: (17.21), 6930: (17.02), 6931: (4.00), 6932: (17.04), 6933: (17.02), 6934: (4.00), 6935: (17.04), 6936: (17.01), 6937: (4.00), 6938: (17.01), 6939: (17.01), 6940: (17.01), 6941: (17.03), 6942: (17.01), 6943: (4.00), 6944: (17.03), 6945: (17.02), 6946: (17.02), 6947: (17.08), 6948: (17.06), 6949: (17.03), 6950: (17.02), 6951: (17.16), 6952: (17.03), 6953: (17.03), 6954: (17.05), 6955: (4.00), 6956: (17.01), 6957: (17.01), 6958: (4.00), 6959: (4.00), 6960: (17.04), 6961: (17.01), 6962: (4.00), 6963: (4.00), 6964: (4.00), 6965: (4.00), 6966: (4.00), 6967: (4.00), 6968: (4.00), 6969: (17.01), 6970: (4.00), 6971: (4.00), 6972: (4.00), 6973: (4.00), 6974: (17.06), 6975: (4.00), 6976: (17.11), 6977: (4.00), 6978: (17.01), 6979: (4.00), 6980: (17.01), 6981: (4.00), 6982: (4.00), 6983: (4.00), 6984: (4.00), 6985: (4.00), 6986: (4.00), 6987: (4.00), 6988: (17.05), 6989: (4.00), 6990: (4.00), 6991: (4.00), 6992: (4.00), 6993: (4.00), 6994: (4.00), 6995: (4.00), 6996: (4.00), 6997: (4.00), 6998: (4.00), 6999: (4.00), 7000: (4.00), 7001: (4.00), 7002: (4.00), 7003: (4.00), 7004: (4.00), 7005: (4.00), 7006: (17.15), 7007: (17.06), 7008: (17.03), 7009: (4.00), 7010: (4.00), 7011: (4.00), 7012: (4.00), 7013: (4.00), 7014: (4.00), 7015: (4.00), 7016: (4.00), 7017: (4.00), 7018: (4.00), 7019: (4.00), 7020: (4.00), 7021: (4.00), 7022: (4.00), 7023: (4.00), 7024: (4.00), 7025: (4.00), 7026: (4.00), 7027: (17.02), 7028: (4.00), 7029: (4.00), 7030: (4.00), 7031: (4.00), 7032: (4.00), 7033: (4.00), 7034: (4.00), 7035: (4.00), 7036: (4.00), 7037: (4.00), 7038: (4.00), 7039: (4.00), 7040: (4.00), 7041: (4.00), 7042: (4.00), 7043: (4.00), 7044: (4.00), 7045: (17.01), 7046: (17.12), 7047: (4.00), 7048: (4.00), 7049: (4.00), 7050: (4.00), 7051: (4.00), 7052: (4.00), 7053: (4.00), 7054: (4.00), 7055: (4.00), 7056: (4.00), 7057: (4.00), 7058: (4.00), 7059: (4.00), 7060: (4.00), 7061: (4.00), 7062: (4.00), 7063: (4.00), 7064: (4.00), 7065: (17.05), 7066: (17.01), 7067: (4.00), 7068: (4.00), 7069: (4.00), 7070: (4.00), 7071: (4.00), 7072: (4.00), 7073: (4.00), 7074: (4.00), 7075: (4.00), 7076: (4.00), 7077: (4.00), 7078: (4.00), 7079: (4.00), 7080: (4.00), 7081: (4.00), 7082: (4.00), 7083: (4.00), 7084: (17.08), 7085: (4.00), 7086: (4.00), 7087: (4.00), 7088: (4.00), 7089: (4.00), 7090: (4.00), 7091: (4.00), 7092: (4.00), 7093: (17.04), 7094: (17.02), 7095: (4.00), 7096: (4.00), 7097: (4.00), 7098: (4.00), 7099: (4.00), 7100: (4.00), 7101: (4.00), 7102: (17.01), 7103: (17.13), 7104: (4.00), 7105: (4.00), 7106: (4.00), 7107: (4.00), 7108: (4.00), 7109: (4.00), 7110: (4.00), 7111: (4.00), 7112: (4.00), 7113: (4.00), 7114: (4.00), 7115: (4.00), 7116: (4.00), 7117: (4.00), 7118: (4.00), 7119: (4.00), 7120: (4.00), 7121: (4.00), 7122: (17.01), 7123: (17.01), 7124: (4.00), 7125: (4.00), 7126: (4.00), 7127: (4.00), 7128: (4.00), 7129: (4.00), 7130: (4.00), 7131: (4.00), 7132: (4.00), 7133: (4.00), 7134: (4.00), 7135: (4.00), 7136: (4.00), 7137: (4.00), 7138: (4.00), 7139: (4.00), 7140: (4.00), 7141: (17.07), 7142: (17.01), 7143: (4.00), 7144: (4.00), 7145: (4.00), 7146: (4.00), 7147: (4.00), 7148: (4.00), 7149: (4.00), 7150: (4.00), 7151: (4.00), 7152: (4.00), 7153: (4.00), 7154: (4.00), 7155: (4.00), 7156: (4.00), 7157: (4.00), 7158: (4.00), 7159: (4.00), 7160: (17.07), 7161: (17.02), 7162: (4.00), 7163: (4.00), 7164: (4.00), 7165: (4.00), 7166: (4.00), 7167: (4.00), 7168: (4.00), 7169: (4.00), 7170: (17.01), 7171: (4.00), 7172: (4.00), 7173: (4.00), 7174: (17.04), 7175: (17.01), 7176: (4.00), 7177: (4.00), 7178: (17.01), 7179: (17.08), 7180: (4.00), 7181: (4.00), 7182: (4.00), 7183: (4.00), 7184: (4.00), 7185: (4.00), 7186: (4.00), 7187: (4.00), 7188: (17.12), 7189: (17.01), 7190: (4.00), 7191: (4.00), 7192: (4.00), 7193: (4.00), 7194: (4.00), 7195: (4.00), 7196: (4.00), 7197: (17.08), 7198: (4.00), 7199: (4.00), 7200: (4.00), 7201: (4.00), 7202: (4.00), 7203: (4.00), 7204: (4.00), 7205: (4.00), 7206: (4.00), 7207: (4.00), 7208: (4.00), 7209: (4.00), 7210: (4.00), 7211: (4.00), 7212: (4.00), 7213: (4.00), 7214: (4.00), 7215: (4.00), 7216: (17.02), 7217: (17.01), 7218: (4.00), 7219: (4.00), 7220: (4.00), 7221: (4.00), 7222: (4.00), 7223: (4.00), 7224: (4.00), 7225: (4.00), 7226: (4.00), 7227: (4.00), 7228: (4.00), 7229: (4.00), 7230: (4.00), 7231: (4.00), 7232: (4.00), 7233: (4.00), 7234: (4.00), 7235: (17.32), 7236: (4.00), 7237: (4.00), 7238: (4.00), 7239: (4.00), 7240: (4.00), 7241: (4.00), 7242: (4.00), 7243: (4.00), 7244: (4.00), 7245: (4.00), 7246: (4.00), 7247: (4.00), 7248: (4.00), 7249: (17.31), 7250: (4.00), 7251: (4.00), 7252: (4.00), 7253: (17.01), 7254: (17.03), 7255: (17.01), 7256: (4.00), 7257: (4.00), 7258: (4.00), 7259: (4.00), 7260: (4.00), 7261: (4.00), 7262: (4.00), 7263: (17.01), 7264: (4.00), 7265: (4.00), 7266: (17.02), 7267: (17.01), 7268: (17.01), 7269: (4.00), 7270: (4.00), 7271: (17.05), 7272: (17.01), 7273: (4.00), 7274: (4.00), 7275: (4.00), 7276: (4.00), 7277: (4.00), 7278: (4.00), 7279: (4.00), 7280: (17.03), 7281: (17.04), 7282: (17.01), 7283: (4.00), 7284: (4.00), 7285: (4.00), 7286: (4.00), 7287: (4.00), 7288: (4.00), 7289: (4.00), 7290: (4.00), 7291: (4.00), 7292: (4.00), 7293: (4.00), 7294: (17.01), 7295: (17.05), 7296: (4.00), 7297: (4.00), 7298: (4.00), 7299: (17.03), 7300: (4.00), 7301: (4.00), 7302: (4.00), 7303: (4.00), 7304: (4.00), 7305: (4.00), 7306: (4.00), 7307: (4.00), 7308: (17.07), 7309: (17.01), 7310: (4.00), 7311: (4.00), 7312: (4.00), 7313: (4.00), 7314: (4.00), 7315: (4.00), 7316: (4.00), 7317: (17.02), 7318: (17.01), 7319: (4.00), 7320: (4.00), 7321: (4.00), 7322: (17.01), 7323: (4.00), 7324: (4.00), 7325: (4.00), 7326: (17.03), 7327: (17.06), 7328: (17.01), 7329: (4.00), 7330: (4.00), 7331: (4.00), 7332: (4.00), 7333: (4.00), 7334: (4.00), 7335: (4.00), 7336: (4.00), 7337: (4.00), 7338: (4.00), 7339: (4.00), 7340: (4.00), 7341: (4.00), 7342: (4.00), 7343: (4.00), 7344: (4.00), 7345: (4.00), 7346: (17.07), 7347: (17.01), 7348: (4.00), 7349: (4.00), 7350: (4.00), 7351: (4.00), 7352: (4.00), 7353: (4.00), 7354: (4.00), 7355: (4.00), 7356: (4.00), 7357: (4.00), 7358: (4.00), 7359: (4.00), 7360: (4.00), 7361: (4.00), 7362: (4.00), 7363: (4.00), 7364: (4.00), 7365: (17.21), 7366: (4.00), 7367: (4.00), 7368: (4.00), 7369: (4.00), 7370: (4.00), 7371: (4.00), 7372: (4.00), 7373: (4.00), 7374: (4.00), 7375: (4.00), 7376: (4.00), 7377: (4.00), 7378: (4.00), 7379: (4.00), 7380: (4.00), 7381: (4.00), 7382: (4.00), 7383: (4.00), 7384: (17.05), 7385: (17.01), 7386: (4.00), 7387: (4.00), 7388: (4.00), 7389: (4.00), 7390: (4.00), 7391: (4.00), 7392: (4.00), 7393: (17.01), 7394: (4.00), 7395: (4.00), 7396: (17.01), 7397: (17.02), 7398: (4.00), 7399: (4.00), 7400: (4.00), 7401: (17.10), 7402: (17.01), 7403: (4.00), 7404: (4.00), 7405: (4.00), 7406: (4.00), 7407: (4.00), 7408: (4.00), 7409: (4.00), 7410: (4.00), 7411: (17.17), 7412: (4.00), 7413: (4.00), 7414: (4.00), 7415: (4.00), 7416: (4.00), 7417: (4.00), 7418: (4.00), 7419: (4.00), 7420: (17.22), 7421: (4.00), 7422: (4.00), 7423: (4.00), 7424: (4.00), 7425: (4.00), 7426: (4.00), 7427: (4.00), 7428: (4.00), 7429: (4.00), 7430: (4.00), 7431: (4.00), 7432: (4.00), 7433: (4.00), 7434: (4.00), 7435: (4.00), 7436: (4.00), 7437: (4.00), 7438: (4.00), 7439: (17.04), 7440: (4.00), 7441: (4.00), 7442: (4.00), 7443: (4.00), 7444: (4.00), 7445: (4.00), 7446: (4.00), 7447: (4.00), 7448: (4.00), 7449: (4.00), 7450: (4.00), 7451: (4.00), 7452: (4.00), 7453: (4.00), 7454: (4.00), 7455: (4.00), 7456: (4.00), 7457: (17.14), 7458: (17.01), 7459: (4.00), 7460: (4.00), 7461: (4.00), 7462: (4.00), 7463: (4.00), 7464: (4.00), 7465: (4.00), 7466: (4.00), 7467: (4.00), 7468: (4.00), 7469: (4.00), 7470: (4.00), 7471: (4.00), 7472: (4.00), 7473: (17.09), 7474: (4.00), 7475: (4.00), 7476: (17.06), 7477: (17.01), 7478: (4.00), 7479: (4.00), 7480: (17.08), 7481: (4.00), 7482: (4.00), 7483: (4.00), 7484: (4.00), 7485: (4.00), 7486: (4.00), 7487: (4.00), 7488: (4.00), 7489: (4.00), 7490: (4.00), 7491: (4.00), 7492: (4.00), 7493: (4.00), 7494: (17.20), 7495: (17.02), 7496: (4.00), 7497: (4.00), 7498: (4.00), 7499: (4.00), 7500: (4.00), 7501: (4.00), 7502: (4.00), 7503: (17.11), 7504: (4.00), 7505: (4.00), 7506: (4.00), 7507: (4.00), 7508: (4.00), 7509: (4.00), 7510: (4.00), 7511: (4.00), 7512: (17.01), 7513: (17.14), 7514: (17.01), 7515: (4.00), 7516: (4.00), 7517: (4.00), 7518: (4.00), 7519: (4.00), 7520: (4.00), 7521: (4.00), 7522: (4.00), 7523: (4.00), 7524: (4.00), 7525: (4.00), 7526: (4.00), 7527: (4.00), 7528: (4.00), 7529: (4.00), 7530: (4.00), 7531: (4.00), 7532: (17.02), 7533: (4.00), 7534: (4.00), 7535: (4.00), 7536: (4.00), 7537: (4.00), 7538: (4.00), 7539: (4.00), 7540: (4.00), 7541: (4.00), 7542: (4.00), 7543: (4.00), 7544: (4.00), 7545: (4.00), 7546: (4.00), 7547: (4.00), 7548: (4.00), 7549: (4.00), 7550: (17.23), 7551: (4.00), 7552: (4.00), 7553: (4.00), 7554: (4.00), 7555: (4.00), 7556: (4.00), 7557: (4.00), 7558: (4.00), 7559: (4.00), 7560: (4.00), 7561: (4.00), 7562: (4.00), 7563: (4.00), 7564: (17.11), 7565: (17.01), 7566: (4.00), 7567: (4.00), 7568: (4.00), 7569: (17.03), 7570: (17.03), 7571: (4.00), 7572: (4.00), 7573: (17.12), 7574: (4.00), 7575: (4.00), 7576: (4.00), 7577: (4.00), 7578: (17.09), 7579: (17.14), 7580: (17.01), 7581: (17.14), 7582: (17.11), 7583: (17.02), 7584: (4.00), 7585: (17.02), 7586: (17.01), 7587: (17.20), 7588: (17.01), 7589: (4.00), 7590: (4.00), 7591: (4.00), 7592: (4.00), 7593: (4.00), 7594: (4.00), 7595: (4.00), 7596: (17.15), 7597: (17.01), 7598: (17.01), 7599: (4.00), 7600: (4.00), 7601: (4.00), 7602: (4.00), 7603: (4.00), 7604: (4.00), 7605: (4.00), 7606: (4.00), 7607: (4.00), 7608: (4.00), 7609: (4.00), 7610: (17.10), 7611: (17.01), 7612: (4.00), 7613: (4.00), 7614: (4.00), 7615: (17.02), 7616: (4.00), 7617: (4.00), 7618: (4.00), 7619: (4.00), 7620: (4.00), 7621: (4.00), 7622: (4.00), 7623: (4.00), 7624: (17.11), 7625: (4.00), 7626: (4.00), 7627: (4.00), 7628: (4.00), 7629: (4.00), 7630: (4.00), 7631: (4.00), 7632: (4.00), 7633: (4.00), 7634: (4.00), 7635: (4.00), 7636: (4.00), 7637: (17.01), 7638: (17.02), 7639: (4.00), 7640: (4.00), 7641: (4.00), 7642: (17.01), 7643: (17.13), 7644: (4.00), 7645: (4.00), 7646: (4.00), 7647: (4.00), 7648: (4.00), 7649: (4.00), 7650: (4.00), 7651: (4.00), 7652: (17.01), 7653: (4.00), 7654: (4.00), 7655: (4.00), 7656: (17.06), 7657: (17.02), 7658: (4.00), 7659: (4.00), 7660: (4.00), 7661: (4.00), 7662: (4.00), 7663: (4.00), 7664: (4.00), 7665: (17.02), 7666: (4.00), 7667: (4.00), 7668: (4.00), 7669: (4.00), 7670: (4.00), 7671: (4.00), 7672: (4.00), 7673: (4.00), 7674: (17.17), 7675: (4.00), 7676: (4.00), 7677: (4.00), 7678: (17.02), 7679: (4.00), 7680: (4.00), 7681: (4.00), 7682: (4.00), 7683: (4.00), 7684: (4.00), 7685: (4.00), 7686: (4.00), 7687: (17.02), 7688: (17.02), 7689: (4.00), 7690: (4.00), 7691: (4.00), 7692: (4.00), 7693: (4.00), 7694: (4.00), 7695: (4.00), 7696: (17.01), 7697: (17.02), 7698: (4.00), 7699: (4.00), 7700: (17.01), 7701: (17.03), 7702: (4.00), 7703: (4.00), 7704: (4.00), 7705: (17.03), 7706: (17.01), 7707: (4.00), 7708: (4.00), 7709: (4.00), 7710: (4.00), 7711: (4.00), 7712: (4.00), 7713: (4.00), 7714: (17.03), 7715: (17.01), 7716: (4.00), 7717: (4.00), 7718: (4.00), 7719: (4.00), 7720: (4.00), 7721: (4.00), 7722: (4.00), 7723: (17.03), 7724: (17.01), 7725: (4.00), 7726: (17.01), 7727: (17.01), 7728: (17.01), 7729: (17.01), 7730: (17.03), 7731: (17.01), 7732: (17.12), 7733: (17.01), 7734: (4.00), 7735: (4.00), 7736: (4.00), 7737: (4.00), 7738: (4.00), 7739: (4.00), 7740: (4.00), 7741: (17.01), 7742: (4.00), 7743: (4.00), 7744: (4.00), 7745: (17.08), 7746: (17.01), 7747: (4.00), 7748: (4.00), 7749: (4.00), 7750: (4.00), 7751: (4.00), 7752: (4.00), 7753: (4.00), 7754: (4.00), 7755: (4.00), 7756: (4.00), 7757: (4.00), 7758: (4.00), 7759: (4.00), 7760: (4.00), 7761: (4.00), 7762: (4.00), 7763: (17.10), 7764: (4.00), 7765: (4.00), 7766: (4.00), 7767: (17.01), 7768: (4.00), 7769: (4.00), 7770: (4.00), 7771: (4.00), 7772: (4.00), 7773: (4.00), 7774: (4.00), 7775: (4.00), 7776: (17.02), 7777: (17.01), 7778: (4.00), 7779: (4.00), 7780: (4.00), 7781: (17.01), 7782: (4.00), 7783: (4.00), 7784: (4.00), 7785: (4.00), 7786: (17.01), 7787: (4.00), 7788: (4.00), 7789: (4.00), 7790: (17.01), 7791: (4.00), 7792: (4.00), 7793: (4.00), 7794: (17.01), 7795: (4.00), 7796: (4.00), 7797: (4.00), 7798: (4.00), 7799: (4.00), 7800: (4.00), 7801: (4.00), 7802: (4.00), 7803: (17.12), 7804: (4.00), 7805: (4.00), 7806: (4.00), 7807: (17.01), 7808: (4.00), 7809: (4.00), 7810: (4.00), 7811: (4.00), 7812: (17.28), 7813: (17.06), 7814: (17.10), 7815: (4.00), 7816: (17.09), 7817: (17.01), 7818: (17.07), 7819: (17.02), 7820: (17.13), 7821: (17.01), 7822: (4.00), 7823: (4.00), 7824: (4.00), 7825: (4.00), 7826: (4.00), 7827: (4.00), 7828: (4.00), 7829: (17.01), 7830: (4.00), 7831: (4.00), 7832: (4.00), 7833: (4.00), 7834: (17.13), 7835: (4.00), 7836: (4.00), 7837: (4.00), 7838: (4.00), 7839: (4.00), 7840: (4.00), 7841: (4.00), 7842: (4.00), 7843: (4.00), 7844: (4.00), 7845: (4.00), 7846: (4.00), 7847: (4.00), 7848: (4.00), 7849: (4.00), 7850: (4.00), 7851: (17.07), 7852: (17.03), 7853: (4.00), 7854: (4.00), 7855: (4.00), 7856: (17.01), 7857: (4.00), 7858: (4.00), 7859: (4.00), 7860: (4.00), 7861: (4.00), 7862: (4.00), 7863: (4.00), 7864: (4.00), 7865: (17.13), 7866: (4.00), 7867: (4.00), 7868: (4.00), 7869: (17.02), 7870: (17.01), 7871: (4.00), 7872: (17.01), 7873: (4.00), 7874: (17.12), 7875: (4.00), 7876: (4.00), 7877: (4.00), 7878: (17.02), 7879: (4.00), 7880: (4.00), 7881: (4.00), 7882: (17.01), 7883: (17.01), 7884: (17.01), 7885: (4.00), 7886: (4.00), 7887: (4.00), 7888: (4.00), 7889: (4.00), 7890: (4.00), 7891: (17.06), 7892: (17.01), 7893: (4.00), 7894: (4.00), 7895: (4.00), 7896: (4.00), 7897: (4.00), 7898: (4.00), 7899: (4.00), 7900: (17.05), 7901: (17.01), 7902: (17.01), 7903: (17.08), 7904: (17.08), 7905: (17.01), 7906: (4.00), 7907: (4.00), 7908: (17.03), 7909: (17.19), 7910: (17.01), 7911: (17.01), 7912: (4.00), 7913: (4.00), 7914: (4.00), 7915: (4.00), 7916: (4.00), 7917: (17.01), 7918: (4.00), 7919: (4.00), 7920: (4.00), 7921: (4.00), 7922: (17.07), 7923: (17.01), 7924: (4.00), 7925: (4.00), 7926: (4.00), 7927: (4.00), 7928: (4.00), 7929: (4.00), 7930: (4.00), 7931: (4.00), 7932: (4.00), 7933: (4.00), 7934: (4.00), 7935: (4.00), 7936: (4.00), 7937: (4.00), 7938: (4.00), 7939: (17.08), 7940: (17.01), 7941: (4.00), 7942: (4.00), 7943: (4.00), 7944: (17.02), 7945: (4.00), 7946: (4.00), 7947: (4.00), 7948: (4.00), 7949: (4.00), 7950: (4.00), 7951: (4.00), 7952: (17.01), 7953: (17.01), 7954: (4.00), 7955: (4.00), 7956: (4.00), 7957: (17.03), 7958: (4.00), 7959: (4.00), 7960: (4.00), 7961: (4.00), 7962: (17.11), 7963: (4.00), 7964: (4.00), 7965: (4.00), 7966: (17.01), 7967: (4.00), 7968: (4.00), 7969: (4.00), 7970: (17.01), 7971: (4.00), 7972: (4.00), 7973: (4.00), 7974: (4.00), 7975: (4.00), 7976: (4.00), 7977: (4.00), 7978: (4.00), 7979: (17.24), 7980: (4.00), 7981: (4.00), 7982: (4.00), 7983: (4.00), 7984: (4.00), 7985: (4.00), 7986: (4.00), 7987: (4.00), 7988: (4.00), 7989: (4.00), 7990: (4.00), 7991: (17.03), 7992: (17.04), 7993: (17.01), 7994: (4.00), 7995: (4.00), 7996: (17.01), 7997: (4.00), 7998: (4.00), 7999: (4.00), 8000: (4.00), 8001: (4.00), 8002: (4.00), 8003: (4.00), 8004: (4.00), 8005: (17.11), 8006: (4.00), 8007: (4.00), 8008: (4.00), 8009: (17.02), 8010: (4.00), 8011: (4.00), 8012: (4.00), 8013: (4.00), 8014: (4.00), 8015: (4.00), 8016: (4.00), 8017: (17.01), 8018: (17.07), 8019: (4.00), 8020: (4.00), 8021: (4.00), 8022: (17.03), 8023: (4.00), 8024: (4.00), 8025: (4.00), 8026: (4.00), 8027: (4.00), 8028: (4.00), 8029: (4.00), 8030: (4.00), 8031: (17.02), 8032: (17.04), 8033: (4.00), 8034: (4.00), 8035: (4.00), 8036: (4.00), 8037: (4.00), 8038: (4.00), 8039: (4.00), 8040: (4.00), 8041: (4.00), 8042: (4.00), 8043: (4.00), 8044: (4.00), 8045: (17.20), 8046: (4.00), 8047: (4.00), 8048: (4.00), 8049: (4.00), 8050: (4.00), 8051: (4.00), 8052: (4.00), 8053: (4.00), 8054: (4.00), 8055: (4.00), 8056: (4.00), 8057: (4.00), 8058: (4.00), 8059: (17.26), 8060: (4.00), 8061: (4.00), 8062: (4.00), 8063: (4.00), 8064: (4.00), 8065: (4.00), 8066: (4.00), 8067: (4.00), 8068: (4.00), 8069: (4.00), 8070: (4.00), 8071: (4.00), 8072: (17.16), 8073: (4.00), 8074: (4.00), 8075: (4.00), 8076: (17.01), 8077: (17.02), 8078: (4.00), 8079: (4.00), 8080: (4.00), 8081: (4.00), 8082: (4.00), 8083: (4.00), 8084: (4.00), 8085: (4.00), 8086: (17.23), 8087: (4.00), 8088: (17.01), 8089: (4.00), 8090: (4.00), 8091: (4.00), 8092: (4.00), 8093: (4.00), 8094: (4.00), 8095: (4.00), 8096: (4.00), 8097: (4.00), 8098: (4.00), 8099: (4.00), 8100: (17.01), 8101: (4.00), 8102: (4.00), 8103: (4.00), 8104: (4.00), 8105: (4.00), 8106: (4.00), 8107: (4.00), 8108: (4.00), 8109: (17.10), 8110: (17.01), 8111: (4.00), 8112: (4.00), 8113: (4.00), 8114: (4.00), 8115: (4.00), 8116: (4.00), 8117: (4.00), 8118: (4.00), 8119: (4.00), 8120: (4.00), 8121: (4.00), 8122: (4.00), 8123: (17.18), 8124: (4.00), 8125: (4.00), 8126: (4.00), 8127: (4.00), 8128: (4.00), 8129: (4.00), 8130: (4.00), 8131: (4.00), 8132: (4.00), 8133: (4.00), 8134: (4.00), 8135: (4.00), 8136: (17.37), 8137: (17.01), 8138: (4.00), 8139: (4.00), 8140: (17.01), 8141: (17.07), 8142: (17.01), 8143: (4.00), 8144: (4.00), 8145: (4.00), 8146: (4.00), 8147: (4.00), 8148: (4.00), 8149: (4.00), 8150: (17.29), 8151: (4.00), 8152: (4.00), 8153: (4.00), 8154: (4.00), 8155: (4.00), 8156: (4.00), 8157: (4.00), 8158: (4.00), 8159: (4.00), 8160: (4.00), 8161: (4.00), 8162: (4.00), 8163: (17.18), 8164: (17.01), 8165: (4.00), 8166: (4.00), 8167: (17.01), 8168: (17.02), 8169: (4.00), 8170: (4.00), 8171: (4.00), 8172: (4.00), 8173: (4.00), 8174: (4.00), 8175: (4.00), 8176: (17.01), 8177: (17.14), 8178: (17.01), 8179: (4.00), 8180: (4.00), 8181: (4.00), 8182: (4.00), 8183: (4.00), 8184: (4.00), 8185: (4.00), 8186: (4.00), 8187: (4.00), 8188: (4.00), 8189: (4.00), 8190: (4.00), 8191: (4.00), 8192: (17.01), 8193: (4.00), 8194: (4.00), 8195: (4.00), 8196: (4.00), 8197: (4.00), 8198: (4.00), 8199: (4.00), 8200: (17.23), 8201: (17.01), 8202: (4.00), 8203: (4.00), 8204: (4.00), 8205: (17.01), 8206: (4.00), 8207: (4.00), 8208: (4.00), 8209: (4.00), 8210: (4.00), 8211: (4.00), 8212: (4.00), 8213: (4.00), 8214: (17.20), 8215: (4.00), 8216: (4.00), 8217: (4.00), 8218: (4.00), 8219: (4.00), 8220: (4.00), 8221: (4.00), 8222: (4.00), 8223: (4.00), 8224: (4.00), 8225: (4.00), 8226: (4.00), 8227: (17.03), 8228: (17.03), 8229: (4.00), 8230: (4.00), 8231: (4.00), 8232: (17.02), 8233: (17.01), 8234: (4.00), 8235: (4.00), 8236: (4.00), 8237: (4.00), 8238: (4.00), 8239: (4.00), 8240: (4.00), 8241: (17.10), 8242: (17.01), 8243: (4.00), 8244: (4.00), 8245: (4.00), 8246: (4.00), 8247: (4.00), 8248: (4.00), 8249: (4.00), 8250: (4.00), 8251: (4.00), 8252: (4.00), 8253: (4.00), 8254: (4.00), 8255: (17.14), 8256: (4.00), 8257: (4.00), 8258: (4.00), 8259: (17.01), 8260: (17.01), 8261: (4.00), 8262: (4.00), 8263: (4.00), 8264: (4.00), 8265: (4.00), 8266: (4.00), 8267: (4.00), 8268: (17.07), 8269: (17.05), 8270: (4.00), 8271: (4.00), 8272: (4.00), 8273: (4.00), 8274: (4.00), 8275: (4.00), 8276: (4.00), 8277: (4.00), 8278: (4.00), 8279: (4.00), 8280: (4.00), 8281: (4.00), 8282: (4.00), 8283: (4.00), 8284: (4.00), 8285: (4.00), 8286: (4.00), 8287: (4.00), 8288: (4.00), 8289: (4.00), 8290: (4.00), 8291: (4.00), 8292: (17.08), 8293: (4.00), 8294: (4.00), 8295: (4.00), 8296: (4.00), 8297: (4.00), 8298: (4.00), 8299: (4.00), 8300: (4.00), 8301: (4.00), 8302: (4.00), 8303: (4.00), 8304: (4.00), 8305: (4.00), 8306: (17.26), 8307: (4.00), 8308: (4.00), 8309: (4.00), 8310: (4.00), 8311: (4.00), 8312: (4.00), 8313: (4.00), 8314: (4.00), 8315: (4.00), 8316: (4.00), 8317: (4.00), 8318: (4.00), 8319: (17.01), 8320: (17.02), 8321: (4.00), 8322: (4.00), 8323: (4.00), 8324: (17.01), 8325: (4.00), 8326: (4.00), 8327: (4.00), 8328: (4.00), 8329: (4.00), 8330: (4.00), 8331: (4.00), 8332: (4.00), 8333: (17.19), 8334: (17.01), 8335: (4.00), 8336: (4.00), 8337: (4.00), 8338: (4.00), 8339: (4.00), 8340: (4.00), 8341: (4.00), 8342: (4.00), 8343: (4.00), 8344: (4.00), 8345: (4.00), 8346: (4.00), 8347: (17.23), 8348: (4.00), 8349: (4.00), 8350: (4.00), 8351: (4.00), 8352: (17.01), 8353: (4.00), 8354: (4.00), 8355: (4.00), 8356: (4.00), 8357: (4.00), 8358: (4.00), 8359: (4.00), 8360: (4.00), 8361: (17.07), 8362: (4.00), 8363: (4.00), 8364: (4.00), 8365: (4.00), 8366: (4.00), 8367: (4.00), 8368: (4.00), 8369: (4.00), 8370: (4.00), 8371: (4.00), 8372: (4.00), 8373: (4.00), 8374: (4.00), 8375: (4.00), 8376: (4.00), 8377: (4.00), 8378: (4.00), 8379: (4.00), 8380: (17.02), 8381: (17.01), 8382: (4.00), 8383: (4.00), 8384: (4.00), 8385: (4.00), 8386: (4.00), 8387: (4.00), 8388: (4.00), 8389: (4.00), 8390: (4.00), 8391: (4.00), 8392: (4.00), 8393: (4.00), 8394: (4.00), 8395: (4.00), 8396: (4.00), 8397: (4.00), 8398: (4.00), 8399: (17.20), 8400: (4.00), 8401: (4.00), 8402: (4.00), 8403: (4.00), 8404: (4.00), 8405: (4.00), 8406: (4.00), 8407: (4.00), 8408: (4.00), 8409: (4.00), 8410: (4.00), 8411: (4.00), 8412: (4.00), 8413: (4.00), 8414: (4.00), 8415: (4.00), 8416: (4.00), 8417: (17.04), 8418: (17.03), 8419: (17.01), 8420: (4.00), 8421: (17.07), 8422: (17.05), 8423: (4.00), 8424: (4.00), 8425: (4.00), 8426: (4.00), 8427: (4.00), 8428: (4.00), 8429: (4.00), 8430: (4.00), 8431: (4.00), 8432: (4.00), 8433: (4.00), 8434: (4.00), 8435: (17.15), 8436: (17.02), 8437: (4.00), 8438: (4.00), 8439: (4.00), 8440: (4.00), 8441: (4.00), 8442: (4.00), 8443: (4.00), 8444: (17.18), 8445: (4.00), 8446: (4.00), 8447: (4.00), 8448: (4.00), 8449: (4.00), 8450: (4.00), 8451: (4.00), 8452: (4.00), 8453: (17.04), 8454: (17.05), 8455: (4.00), 8456: (4.00), 8457: (4.00), 8458: (4.00), 8459: (4.00), 8460: (4.00), 8461: (4.00), 8462: (4.00), 8463: (4.00), 8464: (4.00), 8465: (4.00), 8466: (4.00), 8467: (4.00), 8468: (4.00), 8469: (4.00), 8470: (4.00), 8471: (4.00), 8472: (17.02), 8473: (4.00), 8474: (4.00), 8475: (4.00), 8476: (4.00), 8477: (4.00), 8478: (4.00), 8479: (4.00), 8480: (4.00), 8481: (4.00), 8482: (4.00), 8483: (4.00), 8484: (4.00), 8485: (4.00), 8486: (4.00), 8487: (4.00), 8488: (4.00), 8489: (4.00), 8490: (17.07), 8491: (4.00), 8492: (4.00), 8493: (4.00), 8494: (4.00), 8495: (4.00), 8496: (4.00), 8497: (4.00), 8498: (4.00), 8499: (4.00), 8500: (4.00), 8501: (4.00), 8502: (4.00), 8503: (4.00), 8504: (4.00), 8505: (4.00), 8506: (17.06), 8507: (4.00), 8508: (4.00), 8509: (4.00), 8510: (4.00), 8511: (4.00), 8512: (17.02), 8513: (4.00), 8514: (4.00), 8515: (17.07), 8516: (4.00), 8517: (17.02), 8518: (17.03), 8519: (17.01), 8520: (4.00), 8521: (4.00), 8522: (4.00), 8523: (4.00), 8524: (4.00), 8525: (4.00), 8526: (17.09), 8527: (17.02), 8528: (17.01), 8529: (17.01), 8530: (17.01), 8531: (4.00), 8532: (4.00), 8533: (4.00), 8534: (4.00), 8535: (17.06), 8536: (17.02), 8537: (4.00), 8538: (4.00), 8539: (4.00), 8540: (4.00), 8541: (4.00), 8542: (4.00), 8543: (4.00), 8544: (4.00), 8545: (17.02), 8546: (17.02), 8547: (4.00), 8548: (4.00), 8549: (4.00), 8550: (4.00), 8551: (4.00), 8552: (4.00), 8553: (4.00), 8554: (4.00), 8555: (4.00), 8556: (4.00), 8557: (4.00), 8558: (4.00), 8559: (4.00), 8560: (4.00), 8561: (4.00), 8562: (4.00), 8563: (17.01), 8564: (17.01), 8565: (4.00), 8566: (4.00), 8567: (4.00), 8568: (4.00), 8569: (4.00), 8570: (4.00), 8571: (4.00), 8572: (4.00), 8573: (4.00), 8574: (4.00), 8575: (4.00), 8576: (4.00), 8577: (4.00), 8578: (4.00), 8579: (4.00), 8580: (4.00), 8581: (17.17), 8582: (17.02), 8583: (4.00), 8584: (4.00), 8585: (4.00), 8586: (4.00), 8587: (4.00), 8588: (4.00), 8589: (4.00), 8590: (4.00), 8591: (17.01), 8592: (17.01), 8593: (4.00), 8594: (4.00), 8595: (17.26), 8596: (17.01), 8597: (4.00), 8598: (4.00), 8599: (4.00), 8600: (17.10), 8601: (17.08), 8602: (17.01), 8603: (17.01), 8604: (17.14), 8605: (17.05), 8606: (17.01), 8607: (4.00), 8608: (4.00), 8609: (17.04), 8610: (17.04), 8611: (17.02), 8612: (17.02), 8613: (17.03), 8614: (4.00), 8615: (4.00), 8616: (4.00), 8617: (17.02), 8618: (17.15), 8619: (4.00), 8620: (4.00), 8621: (4.00), 8622: (4.00), 8623: (4.00), 8624: (4.00), 8625: (4.00), 8626: (4.00), 8627: (17.20), 8628: (17.01), 8629: (17.01), 8630: (4.00), 8631: (4.00), 8632: (4.00), 8633: (4.00), 8634: (4.00), 8635: (4.00), 8636: (4.00), 8637: (4.00), 8638: (4.00), 8639: (4.00), 8640: (4.00), 8641: (17.30), 8642: (17.01), 8643: (4.00), 8644: (4.00), 8645: (17.01), 8646: (17.01), 8647: (4.00), 8648: (4.00), 8649: (4.00), 8650: (4.00), 8651: (4.00), 8652: (4.00), 8653: (4.00), 8654: (4.00), 8655: (17.21), 8656: (4.00), 8657: (4.00), 8658: (4.00), 8659: (4.00), 8660: (4.00), 8661: (4.00), 8662: (4.00), 8663: (4.00), 8664: (4.00), 8665: (4.00), 8666: (4.00), 8667: (4.00), 8668: (4.00), 8669: (4.00), 8670: (4.00), 8671: (4.00), 8672: (4.00), 8673: (4.00), 8674: (17.31), 8675: (4.00), 8676: (4.00), 8677: (4.00), 8678: (4.00), 8679: (4.00), 8680: (4.00), 8681: (4.00), 8682: (4.00), 8683: (4.00), 8684: (4.00), 8685: (4.00), 8686: (4.00), 8687: (4.00), 8688: (4.00), 8689: (4.00), 8690: (4.00), 8691: (4.00), 8692: (4.00), 8693: (4.00), 8694: (4.00), 8695: (4.00), 8696: (4.00), 8697: (4.00), 8698: (4.00), 8699: (4.00), 8700: (4.00), 8701: (4.00), 8702: (4.00), 8703: (4.00), 8704: (4.00), 8705: (4.00), 8706: (4.00), 8707: (4.00), 8708: (4.00), 8709: (4.00), 8710: (4.00), 8711: (4.00), 8712: (4.00), 8713: (4.00), 8714: (4.00), 8715: (4.00), 8716: (4.00), 8717: (4.00), 8718: (4.00), 8719: (4.00), 8720: (4.00), 8721: (4.00), 8722: (4.00), 8723: (4.00), 8724: (4.00), 8725: (4.00), 8726: (4.00), 8727: (4.00), 8728: (4.00), 8729: (4.00), 8730: (4.00), 8731: (4.00), 8732: (4.00), 8733: (4.00), 8734: (4.00), 8735: (4.00), 8736: (4.00), 8737: (4.00), 8738: (4.00), 8739: (4.00), 8740: (4.00), 8741: (4.00), 8742: (4.00), 8743: (4.00), 8744: (4.00), 8745: (4.00), 8746: (4.00), 8747: (4.00), 8748: (17.02), 8749: (4.00), 8750: (4.00), 8751: (4.00), 8752: (4.00), 8753: (4.00), 8754: (4.00), 8755: (4.00), 8756: (4.00), 8757: (17.03), 8758: (17.01), 8759: (4.00), 8760: (4.00), 8761: (4.00), 8762: (4.00), 8763: (4.00), 8764: (4.00), 8765: (4.00), 8766: (4.00), 8767: (17.03), 8768: (4.00), 8769: (4.00), 8770: (4.00), 8771: (4.00), 8772: (4.00), 8773: (4.00), 8774: (4.00), 8775: (4.00), 8776: (4.00), 8777: (4.00), 8778: (4.00), 8779: (4.00), 8780: (4.00), 8781: (4.00), 8782: (4.00), 8783: (4.00), 8784: (4.00), 8785: (4.00), 8786: (4.00), 8787: (4.00), 8788: (4.00), 8789: (4.00), 8790: (4.00), 8791: (4.00), 8792: (4.00), 8793: (4.00), 8794: (4.00), 8795: (4.00), 8796: (4.00), 8797: (4.00), 8798: (4.00), 8799: (4.00), 8800: (4.00), 8801: (4.00), 8802: (4.00), 8803: (4.00), 8804: (4.00), 8805: (4.00), 8806: (4.00), 8807: (4.00), 8808: (4.00), 8809: (4.00), 8810: (4.00), 8811: (4.00), 8812: (4.00), 8813: (4.00), 8814: (4.00), 8815: (4.00), 8816: (4.00), 8817: (4.00), 8818: (4.00), 8819: (4.00), 8820: (4.00), 8821: (4.00), 8822: (4.00), 8823: (4.00), 8824: (4.00), 8825: (4.00), 8826: (4.00), 8827: (4.00), 8828: (4.00), 8829: (4.00), 8830: (4.00), 8831: (4.00), 8832: (4.00), 8833: (4.00), 8834: (4.00), 8835: (4.00), 8836: (4.00), 8837: (4.00), 8838: (4.00), 8839: (4.00), 8840: (4.00), 8841: (4.00), 8842: (17.01), 8843: (4.00), 8844: (4.00), 8845: (4.00), 8846: (4.00), 8847: (4.00), 8848: (4.00), 8849: (4.00), 8850: (4.00), 8851: (4.00), 8852: (4.00), 8853: (4.00), 8854: (4.00), 8855: (4.00), 8856: (4.00), 8857: (4.00), 8858: (4.00), 8859: (4.00), 8860: (4.00), 8861: (17.01), 8862: (4.00), 8863: (4.00), 8864: (4.00), 8865: (4.00), 8866: (4.00), 8867: (4.00), 8868: (4.00), 8869: (4.00), 8870: (4.00), 8871: (4.00), 8872: (4.00), 8873: (4.00), 8874: (4.00), 8875: (4.00), 8876: (4.00), 8877: (4.00), 8878: (4.00), 8879: (4.00), 8880: (4.00), 8881: (4.00), 8882: (4.00), 8883: (4.00), 8884: (4.00), 8885: (4.00), 8886: (4.00), 8887: (4.00), 8888: (4.00), 8889: (4.00), 8890: (4.00), 8891: (4.00), 8892: (4.00), 8893: (4.00), 8894: (4.00), 8895: (4.00), 8896: (4.00), 8897: (4.00), 8898: (4.00), 8899: (4.00), 8900: (4.00), 8901: (4.00), 8902: (4.00), 8903: (4.00), 8904: (4.00), 8905: (4.00), 8906: (4.00), 8907: (4.00), 8908: (4.00), 8909: (4.00), 8910: (4.00), 8911: (4.00), 8912: (4.00), 8913: (4.00), 8914: (4.00), 8915: (4.00), 8916: (4.00), 8917: (4.00), 8918: (4.00), 8919: (4.00), 8920: (4.00), 8921: (4.00), 8922: (4.00), 8923: (4.00), 8924: (4.00), 8925: (4.00), 8926: (4.00), 8927: (4.00), 8928: (4.00), 8929: (4.00), 8930: (4.00), 8931: (4.00), 8932: (4.00), 8933: (4.00), 8934: (4.00), 8935: (4.00), 8936: (17.01), 8937: (4.00), 8938: (4.00), 8939: (4.00), 8940: (4.00), 8941: (4.00), 8942: (4.00), 8943: (4.00), 8944: (4.00), 8945: (17.02), 8946: (4.00), 8947: (4.00), 8948: (4.00), 8949: (4.00), 8950: (4.00), 8951: (4.00), 8952: (4.00), 8953: (4.00), 8954: (4.00), 8955: (4.00), 8956: (4.00), 8957: (4.00), 8958: (4.00), 8959: (4.00), 8960: (4.00), 8961: (4.00), 8962: (4.00), 8963: (4.00), 8964: (4.00), 8965: (4.00), 8966: (4.00), 8967: (4.00), 8968: (4.00), 8969: (4.00), 8970: (4.00), 8971: (4.00), 8972: (4.00), 8973: (4.00), 8974: (4.00), 8975: (4.00), 8976: (4.00), 8977: (4.00), 8978: (4.00), 8979: (4.00), 8980: (4.00), 8981: (4.00), 8982: (4.00), 8983: (4.00), 8984: (4.00), 8985: (4.00), 8986: (4.00), 8987: (4.00), 8988: (4.00), 8989: (4.00), 8990: (4.00), 8991: (4.00), 8992: (4.00), 8993: (4.00), 8994: (4.00), 8995: (4.00), 8996: (4.00), 8997: (4.00), 8998: (4.00), 8999: (4.00), 9000: (4.00), 9001: (4.00), 9002: (4.00), 9003: (4.00), 9004: (4.00), 9005: (4.00), 9006: (4.00), 9007: (4.00), 9008: (4.00), 9009: (4.00), 9010: (4.00), 9011: (4.00), 9012: (4.00), 9013: (4.00), 9014: (4.00), 9015: (4.00), 9016: (4.00), 9017: (4.00), 9018: (4.00), 9019: (4.00), 9020: (4.00), 9021: (4.00), 9022: (4.00), 9023: (4.00), 9024: (4.00), 9025: (4.00), 9026: (4.00), 9027: (4.00), 9028: (4.00), 9029: (4.00), 9030: (4.00), 9031: (4.00), 9032: (4.00), 9033: (4.00), 9034: (4.00), 9035: (4.00), 9036: (4.00), 9037: (4.00), 9038: (4.00), 9039: (4.00), 9040: (4.00), 9041: (4.00), 9042: (4.00), 9043: (4.00), 9044: (4.00), 9045: (4.00), 9046: (4.00), 9047: (4.00), 9048: (4.00), 9049: (4.00), 9050: (4.00), 9051: (4.00), 9052: (4.00), 9053: (4.00), 9054: (4.00), 9055: (4.00), 9056: (4.00), 9057: (4.00), 9058: (4.00), 9059: (4.00), 9060: (4.00), 9061: (4.00), 9062: (4.00), 9063: (4.00), 9064: (4.00), 9065: (4.00), 9066: (4.00), 9067: (4.00), 9068: (4.00), 9069: (4.00), 9070: (4.00), 9071: (4.00), 9072: (4.00), 9073: (4.00), 9074: (4.00), 9075: (4.00), 9076: (4.00), 9077: (4.00), 9078: (4.00), 9079: (4.00), 9080: (4.00), 9081: (4.00), 9082: (4.00), 9083: (4.00), 9084: (4.00), 9085: (4.00), 9086: (4.00), 9087: (4.00), 9088: (4.00), 9089: (4.00), 9090: (4.00), 9091: (4.00), 9092: (4.00), 9093: (4.00), 9094: (4.00), 9095: (4.00), 9096: (4.00), 9097: (4.00), 9098: (4.00), 9099: (4.00), 9100: (4.00), 9101: (4.00), 9102: (4.00), 9103: (4.00), 9104: (4.00), 9105: (4.00), 9106: (4.00), 9107: (4.00), 9108: (4.00), 9109: (4.00), 9110: (4.00), 9111: (4.00), 9112: (4.00), 9113: (4.00), 9114: (4.00), 9115: (4.00), 9116: (4.00), 9117: (4.00), 9118: (4.00), 9119: (4.00), 9120: (4.00), 9121: (4.00), 9122: (4.00), 9123: (4.00), 9124: (4.00), 9125: (4.00), 9126: (4.00), 9127: (4.00), 9128: (4.00), 9129: (4.00), 9130: (17.01), 9131: (4.00), 9132: (4.00), 9133: (4.00), 9134: (4.00), 9135: (4.00), 9136: (4.00), 9137: (4.00), 9138: (4.00), 9139: (4.00), 9140: (4.00), 9141: (4.00), 9142: (4.00), 9143: (4.00), 9144: (4.00), 9145: (4.00), 9146: (4.00), 9147: (4.00), 9148: (4.00), 9149: (4.00), 9150: (4.00), 9151: (4.00), 9152: (4.00), 9153: (4.00), 9154: (4.00), 9155: (4.00), 9156: (4.00), 9157: (4.00), 9158: (4.00), 9159: (4.00), 9160: (4.00), 9161: (4.00), 9162: (4.00), 9163: (4.00), 9164: (4.00), 9165: (4.00), 9166: (4.00), 9167: (4.00), 9168: (4.00), 9169: (4.00), 9170: (4.00), 9171: (4.00), 9172: (4.00), 9173: (4.00), 9174: (4.00), 9175: (4.00), 9176: (4.00), 9177: (4.00), 9178: (4.00), 9179: (4.00), 9180: (4.00), 9181: (4.00), 9182: (4.00), 9183: (4.00), 9184: (4.00), 9185: (4.00), 9186: (4.00), 9187: (4.00), 9188: (4.00), 9189: (4.00), 9190: (4.00), 9191: (4.00), 9192: (4.00), 9193: (4.00), 9194: (4.00), 9195: (4.00), 9196: (4.00), 9197: (4.00), 9198: (4.00), 9199: (4.00), 9200: (4.00), 9201: (4.00), 9202: (4.00), 9203: (4.00), 9204: (4.00), 9205: (4.00), 9206: (4.00), 9207: (4.00), 9208: (4.00), 9209: (4.00), 9210: (4.00), 9211: (4.00), 9212: (4.00), 9213: (17.01), 9214: (4.00), 9215: (4.00), 9216: (4.00), 9217: (4.00), 9218: (4.00), 9219: (4.00), 9220: (4.00), 9221: (4.00), 9222: (4.00), 9223: (4.00), 9224: (4.00), 9225: (4.00), 9226: (4.00), 9227: (4.00), 9228: (4.00), 9229: (4.00), 9230: (4.00), 9231: (17.01), 9232: (4.00), 9233: (4.00), 9234: (4.00), 9235: (4.00), 9236: (4.00), 9237: (4.00), 9238: (4.00), 9239: (4.00), 9240: (4.00), 9241: (4.00), 9242: (4.00), 9243: (4.00), 9244: (4.00), 9245: (4.00), 9246: (4.00), 9247: (4.00), 9248: (4.00), 9249: (4.00), 9250: (4.00), 9251: (4.00), 9252: (4.00), 9253: (4.00), 9254: (4.00), 9255: (4.00), 9256: (4.00), 9257: (4.00), 9258: (4.00), 9259: (4.00), 9260: (4.00), 9261: (4.00), 9262: (4.00), 9263: (4.00), 9264: (4.00), 9265: (4.00), 9266: (4.00), 9267: (4.00), 9268: (4.00), 9269: (4.00), 9270: (4.00), 9271: (4.00), 9272: (4.00), 9273: (4.00), 9274: (4.00), 9275: (4.00), 9276: (4.00), 9277: (4.00), 9278: (4.00), 9279: (4.00), 9280: (4.00), 9281: (4.00), 9282: (4.00), 9283: (4.00), 9284: (4.00), 9285: (4.00), 9286: (4.00), 9287: (4.00), 9288: (4.00), 9289: (4.00), 9290: (4.00), 9291: (4.00), 9292: (4.00), 9293: (4.00), 9294: (4.00), 9295: (4.00), 9296: (4.00), 9297: (4.00), 9298: (4.00), 9299: (4.00), 9300: (4.00), 9301: (4.00), 9302: (4.00), 9303: (4.00), 9304: (4.00), 9305: (4.00), 9306: (4.00), 9307: (4.00), 9308: (4.00), 9309: (4.00), 9310: (4.00), 9311: (4.00), 9312: (4.00), 9313: (4.00), 9314: (4.00), 9315: (4.00), 9316: (4.00), 9317: (4.00), 9318: (4.00), 9319: (4.00), 9320: (4.00), 9321: (4.00), 9322: (4.00), 9323: (4.00), 9324: (4.00), 9325: (4.00), 9326: (4.00), 9327: (4.00), 9328: (4.00), 9329: (4.00), 9330: (4.00), 9331: (4.00), 9332: (4.00), 9333: (4.00), 9334: (4.00), 9335: (4.00), 9336: (4.00), 9337: (4.00), 9338: (4.00), 9339: (4.00), 9340: (4.00), 9341: (4.00), 9342: (4.00), 9343: (4.00), 9344: (4.00), 9345: (4.00), 9346: (4.00), 9347: (4.00), 9348: (4.00), 9349: (4.00), 9350: (4.00), 9351: (4.00), 9352: (4.00), 9353: (4.00), 9354: (4.00), 9355: (4.00), 9356: (4.00), 9357: (4.00), 9358: (4.00), 9359: (4.00), 9360: (4.00), 9361: (4.00), 9362: (4.00), 9363: (4.00), 9364: (4.00), 9365: (4.00), 9366: (4.00), 9367: (4.00), 9368: (4.00), 9369: (4.00), 9370: (4.00), 9371: (4.00), 9372: (4.00), 9373: (4.00), 9374: (4.00), 9375: (4.00), 9376: (4.00), 9377: (4.00), 9378: (4.00), 9379: (4.00), 9380: (4.00), 9381: (4.00), 9382: (4.00), 9383: (4.00), 9384: (4.00), 9385: (4.00), 9386: (4.00), 9387: (4.00), 9388: (4.00), 9389: (4.00), 9390: (4.00), 9391: (4.00), 9392: (4.00), 9393: (4.00), 9394: (4.00), 9395: (4.00), 9396: (4.00), 9397: (4.00), 9398: (4.00), 9399: (4.00), 9400: (4.00), 9401: (4.00), 9402: (4.00), 9403: (4.00), 9404: (4.00), 9405: (4.00), 9406: (4.00), 9407: (4.00), 9408: (4.00), 9409: (4.00), 9410: (4.00), 9411: (4.00), 9412: (4.00), 9413: (4.00), 9414: (17.05), 9415: (4.00), 9416: (4.00), 9417: (4.00), 9418: (4.00), 9419: (4.00), 9420: (4.00), 9421: (4.00), 9422: (4.00), 9423: (4.00), 9424: (4.00), 9425: (4.00), 9426: (4.00), 9427: (4.00), 9428: (4.00), 9429: (4.00), 9430: (4.00), 9431: (4.00), 9432: (4.00), 9433: (17.01), 9434: (4.00), 9435: (17.02), 9436: (4.00), 9437: (4.00), 9438: (17.08), 9439: (17.01), 9440: (4.00), 9441: (4.00), 9442: (4.00), 9443: (4.00), 9444: (4.00), 9445: (4.00), 9446: (4.00), 9447: (4.00), 9448: (4.00), 9449: (4.00), 9450: (4.00), 9451: (4.00), 9452: (4.00), 9453: (4.00), 9454: (4.00), 9455: (4.00), 9456: (4.00), 9457: (4.00), 9458: (4.00), 9459: (4.00), 9460: (4.00), 9461: (4.00), 9462: (17.01), 9463: (4.00), 9464: (4.00), 9465: (4.00), 9466: (4.00), 9467: (4.00), 9468: (4.00), 9469: (17.06), 9470: (17.01), 9471: (17.02), 9472: (4.00), 9473: (17.02), 9474: (4.00), 9475: (4.00), 9476: (4.00), 9477: (4.00), 9478: (17.01), 9479: (4.00), 9480: (4.00), 9481: (4.00), 9482: (17.02), 9483: (4.00), 9484: (4.00), 9485: (17.02), 9486: (17.01), 9487: (17.04), 9488: (17.01), 9489: (4.00), 9490: (4.00), 9491: (4.00), 9492: (4.00), 9493: (4.00), 9494: (4.00), 9495: (4.00), 9496: (17.01), 9497: (4.00), 9498: (4.00), 9499: (4.00), 9500: (4.00), 9501: (4.00), 9502: (4.00), 9503: (4.00), 9504: (4.00), 9505: (17.27), 9506: (17.04), 9507: (17.03), 9508: (17.01), 9509: (17.02), 9510: (4.00), 9511: (4.00), 9512: (4.00), 9513: (4.00), 9514: (4.00), 9515: (4.00), 9516: (4.00), 9517: (4.00), 9518: (4.00), 9519: (4.00), 9520: (4.00), 9521: (4.00), 9522: (4.00), 9523: (4.00), 9524: (4.00), 9525: (4.00), 9526: (17.01), 9527: (4.00), 9528: (4.00), 9529: (4.00), 9530: (4.00), 9531: (4.00), 9532: (4.00), 9533: (17.02), 9534: (4.00), 9535: (4.00), 9536: (4.00), 9537: (4.00), 9538: (4.00), 9539: (4.00), 9540: (4.00), 9541: (4.00), 9542: (4.00), 9543: (4.00), 9544: (4.00), 9545: (4.00), 9546: (4.00), 9547: (4.00), 9548: (4.00), 9549: (17.01), 9550: (4.00), 9551: (4.00), 9552: (4.00), 9553: (4.00), 9554: (17.01), 9555: (17.01), 9556: (17.01), 9557: (17.01), 9558: (4.00), 9559: (17.01), 9560: (17.22), 9561: (17.01), 9562: (17.02), 9563: (17.01), 9564: (17.01), 9565: (4.00), 9566: (4.00), 9567: (4.00), 9568: (4.00), 9569: (4.00), 9570: (17.01), 9571: (4.00), 9572: (4.00), 9573: (17.01), 9574: (17.03), 9575: (17.01), 9576: (17.07), 9577: (4.00), 9578: (17.02), 9579: (17.01), 9580: (17.01), 9581: (17.01), 9582: (4.00), 9583: (4.00), 9584: (4.00), 9585: (4.00), 9586: (4.00), 9587: (17.32), 9588: (17.02), 9589: (4.00), 9590: (17.02), 9591: (17.07), 9592: (4.00), 9593: (4.00), 9594: (4.00), 9595: (4.00), 9596: (17.21), 9597: (17.02), 9598: (17.01), 9599: (17.01), 9600: (17.01), 9601: (4.00), 9602: (4.00), 9603: (4.00), 9604: (4.00), 9605: (4.00), 9606: (4.00), 9607: (4.00), 9608: (4.00), 9609: (4.00), 9610: (4.00), 9611: (4.00), 9612: (4.00), 9613: (4.00), 9614: (4.00), 9615: (17.01), 9616: (4.00), 9617: (17.01), 9618: (4.00), 9619: (4.00), 9620: (4.00), 9621: (4.00), 9622: (4.00), 9623: (4.00), 9624: (17.01), 9625: (17.01), 9626: (17.01), 9627: (17.01), 9628: (4.00), 9629: (4.00), 9630: (4.00), 9631: (4.00), 9632: (4.00), 9633: (4.00), 9634: (4.00), 9635: (4.00), 9636: (4.00), 9637: (4.00), 9638: (4.00), 9639: (4.00), 9640: (4.00), 9641: (4.00), 9642: (4.00), 9643: (4.00), 9644: (17.01), 9645: (4.00), 9646: (17.02), 9647: (17.03), 9648: (17.01), 9649: (4.00), 9650: (4.00), 9651: (17.01), 9652: (17.38), 9653: (17.02), 9654: (17.01), 9655: (17.05), 9656: (4.00), 9657: (17.01), 9658: (4.00), 9659: (4.00), 9660: (4.00), 9661: (4.00), 9662: (17.01), 9663: (4.00), 9664: (17.01), 9665: (17.01), 9666: (17.01), 9667: (4.00), 9668: (17.04), 9669: (4.00), 9670: (17.01), 9671: (4.00), 9672: (4.00), 9673: (4.00), 9674: (4.00), 9675: (4.00), 9676: (4.00), 9677: (4.00), 9678: (4.00), 9679: (17.12), 9680: (4.00), 9681: (4.00), 9682: (4.00), 9683: (4.00), 9684: (4.00), 9685: (4.00), 9686: (4.00), 9687: (4.00), 9688: (17.16), 9689: (17.01), 9690: (4.00), 9691: (4.00), 9692: (17.01), 9693: (4.00), 9694: (4.00), 9695: (4.00), 9696: (4.00), 9697: (4.00), 9698: (4.00), 9699: (4.00), 9700: (4.00), 9701: (4.00), 9702: (4.00), 9703: (4.00), 9704: (4.00), 9705: (4.00), 9706: (4.00), 9707: (4.00), 9708: (4.00), 9709: (4.00), 9710: (17.01), 9711: (4.00), 9712: (4.00), 9713: (4.00), 9714: (4.00), 9715: (4.00), 9716: (17.01), 9717: (17.01), 9718: (4.00), 9719: (17.01), 9720: (4.00), 9721: (4.00), 9722: (4.00), 9723: (4.00), 9724: (4.00), 9725: (4.00), 9726: (4.00), 9727: (17.01), 9728: (4.00), 9729: (4.00), 9730: (4.00), 9731: (4.00), 9732: (4.00), 9733: (17.01), 9734: (17.01), 9735: (17.01), 9736: (17.01), 9737: (17.01), 9738: (17.09), 9739: (17.03), 9740: (17.01), 9741: (4.00), 9742: (17.01), 9743: (17.07), 9744: (17.02), 9745: (17.01), 9746: (17.07), 9747: (17.02), 9748: (4.00), 9749: (4.00), 9750: (4.00), 9751: (4.00), 9752: (17.01), 9753: (17.01), 9754: (17.03), 9755: (17.02), 9756: (17.10), 9757: (4.00), 9758: (17.01), 9759: (17.14), 9760: (17.01), 9761: (17.01), 9762: (17.03), 9763: (17.01), 9764: (4.00), 9765: (4.00), 9766: (4.00), 9767: (4.00), 9768: (4.00), 9769: (4.00), 9770: (17.01), 9771: (17.01), 9772: (17.01), 9773: (17.01), 9774: (17.02), 9775: (17.01), 9776: (4.00), 9777: (4.00), 9778: (4.00), 9779: (17.19), 9780: (17.04), 9781: (4.00), 9782: (17.01), 9783: (4.00), 9784: (17.02), 9785: (4.00), 9786: (4.00), 9787: (4.00), 9788: (4.00), 9789: (4.00), 9790: (4.00), 9791: (4.00), 9792: (4.00), 9793: (4.00), 9794: (4.00), 9795: (4.00), 9796: (4.00), 9797: (4.00), 9798: (4.00), 9799: (4.00), 9800: (17.01), 9801: (4.00), 9802: (4.00), 9803: (4.00), 9804: (4.00), 9805: (4.00), 9806: (17.01), 9807: (17.01), 9808: (17.02), 9809: (17.01), 9810: (4.00), 9811: (4.00), 9812: (4.00), 9813: (4.00), 9814: (4.00), 9815: (4.00), 9816: (4.00), 9817: (4.00), 9818: (17.01), 9819: (17.01), 9820: (17.01), 9821: (17.03), 9822: (17.01), 9823: (17.02), 9824: (4.00), 9825: (17.01), 9826: (17.01), 9827: (17.04), 9828: (17.03), 9829: (4.00), 9830: (4.00), 9831: (4.00), 9832: (17.01), 9833: (17.01), 9834: (17.07), 9835: (17.02), 9836: (17.02), 9837: (17.04), 9838: (17.06), 9839: (17.01), 9840: (17.01), 9841: (17.01), 9842: (4.00), 9843: (4.00), 9844: (4.00), 9845: (17.01), 9846: (4.00), 9847: (4.00), 9848: (17.08), 9849: (4.00), 9850: (17.01), 9851: (17.08), 9852: (4.00), 9853: (17.01), 9854: (4.00), 9855: (4.00), 9856: (4.00), 9857: (4.00), 9858: (4.00), 9859: (4.00), 9860: (4.00), 9861: (4.00), 9862: (17.09), 9863: (4.00), 9864: (4.00), 9865: (4.00), 9866: (4.00), 9867: (4.00), 9868: (4.00), 9869: (4.00), 9870: (4.00), 9871: (17.16), 9872: (17.04), 9873: (17.01), 9874: (17.04), 9875: (17.01), 9876: (17.01), 9877: (17.01), 9878: (17.01), 9879: (17.01), 9880: (4.00), 9881: (17.01), 9882: (4.00), 9883: (17.01), 9884: (4.00), 9885: (17.01), 9886: (17.01), 9887: (4.00), 9888: (4.00), 9889: (17.01), 9890: (4.00), 9891: (4.00), 9892: (4.00), 9893: (4.00), 9894: (4.00), 9895: (4.00), 9896: (4.00), 9897: (4.00), 9898: (4.00), 9899: (4.00), 9900: (4.00), 9901: (4.00), 9902: (4.00), 9903: (4.00), 9904: (4.00), 9905: (4.00), 9906: (4.00), 9907: (4.00), 9908: (4.00), 9909: (4.00), 9910: (4.00), 9911: (4.00), 9912: (4.00), 9913: (4.00), 9914: (4.00), 9915: (4.00), 9916: (4.00), 9917: (4.00), 9918: (4.00), 9919: (4.00), 9920: (4.00), 9921: (4.00), 9922: (4.00), 9923: (4.00), 9924: (4.00), 9925: (17.08), 9926: (17.01), 9927: (4.00), 9928: (17.01), 9929: (17.01), 9930: (4.00), 9931: (4.00), 9932: (4.00), 9933: (4.00), 9934: (4.00), 9935: (4.00), 9936: (4.00), 9937: (4.00), 9938: (4.00), 9939: (17.06), 9940: (4.00), 9941: (4.00), 9942: (17.18), 9943: (4.00), 9944: (17.02), 9945: (4.00), 9946: (4.00), 9947: (4.00), 9948: (4.00), 9949: (4.00), 9950: (4.00), 9951: (4.00), 9952: (17.02), 9953: (17.07), 9954: (17.01), 9955: (17.02), 9956: (4.00), 9957: (4.00), 9958: (17.01), 9959: (17.03), 9960: (4.00), 9961: (17.05), 9962: (17.46), 9963: (17.03), 9964: (4.00), 9965: (17.02), 9966: (17.01), 9967: (17.01), 9968: (4.00), 9969: (4.00), 9970: (17.01), 9971: (17.01), 9972: (17.05), 9973: (4.00), 9974: (17.03), 9975: (17.01), 9976: (17.01), 9977: (17.01), 9978: (17.01), 9979: (4.00), 9980: (4.00), 9981: (4.00), 9982: (4.00), 9983: (4.00), 9984: (4.00), 9985: (4.00), 9986: (4.00), 9987: (4.00), 9988: (4.00), 9989: (4.00), 9990: (4.00), 9991: (4.00), 9992: (4.00), 9993: (4.00), 9994: (4.00), 9995: (4.00), 9996: (4.00), 9997: (4.00), 9998: (4.00), 9999: (4.00), 10000: (4.00), 10001: (4.00), 10002: (4.00), 10003: (4.00), 10004: (4.00), 10005: (4.00), 10006: (4.00), 10007: (4.00), 10008: (17.01), 10009: (4.00), 10010: (17.04), 10011: (4.00), 10012: (4.00), 10013: (4.00), 10014: (4.00), 10015: (17.01), 10016: (17.01), 10017: (17.04), 10018: (17.01), 10019: (17.02), 10020: (17.03), 10021: (4.00), 10022: (4.00), 10023: (4.00), 10024: (4.00), 10025: (4.00), 10026: (4.00), 10027: (4.00), 10028: (4.00), 10029: (4.00), 10030: (17.03), 10031: (4.00), 10032: (4.00), 10033: (17.08), 10034: (4.00), 10035: (4.00), 10036: (4.00), 10037: (4.00), 10038: (4.00), 10039: (4.00), 10040: (4.00), 10041: (4.00), 10042: (4.00), 10043: (4.00), 10044: (17.19), 10045: (4.00), 10046: (4.00), 10047: (4.00), 10048: (4.00), 10049: (4.00), 10050: (4.00), 10051: (4.00), 10052: (4.00), 10053: (17.07), 10054: (17.01), 10055: (17.01), 10056: (4.00), 10057: (17.01), 10058: (4.00), 10059: (4.00), 10060: (4.00), 10061: (4.00), 10062: (4.00), 10063: (17.05), 10064: (17.01), 10065: (17.01), 10066: (17.06), 10067: (17.01), 10068: (17.01), 10069: (17.02), 10070: (17.01), 10071: (17.01), 10072: (4.00), 10073: (4.00), 10074: (17.01), 10075: (4.00), 10076: (4.00), 10077: (4.00), 10078: (4.00), 10079: (4.00), 10080: (17.01), 10081: (4.00), 10082: (4.00), 10083: (4.00), 10084: (4.00), 10085: (4.00), 10086: (4.00), 10087: (4.00), 10088: (4.00), 10089: (4.00), 10090: (4.00), 10091: (4.00), 10092: (4.00), 10093: (4.00), 10094: (4.00), 10095: (4.00), 10096: (4.00), 10097: (17.01), 10098: (17.01), 10099: (17.03), 10100: (4.00), 10101: (4.00), 10102: (17.06), 10103: (17.01), 10104: (4.00), 10105: (4.00), 10106: (17.02), 10107: (17.01), 10108: (17.10), 10109: (17.01), 10110: (4.00), 10111: (4.00), 10112: (4.00), 10113: (4.00), 10114: (4.00), 10115: (17.01), 10116: (4.00), 10117: (4.00), 10118: (4.00), 10119: (17.04), 10120: (17.01), 10121: (17.11), 10122: (17.02), 10123: (17.33), 10124: (17.02), 10125: (17.01), 10126: (17.01), 10127: (17.01), 10128: (17.01), 10129: (17.01), 10130: (17.01), 10131: (4.00), 10132: (17.01), 10133: (17.01), 10134: (17.06), 10135: (17.07), 10136: (17.01), 10137: (17.01), 10138: (17.04), 10139: (4.00), 10140: (4.00), 10141: (4.00), 10142: (4.00), 10143: (4.00), 10144: (17.08), 10145: (17.01), 10146: (4.00), 10147: (17.01), 10148: (4.00), 10149: (4.00), 10150: (4.00), 10151: (17.01), 10152: (4.00), 10153: (17.01), 10154: (17.01), 10155: (4.00), 10156: (17.01), 10157: (4.00), 10158: (4.00), 10159: (4.00), 10160: (4.00), 10161: (17.01), 10162: (4.00), 10163: (17.01), 10164: (17.01), 10165: (17.02), 10166: (17.01), 10167: (17.01), 10168: (17.01), 10169: (17.01), 10170: (4.00), 10171: (17.01), 10172: (17.01), 10173: (4.00), 10174: (4.00), 10175: (4.00), 10176: (4.00), 10177: (4.00), 10178: (4.00), 10179: (4.00), 10180: (4.00), 10181: (4.00), 10182: (4.00), 10183: (4.00), 10184: (4.00), 10185: (4.00), 10186: (4.00), 10187: (4.00), 10188: (17.01), 10189: (4.00), 10190: (17.03), 10191: (17.01), 10192: (17.01), 10193: (4.00), 10194: (17.01), 10195: (17.02), 10196: (4.00), 10197: (4.00), 10198: (17.01), 10199: (17.04), 10200: (17.02), 10201: (17.04), 10202: (17.01), 10203: (17.02), 10204: (4.00), 10205: (4.00), 10206: (17.01), 10207: (4.00), 10208: (4.00), 10209: (17.01), 10210: (17.01), 10211: (17.01), 10212: (4.00), 10213: (17.01), 10214: (17.01), 10215: (17.12), 10216: (17.01), 10217: (17.01), 10218: (4.00), 10219: (4.00), 10220: (4.00), 10221: (4.00), 10222: (4.00), 10223: (4.00), 10224: (4.00), 10225: (4.00), 10226: (17.15), 10227: (17.01), 10228: (4.00), 10229: (4.00), 10230: (4.00), 10231: (4.00), 10232: (4.00), 10233: (4.00), 10234: (4.00), 10235: (17.27), 10236: (17.04), 10237: (17.01), 10238: (17.02), 10239: (17.03), 10240: (4.00), 10241: (4.00), 10242: (17.01), 10243: (17.02), 10244: (17.02), 10245: (17.02), 10246: (17.06), 10247: (4.00), 10248: (17.02), 10249: (17.01), 10250: (17.02), 10251: (17.01), 10252: (17.02), 10253: (17.01), 10254: (17.01), 10255: (4.00), 10256: (4.00), 10257: (4.00), 10258: (4.00), 10259: (4.00), 10260: (4.00), 10261: (4.00), 10262: (4.00), 10263: (17.01), 10264: (4.00), 10265: (4.00), 10266: (4.00), 10267: (4.00), 10268: (4.00), 10269: (4.00), 10270: (4.00), 10271: (4.00), 10272: (4.00), 10273: (4.00), 10274: (4.00), 10275: (4.00), 10276: (17.01), 10277: (4.00), 10278: (4.00), 10279: (4.00), 10280: (17.02), 10281: (4.00), 10282: (17.01), 10283: (4.00), 10284: (4.00), 10285: (4.00), 10286: (4.00), 10287: (4.00), 10288: (17.03), 10289: (17.12), 10290: (17.05), 10291: (17.01), 10292: (17.04), 10293: (17.01), 10294: (17.02), 10295: (17.01), 10296: (17.01), 10297: (17.01), 10298: (17.01), 10299: (17.01), 10300: (17.02), 10301: (17.02), 10302: (17.01), 10303: (17.03), 10304: (4.00), 10305: (17.06), 10306: (17.01), 10307: (4.00), 10308: (4.00), 10309: (4.00), 10310: (4.00), 10311: (4.00), 10312: (4.00), 10313: (4.00), 10314: (4.00), 10315: (4.00), 10316: (17.12), 10317: (17.01), 10318: (4.00), 10319: (17.15), 10320: (17.04), 10321: (4.00), 10322: (4.00), 10323: (4.00), 10324: (4.00), 10325: (17.23), 10326: (17.02), 10327: (4.00), 10328: (4.00), 10329: (17.01), 10330: (4.00), 10331: (4.00), 10332: (4.00), 10333: (4.00), 10334: (4.00), 10335: (4.00), 10336: (4.00), 10337: (4.00), 10338: (4.00), 10339: (4.00), 10340: (4.00), 10341: (4.00), 10342: (4.00), 10343: (4.00), 10344: (17.01), 10345: (17.02), 10346: (4.00), 10347: (17.01), 10348: (4.00), 10349: (4.00), 10350: (4.00), 10351: (4.00), 10352: (4.00), 10353: (4.00), 10354: (4.00), 10355: (4.00), 10356: (4.00), 10357: (4.00), 10358: (4.00), 10359: (4.00), 10360: (4.00), 10361: (4.00), 10362: (4.00), 10363: (4.00), 10364: (4.00), 10365: (4.00), 10366: (4.00), 10367: (4.00), 10368: (4.00), 10369: (4.00), 10370: (4.00), 10371: (4.00), 10372: (4.00), 10373: (4.00), 10374: (4.00), 10375: (4.00), 10376: (4.00), 10377: (4.00), 10378: (4.00), 10379: (17.02), 10380: (17.12), 10381: (17.04), 10382: (17.02), 10383: (17.03), 10384: (17.01), 10385: (17.01), 10386: (4.00), 10387: (4.00), 10388: (4.00), 10389: (4.00), 10390: (4.00), 10391: (4.00), 10392: (4.00), 10393: (4.00), 10394: (4.00), 10395: (4.00), 10396: (17.06), 10397: (17.01), 10398: (17.01), 10399: (4.00), 10400: (4.00), 10401: (4.00), 10402: (4.00), 10403: (4.00), 10404: (4.00), 10405: (4.00), 10406: (4.00), 10407: (17.01), 10408: (4.00), 10409: (4.00), 10410: (4.00), 10411: (4.00), 10412: (4.00), 10413: (4.00), 10414: (4.00), 10415: (4.00), 10416: (17.13), 10417: (17.01), 10418: (17.02), 10419: (17.02), 10420: (4.00), 10421: (17.02), 10422: (17.01), 10423: (4.00), 10424: (4.00), 10425: (4.00), 10426: (4.00), 10427: (4.00), 10428: (4.00), 10429: (4.00), 10430: (4.00), 10431: (4.00), 10432: (4.00), 10433: (4.00), 10434: (4.00), 10435: (4.00), 10436: (4.00), 10437: (4.00), 10438: (4.00), 10439: (4.00), 10440: (4.00), 10441: (4.00), 10442: (4.00), 10443: (4.00), 10444: (4.00), 10445: (4.00), 10446: (4.00), 10447: (4.00), 10448: (4.00), 10449: (4.00), 10450: (4.00), 10451: (4.00), 10452: (4.00), 10453: (4.00), 10454: (4.00), 10455: (4.00), 10456: (4.00), 10457: (4.00), 10458: (4.00), 10459: (4.00), 10460: (4.00), 10461: (17.02), 10462: (17.01), 10463: (17.02), 10464: (4.00), 10465: (17.01), 10466: (17.01), 10467: (17.01), 10468: (4.00), 10469: (4.00), 10470: (17.19), 10471: (17.01), 10472: (4.00), 10473: (17.01), 10474: (17.01), 10475: (4.00), 10476: (4.00), 10477: (4.00), 10478: (17.01), 10479: (17.01), 10480: (17.01), 10481: (17.06), 10482: (17.03), 10483: (17.19), 10484: (17.01), 10485: (17.11), 10486: (17.02), 10487: (17.01), 10488: (17.03), 10489: (17.01), 10490: (17.01), 10491: (17.04), 10492: (17.03), 10493: (17.09), 10494: (17.11), 10495: (17.03), 10496: (17.11), 10497: (17.04), 10498: (17.05), 10499: (17.02), 10500: (17.31), 10501: (17.03), 10502: (17.01), 10503: (17.01), 10504: (17.03), 10505: (17.05), 10506: (17.28), 10507: (17.03), 10508: (4.00), 10509: (4.00), 10510: (17.01), 10511: (4.00), 10512: (4.00), 10513: (4.00), 10514: (17.01), 10515: (17.01), 10516: (17.01), 10517: (4.00), 10518: (4.00), 10519: (17.02), 10520: (4.00), 10521: (4.00), 10522: (4.00), 10523: (4.00), 10524: (4.00), 10525: (4.00), 10526: (4.00), 10527: (17.02), 10528: (17.11), 10529: (4.00), 10530: (4.00), 10531: (4.00), 10532: (17.01), 10533: (17.01), 10534: (4.00), 10535: (4.00), 10536: (4.00), 10537: (4.00), 10538: (4.00), 10539: (4.00), 10540: (4.00), 10541: (4.00), 10542: (4.00), 10543: (4.00), 10544: (4.00), 10545: (4.00), 10546: (4.00), 10547: (4.00), 10548: (4.00), 10549: (4.00), 10550: (17.01), 10551: (4.00), 10552: (4.00), 10553: (4.00), 10554: (4.00), 10555: (17.29), 10556: (17.01), 10557: (17.02), 10558: (4.00), 10559: (4.00), 10560: (17.01), 10561: (4.00), 10562: (4.00), 10563: (4.00), 10564: (4.00), 10565: (4.00), 10566: (4.00), 10567: (4.00), 10568: (4.00), 10569: (4.00), 10570: (17.01), 10571: (4.00), 10572: (4.00), 10573: (17.01), 10574: (4.00), 10575: (4.00), 10576: (17.01), 10577: (17.01), 10578: (17.01), 10579: (4.00), 10580: (17.01), 10581: (17.02), 10582: (17.14), 10583: (4.00), 10584: (17.02), 10585: (17.01), 10586: (17.07), 10587: (17.12), 10588: (17.03), 10589: (17.01), 10590: (17.16), 10591: (17.03), 10592: (17.01), 10593: (17.02), 10594: (4.00), 10595: (17.05), 10596: (17.04), 10597: (17.01), 10598: (17.01), 10599: (4.00), 10600: (4.00), 10601: (4.00), 10602: (4.00), 10603: (4.00), 10604: (4.00), 10605: (4.00), 10606: (4.00), 10607: (4.00), 10608: (4.00), 10609: (4.00), 10610: (4.00), 10611: (4.00), 10612: (4.00), 10613: (4.00), 10614: (4.00), 10615: (4.00), 10616: (4.00), 10617: (4.00), 10618: (17.13), 10619: (4.00), 10620: (4.00), 10621: (4.00), 10622: (4.00), 10623: (17.01), 10624: (17.01), 10625: (4.00), 10626: (4.00), 10627: (4.00), 10628: (4.00), 10629: (4.00), 10630: (4.00), 10631: (4.00), 10632: (4.00), 10633: (4.00), 10634: (4.00), 10635: (4.00), 10636: (4.00), 10637: (17.01), 10638: (4.00), 10639: (4.00), 10640: (4.00), 10641: (4.00), 10642: (4.00), 10643: (17.01), 10644: (17.01), 10645: (17.01), 10646: (17.08), 10647: (4.00), 10648: (4.00), 10649: (4.00), 10650: (4.00), 10651: (17.04), 10652: (17.01), 10653: (4.00), 10654: (4.00), 10655: (4.00), 10656: (4.00), 10657: (4.00), 10658: (4.00), 10659: (4.00), 10660: (4.00), 10661: (4.00), 10662: (4.00), 10663: (4.00), 10664: (4.00), 10665: (4.00), 10666: (4.00), 10667: (4.00), 10668: (4.00), 10669: (17.01), 10670: (17.05), 10671: (17.01), 10672: (17.05), 10673: (4.00), 10674: (4.00), 10675: (4.00), 10676: (4.00), 10677: (17.01), 10678: (17.11), 10679: (17.07), 10680: (17.03), 10681: (4.00), 10682: (4.00), 10683: (17.01), 10684: (17.03), 10685: (4.00), 10686: (17.07), 10687: (17.04), 10688: (17.01), 10689: (4.00), 10690: (4.00), 10691: (4.00), 10692: (4.00), 10693: (4.00), 10694: (4.00), 10695: (4.00), 10696: (4.00), 10697: (4.00), 10698: (4.00), 10699: (4.00), 10700: (4.00), 10701: (4.00), 10702: (4.00), 10703: (4.00), 10704: (4.00), 10705: (4.00), 10706: (4.00), 10707: (4.00), 10708: (4.00), 10709: (17.19), 10710: (17.02), 10711: (4.00), 10712: (4.00), 10713: (4.00), 10714: (17.01), 10715: (4.00), 10716: (4.00), 10717: (4.00), 10718: (4.00), 10719: (4.00), 10720: (4.00), 10721: (4.00), 10722: (4.00), 10723: (4.00), 10724: (4.00), 10725: (4.00), 10726: (4.00), 10727: (4.00), 10728: (17.01), 10729: (4.00), 10730: (4.00), 10731: (4.00), 10732: (17.01), 10733: (17.01), 10734: (17.06), 10735: (4.00), 10736: (17.01), 10737: (17.16), 10738: (17.01), 10739: (4.00), 10740: (4.00), 10741: (4.00), 10742: (4.00), 10743: (4.00), 10744: (4.00), 10745: (4.00), 10746: (4.00), 10747: (4.00), 10748: (4.00), 10749: (4.00), 10750: (4.00), 10751: (17.01), 10752: (4.00), 10753: (4.00), 10754: (4.00), 10755: (4.00), 10756: (4.00), 10757: (4.00), 10758: (4.00), 10759: (4.00), 10760: (17.01), 10761: (17.01), 10762: (4.00), 10763: (17.02), 10764: (17.07), 10765: (4.00), 10766: (4.00), 10767: (17.01), 10768: (17.02), 10769: (17.05), 10770: (17.01), 10771: (4.00), 10772: (4.00), 10773: (4.00), 10774: (4.00), 10775: (4.00), 10776: (4.00), 10777: (17.04), 10778: (17.04), 10779: (17.01), 10780: (17.01), 10781: (4.00), 10782: (4.00), 10783: (4.00), 10784: (4.00), 10785: (4.00), 10786: (4.00), 10787: (4.00), 10788: (4.00), 10789: (4.00), 10790: (4.00), 10791: (4.00), 10792: (4.00), 10793: (4.00), 10794: (4.00), 10795: (4.00), 10796: (4.00), 10797: (4.00), 10798: (4.00), 10799: (4.00), 10800: (17.01), 10801: (17.01), 10802: (4.00), 10803: (4.00), 10804: (4.00), 10805: (17.01), 10806: (4.00), 10807: (4.00), 10808: (4.00), 10809: (4.00), 10810: (4.00), 10811: (4.00), 10812: (4.00), 10813: (4.00), 10814: (4.00), 10815: (4.00), 10816: (4.00), 10817: (4.00), 10818: (4.00), 10819: (17.02), 10820: (4.00), 10821: (4.00), 10822: (4.00), 10823: (17.02), 10824: (17.01), 10825: (17.03), 10826: (4.00), 10827: (4.00), 10828: (17.11), 10829: (17.01), 10830: (17.01), 10831: (17.01), 10832: (17.02), 10833: (17.01), 10834: (17.01), 10835: (17.02), 10836: (17.01), 10837: (4.00), 10838: (4.00), 10839: (4.00), 10840: (4.00), 10841: (4.00), 10842: (4.00), 10843: (4.00), 10844: (4.00), 10845: (4.00), 10846: (4.00), 10847: (4.00), 10848: (4.00), 10849: (4.00), 10850: (4.00), 10851: (17.01), 10852: (17.05), 10853: (17.02), 10854: (17.01), 10855: (17.04), 10856: (17.01), 10857: (4.00), 10858: (4.00), 10859: (17.02), 10860: (17.02), 10861: (17.06), 10862: (17.09), 10863: (17.03), 10864: (17.03), 10865: (4.00), 10866: (4.00), 10867: (4.00), 10868: (17.06), 10869: (17.10), 10870: (17.01), 10871: (17.01), 10872: (17.02), 10873: (4.00), 10874: (4.00), 10875: (4.00), 10876: (4.00), 10877: (4.00), 10878: (17.02), 10879: (4.00), 10880: (17.02), 10881: (4.00), 10882: (17.01), 10883: (17.01), 10884: (17.01), 10885: (17.02), 10886: (4.00), 10887: (4.00), 10888: (4.00), 10889: (4.00), 10890: (17.06), 10891: (17.27), 10892: (17.09), 10893: (4.00), 10894: (17.07), 10895: (17.01), 10896: (17.01), 10897: (4.00), 10898: (4.00), 10899: (4.00), 10900: (17.03), 10901: (17.01), 10902: (17.08), 10903: (17.02), 10904: (17.13), 10905: (17.06), 10906: (17.01), 10907: (17.01), 10908: (17.01), 10909: (17.02), 10910: (17.03), 10911: (17.01), 10912: (17.01), 10913: (17.01), 10914: (17.01), 10915: (17.02), 10916: (4.00), 10917: (17.04), 10918: (17.12), 10919: (17.03), 10920: (17.02), 10921: (17.13), 10922: (17.01), 10923: (17.11), 10924: (17.09), 10925: (17.07), 10926: (17.08), 10927: (17.01), 10928: (17.08), 10929: (17.07), 10930: (17.03), 10931: (17.01), 10932: (17.10), 10933: (17.06), 10934: (17.07), 10935: (17.15), 10936: (17.01), 10937: (17.01), 10938: (17.01), 10939: (17.01), 10940: (17.01), 10941: (17.07), 10942: (17.03), 10943: (17.08), 10944: (17.01), 10945: (4.00), 10946: (17.04), 10947: (17.01), 10948: (17.08), 10949: (17.03), 10950: (17.05), 10951: (17.15), 10952: (17.25), 10953: (17.08), 10954: (17.01), 10955: (17.02), 10956: (17.01), 10957: (17.01), 10958: (17.03), 10959: (17.15), 10960: (17.03), 10961: (17.01), 10962: (17.03), 10963: (17.01), 10964: (4.00), 10965: (4.00), 10966: (17.01), 10967: (4.00), 10968: (4.00), 10969: (17.02), 10970: (17.02), 10971: (17.01), 10972: (17.02), 10973: (17.02), 10974: (17.13), 10975: (17.03), 10976: (17.15), 10977: (17.01), 10978: (4.00), 10979: (17.01), 10980: (17.02), 10981: (17.11), 10982: (17.03), 10983: (17.06), 10984: (17.03), 10985: (17.08), 10986: (17.05), 10987: (17.01), 10988: (17.01), 10989: (17.01), 10990: (17.14), 10991: (17.02), 10992: (17.12), 10993: (17.02), 10994: (17.10), 10995: (17.01), 10996: (17.01), 10997: (17.12), 10998: (17.01), 10999: (17.02), 11000: (17.03), 11001: (17.04), 11002: (17.01), 11003: (17.01), 11004: (17.01), 11005: (17.16), 11006: (17.02), 11007: (17.05), 11008: (17.04), 11009: (17.07), 11010: (17.11), 11011: (17.13), 11012: (17.11), 11013: (17.17), 11014: (17.09), 11015: (17.04), 11016: (17.08), 11017: (17.03), 11018: (17.01), 11019: (17.02), 11020: (17.01), 11021: (17.02), 11022: (17.25), 11023: (17.02), 11024: (17.13), 11025: (17.04), 11026: (17.01), 11027: (17.02), 11028: (17.01), 11029: (17.03), 11030: (17.02), 11031: (17.02), 11032: (17.01), 11033: (4.00), 11034: (17.01), 11035: (17.01), 11036: (17.01), 11037: (17.01), 11038: (17.01), 11039: (17.02), 11040: (17.04), 11041: (17.01), 11042: (17.02), 11043: (17.01), 11044: (17.01), 11045: (4.00), 11046: (17.01), 11047: (17.03), 11048: (17.06), 11049: (17.24), 11050: (17.02), 11051: (17.09), 11052: (4.00), 11053: (17.01), 11054: (4.00), 11055: (17.02), 11056: (4.00), 11057: (17.05), 11058: (17.08), 11059: (17.01), 11060: (17.06), 11061: (17.02), 11062: (17.07), 11063: (17.09), 11064: (17.04), 11065: (17.19), 11066: (17.06), 11067: (4.00), 11068: (17.02), 11069: (17.02), 11070: (17.01), 11071: (17.06), 11072: (17.48), 11073: (17.01), 11074: (17.07), 11075: (17.11), 11076: (17.07), 11077: (17.02), 11078: (17.10), 11079: (17.01), 11080: (17.02), 11081: (17.03), 11082: (17.12), 11083: (17.02), 11084: (17.28), 11085: (17.19), 11086: (17.04), 11087: (17.02), 11088: (17.03), 11089: (17.01), 11090: (17.01), 11091: (4.00), 11092: (17.01), 11093: (17.01), 11094: (17.19), 11095: (17.08), 11096: (17.04), 11097: (17.05), 11098: (17.04), 11099: (17.02), 11100: (17.02), 11101: (17.02), 11102: (17.10), 11103: (17.30), 11104: (17.11), 11105: (17.03), 11106: (17.23), 11107: (17.03), 11108: (17.04), 11109: (17.04), 11110: (17.03), 11111: (17.03), 11112: (17.06), 11113: (17.01), 11114: (17.06), 11115: (17.09), 11116: (17.16), 11117: (17.07), 11118: (17.03), 11119: (17.05), 11120: (17.01), 11121: (17.03), 11122: (4.00), 11123: (4.00), 11124: (17.01), 11125: (17.06), 11126: (17.06), 11127: (17.03), 11128: (17.01), 11129: (17.01), 11130: (17.22), 11131: (17.04), 11132: (17.04), 11133: (17.01), 11134: (17.01), 11135: (17.07), 11136: (17.06), 11137: (17.02), 11138: (17.08), 11139: (17.30), 11140: (17.16), 11141: (17.18), 11142: (17.04), 11143: (17.12), 11144: (17.01), 11145: (17.01), 11146: (17.03), 11147: (4.00), 11148: (17.01), 11149: (17.05), 11150: (17.02), 11151: (17.12), 11152: (17.06), 11153: (17.10), 11154: (17.03), 11155: (4.00), 11156: (17.14), 11157: (17.03), 11158: (17.01), 11159: (17.01), 11160: (17.02), 11161: (17.03), 11162: (17.05), 11163: (17.04), 11164: (17.14), 11165: (17.02), 11166: (17.01), 11167: (17.01), 11168: (17.04), 11169: (17.02), 11170: (17.03), 11171: (17.04), 11172: (17.03), 11173: (17.01), 11174: (17.06), 11175: (17.05), 11176: (17.03), 11177: (17.10), 11178: (17.02), 11179: (17.02), 11180: (17.01), 11181: (17.01), 11182: (17.01), 11183: (17.03), 11184: (17.01), 11185: (17.01), 11186: (4.00), 11187: (4.00), 11188: (17.07), 11189: (17.01), 11190: (17.04), 11191: (4.00), 11192: (17.01), 11193: (17.08), 11194: (17.01), 11195: (17.12), 11196: (17.02), 11197: (17.02), 11198: (17.01), 11199: (17.01), 11200: (4.00), 11201: (17.10), 11202: (17.01), 11203: (4.00), 11204: (17.12), 11205: (17.02), 11206: (17.01), 11207: (17.02), 11208: (17.02), 11209: (17.02), 11210: (17.02), 11211: (17.05), 11212: (17.08), 11213: (17.14), 11214: (17.07), 11215: (17.02), 11216: (17.08), 11217: (17.04), 11218: (17.13), 11219: (17.04), 11220: (17.04), 11221: (17.08), 11222: (17.04), 11223: (17.01), 11224: (17.02), 11225: (17.03), 11226: (17.05), 11227: (17.02), 11228: (17.10), 11229: (17.01), 11230: (4.00), 11231: (4.00), 11232: (4.00), 11233: (4.00), 11234: (4.00), 11235: (4.00), 11236: (4.00), 11237: (4.00), 11238: (17.03), 11239: (17.02), 11240: (17.05), 11241: (17.01), 11242: (17.25), 11243: (17.01), 11244: (17.11), 11245: (17.03), 11246: (17.03), 11247: (17.01), 11248: (17.01), 11249: (17.01), 11250: (17.06), 11251: (17.11), 11252: (4.00), 11253: (17.04), 11254: (17.01), 11255: (17.05), 11256: (17.01), 11257: (4.00), 11258: (17.01), 11259: (4.00), 11260: (4.00), 11261: (17.01), 11262: (4.00), 11263: (4.00), 11264: (17.03), 11265: (17.02), 11266: (17.02), 11267: (17.04), 11268: (17.01), 11269: (17.01), 11270: (17.02), 11271: (17.02), 11272: (4.00), 11273: (17.01), 11274: (4.00), 11275: (17.01), 11276: (4.00), 11277: (17.01), 11278: (17.01), 11279: (17.19), 11280: (17.02), 11281: (17.01), 11282: (17.05), 11283: (17.09), 11284: (17.02), 11285: (17.04), 11286: (17.15), 11287: (4.00), 11288: (4.00), 11289: (17.02), 11290: (17.01), 11291: (17.26), 11292: (17.02), 11293: (4.00), 11294: (17.07), 11295: (17.07), 11296: (17.02), 11297: (17.01), 11298: (17.01), 11299: (17.02), 11300: (17.04), 11301: (17.13), 11302: (4.00), 11303: (17.22), 11304: (17.08), 11305: (17.05), 11306: (17.05), 11307: (17.01), 11308: (17.01), 11309: (17.04), 11310: (17.03), 11311: (17.05), 11312: (17.02), 11313: (17.03), 11314: (17.02), 11315: (17.01), 11316: (17.01), 11317: (4.00), 11318: (17.09), 11319: (17.01), 11320: (4.00), 11321: (4.00), 11322: (4.00), 11323: (4.00), 11324: (4.00), 11325: (4.00), 11326: (4.00), 11327: (4.00), 11328: (17.02), 11329: (4.00), 11330: (17.01), 11331: (17.01), 11332: (17.07), 11333: (17.01), 11334: (17.04), 11335: (17.08), 11336: (17.01), 11337: (4.00), 11338: (4.00), 11339: (4.00), 11340: (17.01), 11341: (17.05), 11342: (4.00), 11343: (17.07), 11344: (17.01), 11345: (17.11), 11346: (4.00), 11347: (17.01), 11348: (4.00), 11349: (4.00), 11350: (4.00), 11351: (4.00), 11352: (4.00), 11353: (4.00), 11354: (17.09), 11355: (17.03), 11356: (17.01), 11357: (17.06), 11358: (17.15), 11359: (4.00), 11360: (17.01), 11361: (17.02), 11362: (17.02), 11363: (17.11), 11364: (17.01), 11365: (17.03), 11366: (17.03), 11367: (17.06), 11368: (17.16), 11369: (17.03), 11370: (17.06), 11371: (17.07), 11372: (17.02), 11373: (17.01), 11374: (17.01), 11375: (4.00), 11376: (4.00), 11377: (4.00), 11378: (4.00), 11379: (4.00), 11380: (4.00), 11381: (17.14), 11382: (17.04), 11383: (17.09), 11384: (17.06), 11385: (17.01), 11386: (17.01), 11387: (17.01), 11388: (17.01), 11389: (4.00), 11390: (17.09), 11391: (17.04), 11392: (17.03), 11393: (17.12), 11394: (17.01), 11395: (17.04), 11396: (17.02), 11397: (17.04), 11398: (17.12), 11399: (17.02), 11400: (17.01), 11401: (4.00), 11402: (4.00), 11403: (4.00), 11404: (4.00), 11405: (4.00), 11406: (4.00), 11407: (4.00), 11408: (17.13), 11409: (4.00), 11410: (4.00), 11411: (4.00), 11412: (4.00), 11413: (4.00), 11414: (4.00), 11415: (4.00), 11416: (4.00), 11417: (17.01), 11418: (4.00), 11419: (17.09), 11420: (17.01), 11421: (17.02), 11422: (4.00), 11423: (17.01), 11424: (17.02), 11425: (4.00), 11426: (4.00), 11427: (4.00), 11428: (4.00), 11429: (4.00), 11430: (17.01), 11431: (17.03), 11432: (17.01), 11433: (17.18), 11434: (17.01), 11435: (17.03), 11436: (17.01), 11437: (17.01), 11438: (4.00), 11439: (17.01), 11440: (17.05), 11441: (17.03), 11442: (17.01), 11443: (17.01), 11444: (17.06), 11445: (17.04), 11446: (17.02), 11447: (17.04), 11448: (17.02), 11449: (17.01), 11450: (4.00), 11451: (4.00), 11452: (4.00), 11453: (17.11), 11454: (17.07), 11455: (17.02), 11456: (17.08), 11457: (17.01), 11458: (17.01), 11459: (17.01), 11460: (4.00), 11461: (17.02), 11462: (17.09), 11463: (17.04), 11464: (17.02), 11465: (17.01), 11466: (4.00), 11467: (4.00), 11468: (4.00), 11469: (17.01), 11470: (17.03), 11471: (17.37), 11472: (17.03), 11473: (4.00), 11474: (17.05), 11475: (17.04), 11476: (17.01), 11477: (17.01), 11478: (17.02), 11479: (17.01), 11480: (17.10), 11481: (4.00), 11482: (4.00), 11483: (17.02), 11484: (4.00), 11485: (17.01), 11486: (17.01), 11487: (4.00), 11488: (4.00), 11489: (17.01), 11490: (17.05), 11491: (17.01), 11492: (17.05), 11493: (4.00), 11494: (17.02), 11495: (17.01), 11496: (17.01), 11497: (17.01), 11498: (17.07), 11499: (17.01), 11500: (4.00), 11501: (4.00), 11502: (4.00), 11503: (4.00), 11504: (4.00), 11505: (4.00), 11506: (4.00), 11507: (4.00), 11508: (17.01), 11509: (4.00), 11510: (17.01), 11511: (17.01), 11512: (17.07), 11513: (17.02), 11514: (17.01), 11515: (17.08), 11516: (4.00), 11517: (4.00), 11518: (4.00), 11519: (17.01), 11520: (17.03), 11521: (17.03), 11522: (17.04), 11523: (17.11), 11524: (17.01), 11525: (17.07), 11526: (4.00), 11527: (4.00), 11528: (17.01), 11529: (17.03), 11530: (17.02), 11531: (17.07), 11532: (17.02), 11533: (17.02), 11534: (17.25), 11535: (17.06), 11536: (17.06), 11537: (17.09), 11538: (17.01), 11539: (17.01), 11540: (17.01), 11541: (17.01), 11542: (17.06), 11543: (17.16), 11544: (17.03), 11545: (17.03), 11546: (17.04), 11547: (17.03), 11548: (17.01), 11549: (17.04), 11550: (17.04), 11551: (17.07), 11552: (17.08), 11553: (17.02), 11554: (4.00), 11555: (4.00), 11556: (4.00), 11557: (4.00), 11558: (4.00), 11559: (4.00), 11560: (17.03), 11561: (17.23), 11562: (17.04), 11563: (17.01), 11564: (17.14), 11565: (17.01), 11566: (17.02), 11567: (17.01), 11568: (4.00), 11569: (17.02), 11570: (17.07), 11571: (17.09), 11572: (17.02), 11573: (4.00), 11574: (17.01), 11575: (17.03), 11576: (17.01), 11577: (4.00), 11578: (17.03), 11579: (17.05), 11580: (17.01), 11581: (4.00), 11582: (4.00), 11583: (4.00), 11584: (4.00), 11585: (4.00), 11586: (4.00), 11587: (4.00), 11588: (17.01), 11589: (17.07), 11590: (4.00), 11591: (4.00), 11592: (4.00), 11593: (4.00), 11594: (4.00), 11595: (4.00), 11596: (4.00), 11597: (4.00), 11598: (4.00), 11599: (4.00), 11600: (4.00), 11601: (4.00), 11602: (4.00), 11603: (4.00), 11604: (4.00), 11605: (4.00), 11606: (4.00), 11607: (4.00), 11608: (17.25), 11609: (17.01), 11610: (4.00), 11611: (4.00), 11612: (4.00), 11613: (4.00), 11614: (4.00), 11615: (4.00), 11616: (4.00), 11617: (4.00), 11618: (4.00), 11619: (4.00), 11620: (4.00), 11621: (4.00), 11622: (4.00), 11623: (4.00), 11624: (4.00), 11625: (4.00), 11626: (4.00), 11627: (17.11), 11628: (4.00), 11629: (4.00), 11630: (4.00), 11631: (4.00), 11632: (4.00), 11633: (4.00), 11634: (4.00), 11635: (4.00), 11636: (17.04), 11637: (17.01), 11638: (4.00), 11639: (4.00), 11640: (4.00), 11641: (4.00), 11642: (4.00), 11643: (4.00), 11644: (4.00), 11645: (4.00), 11646: (17.14), 11647: (4.00), 11648: (4.00), 11649: (4.00), 11650: (4.00), 11651: (4.00), 11652: (4.00), 11653: (4.00), 11654: (4.00), 11655: (17.07), 11656: (4.00), 11657: (4.00), 11658: (4.00), 11659: (4.00), 11660: (4.00), 11661: (4.00), 11662: (4.00), 11663: (4.00), 11664: (17.02), 11665: (17.09), 11666: (4.00), 11667: (4.00), 11668: (4.00), 11669: (4.00), 11670: (17.01), 11671: (17.02), 11672: (4.00), 11673: (4.00), 11674: (4.00), 11675: (4.00), 11676: (4.00), 11677: (17.04), 11678: (4.00), 11679: (4.00), 11680: (4.00), 11681: (4.00), 11682: (4.00), 11683: (17.05), 11684: (17.02), 11685: (4.00), 11686: (4.00), 11687: (4.00), 11688: (4.00), 11689: (17.13), 11690: (4.00), 11691: (4.00), 11692: (4.00), 11693: (4.00), 11694: (4.00), 11695: (17.01), 11696: (17.01), 11697: (4.00), 11698: (4.00), 11699: (4.00), 11700: (4.00), 11701: (4.00), 11702: (17.09), 11703: (17.01), 11704: (4.00), 11705: (4.00), 11706: (17.01), 11707: (17.06), 11708: (4.00), 11709: (4.00), 11710: (4.00), 11711: (17.03), 11712: (4.00), 11713: (4.00), 11714: (4.00), 11715: (4.00), 11716: (17.02), 11717: (4.00), 11718: (4.00), 11719: (4.00), 11720: (17.01), 11721: (17.04), 11722: (4.00), 11723: (4.00), 11724: (4.00), 11725: (17.02), 11726: (4.00), 11727: (4.00), 11728: (4.00), 11729: (4.00), 11730: (17.01), 11731: (4.00), 11732: (4.00), 11733: (4.00), 11734: (17.01), 11735: (4.00), 11736: (4.00), 11737: (4.00), 11738: (4.00), 11739: (17.08), 11740: (4.00), 11741: (4.00), 11742: (17.01), 11743: (17.05), 11744: (4.00), 11745: (4.00), 11746: (4.00), 11747: (17.03), 11748: (4.00), 11749: (4.00), 11750: (4.00), 11751: (17.12), 11752: (4.00), 11753: (4.00), 11754: (4.00), 11755: (17.03), 11756: (17.15), 11757: (4.00), 11758: (17.02), 11759: (17.02), 11760: (17.20), 11761: (4.00), 11762: (4.00), 11763: (17.01), 11764: (17.13), 11765: (17.02), 11766: (4.00), 11767: (17.07), 11768: (17.05), 11769: (17.14), 11770: (4.00), 11771: (17.17), 11772: (17.02), 11773: (17.05), 11774: (17.05), 11775: (17.02), 11776: (17.13), 11777: (17.01), 11778: (17.01), 11779: (4.00), 11780: (4.00), 11781: (4.00), 11782: (4.00), 11783: (4.00), 11784: (4.00), 11785: (4.00), 11786: (4.00), 11787: (4.00), 11788: (4.00), 11789: (4.00), 11790: (4.00), 11791: (4.00), 11792: (4.00), 11793: (4.00), 11794: (4.00), 11795: (4.00), 11796: (4.00), 11797: (4.00), 11798: (4.00), 11799: (4.00), 11800: (4.00), 11801: (4.00), 11802: (4.00), 11803: (4.00), 11804: (4.00), 11805: (4.00), 11806: (4.00), 11807: (4.00), 11808: (4.00), 11809: (4.00), 11810: (4.00), 11811: (4.00), 11812: (4.00), 11813: (4.00), 11814: (4.00), 11815: (4.00), 11816: (4.00), 11817: (4.00), 11818: (4.00), 11819: (4.00), 11820: (4.00), 11821: (4.00), 11822: (4.00), 11823: (4.00), 11824: (4.00), 11825: (4.00), 11826: (4.00), 11827: (4.00), 11828: (4.00), 11829: (4.00), 11830: (4.00), 11831: (4.00), 11832: (4.00), 11833: (4.00), 11834: (4.00), 11835: (4.00), 11836: (4.00), 11837: (4.00), 11838: (4.00), 11839: (4.00), 11840: (4.00), 11841: (4.00), 11842: (4.00), 11843: (4.00), 11844: (4.00), 11845: (4.00), 11846: (4.00), 11847: (4.00), 11848: (4.00), 11849: (4.00), 11850: (4.00), 11851: (4.00), 11852: (4.00), 11853: (4.00), 11854: (4.00), 11855: (4.00), 11856: (4.00), 11857: (4.00), 11858: (4.00), 11859: (4.00), 11860: (4.00), 11861: (4.00), 11862: (4.00), 11863: (4.00), 11864: (4.00), 11865: (4.00), 11866: (4.00), 11867: (4.00), 11868: (4.00), 11869: (4.00), 11870: (4.00), 11871: (4.00), 11872: (4.00), 11873: (4.00), 11874: (4.00), 11875: (4.00), 11876: (4.00), 11877: (4.00), 11878: (4.00), 11879: (4.00), 11880: (4.00), 11881: (4.00), 11882: (4.00), 11883: (4.00), 11884: (4.00), 11885: (4.00), 11886: (4.00), 11887: (4.00), 11888: (4.00), 11889: (4.00), 11890: (4.00), 11891: (4.00), 11892: (4.00), 11893: (4.00), 11894: (4.00), 11895: (4.00), 11896: (4.00), 11897: (4.00), 11898: (4.00), 11899: (4.00), 11900: (4.00), 11901: (4.00), 11902: (4.00), 11903: (4.00), 11904: (4.00), 11905: (4.00), 11906: (4.00), 11907: (4.00), 11908: (4.00), 11909: (4.00), 11910: (4.00), 11911: (4.00), 11912: (4.00), 11913: (4.00), 11914: (4.00), 11915: (4.00), 11916: (4.00), 11917: (4.00), 11918: (4.00), 11919: (4.00), 11920: (4.00), 11921: (4.00), 11922: (4.00), 11923: (4.00), 11924: (4.00), 11925: (4.00), 11926: (4.00), 11927: (4.00), 11928: (4.00), 11929: (4.00), 11930: (4.00), 11931: (4.00), 11932: (4.00), 11933: (4.00), 11934: (4.00), 11935: (4.00), 11936: (4.00), 11937: (4.00), 11938: (4.00), 11939: (4.00), 11940: (4.00), 11941: (4.00), 11942: (4.00), 11943: (4.00), 11944: (4.00), 11945: (4.00), 11946: (4.00), 11947: (4.00), 11948: (4.00), 11949: (4.00), 11950: (4.00), 11951: (4.00), 11952: (4.00), 11953: (4.00), 11954: (4.00), 11955: (4.00), 11956: (4.00), 11957: (4.00), 11958: (4.00), 11959: (4.00), 11960: (4.00), 11961: (4.00), 11962: (4.00), 11963: (4.00), 11964: (4.00), 11965: (4.00), 11966: (4.00), 11967: (4.00), 11968: (4.00), 11969: (4.00), 11970: (4.00), 11971: (4.00), 11972: (4.00), 11973: (4.00), 11974: (4.00), 11975: (4.00), 11976: (4.00), 11977: (4.00), 11978: (4.00), 11979: (4.00), 11980: (4.00), 11981: (4.00), 11982: (4.00), 11983: (4.00), 11984: (4.00), 11985: (4.00), 11986: (4.00), 11987: (4.00), 11988: (4.00), 11989: (4.00), 11990: (4.00), 11991: (4.00), 11992: (4.00), 11993: (4.00), 11994: (4.00), 11995: (4.00), 11996: (4.00), 11997: (4.00), 11998: (4.00), 11999: (4.00), 12000: (4.00), 12001: (4.00), 12002: (4.00), 12003: (4.00), 12004: (4.00), 12005: (4.00), 12006: (4.00), 12007: (4.00), 12008: (4.00), 12009: (4.00), 12010: (4.00), 12011: (4.00), 12012: (4.00), 12013: (4.00), 12014: (4.00), 12015: (4.00), 12016: (4.00), 12017: (4.00), 12018: (4.00), 12019: (4.00), 12020: (4.00), 12021: (4.00), 12022: (4.00), 12023: (4.00), 12024: (4.00), 12025: (4.00), 12026: (4.00), 12027: (4.00), 12028: (4.00), 12029: (4.00), 12030: (4.00), 12031: (4.00), 12032: (4.00), 12033: (4.00), 12034: (4.00), 12035: (4.00), 12036: (4.00), 12037: (4.00), 12038: (4.00), 12039: (4.00), 12040: (4.00), 12041: (4.00), 12042: (4.00), 12043: (4.00), 12044: (4.00), 12045: (4.00), 12046: (4.00), 12047: (4.00), 12048: (4.00), 12049: (4.00), 12050: (4.00), 12051: (4.00), 12052: (4.00), 12053: (4.00), 12054: (4.00), 12055: (4.00), 12056: (4.00), 12057: (4.00), 12058: (4.00), 12059: (4.00), 12060: (4.00), 12061: (4.00), 12062: (4.00), 12063: (4.00), 12064: (4.00), 12065: (4.00), 12066: (4.00), 12067: (4.00), 12068: (4.00), 12069: (4.00), 12070: (4.00), 12071: (4.00), 12072: (4.00), 12073: (4.00), 12074: (4.00), 12075: (4.00), 12076: (4.00), 12077: (4.00), 12078: (4.00), 12079: (4.00), 12080: (4.00), 12081: (4.00), 12082: (4.00), 12083: (4.00), 12084: (4.00), 12085: (4.00), 12086: (4.00), 12087: (4.00), 12088: (4.00), 12089: (4.00), 12090: (4.00), 12091: (4.00), 12092: (4.00), 12093: (4.00), 12094: (4.00), 12095: (4.00), 12096: (4.00), 12097: (4.00), 12098: (4.00), 12099: (4.00), 12100: (4.00), 12101: (4.00), 12102: (4.00), 12103: (4.00), 12104: (4.00), 12105: (4.00), 12106: (4.00), 12107: (4.00), 12108: (4.00), 12109: (4.00), 12110: (4.00), 12111: (4.00), 12112: (4.00), 12113: (4.00), 12114: (4.00), 12115: (4.00), 12116: (4.00), 12117: (4.00), 12118: (4.00), 12119: (4.00), 12120: (4.00), 12121: (4.00), 12122: (4.00), 12123: (4.00), 12124: (4.00), 12125: (4.00), 12126: (4.00), 12127: (4.00), 12128: (4.00), 12129: (4.00), 12130: (4.00), 12131: (4.00), 12132: (4.00), 12133: (4.00), 12134: (4.00), 12135: (4.00), 12136: (4.00), 12137: (4.00), 12138: (4.00), 12139: (4.00), 12140: (4.00), 12141: (4.00), 12142: (4.00), 12143: (4.00), 12144: (4.00), 12145: (4.00), 12146: (4.00), 12147: (4.00), 12148: (4.00), 12149: (4.00), 12150: (4.00), 12151: (4.00), 12152: (4.00), 12153: (4.00), 12154: (4.00), 12155: (4.00), 12156: (4.00), 12157: (4.00), 12158: (4.00), 12159: (4.00), 12160: (4.00), 12161: (4.00), 12162: (4.00), 12163: (4.00), 12164: (4.00), 12165: (4.00), 12166: (4.00), 12167: (4.00), 12168: (4.00), 12169: (4.00), 12170: (4.00), 12171: (4.00), 12172: (4.00), 12173: (4.00), 12174: (4.00), 12175: (4.00), 12176: (4.00), 12177: (4.00), 12178: (4.00), 12179: (4.00), 12180: (4.00), 12181: (4.00), 12182: (4.00), 12183: (4.00), 12184: (4.00), 12185: (4.00), 12186: (4.00), 12187: (4.00), 12188: (4.00), 12189: (4.00), 12190: (4.00), 12191: (4.00), 12192: (4.00), 12193: (4.00), 12194: (4.00), 12195: (4.00), 12196: (4.00), 12197: (4.00), 12198: (4.00), 12199: (4.00), 12200: (4.00), 12201: (4.00), 12202: (4.00), 12203: (4.00), 12204: (4.00), 12205: (4.00), 12206: (4.00), 12207: (4.00), 12208: (4.00), 12209: (4.00), 12210: (4.00), 12211: (4.00), 12212: (4.00), 12213: (4.00), 12214: (4.00), 12215: (4.00), 12216: (4.00), 12217: (4.00), 12218: (4.00), 12219: (4.00), 12220: (4.00), 12221: (4.00), 12222: (4.00), 12223: (4.00), 12224: (4.00), 12225: (4.00), 12226: (4.00), 12227: (4.00), 12228: (4.00), 12229: (4.00), 12230: (4.00), 12231: (4.00), 12232: (4.00), 12233: (4.00), 12234: (4.00), 12235: (4.00), 12236: (4.00), 12237: (4.00), 12238: (4.00), 12239: (4.00), 12240: (4.00), 12241: (4.00), 12242: (4.00), 12243: (4.00), 12244: (4.00), 12245: (4.00), 12246: (4.00), 12247: (4.00), 12248: (4.00), 12249: (4.00), 12250: (4.00), 12251: (4.00), 12252: (4.00), 12253: (4.00), 12254: (4.00), 12255: (4.00), 12256: (4.00), 12257: (4.00), 12258: (4.00), 12259: (4.00), 12260: (4.00), 12261: (4.00), 12262: (4.00), 12263: (4.00), 12264: (4.00), 12265: (4.00), 12266: (4.00), 12267: (4.00), 12268: (4.00), 12269: (4.00), 12270: (4.00), 12271: (4.00), 12272: (4.00), 12273: (4.00), 12274: (4.00), 12275: (4.00), 12276: (4.00), 12277: (4.00), 12278: (4.00), 12279: (4.00), 12280: (4.00), 12281: (4.00), 12282: (4.00), 12283: (4.00), 12284: (4.00), 12285: (4.00), 12286: (4.00), 12287: (4.00), 12288: (4.00), 12289: (4.00), 12290: (4.00), 12291: (4.00), 12292: (4.00), 12293: (4.00), 12294: (4.00), 12295: (4.00), 12296: (4.00), 12297: (4.00), 12298: (4.00), 12299: (4.00), 12300: (4.00), 12301: (4.00), 12302: (4.00), 12303: (4.00), 12304: (4.00), 12305: (4.00), 12306: (4.00), 12307: (4.00), 12308: (4.00), 12309: (4.00), 12310: (4.00), 12311: (4.00), 12312: (4.00), 12313: (4.00), 12314: (4.00), 12315: (4.00), 12316: (4.00), 12317: (4.00), 12318: (4.00), 12319: (4.00), 12320: (4.00), 12321: (4.00), 12322: (4.00), 12323: (4.00), 12324: (4.00), 12325: (4.00), 12326: (4.00), 12327: (4.00), 12328: (4.00), 12329: (4.00), 12330: (4.00), 12331: (4.00), 12332: (4.00), 12333: (4.00), 12334: (4.00), 12335: (4.00), 12336: (4.00), 12337: (4.00), 12338: (4.00), 12339: (4.00), 12340: (4.00), 12341: (4.00), 12342: (4.00), 12343: (4.00), 12344: (4.00), 12345: (4.00), 12346: (4.00), 12347: (4.00), 12348: (4.00), 12349: (4.00), 12350: (4.00), 12351: (4.00), 12352: (4.00), 12353: (4.00), 12354: (4.00), 12355: (4.00), 12356: (4.00), 12357: (4.00), 12358: (4.00), 12359: (4.00), 12360: (4.00), 12361: (4.00), 12362: (4.00), 12363: (4.00), 12364: (4.00), 12365: (4.00), 12366: (4.00), 12367: (4.00), 12368: (4.00), 12369: (4.00), 12370: (4.00), 12371: (4.00), 12372: (4.00), 12373: (4.00), 12374: (4.00), 12375: (4.00), 12376: (4.00), 12377: (4.00), 12378: (4.00), 12379: (4.00), 12380: (4.00), 12381: (4.00), 12382: (4.00), 12383: (4.00), 12384: (4.00), 12385: (4.00), 12386: (4.00), 12387: (4.00), 12388: (4.00), 12389: (4.00), 12390: (4.00), 12391: (4.00), 12392: (4.00), 12393: (4.00), 12394: (4.00), 12395: (4.00), 12396: (4.00), 12397: (4.00), 12398: (4.00), 12399: (4.00), 12400: (4.00), 12401: (4.00), 12402: (4.00), 12403: (4.00), 12404: (4.00), 12405: (4.00), 12406: (4.00), 12407: (4.00), 12408: (4.00), 12409: (4.00), 12410: (4.00), 12411: (4.00), 12412: (4.00), 12413: (4.00), 12414: (4.00), 12415: (4.00), 12416: (4.00), 12417: (4.00), 12418: (4.00), 12419: (4.00), 12420: (4.00), 12421: (4.00), 12422: (4.00), 12423: (4.00), 12424: (4.00), 12425: (4.00), 12426: (4.00), 12427: (4.00), 12428: (4.00), 12429: (4.00), 12430: (4.00), 12431: (4.00), 12432: (4.00), 12433: (4.00), 12434: (4.00), 12435: (4.00), 12436: (4.00), 12437: (4.00), 12438: (4.00), 12439: (4.00), 12440: (4.00), 12441: (4.00), 12442: (4.00), 12443: (4.00), 12444: (4.00), 12445: (4.00), 12446: (4.00), 12447: (4.00), 12448: (4.00), 12449: (4.00), 12450: (4.00), 12451: (4.00), 12452: (4.00), 12453: (4.00), 12454: (4.00), 12455: (4.00), 12456: (4.00), 12457: (4.00), 12458: (4.00), 12459: (4.00), 12460: (4.00), 12461: (4.00), 12462: (4.00), 12463: (4.00), 12464: (4.00), 12465: (4.00), 12466: (4.00), 12467: (4.00), 12468: (4.00), 12469: (4.00), 12470: (4.00), 12471: (4.00), 12472: (4.00), 12473: (4.00), 12474: (4.00), 12475: (4.00), 12476: (4.00), 12477: (4.00), 12478: (4.00), 12479: (4.00), 12480: (4.00), 12481: (4.00), 12482: (4.00), 12483: (4.00), 12484: (4.00), 12485: (4.00), 12486: (4.00), 12487: (4.00), 12488: (4.00), 12489: (4.00), 12490: (4.00), 12491: (4.00), 12492: (4.00), 12493: (4.00), 12494: (4.00), 12495: (4.00), 12496: (4.00), 12497: (4.00), 12498: (4.00), 12499: (4.00), 12500: (4.00), 12501: (4.00), 12502: (4.00), 12503: (4.00), 12504: (4.00), 12505: (4.00), 12506: (4.00), 12507: (4.00), 12508: (4.00), 12509: (4.00), 12510: (4.00), 12511: (4.00), 12512: (4.00), 12513: (4.00), 12514: (4.00), 12515: (4.00), 12516: (4.00), 12517: (4.00), 12518: (4.00), 12519: (4.00), 12520: (4.00), 12521: (4.00), 12522: (4.00), 12523: (4.00), 12524: (4.00), 12525: (4.00), 12526: (4.00), 12527: (4.00), 12528: (4.00), 12529: (4.00), 12530: (4.00), 12531: (4.00), 12532: (4.00), 12533: (4.00), 12534: (4.00), 12535: (4.00), 12536: (4.00), 12537: (4.00), 12538: (4.00), 12539: (4.00), 12540: (4.00), 12541: (4.00), 12542: (4.00), 12543: (4.00), 12544: (4.00), 12545: (4.00), 12546: (4.00), 12547: (4.00), 12548: (4.00), 12549: (4.00), 12550: (4.00), 12551: (4.00), 12552: (4.00), 12553: (4.00), 12554: (4.00), 12555: (4.00), 12556: (4.00), 12557: (4.00), 12558: (4.00), 12559: (4.00), 12560: (4.00), 12561: (4.00), 12562: (4.00), 12563: (4.00), 12564: (4.00), 12565: (4.00), 12566: (4.00), 12567: (4.00), 12568: (4.00), 12569: (4.00), 12570: (4.00), 12571: (4.00), 12572: (4.00), 12573: (4.00), 12574: (4.00), 12575: (4.00), 12576: (4.00), 12577: (4.00), 12578: (4.00), 12579: (4.00), 12580: (4.00), 12581: (4.00), 12582: (4.00), 12583: (4.00), 12584: (4.00), 12585: (4.00), 12586: (4.00), 12587: (4.00), 12588: (4.00), 12589: (4.00), 12590: (4.00), 12591: (4.00), 12592: (4.00), 12593: (4.00), 12594: (4.00), 12595: (4.00), 12596: (4.00), 12597: (4.00), 12598: (4.00), 12599: (4.00), 12600: (4.00), 12601: (4.00), 12602: (4.00), 12603: (4.00), 12604: (4.00), 12605: (4.00), 12606: (4.00), 12607: (4.00), 12608: (4.00), 12609: (4.00), 12610: (17.01), 12611: (17.14), 12612: (17.08), 12613: (17.02), 12614: (17.02), 12615: (17.01), 12616: (17.01), 12617: (17.01), 12618: (17.04), 12619: (17.05), 12620: (17.05), 12621: (17.02), 12622: (17.04), 12623: (17.02), 12624: (17.01), 12625: (17.36), 12626: (17.05), 12627: (17.08), 12628: (17.03), 12629: (4.00), 12630: (4.00), 12631: (17.01), 12632: (4.00), 12633: (4.00), 12634: (17.03), 12635: (17.05), 12636: (17.02), 12637: (17.04), 12638: (17.02), 12639: (17.01), 12640: (4.00), 12641: (17.01), 12642: (17.03), 12643: (17.01), 12644: (17.01), 12645: (17.02), 12646: (17.01), 12647: (4.00), 12648: (17.03), 12649: (17.04), 12650: (17.03), 12651: (17.02), 12652: (17.03), 12653: (17.01), 12654: (17.03), 12655: (17.03), 12656: (17.03), 12657: (17.19), 12658: (17.01), 12659: (17.02), 12660: (17.01), 12661: (4.00), 12662: (17.01), 12663: (17.03), 12664: (17.01), 12665: (17.01), 12666: (17.04), 12667: (17.05), 12668: (4.00), 12669: (4.00), 12670: (17.01), 12671: (17.16), 12672: (17.01), 12673: (17.02), 12674: (17.01), 12675: (4.00), 12676: (4.00), 12677: (4.00), 12678: (17.01), 12679: (4.00), 12680: (17.02), 12681: (17.01), 12682: (17.01), 12683: (17.01), 12684: (17.02), 12685: (17.01), 12686: (17.01), 12687: (17.05), 12688: (17.02), 12689: (17.02), 12690: (4.00), 12691: (4.00), 12692: (4.00), 12693: (4.00), 12694: (17.03), 12695: (17.04), 12696: (17.01), 12697: (4.00), 12698: (4.00), 12699: (4.00), 12700: (4.00), 12701: (4.00), 12702: (4.00), 12703: (17.34), 12704: (17.02), 12705: (4.00), 12706: (4.00), 12707: (17.01), 12708: (17.05), 12709: (17.04), 12710: (17.01), 12711: (17.01), 12712: (17.02), 12713: (17.01), 12714: (17.01), 12715: (4.00), 12716: (17.01), 12717: (17.28), 12718: (17.01), 12719: (17.06), 12720: (17.01), 12721: (4.00), 12722: (4.00), 12723: (4.00), 12724: (4.00), 12725: (4.00), 12726: (17.03), 12727: (17.01), 12728: (17.02), 12729: (4.00), 12730: (4.00), 12731: (17.03), 12732: (17.02), 12733: (4.00), 12734: (17.02), 12735: (17.02), 12736: (17.01), 12737: (17.01), 12738: (4.00), 12739: (17.02), 12740: (4.00), 12741: (4.00), 12742: (17.02), 12743: (17.01), 12744: (4.00), 12745: (17.01), 12746: (4.00), 12747: (4.00), 12748: (17.19), 12749: (17.01), 12750: (17.02), 12751: (17.01), 12752: (4.00), 12753: (4.00), 12754: (17.02), 12755: (17.04), 12756: (17.01), 12757: (17.03), 12758: (17.02), 12759: (4.00), 12760: (17.01), 12761: (17.01), 12762: (17.13), 12763: (17.02), 12764: (17.01), 12765: (4.00), 12766: (4.00), 12767: (4.00), 12768: (4.00), 12769: (4.00), 12770: (4.00), 12771: (17.01), 12772: (17.02), 12773: (4.00), 12774: (4.00), 12775: (4.00), 12776: (4.00), 12777: (4.00), 12778: (4.00), 12779: (4.00), 12780: (4.00), 12781: (4.00), 12782: (4.00), 12783: (4.00), 12784: (17.01), 12785: (17.01), 12786: (4.00), 12787: (4.00), 12788: (4.00), 12789: (17.02), 12790: (4.00), 12791: (4.00), 12792: (4.00), 12793: (4.00), 12794: (17.02), 12795: (4.00), 12796: (17.01), 12797: (17.04), 12798: (17.03), 12799: (4.00), 12800: (4.00), 12801: (4.00), 12802: (4.00), 12803: (17.10), 12804: (17.01), 12805: (4.00), 12806: (4.00), 12807: (4.00), 12808: (4.00), 12809: (4.00), 12810: (17.02), 12811: (4.00), 12812: (17.01), 12813: (17.01), 12814: (4.00), 12815: (4.00), 12816: (17.01), 12817: (17.07), 12818: (17.01), 12819: (17.01), 12820: (4.00), 12821: (4.00), 12822: (4.00), 12823: (4.00), 12824: (4.00), 12825: (4.00), 12826: (17.01), 12827: (17.05), 12828: (17.01), 12829: (4.00), 12830: (4.00), 12831: (4.00), 12832: (4.00), 12833: (4.00), 12834: (4.00), 12835: (4.00), 12836: (4.00), 12837: (4.00), 12838: (17.01), 12839: (4.00), 12840: (17.06), 12841: (17.02), 12842: (17.01), 12843: (17.03), 12844: (4.00), 12845: (4.00), 12846: (4.00), 12847: (4.00), 12848: (4.00), 12849: (17.17), 12850: (17.04), 12851: (17.01), 12852: (4.00), 12853: (17.01), 12854: (17.01), 12855: (17.01), 12856: (17.01), 12857: (4.00), 12858: (17.06), 12859: (17.02), 12860: (17.01), 12861: (17.02), 12862: (17.01), 12863: (17.33), 12864: (17.01), 12865: (17.02), 12866: (17.01), 12867: (4.00), 12868: (4.00), 12869: (17.01), 12870: (4.00), 12871: (4.00), 12872: (17.01), 12873: (17.12), 12874: (17.05), 12875: (17.02), 12876: (17.04), 12877: (17.02), 12878: (17.02), 12879: (17.03), 12880: (17.01), 12881: (17.01), 12882: (17.01), 12883: (17.02), 12884: (17.01), 12885: (17.01), 12886: (17.02), 12887: (4.00), 12888: (4.00), 12889: (4.00), 12890: (4.00), 12891: (4.00), 12892: (4.00), 12893: (4.00), 12894: (4.00), 12895: (17.18), 12896: (17.02), 12897: (17.01), 12898: (4.00), 12899: (4.00), 12900: (4.00), 12901: (4.00), 12902: (17.01), 12903: (17.01), 12904: (17.02), 12905: (4.00), 12906: (17.01), 12907: (17.01), 12908: (17.13), 12909: (17.03), 12910: (17.02), 12911: (4.00), 12912: (17.01), 12913: (17.01), 12914: (17.01), 12915: (17.02), 12916: (4.00), 12917: (17.03), 12918: (17.02), 12919: (17.07), 12920: (4.00), 12921: (17.01), 12922: (4.00), 12923: (17.01), 12924: (4.00), 12925: (17.02), 12926: (17.01), 12927: (17.05), 12928: (4.00), 12929: (17.01), 12930: (4.00), 12931: (17.02), 12932: (4.00), 12933: (4.00), 12934: (4.00), 12935: (4.00), 12936: (4.00), 12937: (4.00), 12938: (4.00), 12939: (4.00), 12940: (17.05), 12941: (4.00), 12942: (4.00), 12943: (4.00), 12944: (4.00), 12945: (17.01), 12946: (4.00), 12947: (17.01), 12948: (17.01), 12949: (4.00), 12950: (4.00), 12951: (4.00), 12952: (4.00), 12953: (4.00), 12954: (17.16), 12955: (17.03), 12956: (17.01), 12957: (17.01), 12958: (4.00), 12959: (17.02), 12960: (17.02), 12961: (17.01), 12962: (4.00), 12963: (4.00), 12964: (4.00), 12965: (17.01), 12966: (17.01), 12967: (17.01), 12968: (17.01), 12969: (4.00), 12970: (4.00), 12971: (4.00), 12972: (17.01), 12973: (4.00), 12974: (4.00), 12975: (4.00), 12976: (4.00), 12977: (17.01), 12978: (17.01), 12979: (17.03), 12980: (17.01), 12981: (17.01), 12982: (4.00), 12983: (4.00), 12984: (4.00), 12985: (4.00), 12986: (17.01), 12987: (4.00), 12988: (4.00), 12989: (4.00), 12990: (4.00), 12991: (4.00), 12992: (4.00), 12993: (4.00), 12994: (4.00), 12995: (17.21), 12996: (17.04), 12997: (17.08), 12998: (17.02), 12999: (17.01), 13000: (17.02), 13001: (17.02), 13002: (17.01), 13003: (17.02), 13004: (17.01), 13005: (17.01), 13006: (4.00), 13007: (17.02), 13008: (17.01), 13009: (17.19), 13010: (4.00), 13011: (17.01), 13012: (17.01), 13013: (4.00), 13014: (17.01), 13015: (17.01), 13016: (4.00), 13017: (17.01), 13018: (17.02), 13019: (17.01), 13020: (17.02), 13021: (17.02), 13022: (17.01), 13023: (17.01), 13024: (4.00), 13025: (4.00), 13026: (4.00), 13027: (17.02), 13028: (17.01), 13029: (17.01), 13030: (4.00), 13031: (4.00), 13032: (17.01), 13033: (4.00), 13034: (4.00), 13035: (4.00), 13036: (4.00), 13037: (4.00), 13038: (4.00), 13039: (4.00), 13040: (4.00), 13041: (17.13), 13042: (17.01), 13043: (17.05), 13044: (17.01), 13045: (4.00), 13046: (4.00), 13047: (4.00), 13048: (17.01), 13049: (17.01), 13050: (17.02), 13051: (17.01), 13052: (17.01), 13053: (17.01), 13054: (17.01), 13055: (17.13), 13056: (17.02), 13057: (17.02), 13058: (17.02), 13059: (4.00), 13060: (17.01), 13061: (17.02), 13062: (17.01), 13063: (4.00), 13064: (17.01), 13065: (17.07), 13066: (17.04), 13067: (17.01), 13068: (17.02), 13069: (17.01), 13070: (17.01), 13071: (4.00), 13072: (4.00), 13073: (17.02), 13074: (17.04), 13075: (4.00), 13076: (17.03), 13077: (4.00), 13078: (17.02), 13079: (17.03), 13080: (17.04), 13081: (17.03), 13082: (4.00), 13083: (4.00), 13084: (4.00), 13085: (4.00), 13086: (4.00), 13087: (17.10), 13088: (17.01), 13089: (17.02), 13090: (17.01), 13091: (4.00), 13092: (4.00), 13093: (17.01), 13094: (17.01), 13095: (17.02), 13096: (17.02), 13097: (17.04), 13098: (17.01), 13099: (4.00), 13100: (17.01), 13101: (17.09), 13102: (17.03), 13103: (17.03), 13104: (4.00), 13105: (17.02), 13106: (17.01), 13107: (17.01), 13108: (4.00), 13109: (4.00), 13110: (17.03), 13111: (17.05), 13112: (17.01), 13113: (17.05), 13114: (17.01), 13115: (17.02), 13116: (4.00), 13117: (17.01), 13118: (17.01), 13119: (4.00), 13120: (17.01), 13121: (17.01), 13122: (17.01), 13123: (17.03), 13124: (17.01), 13125: (17.02), 13126: (17.01), 13127: (4.00), 13128: (17.01), 13129: (17.01), 13130: (4.00), 13131: (4.00), 13132: (17.09), 13133: (17.01), 13134: (17.01), 13135: (4.00), 13136: (4.00), 13137: (4.00), 13138: (4.00), 13139: (4.00), 13140: (4.00), 13141: (17.02), 13142: (17.02), 13143: (17.01), 13144: (17.01), 13145: (4.00), 13146: (17.04), 13147: (17.04), 13148: (17.01), 13149: (17.01), 13150: (17.01), 13151: (17.01), 13152: (4.00), 13153: (4.00), 13154: (4.00), 13155: (17.01), 13156: (17.02), 13157: (17.01), 13158: (17.04), 13159: (4.00), 13160: (17.01), 13161: (4.00), 13162: (17.01), 13163: (4.00), 13164: (4.00), 13165: (4.00), 13166: (4.00), 13167: (4.00), 13168: (4.00), 13169: (17.05), 13170: (17.02), 13171: (17.01), 13172: (4.00), 13173: (4.00), 13174: (4.00), 13175: (4.00), 13176: (4.00), 13177: (4.00), 13178: (17.04), 13179: (17.01), 13180: (17.01), 13181: (4.00), 13182: (17.01), 13183: (17.01), 13184: (4.00), 13185: (4.00), 13186: (4.00), 13187: (17.19), 13188: (17.02), 13189: (17.01), 13190: (4.00), 13191: (4.00), 13192: (4.00), 13193: (4.00), 13194: (17.01), 13195: (17.01), 13196: (17.01), 13197: (17.01), 13198: (4.00), 13199: (17.01), 13200: (17.01), 13201: (17.08), 13202: (17.03), 13203: (17.04), 13204: (17.01), 13205: (4.00), 13206: (17.01), 13207: (17.01), 13208: (4.00), 13209: (4.00), 13210: (17.01), 13211: (17.05), 13212: (17.01), 13213: (17.02), 13214: (4.00), 13215: (17.01), 13216: (4.00), 13217: (4.00), 13218: (17.01), 13219: (17.01), 13220: (17.01), 13221: (4.00), 13222: (17.01), 13223: (17.01), 13224: (17.04), 13225: (17.01), 13226: (17.12), 13227: (17.07), 13228: (4.00), 13229: (4.00), 13230: (4.00), 13231: (4.00), 13232: (4.00), 13233: (17.21), 13234: (17.01), 13235: (4.00), 13236: (4.00), 13237: (4.00), 13238: (4.00), 13239: (4.00), 13240: (4.00), 13241: (17.01), 13242: (17.01), 13243: (17.02), 13244: (17.01), 13245: (4.00), 13246: (4.00), 13247: (17.16), 13248: (17.01), 13249: (17.03), 13250: (17.01), 13251: (4.00), 13252: (4.00), 13253: (4.00), 13254: (4.00), 13255: (4.00), 13256: (17.04), 13257: (17.11), 13258: (17.01), 13259: (17.02), 13260: (17.02), 13261: (17.01), 13262: (4.00), 13263: (4.00), 13264: (4.00), 13265: (4.00), 13266: (17.01), 13267: (17.01), 13268: (17.02), 13269: (17.06), 13270: (4.00), 13271: (4.00), 13272: (17.01), 13273: (4.00), 13274: (17.01), 13275: (4.00), 13276: (4.00), 13277: (4.00), 13278: (17.02), 13279: (17.05), 13280: (17.01), 13281: (17.01), 13282: (4.00), 13283: (4.00), 13284: (4.00), 13285: (17.02), 13286: (17.02), 13287: (17.01), 13288: (17.01), 13289: (17.02), 13290: (4.00), 13291: (4.00), 13292: (17.04), 13293: (17.14), 13294: (17.01), 13295: (17.02), 13296: (17.02), 13297: (17.01), 13298: (17.01), 13299: (4.00), 13300: (4.00), 13301: (17.02), 13302: (17.07), 13303: (17.01), 13304: (17.03), 13305: (4.00), 13306: (17.01), 13307: (4.00), 13308: (4.00), 13309: (4.00), 13310: (17.05), 13311: (17.08), 13312: (4.00), 13313: (17.02), 13314: (17.01), 13315: (17.02), 13316: (4.00), 13317: (17.02), 13318: (17.02), 13319: (4.00), 13320: (4.00), 13321: (4.00), 13322: (4.00), 13323: (4.00), 13324: (17.08), 13325: (17.04), 13326: (4.00), 13327: (4.00), 13328: (4.00), 13329: (4.00), 13330: (4.00), 13331: (4.00), 13332: (4.00), 13333: (17.04), 13334: (4.00), 13335: (17.01), 13336: (4.00), 13337: (4.00), 13338: (17.17), 13339: (17.02), 13340: (17.03), 13341: (17.01), 13342: (4.00), 13343: (17.01), 13344: (4.00), 13345: (4.00), 13346: (4.00), 13347: (17.01), 13348: (17.01), 13349: (17.01), 13350: (17.01), 13351: (17.01), 13352: (4.00), 13353: (17.02), 13354: (17.01), 13355: (17.01), 13356: (17.01), 13357: (4.00), 13358: (4.00), 13359: (4.00), 13360: (17.02), 13361: (17.02), 13362: (4.00), 13363: (17.01), 13364: (17.01), 13365: (17.01), 13366: (17.01), 13367: (4.00), 13368: (4.00), 13369: (17.02), 13370: (17.11), 13371: (17.05), 13372: (17.04), 13373: (4.00), 13374: (17.06), 13375: (17.01), 13376: (17.08), 13377: (17.01), 13378: (17.06), 13379: (17.63), 13380: (17.07), 13381: (17.02), 13382: (17.06), 13383: (17.06), 13384: (17.01), 13385: (4.00), 13386: (17.01), 13387: (17.01), 13388: (17.04), 13389: (17.11), 13390: (17.09), 13391: (17.05), 13392: (17.32), 13393: (17.06), 13394: (17.01), 13395: (17.02), 13396: (17.01), 13397: (4.00), 13398: (4.00), 13399: (17.02), 13400: (4.00), 13401: (4.00), 13402: (17.02), 13403: (17.02), 13404: (17.03), 13405: (17.03), 13406: (4.00), 13407: (17.03), 13408: (17.03), 13409: (4.00), 13410: (4.00), 13411: (4.00), 13412: (4.00), 13413: (4.00), 13414: (17.01), 13415: (17.02), 13416: (17.08), 13417: (17.01), 13418: (4.00), 13419: (4.00), 13420: (17.01), 13421: (17.02), 13422: (17.01), 13423: (4.00), 13424: (4.00), 13425: (17.14), 13426: (17.01), 13427: (17.03), 13428: (17.03), 13429: (17.01), 13430: (17.07), 13431: (4.00), 13432: (4.00), 13433: (4.00), 13434: (4.00), 13435: (17.01), 13436: (17.07), 13437: (17.01), 13438: (17.02), 13439: (17.01), 13440: (17.01), 13441: (4.00), 13442: (4.00), 13443: (4.00), 13444: (4.00), 13445: (4.00), 13446: (4.00), 13447: (4.00), 13448: (17.05), 13449: (17.01), 13450: (17.09), 13451: (17.03), 13452: (17.04), 13453: (17.14), 13454: (17.04), 13455: (17.01), 13456: (4.00), 13457: (17.03), 13458: (17.01), 13459: (17.01), 13460: (17.03), 13461: (17.02), 13462: (17.04), 13463: (17.01), 13464: (17.01), 13465: (4.00), 13466: (17.01), 13467: (17.01), 13468: (4.00), 13469: (4.00), 13470: (17.01), 13471: (17.30), 13472: (4.00), 13473: (4.00), 13474: (17.01), 13475: (4.00), 13476: (4.00), 13477: (4.00), 13478: (4.00), 13479: (4.00), 13480: (17.05), 13481: (17.03), 13482: (4.00), 13483: (17.02), 13484: (4.00), 13485: (4.00), 13486: (4.00), 13487: (4.00), 13488: (4.00), 13489: (4.00), 13490: (4.00), 13491: (4.00), 13492: (4.00), 13493: (17.03), 13494: (17.18), 13495: (17.01), 13496: (17.04), 13497: (4.00), 13498: (4.00), 13499: (17.01), 13500: (4.00), 13501: (4.00), 13502: (4.00), 13503: (4.00), 13504: (4.00), 13505: (4.00), 13506: (4.00), 13507: (17.02), 13508: (4.00), 13509: (4.00), 13510: (4.00), 13511: (4.00), 13512: (4.00), 13513: (4.00), 13514: (4.00), 13515: (4.00), 13516: (17.07), 13517: (17.03), 13518: (4.00), 13519: (17.03), 13520: (4.00), 13521: (17.01), 13522: (4.00), 13523: (4.00), 13524: (4.00), 13525: (17.01), 13526: (17.06), 13527: (17.02), 13528: (4.00), 13529: (4.00), 13530: (4.00), 13531: (4.00), 13532: (4.00), 13533: (4.00), 13534: (4.00), 13535: (4.00), 13536: (4.00), 13537: (4.00), 13538: (4.00), 13539: (17.01), 13540: (17.01), 13541: (4.00), 13542: (4.00), 13543: (4.00), 13544: (4.00), 13545: (4.00), 13546: (4.00), 13547: (4.00), 13548: (4.00), 13549: (4.00), 13550: (4.00), 13551: (17.01), 13552: (17.01), 13553: (17.01), 13554: (17.01), 13555: (17.03), 13556: (17.01), 13557: (4.00), 13558: (4.00), 13559: (17.01), 13560: (17.01), 13561: (17.01), 13562: (17.51), 13563: (17.02), 13564: (4.00), 13565: (17.01), 13566: (17.01), 13567: (4.00), 13568: (17.01), 13569: (4.00), 13570: (4.00), 13571: (17.04), 13572: (17.06), 13573: (17.03), 13574: (17.06), 13575: (17.01), 13576: (17.05), 13577: (17.02), 13578: (4.00), 13579: (4.00), 13580: (4.00), 13581: (4.00), 13582: (4.00), 13583: (4.00), 13584: (17.01), 13585: (17.03), 13586: (17.03), 13587: (17.35), 13588: (17.04), 13589: (17.02), 13590: (17.03), 13591: (17.01), 13592: (4.00), 13593: (4.00), 13594: (17.02), 13595: (17.01), 13596: (4.00), 13597: (17.01), 13598: (17.03), 13599: (17.01), 13600: (17.01), 13601: (4.00), 13602: (4.00), 13603: (4.00), 13604: (4.00), 13605: (17.01), 13606: (17.02), 13607: (17.15), 13608: (17.04), 13609: (17.01), 13610: (17.02), 13611: (17.01), 13612: (17.02), 13613: (4.00), 13614: (4.00), 13615: (4.00), 13616: (17.01), 13617: (17.02), 13618: (17.01), 13619: (4.00), 13620: (17.01), 13621: (17.02), 13622: (17.03), 13623: (4.00), 13624: (4.00), 13625: (4.00), 13626: (4.00), 13627: (4.00), 13628: (4.00), 13629: (4.00), 13630: (17.01), 13631: (4.00), 13632: (4.00), 13633: (4.00), 13634: (17.01), 13635: (4.00), 13636: (17.01), 13637: (4.00), 13638: (4.00), 13639: (4.00), 13640: (17.01), 13641: (4.00), 13642: (4.00), 13643: (17.01), 13644: (17.02), 13645: (17.04), 13646: (4.00), 13647: (4.00), 13648: (4.00), 13649: (4.00), 13650: (4.00), 13651: (4.00), 13652: (4.00), 13653: (17.14), 13654: (17.17), 13655: (17.01), 13656: (17.01), 13657: (17.01), 13658: (4.00), 13659: (4.00), 13660: (4.00), 13661: (4.00), 13662: (17.02), 13663: (17.02), 13664: (17.01), 13665: (17.03), 13666: (17.01), 13667: (17.03), 13668: (17.06), 13669: (17.01), 13670: (4.00), 13671: (4.00), 13672: (4.00), 13673: (4.00), 13674: (4.00), 13675: (4.00), 13676: (17.01), 13677: (17.01), 13678: (17.04), 13679: (17.01), 13680: (17.01), 13681: (17.01), 13682: (17.06), 13683: (17.02), 13684: (4.00), 13685: (4.00), 13686: (4.00), 13687: (4.00), 13688: (4.00), 13689: (17.02), 13690: (17.08), 13691: (17.01), 13692: (17.02), 13693: (4.00), 13694: (17.01), 13695: (4.00), 13696: (4.00), 13697: (4.00), 13698: (17.01), 13699: (17.37), 13700: (17.03), 13701: (17.13), 13702: (17.02), 13703: (17.04), 13704: (4.00), 13705: (4.00), 13706: (4.00), 13707: (4.00), 13708: (17.06), 13709: (17.04), 13710: (17.02), 13711: (17.03), 13712: (4.00), 13713: (17.01), 13714: (17.01), 13715: (4.00), 13716: (4.00), 13717: (4.00), 13718: (4.00), 13719: (4.00), 13720: (4.00), 13721: (17.01), 13722: (17.02), 13723: (17.03), 13724: (17.01), 13725: (17.01), 13726: (4.00), 13727: (4.00), 13728: (4.00), 13729: (4.00), 13730: (4.00), 13731: (4.00), 13732: (4.00), 13733: (4.00), 13734: (4.00), 13735: (17.02), 13736: (17.01), 13737: (4.00), 13738: (4.00), 13739: (4.00), 13740: (4.00), 13741: (4.00), 13742: (4.00), 13743: (4.00), 13744: (17.36), 13745: (17.04), 13746: (17.05), 13747: (4.00), 13748: (4.00), 13749: (4.00), 13750: (4.00), 13751: (4.00), 13752: (4.00), 13753: (17.01), 13754: (17.01), 13755: (17.05), 13756: (17.01), 13757: (17.02), 13758: (17.02), 13759: (17.05), 13760: (17.01), 13761: (4.00), 13762: (4.00), 13763: (4.00), 13764: (4.00), 13765: (4.00), 13766: (4.00), 13767: (17.04), 13768: (17.02), 13769: (4.00), 13770: (17.01), 13771: (17.01), 13772: (4.00), 13773: (4.00), 13774: (4.00), 13775: (4.00), 13776: (4.00), 13777: (4.00), 13778: (4.00), 13779: (4.00), 13780: (17.01), 13781: (17.05), 13782: (17.01), 13783: (17.04), 13784: (17.02), 13785: (4.00), 13786: (4.00), 13787: (4.00), 13788: (4.00), 13789: (17.01), 13790: (17.25), 13791: (17.12), 13792: (17.01), 13793: (17.02), 13794: (17.01), 13795: (17.01), 13796: (17.04), 13797: (4.00), 13798: (4.00), 13799: (17.03), 13800: (17.01), 13801: (17.05), 13802: (17.01), 13803: (17.01), 13804: (17.01), 13805: (4.00), 13806: (4.00), 13807: (4.00), 13808: (4.00), 13809: (4.00), 13810: (4.00), 13811: (4.00), 13812: (17.01), 13813: (17.01), 13814: (17.02), 13815: (17.01), 13816: (17.01), 13817: (4.00), 13818: (4.00), 13819: (4.00), 13820: (4.00), 13821: (4.00), 13822: (4.00), 13823: (4.00), 13824: (4.00), 13825: (4.00), 13826: (17.01), 13827: (17.01), 13828: (4.00), 13829: (17.02), 13830: (4.00), 13831: (17.01), 13832: (17.01), 13833: (17.01), 13834: (4.00), 13835: (17.11), 13836: (17.04), 13837: (17.02), 13838: (17.02), 13839: (4.00), 13840: (17.01), 13841: (17.01), 13842: (4.00), 13843: (4.00), 13844: (17.01), 13845: (17.01), 13846: (17.06), 13847: (17.02), 13848: (17.03), 13849: (17.02), 13850: (17.01), 13851: (17.01), 13852: (4.00), 13853: (4.00), 13854: (4.00), 13855: (4.00), 13856: (4.00), 13857: (4.00), 13858: (17.01), 13859: (17.06), 13860: (4.00), 13861: (17.03), 13862: (17.06), 13863: (17.01), 13864: (4.00), 13865: (4.00), 13866: (4.00), 13867: (4.00), 13868: (4.00), 13869: (4.00), 13870: (4.00), 13871: (4.00), 13872: (17.01), 13873: (4.00), 13874: (4.00), 13875: (4.00), 13876: (4.00), 13877: (4.00), 13878: (4.00), 13879: (4.00), 13880: (4.00), 13881: (17.34), 13882: (17.05), 13883: (17.02), 13884: (17.01), 13885: (17.01), 13886: (17.01), 13887: (4.00), 13888: (4.00), 13889: (4.00), 13890: (17.01), 13891: (17.01), 13892: (17.01), 13893: (4.00), 13894: (4.00), 13895: (17.02), 13896: (17.02), 13897: (4.00), 13898: (17.01), 13899: (4.00), 13900: (4.00), 13901: (4.00), 13902: (4.00), 13903: (4.00), 13904: (17.04), 13905: (4.00), 13906: (4.00), 13907: (4.00), 13908: (4.00), 13909: (4.00), 13910: (17.01), 13911: (4.00), 13912: (4.00), 13913: (17.02), 13914: (4.00), 13915: (4.00), 13916: (4.00), 13917: (4.00), 13918: (17.01), 13919: (4.00), 13920: (17.02), 13921: (17.01), 13922: (17.01), 13923: (4.00), 13924: (4.00), 13925: (4.00), 13926: (4.00), 13927: (17.19), 13928: (17.04), 13929: (17.01), 13930: (4.00), 13931: (4.00), 13932: (17.01), 13933: (17.01), 13934: (4.00), 13935: (4.00), 13936: (17.01), 13937: (17.01), 13938: (17.14), 13939: (4.00), 13940: (17.03), 13941: (17.02), 13942: (17.01), 13943: (4.00), 13944: (4.00), 13945: (4.00), 13946: (4.00), 13947: (4.00), 13948: (4.00), 13949: (4.00), 13950: (17.01), 13951: (17.04), 13952: (17.02), 13953: (17.02), 13954: (17.02), 13955: (17.01), 13956: (17.02), 13957: (17.01), 13958: (4.00), 13959: (4.00), 13960: (4.00), 13961: (4.00), 13962: (4.00), 13963: (4.00), 13964: (17.05), 13965: (17.04), 13966: (17.01), 13967: (17.03), 13968: (17.03), 13969: (4.00), 13970: (4.00), 13971: (4.00), 13972: (4.00), 13973: (17.36), 13974: (17.03), 13975: (17.04), 13976: (17.02), 13977: (17.01), 13978: (17.01), 13979: (4.00), 13980: (4.00), 13981: (4.00), 13982: (17.03), 13983: (17.14), 13984: (17.02), 13985: (17.02), 13986: (17.02), 13987: (17.02), 13988: (17.01), 13989: (4.00), 13990: (4.00), 13991: (4.00), 13992: (4.00), 13993: (4.00), 13994: (4.00), 13995: (4.00), 13996: (17.04), 13997: (17.01), 13998: (17.01), 13999: (17.01), 14000: (17.02), 14001: (17.02), 14002: (4.00), 14003: (4.00), 14004: (4.00), 14005: (17.01), 14006: (4.00), 14007: (4.00), 14008: (4.00), 14009: (4.00), 14010: (17.09), 14011: (17.01), 14012: (17.03), 14013: (17.02), 14014: (17.02), 14015: (17.02), 14016: (17.01), 14017: (4.00), 14018: (4.00), 14019: (17.28), 14020: (17.08), 14021: (17.01), 14022: (17.01), 14023: (4.00), 14024: (4.00), 14025: (4.00), 14026: (4.00), 14027: (4.00), 14028: (17.06), 14029: (17.03), 14030: (17.01), 14031: (17.04), 14032: (4.00), 14033: (17.01), 14034: (17.04), 14035: (17.02), 14036: (4.00), 14037: (4.00), 14038: (17.01), 14039: (4.00), 14040: (4.00), 14041: (17.01), 14042: (17.02), 14043: (17.01), 14044: (17.07), 14045: (17.03), 14046: (17.03), 14047: (17.02), 14048: (17.01), 14049: (4.00), 14050: (4.00), 14051: (17.01), 14052: (4.00), 14053: (4.00), 14054: (4.00), 14055: (4.00), 14056: (17.06), 14057: (17.01), 14058: (17.01), 14059: (4.00), 14060: (17.02), 14061: (4.00), 14062: (4.00), 14063: (4.00), 14064: (17.01), 14065: (17.48), 14066: (17.04), 14067: (17.05), 14068: (17.01), 14069: (17.01), 14070: (4.00), 14071: (4.00), 14072: (4.00), 14073: (4.00), 14074: (4.00), 14075: (17.02), 14076: (17.01), 14077: (17.01), 14078: (4.00), 14079: (17.01), 14080: (17.01), 14081: (4.00), 14082: (4.00), 14083: (4.00), 14084: (4.00), 14085: (4.00), 14086: (4.00), 14087: (4.00), 14088: (17.01), 14089: (17.01), 14090: (17.01), 14091: (4.00), 14092: (17.01), 14093: (17.03), 14094: (17.02), 14095: (17.02), 14096: (4.00), 14097: (17.01), 14098: (4.00), 14099: (4.00), 14100: (4.00), 14101: (17.01), 14102: (17.01), 14103: (17.01), 14104: (17.03), 14105: (17.01), 14106: (17.01), 14107: (4.00), 14108: (4.00), 14109: (4.00), 14110: (4.00), 14111: (17.13), 14112: (17.03), 14113: (17.03), 14114: (17.03), 14115: (4.00), 14116: (17.07), 14117: (17.03), 14118: (17.01), 14119: (4.00), 14120: (17.04), 14121: (17.02), 14122: (17.01), 14123: (17.02), 14124: (17.01), 14125: (17.01), 14126: (17.01), 14127: (4.00), 14128: (4.00), 14129: (4.00), 14130: (17.01), 14131: (17.02), 14132: (4.00), 14133: (17.01), 14134: (17.03), 14135: (17.01), 14136: (17.02), 14137: (4.00), 14138: (4.00), 14139: (4.00), 14140: (4.00), 14141: (4.00), 14142: (4.00), 14143: (17.02), 14144: (17.04), 14145: (17.02), 14146: (17.02), 14147: (4.00), 14148: (17.03), 14149: (17.05), 14150: (17.01), 14151: (17.04), 14152: (17.04), 14153: (17.02), 14154: (17.03), 14155: (17.01), 14156: (17.01), 14157: (17.14), 14158: (17.05), 14159: (17.08), 14160: (17.04), 14161: (17.03), 14162: (17.01), 14163: (17.01), 14164: (4.00), 14165: (4.00), 14166: (17.55), 14167: (17.05), 14168: (17.04), 14169: (17.09), 14170: (17.01), 14171: (17.04), 14172: (4.00), 14173: (4.00), 14174: (4.00), 14175: (17.02), 14176: (4.00), 14177: (4.00), 14178: (4.00), 14179: (17.01), 14180: (17.10), 14181: (17.05), 14182: (17.03), 14183: (17.02), 14184: (17.02), 14185: (17.01), 14186: (17.01), 14187: (4.00), 14188: (17.01), 14189: (17.03), 14190: (17.03), 14191: (17.05), 14192: (17.02), 14193: (17.06), 14194: (17.04), 14195: (17.02), 14196: (17.02), 14197: (17.04), 14198: (17.02), 14199: (17.02), 14200: (4.00), 14201: (17.30), 14202: (17.13), 14203: (17.01), 14204: (17.08), 14205: (17.01), 14206: (17.07), 14207: (17.02), 14208: (4.00), 14209: (17.01), 14210: (17.03), 14211: (17.14), 14212: (17.02), 14213: (17.02), 14214: (17.01), 14215: (17.04), 14216: (17.02), 14217: (17.01), 14218: (4.00), 14219: (4.00), 14220: (17.04), 14221: (17.01), 14222: (17.05), 14223: (17.01), 14224: (17.04), 14225: (17.05), 14226: (17.07), 14227: (17.06), 14228: (17.03), 14229: (17.02), 14230: (4.00), 14231: (17.01), 14232: (4.00), 14233: (17.04), 14234: (17.30), 14235: (17.03), 14236: (17.04), 14237: (17.02), 14238: (17.13), 14239: (17.01), 14240: (17.02), 14241: (4.00), 14242: (17.02), 14243: (4.00), 14244: (17.04), 14245: (4.00), 14246: (4.00), 14247: (17.17), 14248: (17.03), 14249: (17.02), 14250: (17.02), 14251: (17.03), 14252: (4.00), 14253: (17.01), 14254: (4.00), 14255: (17.01), 14256: (17.28), 14257: (17.02), 14258: (17.12), 14259: (17.02), 14260: (17.03), 14261: (17.01), 14262: (17.03), 14263: (4.00), 14264: (4.00), 14265: (17.01), 14266: (17.02), 14267: (17.01), 14268: (4.00), 14269: (17.11), 14270: (17.02), 14271: (17.02), 14272: (17.11), 14273: (17.02), 14274: (17.02), 14275: (17.02), 14276: (4.00), 14277: (4.00), 14278: (17.02), 14279: (17.04), 14280: (17.06), 14281: (17.02), 14282: (17.11), 14283: (17.05), 14284: (17.01), 14285: (17.02), 14286: (17.01), 14287: (17.03), 14288: (17.03), 14289: (4.00), 14290: (17.01), 14291: (17.31), 14292: (17.04), 14293: (17.03), 14294: (17.01), 14295: (17.01), 14296: (17.02), 14297: (17.01), 14298: (17.01), 14299: (4.00), 14300: (17.14), 14301: (17.13), 14302: (17.04), 14303: (17.03), 14304: (17.01), 14305: (17.02), 14306: (4.00), 14307: (4.00), 14308: (4.00), 14309: (4.00), 14310: (4.00), 14311: (4.00), 14312: (4.00), 14313: (4.00), 14314: (17.02), 14315: (17.05), 14316: (17.01), 14317: (4.00), 14318: (4.00), 14319: (4.00), 14320: (4.00), 14321: (4.00), 14322: (4.00), 14323: (17.01), 14324: (17.01), 14325: (17.03), 14326: (17.01), 14327: (17.01), 14328: (17.02), 14329: (17.01), 14330: (4.00), 14331: (4.00), 14332: (17.01), 14333: (4.00), 14334: (17.01), 14335: (4.00), 14336: (17.16), 14337: (17.09), 14338: (17.01), 14339: (17.01), 14340: (17.01), 14341: (4.00), 14342: (4.00), 14343: (4.00), 14344: (4.00), 14345: (17.16), 14346: (17.08), 14347: (17.02), 14348: (17.02), 14349: (17.03), 14350: (17.02), 14351: (4.00), 14352: (4.00), 14353: (4.00), 14354: (4.00), 14355: (4.00), 14356: (4.00), 14357: (4.00), 14358: (4.00), 14359: (17.07), 14360: (17.02), 14361: (17.07), 14362: (17.02), 14363: (17.02), 14364: (17.02), 14365: (17.01), 14366: (4.00), 14367: (4.00), 14368: (17.06), 14369: (17.04), 14370: (17.01), 14371: (17.02), 14372: (17.01), 14373: (17.04), 14374: (4.00), 14375: (17.04), 14376: (17.01), 14377: (17.02), 14378: (17.02), 14379: (4.00), 14380: (4.00), 14381: (17.66), 14382: (17.06), 14383: (17.02), 14384: (17.04), 14385: (17.01), 14386: (17.02), 14387: (17.01), 14388: (17.01), 14389: (4.00), 14390: (17.16), 14391: (17.13), 14392: (17.02), 14393: (17.02), 14394: (17.02), 14395: (17.01), 14396: (17.01), 14397: (4.00), 14398: (4.00), 14399: (17.02), 14400: (17.02), 14401: (17.02), 14402: (4.00), 14403: (17.01), 14404: (17.03), 14405: (17.04), 14406: (17.01), 14407: (17.01), 14408: (17.02), 14409: (17.01), 14410: (17.01), 14411: (4.00), 14412: (4.00), 14413: (17.02), 14414: (17.04), 14415: (17.03), 14416: (17.02), 14417: (17.01), 14418: (17.02), 14419: (17.03), 14420: (17.02), 14421: (4.00), 14422: (17.01), 14423: (17.02), 14424: (4.00), 14425: (4.00), 14426: (17.22), 14427: (17.03), 14428: (4.00), 14429: (17.05), 14430: (17.03), 14431: (4.00), 14432: (17.01), 14433: (17.01), 14434: (4.00), 14435: (17.14), 14436: (17.02), 14437: (17.01), 14438: (17.04), 14439: (17.01), 14440: (17.02), 14441: (17.01), 14442: (4.00), 14443: (17.01), 14444: (17.02), 14445: (17.01), 14446: (17.08), 14447: (17.02), 14448: (17.02), 14449: (17.13), 14450: (17.01), 14451: (17.04), 14452: (17.01), 14453: (17.02), 14454: (17.01), 14455: (17.02), 14456: (4.00), 14457: (4.00), 14458: (17.02), 14459: (17.06), 14460: (17.02), 14461: (17.01), 14462: (17.02), 14463: (17.07), 14464: (17.01), 14465: (17.02), 14466: (17.01), 14467: (17.05), 14468: (17.03), 14469: (17.04), 14470: (17.03), 14471: (17.09), 14472: (17.21), 14473: (17.01), 14474: (4.00), 14475: (17.01), 14476: (17.01), 14477: (4.00), 14478: (4.00), 14479: (4.00), 14480: (4.00), 14481: (17.09), 14482: (17.01), 14483: (4.00), 14484: (4.00), 14485: (17.01), 14486: (17.06), 14487: (17.01), 14488: (4.00), 14489: (4.00), 14490: (4.00), 14491: (4.00), 14492: (4.00), 14493: (4.00), 14494: (17.13), 14495: (17.01), 14496: (17.01), 14497: (4.00), 14498: (4.00), 14499: (4.00), 14500: (4.00), 14501: (4.00), 14502: (4.00), 14503: (17.08), 14504: (4.00), 14505: (4.00), 14506: (4.00), 14507: (4.00), 14508: (17.04), 14509: (17.02), 14510: (4.00), 14511: (4.00), 14512: (4.00), 14513: (4.00), 14514: (4.00), 14515: (4.00), 14516: (4.00), 14517: (17.23), 14518: (4.00), 14519: (4.00), 14520: (4.00), 14521: (4.00), 14522: (4.00), 14523: (4.00), 14524: (4.00), 14525: (4.00), 14526: (17.06), 14527: (17.01), 14528: (4.00), 14529: (4.00), 14530: (4.00), 14531: (4.00), 14532: (4.00), 14533: (4.00), 14534: (4.00), 14535: (17.01), 14536: (17.01), 14537: (4.00), 14538: (4.00), 14539: (4.00), 14540: (17.04), 14541: (4.00), 14542: (4.00), 14543: (4.00), 14544: (4.00), 14545: (4.00), 14546: (4.00), 14547: (4.00), 14548: (17.01), 14549: (17.13), 14550: (4.00), 14551: (4.00), 14552: (4.00), 14553: (4.00), 14554: (4.00), 14555: (4.00), 14556: (4.00), 14557: (4.00), 14558: (17.08), 14559: (17.01), 14560: (4.00), 14561: (4.00), 14562: (4.00), 14563: (17.03), 14564: (4.00), 14565: (4.00), 14566: (4.00), 14567: (17.01), 14568: (4.00), 14569: (17.10), 14570: (17.01), 14571: (4.00), 14572: (4.00), 14573: (4.00), 14574: (4.00), 14575: (4.00), 14576: (17.06), 14577: (4.00), 14578: (4.00), 14579: (4.00), 14580: (4.00), 14581: (4.00), 14582: (4.00), 14583: (4.00), 14584: (4.00), 14585: (4.00), 14586: (4.00), 14587: (4.00), 14588: (4.00), 14589: (4.00), 14590: (4.00), 14591: (4.00), 14592: (4.00), 14593: (4.00), 14594: (17.12), 14595: (17.01), 14596: (4.00), 14597: (4.00), 14598: (4.00), 14599: (4.00), 14600: (4.00), 14601: (4.00), 14602: (4.00), 14603: (4.00), 14604: (4.00), 14605: (4.00), 14606: (4.00), 14607: (17.07), 14608: (4.00), 14609: (17.01), 14610: (4.00), 14611: (17.01), 14612: (4.00), 14613: (4.00), 14614: (4.00), 14615: (17.01), 14616: (17.06), 14617: (4.00), 14618: (4.00), 14619: (4.00), 14620: (4.00), 14621: (4.00), 14622: (4.00), 14623: (4.00), 14624: (4.00), 14625: (17.01), 14626: (4.00), 14627: (4.00), 14628: (4.00), 14629: (17.11), 14630: (17.01), 14631: (4.00), 14632: (4.00), 14633: (4.00), 14634: (4.00), 14635: (4.00), 14636: (4.00), 14637: (4.00), 14638: (17.08), 14639: (17.01), 14640: (4.00), 14641: (4.00), 14642: (4.00), 14643: (4.00), 14644: (4.00), 14645: (4.00), 14646: (4.00), 14647: (17.02), 14648: (17.01), 14649: (4.00), 14650: (4.00), 14651: (4.00), 14652: (17.01), 14653: (17.01), 14654: (4.00), 14655: (4.00), 14656: (4.00), 14657: (17.02), 14658: (17.02), 14659: (17.15), 14660: (4.00), 14661: (4.00), 14662: (4.00), 14663: (4.00), 14664: (4.00), 14665: (17.02), 14666: (17.02), 14667: (4.00), 14668: (4.00), 14669: (4.00), 14670: (4.00), 14671: (4.00), 14672: (4.00), 14673: (4.00), 14674: (4.00), 14675: (17.01), 14676: (4.00), 14677: (4.00), 14678: (4.00), 14679: (4.00), 14680: (4.00), 14681: (4.00), 14682: (4.00), 14683: (17.03), 14684: (17.02), 14685: (17.01), 14686: (4.00), 14687: (4.00), 14688: (17.01), 14689: (4.00), 14690: (4.00), 14691: (4.00), 14692: (4.00), 14693: (4.00), 14694: (4.00), 14695: (4.00), 14696: (17.02), 14697: (17.01), 14698: (17.01), 14699: (17.01), 14700: (17.02), 14701: (17.05), 14702: (4.00), 14703: (17.01), 14704: (4.00), 14705: (17.51), 14706: (17.07), 14707: (17.01), 14708: (17.01), 14709: (4.00), 14710: (17.01), 14711: (4.00), 14712: (4.00), 14713: (4.00), 14714: (4.00), 14715: (17.04), 14716: (17.01), 14717: (4.00), 14718: (17.01), 14719: (17.09), 14720: (4.00), 14721: (17.01), 14722: (17.02), 14723: (17.01), 14724: (4.00), 14725: (4.00), 14726: (4.00), 14727: (17.09), 14728: (17.02), 14729: (4.00), 14730: (17.01), 14731: (4.00), 14732: (4.00), 14733: (4.00), 14734: (4.00), 14735: (4.00), 14736: (4.00), 14737: (17.10), 14738: (4.00), 14739: (4.00), 14740: (4.00), 14741: (4.00), 14742: (4.00), 14743: (4.00), 14744: (4.00), 14745: (4.00), 14746: (4.00), 14747: (17.01), 14748: (17.31), 14749: (17.01), 14750: (4.00), 14751: (4.00), 14752: (4.00), 14753: (4.00), 14754: (4.00), 14755: (17.08), 14756: (4.00), 14757: (4.00), 14758: (4.00), 14759: (4.00), 14760: (4.00), 14761: (4.00), 14762: (4.00), 14763: (4.00), 14764: (4.00), 14765: (4.00), 14766: (4.00), 14767: (4.00), 14768: (4.00), 14769: (4.00), 14770: (4.00), 14771: (4.00), 14772: (4.00), 14773: (17.03), 14774: (4.00), 14775: (4.00), 14776: (4.00), 14777: (4.00), 14778: (4.00), 14779: (4.00), 14780: (4.00), 14781: (4.00), 14782: (4.00), 14783: (4.00), 14784: (4.00), 14785: (4.00), 14786: (17.08), 14787: (4.00), 14788: (4.00), 14789: (4.00), 14790: (4.00), 14791: (4.00), 14792: (4.00), 14793: (4.00), 14794: (4.00), 14795: (17.03), 14796: (4.00), 14797: (4.00), 14798: (4.00), 14799: (4.00), 14800: (4.00), 14801: (4.00), 14802: (4.00), 14803: (4.00), 14804: (4.00), 14805: (4.00), 14806: (4.00), 14807: (4.00), 14808: (4.00), 14809: (4.00), 14810: (4.00), 14811: (4.00), 14812: (4.00), 14813: (4.00), 14814: (4.00), 14815: (4.00), 14816: (4.00), 14817: (4.00), 14818: (4.00), 14819: (4.00), 14820: (4.00), 14821: (4.00), 14822: (4.00), 14823: (4.00), 14824: (4.00), 14825: (4.00), 14826: (4.00), 14827: (4.00), 14828: (4.00), 14829: (4.00), 14830: (4.00), 14831: (4.00), 14832: (4.00), 14833: (4.00), 14834: (4.00), 14835: (4.00), 14836: (17.01), 14837: (17.03), 14838: (17.01), 14839: (17.01), 14840: (17.10), 14841: (17.01), 14842: (17.02), 14843: (4.00), 14844: (17.01), 14845: (17.02), 14846: (4.00), 14847: (4.00), 14848: (4.00), 14849: (17.01), 14850: (17.12), 14851: (4.00), 14852: (4.00), 14853: (4.00), 14854: (4.00), 14855: (4.00), 14856: (4.00), 14857: (4.00), 14858: (17.01), 14859: (17.01), 14860: (4.00), 14861: (4.00), 14862: (4.00), 14863: (17.08), 14864: (17.01), 14865: (17.01), 14866: (4.00), 14867: (4.00), 14868: (17.01), 14869: (4.00), 14870: (17.02), 14871: (4.00), 14872: (17.03), 14873: (4.00), 14874: (17.01), 14875: (4.00), 14876: (4.00), 14877: (4.00), 14878: (4.00), 14879: (4.00), 14880: (4.00), 14881: (17.08), 14882: (17.01), 14883: (4.00), 14884: (4.00), 14885: (4.00), 14886: (17.01), 14887: (4.00), 14888: (4.00), 14889: (4.00), 14890: (17.01), 14891: (4.00), 14892: (17.01), 14893: (4.00), 14894: (17.01), 14895: (17.05), 14896: (17.01), 14897: (17.02), 14898: (4.00), 14899: (17.01), 14900: (17.01), 14901: (17.01), 14902: (17.02), 14903: (17.01), 14904: (17.01), 14905: (17.02), 14906: (17.01), 14907: (17.04), 14908: (17.06), 14909: (17.01), 14910: (17.03), 14911: (17.02), 14912: (4.00), 14913: (17.12), 14914: (17.09), 14915: (17.01), 14916: (17.04), 14917: (17.36), 14918: (17.07), 14919: (17.05), 14920: (17.03), 14921: (17.04), 14922: (17.23), 14923: (17.05), 14924: (17.02), 14925: (17.07), 14926: (17.10), 14927: (17.01), 14928: (4.00), 14929: (4.00), 14930: (17.02), 14931: (17.03), 14932: (17.02), 14933: (17.02), 14934: (17.03), 14935: (17.06), 14936: (17.02), 14937: (17.01), 14938: (4.00), 14939: (4.00), 14940: (17.04), 14941: (4.00), 14942: (4.00), 14943: (4.00), 14944: (4.00), 14945: (4.00), 14946: (4.00), 14947: (4.00), 14948: (4.00), 14949: (4.00), 14950: (4.00), 14951: (4.00), 14952: (4.00), 14953: (4.00), 14954: (17.04), 14955: (17.01), 14956: (4.00), 14957: (4.00), 14958: (4.00), 14959: (17.01), 14960: (4.00), 14961: (17.12), 14962: (17.01), 14963: (4.00), 14964: (4.00), 14965: (4.00), 14966: (4.00), 14967: (17.01), 14968: (4.00), 14969: (4.00), 14970: (4.00), 14971: (4.00), 14972: (17.03), 14973: (4.00), 14974: (4.00), 14975: (4.00), 14976: (4.00), 14977: (4.00), 14978: (4.00), 14979: (4.00), 14980: (4.00), 14981: (4.00), 14982: (4.00), 14983: (4.00), 14984: (4.00), 14985: (17.05), 14986: (4.00), 14987: (4.00), 14988: (4.00), 14989: (4.00), 14990: (4.00), 14991: (4.00), 14992: (4.00), 14993: (4.00), 14994: (4.00), 14995: (17.04), 14996: (4.00), 14997: (4.00), 14998: (4.00), 14999: (17.01), 15000: (4.00), 15001: (4.00), 15002: (4.00), 15003: (4.00), 15004: (4.00), 15005: (17.02), 15006: (17.03), 15007: (17.04), 15008: (17.02), 15009: (4.00), 15010: (4.00), 15011: (4.00), 15012: (4.00), 15013: (17.01), 15014: (4.00), 15015: (4.00), 15016: (17.02), 15017: (17.02), 15018: (4.00), 15019: (4.00), 15020: (4.00), 15021: (4.00), 15022: (4.00), 15023: (4.00), 15024: (4.00), 15025: (4.00), 15026: (4.00), 15027: (4.00), 15028: (4.00), 15029: (4.00), 15030: (17.05), 15031: (17.02), 15032: (4.00), 15033: (4.00), 15034: (4.00), 15035: (4.00), 15036: (4.00), 15037: (4.00), 15038: (4.00), 15039: (4.00), 15040: (17.04), 15041: (4.00), 15042: (4.00), 15043: (4.00), 15044: (4.00), 15045: (17.01), 15046: (4.00), 15047: (4.00), 15048: (4.00), 15049: (4.00), 15050: (4.00), 15051: (17.11), 15052: (17.01), 15053: (4.00), 15054: (4.00), 15055: (4.00), 15056: (4.00), 15057: (4.00), 15058: (17.01), 15059: (4.00), 15060: (4.00), 15061: (4.00), 15062: (17.02), 15063: (17.02), 15064: (4.00), 15065: (4.00), 15066: (4.00), 15067: (4.00), 15068: (4.00), 15069: (4.00), 15070: (4.00), 15071: (4.00), 15072: (4.00), 15073: (17.32), 15074: (17.01), 15075: (4.00), 15076: (4.00), 15077: (4.00), 15078: (4.00), 15079: (4.00), 15080: (4.00), 15081: (4.00), 15082: (4.00), 15083: (4.00), 15084: (4.00), 15085: (4.00), 15086: (17.02), 15087: (4.00), 15088: (17.19), 15089: (17.01), 15090: (4.00), 15091: (4.00), 15092: (4.00), 15093: (4.00), 15094: (4.00), 15095: (4.00), 15096: (4.00), 15097: (4.00), 15098: (4.00), 15099: (17.05), 15100: (4.00), 15101: (17.17), 15102: (17.02), 15103: (17.01), 15104: (17.03), 15105: (17.01), 15106: (17.03), 15107: (17.32), 15108: (4.00), 15109: (17.03), 15110: (4.00), 15111: (4.00), 15112: (4.00), 15113: (4.00), 15114: (4.00), 15115: (4.00), 15116: (4.00), 15117: (4.00), 15118: (4.00), 15119: (17.04), 15120: (4.00), 15121: (4.00), 15122: (4.00), 15123: (17.18), 15124: (17.01), 15125: (4.00), 15126: (4.00), 15127: (4.00), 15128: (4.00), 15129: (4.00), 15130: (4.00), 15131: (4.00), 15132: (17.07), 15133: (4.00), 15134: (4.00), 15135: (4.00), 15136: (4.00), 15137: (4.00), 15138: (4.00), 15139: (4.00), 15140: (4.00), 15141: (4.00), 15142: (4.00), 15143: (17.01), 15144: (17.20), 15145: (4.00), 15146: (4.00), 15147: (17.01), 15148: (4.00), 15149: (4.00), 15150: (4.00), 15151: (4.00), 15152: (4.00), 15153: (4.00), 15154: (4.00), 15155: (17.10), 15156: (4.00), 15157: (4.00), 15158: (4.00), 15159: (4.00), 15160: (4.00), 15161: (4.00), 15162: (4.00), 15163: (4.00), 15164: (17.04), 15165: (4.00), 15166: (4.00), 15167: (4.00), 15168: (4.00), 15169: (17.10), 15170: (4.00), 15171: (4.00), 15172: (4.00), 15173: (4.00), 15174: (4.00), 15175: (4.00), 15176: (4.00), 15177: (4.00), 15178: (17.10), 15179: (4.00), 15180: (4.00), 15181: (4.00), 15182: (4.00), 15183: (17.02), 15184: (4.00), 15185: (4.00), 15186: (4.00), 15187: (17.04), 15188: (4.00), 15189: (4.00), 15190: (4.00), 15191: (4.00), 15192: (17.01), 15193: (4.00), 15194: (17.01), 15195: (17.02), 15196: (17.02), 15197: (17.02), 15198: (4.00), 15199: (4.00), 15200: (4.00), 15201: (17.03), 15202: (17.01), 15203: (17.01), 15204: (17.01), 15205: (17.02), 15206: (17.01), 15207: (17.18), 15208: (17.34), 15209: (17.05), 15210: (17.01), 15211: (17.02), 15212: (4.00), 15213: (4.00), 15214: (17.01), 15215: (17.01), 15216: (17.01), 15217: (17.18), 15218: (17.03), 15219: (17.01), 15220: (17.03), 15221: (17.01), 15222: (17.04), 15223: (17.01), 15224: (17.01), 15225: (17.02), 15226: (17.04), 15227: (17.01), 15228: (17.01), 15229: (17.16), 15230: (17.06), 15231: (17.20), 15232: (17.01), 15233: (4.00), 15234: (17.02), 15235: (17.01), 15236: (4.00), 15237: (17.01), 15238: (17.02), 15239: (17.01), 15240: (17.02), 15241: (17.01), 15242: (17.01), 15243: (17.14), 15244: (17.01), 15245: (17.12), 15246: (17.03), 15247: (4.00), 15248: (17.03), 15249: (17.03), 15250: (17.01), 15251: (17.01), 15252: (17.06), 15253: (17.15), 15254: (17.03), 15255: (17.30), 15256: (17.03), 15257: (17.01), 15258: (17.03), 15259: (17.02), 15260: (17.03), 15261: (17.14), 15262: (17.05), 15263: (4.00), 15264: (17.19), 15265: (17.02), 15266: (4.00), 15267: (17.01), 15268: (17.02), 15269: (17.05), 15270: (17.03), 15271: (17.01), 15272: (17.09), 15273: (17.18), 15274: (17.03), 15275: (17.04), 15276: (17.01), 15277: (17.07), 15278: (17.01), 15279: (17.27), 15280: (17.01), 15281: (17.02), 15282: (17.01), 15283: (4.00), 15284: (4.00), 15285: (4.00), 15286: (17.01), 15287: (17.02), 15288: (17.03), 15289: (17.01), 15290: (17.01), 15291: (17.11), 15292: (17.02), 15293: (17.45), 15294: (17.01), 15295: (4.00), 15296: (17.15), 15297: (17.01), 15298: (17.01), 15299: (17.01), 15300: (17.07), 15301: (17.11), 15302: (17.05), 15303: (17.01), 15304: (4.00), 15305: (17.03), 15306: (17.06), 15307: (4.00), 15308: (4.00), 15309: (4.00), 15310: (17.08), 15311: (17.03), 15312: (17.02), 15313: (17.01), 15314: (17.09), 15315: (17.04), 15316: (17.03), 15317: (17.02), 15318: (17.02), 15319: (17.08), 15320: (17.02), 15321: (17.02), 15322: (17.03), 15323: (17.28), 15324: (17.07), 15325: (17.05), 15326: (4.00), 15327: (4.00), 15328: (17.01), 15329: (17.02), 15330: (17.01), 15331: (4.00), 15332: (17.01), 15333: (17.02), 15334: (17.03), 15335: (17.01), 15336: (17.03), 15337: (17.04), 15338: (17.47), 15339: (17.07), 15340: (17.04), 15341: (17.01), 15342: (17.11), 15343: (17.27), 15344: (17.01), 15345: (17.10), 15346: (17.01), 15347: (17.14), 15348: (17.07), 15349: (4.00), 15350: (17.01), 15351: (17.08), 15352: (17.27), 15353: (17.03), 15354: (17.14), 15355: (17.03), 15356: (17.03), 15357: (17.06), 15358: (17.02), 15359: (17.02), 15360: (17.04), 15361: (17.27), 15362: (17.01), 15363: (4.00), 15364: (4.00), 15365: (4.00), 15366: (17.06), 15367: (17.02), 15368: (17.01), 15369: (17.01), 15370: (17.03), 15371: (17.09), 15372: (4.00), 15373: (4.00), 15374: (17.01), 15375: (17.03), 15376: (17.03), 15377: (4.00), 15378: (4.00), 15379: (17.01), 15380: (17.02), 15381: (17.01), 15382: (17.02), 15383: (17.01), 15384: (17.01), 15385: (4.00), 15386: (17.01), 15387: (17.02), 15388: (17.01), 15389: (4.00), 15390: (4.00), 15391: (4.00), 15392: (4.00), 15393: (4.00), 15394: (4.00), 15395: (4.00), 15396: (4.00), 15397: (4.00), 15398: (4.00), 15399: (4.00), 15400: (4.00), 15401: (4.00), 15402: (4.00), 15403: (4.00), 15404: (4.00), 15405: (4.00), 15406: (4.00), 15407: (4.00), 15408: (4.00), 15409: (4.00), 15410: (4.00), 15411: (4.00), 15412: (4.00), 15413: (4.00), 15414: (4.00), 15415: (4.00), 15416: (4.00), 15417: (4.00), 15418: (4.00), 15419: (4.00), 15420: (4.00), 15421: (4.00), 15422: (4.00), 15423: (4.00), 15424: (4.00), 15425: (4.00), 15426: (4.00), 15427: (4.00), 15428: (4.00), 15429: (4.00), 15430: (4.00), 15431: (4.00), 15432: (4.00), 15433: (4.00), 15434: (4.00), 15435: (4.00), 15436: (4.00), 15437: (4.00), 15438: (4.00), 15439: (4.00), 15440: (4.00), 15441: (4.00), 15442: (4.00), 15443: (4.00), 15444: (4.00), 15445: (4.00), 15446: (4.00), 15447: (4.00), 15448: (4.00), 15449: (4.00), 15450: (4.00), 15451: (4.00), 15452: (4.00), 15453: (4.00), 15454: (4.00), 15455: (4.00), 15456: (4.00), 15457: (4.00), 15458: (4.00), 15459: (4.00), 15460: (4.00), 15461: (4.00), 15462: (4.00), 15463: (4.00), 15464: (4.00), 15465: (4.00), 15466: (4.00), 15467: (4.00), 15468: (4.00), 15469: (4.00), 15470: (4.00), 15471: (4.00), 15472: (4.00), 15473: (4.00), 15474: (4.00), 15475: (4.00), 15476: (4.00), 15477: (4.00), 15478: (4.00), 15479: (4.00), 15480: (4.00), 15481: (4.00), 15482: (4.00), 15483: (4.00), 15484: (4.00), 15485: (4.00), 15486: (4.00), 15487: (4.00), 15488: (4.00), 15489: (4.00), 15490: (4.00), 15491: (4.00), 15492: (4.00), 15493: (4.00), 15494: (4.00), 15495: (4.00), 15496: (4.00), 15497: (4.00), 15498: (4.00), 15499: (4.00), 15500: (4.00)"#@param {type:"string"}
    rotation_3d_x = "0: (0), 5: ((-3*(sin(3.141*t/600)**1000)+0.5) + (+0.1 *(sin(3.141*t/900)**5)-0.5))"#@param {type:"string"}
    rotation_3d_y = "0: (0), 5: (0.5*(sin(3.141*t/300)))"#@param {type:"string"}
    rotation_3d_z = "0: (0.3*(cos(3.141*t/300)**900)+0.30)"#@param {type:"string"}
    rotation_3d_w = "0: (-0.3*(cos(3.141*t/150)**900)+0.30)" #@param {type:"string"}
    flip_2d_perspective = False #@param {type:"boolean"}
    perspective_flip_theta = "0:(0)"#@param {type:"string"}
    perspective_flip_phi = "0:(t%15)"#@param {type:"string"}
    perspective_flip_gamma = "0:(0)"#@param {type:"string"}
    perspective_flip_fv = "0:(53)"#@param {type:"string"}
    enable_cfg_scale_schedule = True #@param {type:"boolean"}
    cfg_scale_schedule = "0:(9), 30: (15*(cos(3.141*t/90)**20000)+9), 4026: (12), 4040: (15*(cos(3.141*t/90)**20000)+9)" #@param {type: "string"}
    noise_schedule = "0:(.02)"#@param {type:"string"}
    strength_schedule = "0:(.58), 5: (-4*(cos(3.141*t/600)**20000)+0.58)"#@param {type:"string"}
    contrast_schedule = "0: (1.0)"#@param {type:"string"}
    hybrid_comp_alpha_schedule = "0:(1)" #@param {type:"string"}
    hybrid_comp_mask_blend_alpha_schedule = "0:(0.5)" #@param {type:"string"}
    hybrid_comp_mask_contrast_schedule = "0:(1)" #@param {type:"string"}
    hybrid_comp_mask_auto_contrast_cutoff_high_schedule =  "0:(100)" #@param {type:"string"}
    hybrid_comp_mask_auto_contrast_cutoff_low_schedule =  "0:(0)" #@param {type:"string"}

    # #@markdown ####**Checkpoint Scheduling:**
    # enable_checkpoint_scheduling = False#@param {type:"boolean"}
    # checkpoint_schedule = '0: ("model1.ckpt"), 100: ("model2.ckpt")'

    #@markdown ####**Cadence Scheduling:**
    enable_cadence_schedule = False #@param {type:"boolean"}
    cadence_schedule = "0:(4),300:(6), 600:(8), 900:(4), 1200:(4),1400:(6), 1600:(8), 1900:(4), 14100:(8),14140:(2), 14150:(5)" #@param{type:'string'}

    #@markdown ####**Sampler Scheduling:**
    enable_schedule_samplers = False #@param {type:"boolean"}
    sampler_schedule = "0:('euler'),10:('dpm2'),20:('dpm2_ancestral'),30:('heun'),40:('euler'),50:('euler_ancestral'),60:('dpm_fast'),70:('dpm_adaptive'),80:('dpmpp_2s_a'),90:('dpmpp_2m')" #@param {type:"string"}

    #@markdown ####**Unsharp mask (anti-blur) Parameters:**
    kernel_schedule = "0: (5)"#@param {type:"string"}
    sigma_schedule = "0: (1.0)"#@param {type:"string"}
    amount_schedule = "0: (0.2)"#@param {type:"string"}
    threshold_schedule = "0: (0.0)"#@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = 'Match Frame 0 RGB' #@param ['None', 'Match Frame 0 HSV', 'Match Frame 0 LAB', 'Match Frame 0 RGB', 'Video Input'] {type:'string'}
    color_coherence_video_every_N_frames = 1 #@param {type:"integer"}
    color_force_grayscale = False #@param {type:"boolean"}
    diffusion_cadence = '5' #@param ['1','2','3','4','5','6','7','8'] {type:'string'}

    #@markdown ####**3D Depth Warping:**
    use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3#@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov = 40#@param {type:"number"}
    padding_mode = 'border'#@param ['border', 'reflection', 'zeros'] {type:'string'}
    sampling_mode = 'bicubic'#@param ['bicubic', 'bilinear', 'nearest'] {type:'string'}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path ='/content/video_in.mp4'#@param {type:"string"}
    extract_nth_frame = 1#@param {type:"number"}
    overwrite_extracted_frames = True #@param {type:"boolean"}
    use_mask_video = False #@param {type:"boolean"}
    video_mask_path ='/content/video_in.mp4'#@param {type:"string"}

    #@markdown ####**Hybrid Video for 2D/3D Animation Mode:**
    hybrid_generate_inputframes = False #@param {type:"boolean"}
    hybrid_use_first_frame_as_init_image = False #@param {type:"boolean"}
    hybrid_motion = "None" #@param ['None','Optical Flow','Perspective','Affine']
    hybrid_motion_use_prev_img = False #@param {type:"boolean"}
    hybrid_flow_method = "DIS Medium" #@param ['DenseRLOF','DIS Medium','Farneback','SF']
    hybrid_composite = False #@param {type:"boolean"}
    hybrid_comp_mask_type = "None" #@param ['None', 'Depth', 'Video Depth', 'Blend', 'Difference']
    hybrid_comp_mask_inverse = False #@param {type:"boolean"}
    hybrid_comp_mask_equalize = "None" #@param  ['None','Before','After','Both']
    hybrid_comp_mask_auto_contrast = False #@param {type:"boolean"}
    hybrid_comp_save_extra_frames = False #@param {type:"boolean"}
    hybrid_use_video_as_mse_image = False #@param {type:"boolean"}

    #@markdown ####**Interpolation:**
    interpolate_key_frames = False #@param {type:"boolean"}
    interpolate_x_frames = 4 #@param {type:"number"}
    
    #@markdown ####**Resume Animation:**
    resume_from_timestring = True #@param {type:"boolean"}
    resume_timestring = "20230228001756" #@param {type:"string"}

    return locals()

In [ ]:
#@title $ {\large \textsf{Simple Prompt Modifier Generator}}$
#@markdown $ {\large \textsf{Run the Cell to get popular artists, UHD words, art styles.}}$
#@markdown $ {\large \textsf{The list will be developed more as we go along..}}$
import random
from ipywidgets import widgets, Label
from IPython import display
#artists
psychedelic_artists = ["Pablo Amaringo", "Chuck Arnett", "Chris Dyer", "Doug Binder",  "Brummbaer", "Mark Boyle", "Joan Hills",]
surrealist_artists = ["Genevieve Leavold", "Sam Wilde", "Adam Lawrence", "Violet Polsangi",  "Kazuhiro Higashi", "Kim Marra", "Ricardo Harris-Fuentes",]
anime_artists = ["George Morikaw", "Keisuke Itagaki", "Yoichi Takahash", "Hirohiko Araki", "Masashi Kishimoto", "Yoshihiro Togashi", "Hajime Isayama", "Gosho Aoyama", "Akira Toriyama", "Eiichiro Oda",]

#high definition
high_definition = ["4K","High-resolution","High-quality","Sharp","Detailed","Vibrant","Clear","Realistic","Immersive","Stunning","Crisp","Bright","Colorful","Smooth","Life-like","Rich","Lifelike","Vivid","Gorgeous","Beautiful","Striking","Sharpness","Depth","Intense","Brilliant","Extraordinary","Dynamic","Elegant","Expansive","Fantastic","Fine","Outstanding","Magnificent","Mesmerizing","Incredible","Impressive","Sensational","Spectacular","Superb","Stupendous","Tremendous","Unbelievable","Unreal"]

#art style
art_style = ["Renaissance","Baroque","Rococo","Gothic","Impressionism","Post-Impressionism","Expressionism","Art Nouveau","Futurism","Cubism","Surrealism","Abstract Expressionism","Pop Art","Minimalism","Conceptual Art","Realism","Romanticism","Neoclassicism","Pre-Raphaelitism","Fauvism","Orphism","De Stijl","Suprematism","Constructivism","Dadaism","Expressionist Architecture","Bauhaus","Art Deco","Futurist Architecture","Concrete Art","Arte Povera","New Objectivity","Op Art","Hyperrealism","Photorealism","Magic Realism","Social Realism","Early Netherlandish Painting","High Renaissance","Mannerism","Northern Renaissance","Baroque Classicism","Caravaggism","Dutch Golden Age Painting","Romanticism in Art","American Regionalism","Abstract Illusionism","Abstract Impressionism","Neo-Expressionism","Hyperrealistic Sculpture","Environmental Art","New Media Art","Contemporary Realism","Neo-Pop Art"]

button1 = widgets.Button(description="Psychedelic Artists", button_style="danger")

def on_button1_clicked(b):

  selected_artists = random.sample(psychedelic_artists, 3)
  print("Psychedelic Artists: " + ", ".join(selected_artists))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button1.on_click(on_button1_clicked)

button2 = widgets.Button(description="Surrealist Artists", button_style="success")

def on_button2_clicked(b):

  selected_artists = random.sample(surrealist_artists, 3)
  print("Surrealist Artists: " + ", ".join(selected_artists))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button2.on_click(on_button2_clicked)

button3 = widgets.Button(description="Anime Artists", button_style="info")

def on_button3_clicked(b):

  selected_artists = random.sample(anime_artists, 3)
  print("Anime Artists: " + ", ".join(selected_artists))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button3.on_click(on_button3_clicked)

button_all = widgets.Button(description="Remix", button_style="Warning")

def on_button_all_clicked(b):

  first_artist = random.choice(psychedelic_artists)
  second_artist = random.choice(surrealist_artists)
  third_artist = random.choice(anime_artists)
  art_styles = random.choice(art_style)
  hd_word = random.choice(high_definition)
  print("Remix of Prompt Words: {}, {}, {}, {}, {}".format(first_artist, second_artist, third_artist, art_styles, hd_word))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button_all.on_click(on_button_all_clicked)

button4 = widgets.Button(description="High Definition Words", button_style="success")

def on_button4_clicked(b):

  selected_hd = random.sample(high_definition, 4)
  print("High Definition Words: " + ", ".join(selected_hd))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button4.on_click(on_button4_clicked)

button5 = widgets.Button(description="Art Styles", button_style="info")

def on_button5_clicked(b):

  selected_style = random.sample(art_style, 4)
  print("Art Styles: " + ", ".join(selected_style))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button5.on_click(on_button5_clicked)

display.display(button1, button2, button3, button4, button5, button_all)

In [ ]:
# conditional (postitive) prompts
cond_prompts = {
0: "perfect drawing, conceptart, psychedelic glowing colors, eternal mortality, illusion, zombified mummy grudge, psychedelic grudge, armageddon apocalyptic cityscape, zombified mummy portrait, eternal, parabola spirit, celestial body, tool, progressive rock, post apocalyptic, armageddon, very very detailed illustration of a parabolic psychedelic rockstar, UHD, clear, crisp, high resolution, high quality, octane render, very coherent, perfect art",

}

# unconditional (negative) prompts
uncond_prompts = {
    0: "streaks, artefacts, bad drawing, incoherent, messy, low resolution, pixelated, nude, nsfw, naked, low quality, multiple",
}


In [ ]:
#@title $ {\large \textsf{Load Settings}}$
from IPython import display
import time
start_time = time.time()
#@markdown **NLTK Random Word Generator, Use This to Spice Up Your Exisiting Prompts!**
use_random_words_nltk = True #@param {type: "boolean"}
if use_random_words_nltk:
    # function to get a random adjective or noun
    def get_random_word(pos):
        synsets = wordnet.all_synsets(pos=pos)
        lemmas = [l for s in synsets for l in s.lemmas()]
        return random.choice(lemmas).name()

    # loop through the dictionary values and add random adjectives and nouns
    for key in cond_prompts:
        tokens = nltk.word_tokenize(cond_prompts[key])
        tagged_tokens = nltk.pos_tag(tokens)
        new_tokens = []
        for token, pos in tagged_tokens:
            new_tokens.append(token)
            if pos.startswith('N'):
                new_token = get_random_word('n')
                new_tokens.append(new_token)
            elif pos.startswith('J'):
                new_token = get_random_word('a')
                new_tokens.append(new_token)
        new_value = ' '.join(new_tokens)
        cond_prompts[key] = new_value

    # print the updated dictionary
    print(cond_prompts)
#@markdown **Deforum Args**
override_settings_with_file = False #@param {type:"boolean"}
settings_file = "custom" #@param ["custom", "512x512_aesthetic_0.json","512x512_aesthetic_1.json","512x512_colormatch_0.json","512x512_colormatch_1.json","512x512_colormatch_2.json","512x512_colormatch_3.json"]
custom_settings_file = "/content/drive/MyDrive/Settings.txt"#@param {type:"string"}

def DeforumArgs():
    #@markdown **Image Settings**
    W = 512 #@param
    H = 896 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    bit_depth_output = 8 #@param [8, 16, 32] {type:"raw"}
    render_video_every = 30 #@param{type: "number"}
    #@markdown **Sampling Settings**
    seed = -1 #@param
    sampler = 'dpm2_ancestral' #@param ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral","plms", "ddim", "dpm_fast", "dpm_adaptive", "dpmpp_2s_a", "dpmpp_2m"]
    steps = 120 #@param
    scale = 9 #@param
    ddim_eta = 0.0 #@param
    dynamic_threshold = None
    static_threshold = None   

    #@markdown **Save & Display Settings**
    save_samples = True #@param {type:"boolean"}
    save_settings = True #@param {type:"boolean"}
    display_samples = True #@param {type:"boolean"}
    save_sample_per_step = False #@param {type:"boolean"}
    show_sample_per_step = False #@param {type:"boolean"}

    #@markdown **Batch Settings**
    n_batch = 1 #@param
    n_samples = 1 #@param
    batch_name = "grudgetok" #@param {type:"string"}
    filename_format = "{timestring}_{index}_{prompt}.png" #@param ["{timestring}_{index}_{seed}.png","{timestring}_{index}_{prompt}.png"]
    seed_behavior = "iter" #@param ["iter","fixed","random","ladder","alternate"]
    seed_iter_N = 1 #@param {type:'integer'}
    make_grid = False #@param {type:"boolean"}
    grid_rows = 2 #@param 
    outdir = get_output_folder(root.output_path, batch_name)

    #@markdown **Init Settings**
    use_init = False #@param {type:"boolean"}
    strength = 0.65 #@param {type:"number"}
    strength_0_no_init = True # Set the strength to 0 automatically when no init image is used
    init_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    add_init_noise = True #@param {type:"boolean"}
    init_noise = 0.015 #@param
    # Whiter areas of the mask are areas that change more
    use_mask = False #@param {type:"boolean"}
    use_alpha_as_mask = False # use the alpha channel of the init image as the mask
    mask_file = "https://www.filterforge.com/wiki/images/archive/b/b7/20080927223728%21Polygonal_gradient_thumb.jpg" #@param {type:"string"}
    invert_mask = False #@param {type:"boolean"}
    # Adjust mask image, 1.0 is no adjustment. Should be positive numbers.
    mask_brightness_adjust = 1.0  #@param {type:"number"}
    mask_contrast_adjust = 1.0  #@param {type:"number"}
    # Overlay the masked image at the end of the generation so it does not get degraded by encoding and decoding
    overlay_mask = True  # {type:"boolean"}
    # Blur edges of final overlay mask, if used. Minimum = 0 (no blur)
    mask_overlay_blur = 5 # {type:"number"}

    #@markdown **Exposure/Contrast Conditional Settings**
    mean_scale = 0 #@param {type:"number"}
    var_scale = 0 #@param {type:"number"}
    exposure_scale = 0 #@param {type:"number"}
    exposure_target = 0.5 #@param {type:"number"}

    #@markdown **Color Match Conditional Settings**
    colormatch_scale = 0 #@param {type:"number"}
    colormatch_image = "https://www.saasdesign.io/wp-content/uploads/2021/02/palette-3-min-980x588.png" #@param {type:"string"}
    colormatch_n_colors = 4 #@param {type:"number"}
    ignore_sat_weight = 0 #@param {type:"number"}

    #@markdown **CLIP\Aesthetics Conditional Settings**
    clip_name = 'ViT-L/14' #@param ['ViT-L/14', 'ViT-L/14@336px', 'ViT-B/16', 'ViT-B/32']
    clip_scale = 0 #@param {type:"number"}
    aesthetics_scale = 0 #@param {type:"number"}
    cutn = 1 #@param {type:"number"}
    cut_pow = 0.0001 #@param {type:"number"}

    #@markdown **Other Conditional Settings**
    init_mse_scale = 0 #@param {type:"number"}
    init_mse_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    blue_scale = 0 #@param {type:"number"}
    
    #@markdown **Conditional Gradient Settings**
    gradient_wrt = 'x0_pred' #@param ["x", "x0_pred"]
    gradient_add_to = 'both' #@param ["cond", "uncond", "both"]
    decode_method = 'linear' #@param ["autoencoder","linear"]
    grad_threshold_type = 'dynamic' #@param ["dynamic", "static", "mean", "schedule"]
    clamp_grad_threshold = 0.2 #@param {type:"number"}
    clamp_start = 0.2 #@param
    clamp_stop = 0.01 #@param
    grad_inject_timing = list(range(1,10)) #@param

    #@markdown **Speed vs VRAM Settings**
    cond_uncond_sync = False #@param {type:"boolean"}
    precision = 'autocast' 
    C = 4
    f = 8

    cond_prompt = ""
    cond_prompts = ""
    uncond_prompt = ""
    uncond_prompts = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_sample_raw = None
    mask_sample = None
    init_c = None
    seed_internal = 0

    return locals()

args_dict = DeforumArgs()
anim_args_dict = DeforumAnimArgs()

if override_settings_with_file:
    load_args(args_dict, anim_args_dict, settings_file, custom_settings_file, verbose=False)

args = SimpleNamespace(**args_dict)
anim_args = SimpleNamespace(**anim_args_dict)

args.timestring = time.strftime('%Y%m%d%H%M%S')
args.strength = max(0.0, min(1.0, args.strength))

# Load clip model if using clip guidance
if (args.clip_scale > 0) or (args.aesthetics_scale > 0):
    root.clip_model = clip.load(args.clip_name, jit=False)[0].eval().requires_grad_(False).to(root.device)
    if (args.aesthetics_scale > 0):
        root.aesthetics_model = load_aesthetics_model(args, root)

if args.seed == -1:
    args.seed = random.randint(0, 2**32 - 1)
if not args.use_init:
    args.init_image = None
if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
    print(f"Init images aren't supported with PLMS yet, switching to KLMS")
    args.sampler = 'klms'
if args.sampler != 'ddim':
    args.ddim_eta = 0

if anim_args.animation_mode == 'None':
    anim_args.max_frames = 1
elif anim_args.animation_mode == 'Video Input':
    args.use_init = True

# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

with open("Deforum.yaml", "w") as file:
  yaml.dump(DeforumAnimArgs(), file)
  yaml.dump(DeforumArgs(), file)
  yaml.dump(cond_prompts, file)
  yaml.dump(uncond_prompts, file)

# dispatch to appropriate renderer
if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
    render_animation(root, anim_args, args, cond_prompts, uncond_prompts)
elif anim_args.animation_mode == 'Video Input':
    render_input_video(root, anim_args, args, cond_prompts, uncond_prompts)
elif anim_args.animation_mode == 'Interpolation':
    render_interpolation(root, anim_args, args, cond_prompts, uncond_prompts)
else:
    render_image_batch(root, args, anim_args, cond_prompts, uncond_prompts)

print("\033[0;34m..Your Animation Has Completed, Congratulations..")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

# $ {\large \textsf{Create Video From Frames}}$


In [ ]:
#@title $ { \textsf{COMPILE FRAMES TO VIDEO(SUBPROCESS--SOMETIMES FASTER)}}$
import time
import subprocess
from IPython import display

start_time = time.time()
#@markdown **This cell will compile ALL images in the folder you provide, so make sure that the folder you specify has the correct images and do not have any unwanted images.**
vid_fps = 30 #@param {type:'number'}
image_ext = '*.png' #@param ['*.png', '*.PNG', '*.jpg', '*.jpeg', '*.JPEG', '*.JPG']
input_path = '/content/drive/MyDrive/Looping_out/merged_images' # @param {type: 'string'}
#@markdown **The input folder with the frames you want to create a video with.**

raw_vid = '/content/temp.mp4' # @param {type: 'string'}
#@markdown **This `raw_vid` param results in an uncompressed, unformatted video, so to enable compatibility with most video players, we compress it to `compressed_vid`**

compressed_vid = '/content/looping_merged.mp4' # @param {type: 'string'}
#@markdown **This `compressed_vid` is now compatible with most video players and should play properly.**
make_gif = False #@param {type:"boolean"}
#@markdown **Please ensure you change the extension of your video to `.gif` if you intend to make gifs**

%cd $input_path

process = subprocess.Popen(['ffmpeg', '-framerate', f"{vid_fps}", '-pattern_type', 'glob', '-i', image_ext, '-crf', '17', '-preset', 'veryfast', '-y', raw_vid], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

output, error = process.communicate()

display_ffmpeg_output=True #@param {type:"boolean"}

if display_ffmpeg_output:
  display.display(output)
  display.display(error)

%cd "/content/"
if not make_gif:
  process = subprocess.Popen(['ffmpeg', '-i', raw_vid, '-c:v', 'libx264', '-rc', 'vbr', '-crf', '17', '-pix_fmt', 'yuv420p', '-b:v', '0', '-preset', 'veryfast', '-y', compressed_vid], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

  output, error = process.communicate()

else:

  process = subprocess.Popen(['ffmpeg', '-i', raw_vid, '-r', str(vid_fps), '-y', compressed_vid], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

  output, error = process.communicate()

if display_ffmpeg_output:
  display.display(output)
  display.display(error)

elapsed_time = time.time() - start_time

print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $ { \textsf{COMPILE FRAMES TO VIDEO(FFMPEG)}}$
import time

start_time = time.time()

#@markdown **This cell will compile ALL images in the folder you provide, so make sure that the folder you specify has the correct images and do not have any unwanted images.**
vid_fps = 30 #@param {type:'number'}
image_ext = "'*.png'" #@param ['*.png', '*.PNG', '*.jpg', '*.jpeg', '*.JPEG', '*.JPG']
input_path = '/content/drive/MyDrive/AI/StableDiffusion/2023-02/NeonMutation_SeventhHell ' # @param {type: 'string'}
#@markdown **The input folder with the frames you want to create a video with.**
raw_vid = '/content/temp.mp4' # @param {type: 'string'}
#@markdown **This `raw_vid` param results in an uncompressed, unformatted video, so to enable compatibility with most video players, we compress it to `compressed_vid`**

compressed_vid = '/content/NeonMutation_SeventhHell.mp4' # @param {type: 'string'}
#@markdown **This `compressed_vid` is now compatible with most video players and should play properly.**

make_gif = False #@param {type:"boolean"}
#@markdown **Please ensure you change the extension of your video to `.gif` if you intend to make gifs**

%cd $input_path

!ffmpeg -framerate $vid_fps \
 -pattern_type glob \
 -i  $image_ext \
 -y $raw_vid

%cd '/content'
if not make_gif:
  !ffmpeg -i $raw_vid \
  -c:v libx264 \
  -rc vbr \
  -cq 19 \
  -pix_fmt yuv420p \
  -b:v 0 \
  -y $compressed_vid

else:
  !ffmpeg \
  -y \
  -i $raw_vid \
  -r $vid_fps \
  $compressed_vid

elapsed_time = time.time() - start_time

print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $ {\textsf{Display The Video Using kora(may error out, try uninstalling with "pip uninstall kora" if it doesn't work)!}}$
from kora.drive import upload_public
from IPython.display import HTML
url = '/content/grudgeTOK.mp4' #@param{type:'string'}
url = upload_public(url)
# mp4_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/kong/20230123004448.mp4"
# mp4 = open(mp4_path,'rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# then display it
HTML(f"""<video src={url} width=500 controls/>""")

# $ \color{cyan} {\large \textsf{Model Merger}}$

In [ ]:
#@title **Model Download**
#@markdown **Download CivitAI Models Here**
model_URL = "https://civitai.com/api/download/models/6987?type=Model&format=PickleTensor" #@param {type:"string"}
folder_path = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion" #@param {type:"string"}
# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
#Name for the model
filename = "Realistic_Vision_V1.3.ckpt" #@param{type:"string"}

# Download the file and save it to the folder
response = requests.get(model_URL)
with open(os.path.join(folder_path, filename), "wb") as f:
    f.write(response.content)

print(f"File {filename} downloaded and saved to {folder_path}")

In [ ]:
#@title $ \color{cyan} {\large \textsf{Merge Model!}}$
start_time = time.time()

#@markdown **It Appears That When Merging Safetensors, `map_location` must be `cpu`. Please ensure to follow this. You can also try `cuda` but it didn't work for me.**
def Load_checkpoints_to_be_merged():
  ckpt_dir = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion" #@param{type:'string'}
  primary_model_name = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/deliberate_2_28.ckpt" #@param{type:'string'}
  secondary_model_name = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/Pastel_Mix.ckpt" #@param{type:'string'}
  tertiary_model_name = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/Realistic_Vision_V1.3.ckpt" #@param{type:'string'}
  output_modelname = "Pastel_Deliberate_Realistic" #@param{type:'string'}
  config_source = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/DeliberateDream.yaml" #@param{type:'string'}
  map_location = "cuda" #@param['cuda', 'cpu']
  interp_method = "Add difference" #@param["No interpolation", "Weighted sum", "Add difference"]
  multiplier = 0.7 #@param{type:'slider', min:0.1, max:1, step:0.1}
  save_as_half = False #@param{type:'boolean'}
  custom_name = "cuda" #@param['cuda', 'cpu']
  checkpoint_format = "ckpt" #@param['ckpt', 'safetensor']
  bake_in_vae = "/content/drive/MyDrive/AI/models/vae-ft-mse-840000-ema-pruned.ckpt" #@param{type:'string'}
  discard_weights = True #@param{type:'boolean'}
  return locals()

load_checkpoints_to_be_merged = Load_checkpoints_to_be_merged()
load_checkpoints_to_be_merged = SimpleNamespace(**load_checkpoints_to_be_merged)

chckpoint_dict_replacements = {
    'cond_stage_model.transformer.embeddings.': 'cond_stage_model.transformer.text_model.embeddings.',
    'cond_stage_model.transformer.encoder.': 'cond_stage_model.transformer.text_model.encoder.',
    'cond_stage_model.transformer.final_layer_norm.': 'cond_stage_model.transformer.text_model.final_layer_norm.',
}

def _load_vae_dict(model, vae_dict_1):
    model.first_stage_model.load_state_dict(vae_dict_1)
    model.first_stage_model.to("cuda")

def load_vae_dict(filename, map_location):
    vae_ckpt = read_state_dict(filename, map_location=map_location)
    vae_dict_1 = {k: v for k, v in vae_ckpt.items() if k[0:4] != "loss" and k not in vae_ignore_keys}
    return vae_dict_1


def load_vae(model, vae_file=None, vae_source="from unknown source"):
    global vae_dict, loaded_vae_file
    # save_settings = False


    if vae_file:
      _load_vae_dict(model, vae_file)
    else:
      assert os.path.isfile(vae_file), f"VAE {vae_source} doesn't exist: {vae_file}"
      print(f"Loading VAE weights {vae_source}: {vae_file}")
            

    vae_dict_1 = load_vae_dict(vae_file, load_checkpoints_to_be_merged.map_location)
    _load_vae_dict(model, vae_dict_1)

    vae_file = vae_dict_1.copy()

    loaded_vae_file = vae_file



def find_checkpoint_config(info):
    config = os.path.splitext(load_checkpoints_to_be_merged.primary_model_name) + ".yaml"
    if os.path.exists(config):
        return config

    return config

def transform_checkpoint_dict_key(k):
    for text, replacement in chckpoint_dict_replacements.items():
        if k.startswith(text):
            k = replacement + k[len(text):]

    return k

def get_state_dict_from_checkpoint(pl_sd):
    pl_sd = pl_sd.pop("state_dict", pl_sd)
    pl_sd.pop("state_dict", None)

    sd = {}
    for k, v in pl_sd.items():
        new_key = transform_checkpoint_dict_key(k)

        if new_key is not None:
            sd[new_key] = v

    pl_sd.clear()
    pl_sd.update(sd)

    return pl_sd

def read_state_dict(checkpoint_file, print_global_state=False, map_location=None):
    _, extension = os.path.splitext(checkpoint_file)
    if extension.lower() == ".safetensors":
        device = load_checkpoints_to_be_merged.map_location
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        pl_sd = safetensors.torch.load_file(checkpoint_file, device=device)
    else:
        pl_sd = torch.load(checkpoint_file, map_location=map_location)

    if print_global_state and "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")

    sd = get_state_dict_from_checkpoint(pl_sd)
    return sd

def create_config(ckpt_result, config_source, a, b, c):

    cfg = load_checkpoints_to_be_merged.config_source

    filename, _ = os.path.splitext(load_checkpoints_to_be_merged.output_modelname)
    checkpoint_filename = os.path.dirname(load_checkpoints_to_be_merged.primary_model_name) + "/" + ckpt_result + ".yaml"
    print("Copying config:")
    print("   from:", cfg)
    print("     to:", checkpoint_filename)
    shutil.copyfile(cfg, checkpoint_filename)


checkpoint_dict_skip_on_merge = ["cond_stage_model.transformer.text_model.embeddings.position_ids"]


def to_half(tensor, enable):
    if enable and tensor.dtype == torch.float:
        return tensor.half()

    return tensor

vae_ignore_keys = {"model_ema.decay", "model_ema.num_updates"}
vae_dict = {}
def run_modelmerger(primary_model_name, secondary_model_name, tertiary_model_name, interp_method, multiplier, save_as_half, custom_name, checkpoint_format, config_source, bake_in_vae, discard_weights):

    def fail(message):
        textinfo = message
        return message

    def weighted_sum(theta0, theta1, alpha):
        return ((1 - alpha) * theta0) + (alpha * theta1)

    def get_difference(theta1, theta2):
        return theta1 - theta2

    def add_difference(theta0, theta1_2_diff, alpha):
        return theta0 + (alpha * theta1_2_diff)

    def filename_weighted_sum():
        a = load_checkpoints_to_be_merged.primary_model_name
        b = load_checkpoints_to_be_merged.secondary_model_name
        Ma = round(1 - load_checkpoints_to_be_merged.multiplier, 2)
        Mb = round(load_checkpoints_to_be_merged.multiplier, 2)

        return f"{Ma}({a}) + {Mb}({b})"

    def filename_add_difference():
        a = load_checkpoints_to_be_merged.primary_model_name
        b = load_checkpoints_to_be_merged.secondary_model_name
        c = load_checkpoints_to_be_merged.tertiary_model_name
        M = round(load_checkpoints_to_be_merged.multiplier, 2)

        return f"{a} + {M}({b} - {c})"

    def filename_nothing():
        return load_checkpoints_to_be_merged.primary_model_name

    theta_funcs = {
        "Weighted sum": (filename_weighted_sum, None, weighted_sum),
        "Add difference": (filename_add_difference, get_difference, add_difference),
        "No interpolation": (filename_nothing, None, None),
    }
    filename_generator, theta_func1, theta_func2 = theta_funcs[load_checkpoints_to_be_merged.interp_method]
    job_count = (1 if theta_func1 else 0) + (1 if theta_func2 else 0)

    if not primary_model_name:
        return fail("Failed: Merging requires a primary model.")

    primary_model_info = load_checkpoints_to_be_merged.primary_model_name

    if theta_func2 and not secondary_model_name:
        return fail("Failed: Merging requires a secondary model.")

    secondary_model_info = load_checkpoints_to_be_merged.secondary_model_name if theta_func2 else None

    if theta_func1 and not tertiary_model_name:
        return fail(f"Failed: Interpolation method ({interp_method}) requires a tertiary model.")

    tertiary_model_info = load_checkpoints_to_be_merged.tertiary_model_name if theta_func1 else None

    result_is_inpainting_model = False

    if theta_func2:
        textinfo = f"Loading B"
        print(f"Loading {secondary_model_info}...")
        theta_1 = read_state_dict(secondary_model_info, map_location='cpu')
    else:
        theta_1 = None

    if theta_func1:
        textinfo = f"Loading C"
        print(f"Loading {tertiary_model_info}...")
        theta_2 = read_state_dict(tertiary_model_info, map_location='cpu')

        textinfo = 'Merging B and C'
        sampling_steps = len(theta_1.keys())
        for key in tqdm.tqdm(theta_1.keys()):
            if key in checkpoint_dict_skip_on_merge:
                continue

            if 'model' in key:
                if key in theta_2:
                    t2 = theta_2.get(key, torch.zeros_like(theta_1[key]))
                    theta_1[key] = theta_func1(theta_1[key], t2)
                else:
                    theta_1[key] = torch.zeros_like(theta_1[key])

            sampling_steps += 1
        del theta_2


    textinfo = f"Loading {primary_model_info}..."
    print(f"Loading {primary_model_info}...")
    theta_0 = read_state_dict(primary_model_info, map_location='cpu')

    print("Merging...")
    textinfo = 'Merging A and B'
    sampling_steps = len(theta_0.keys())
    for key in tqdm.tqdm(theta_0.keys()):
        if theta_1 and 'model' in key and key in theta_1:

            if key in checkpoint_dict_skip_on_merge:
                continue

            a = theta_0[key]
            b = theta_1[key]

            # this enables merging an inpainting model (A) with another one (B);
            # where normal model would have 4 channels, for latenst space, inpainting model would
            # have another 4 channels for unmasked picture's latent space, plus one channel for mask, for a total of 9
            if a.shape != b.shape and a.shape[0:1] + a.shape[2:] == b.shape[0:1] + b.shape[2:]:
                if a.shape[1] == 4 and b.shape[1] == 9:
                    raise RuntimeError("When merging inpainting model with a normal one, A must be the inpainting model.")

                assert a.shape[1] == 9 and b.shape[1] == 4, f"Bad dimensions for merged layer {key}: A={a.shape}, B={b.shape}"

                theta_0[key][:, 0:4, :, :] = theta_func2(a[:, 0:4, :, :], b, multiplier)
                result_is_inpainting_model = True
            else:
                theta_0[key] = theta_func2(a, b, multiplier)

            theta_0[key] = to_half(theta_0[key], save_as_half)

        sampling_steps += 1

    del theta_1

    bake_in_vae_filename = load_checkpoints_to_be_merged.bake_in_vae
    if bake_in_vae_filename is not None:
        print(f"Baking in VAE from {bake_in_vae_filename}")
        textinfo = 'Baking in VAE'
        vae_dict = load_vae_dict(bake_in_vae_filename, map_location='cpu')

        for key in vae_dict.keys():
            theta_0_key = 'first_stage_model.' + key
            if theta_0_key in theta_0:
                theta_0[theta_0_key] = to_half(vae_dict[key], save_as_half)

        del vae_dict

    if save_as_half and not theta_func2:
        for key in theta_0.keys():
            theta_0[key] = to_half(theta_0[key], save_as_half)

    # if discard_weights:
    #     regex = re.compile(discard_weights)
    #     for key in list(theta_0):
    #         if re.search(regex, key):
    #             theta_0.pop(key, None)

    ckpt_dir = load_checkpoints_to_be_merged.ckpt_dir
    filename = filename_generator() if custom_name == '' else custom_name
    filename += ".inpainting" if result_is_inpainting_model else ""
    filename += "." + checkpoint_format

    output_modelname = os.path.join(ckpt_dir, filename)

    textinfo = "Saving"
    print(f"Saving to {output_modelname}...")

    _, extension = os.path.splitext(output_modelname)
    if extension.lower() == ".safetensors":
        safetensors.torch.save_file(theta_0, output_modelname, metadata={"format": "pt"})
    else:
        torch.save(theta_0, output_modelname)


    ckpt_cfg = create_config(load_checkpoints_to_be_merged.output_modelname,
                  load_checkpoints_to_be_merged.config_source,
                  primary_model_info,
                  secondary_model_info,
                  tertiary_model_info)

    print(f"Checkpoint saved to {output_modelname}.")
    textinfo = "Checkpoint saved"
    

    return load_checkpoints_to_be_merged.output_modelname, ckpt_cfg

run_modelmerger(load_checkpoints_to_be_merged.primary_model_name,
                load_checkpoints_to_be_merged.secondary_model_name, 
                load_checkpoints_to_be_merged.tertiary_model_name, 
                load_checkpoints_to_be_merged.interp_method, 
                load_checkpoints_to_be_merged.multiplier, 
                load_checkpoints_to_be_merged.save_as_half, 
                load_checkpoints_to_be_merged.output_modelname, 
                load_checkpoints_to_be_merged.checkpoint_format, 
                load_checkpoints_to_be_merged.config_source, 
                load_checkpoints_to_be_merged.bake_in_vae, 
                load_checkpoints_to_be_merged.discard_weights)

print("\033[0;34m..Model Merging Has Completed, Congratulations..")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

# $\color{salmon} {\large \textsf{REAL-ESRGAN Video Upscaling!}}$

Please Be Sure to Run The Enviornment Setup Cell!


In [ ]:
import os
import cv2
import time

print("\033[0;34m")

start_time = time.time()

#@title $\color{salmon} {\large \textsf { 📹 RUN REAL-ESRGAN VIDEO UPSCALE PHASE (from video) 📹}}$ 
model_name = 'realesr-general-x4v3' #@param ['realesr-general-x4v3', 'realesr-general-wdn-x4v3', 'RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus'] {type:"string"}
lowres_vid = '/content/looping_merged.mp4' #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input your lowres video path here to be upscaled}}$
highres_vid_dir = '/content/drive/MyDrive/LINUM/4k/' #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input your high res video output path here}}$
if not os.path.exists(highres_vid_dir):
  os.makedirs(highres_vid_dir)
outscale = "4" #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
denoising_strength = 1 #@param{type:'slider', step:0.01, min:0.01, max:1}
target_fps = 30 #@param{type:'number'}
fp32 = 'fp32'
gpu_id = 0 #@param ['0', '1', '2', '3', '4']
use_face_enhance = False #@param {type:"boolean"}
if not use_face_enhance:
  %cd '/content/Real-ESRGAN/'
  !python3 /content/Real-ESRGAN/inference_realesrgan_video.py \
  --model_name $model_name \
  --outscale $outscale \
  -dn $denoising_strength \
  --fp32 \
  --input $lowres_vid \
  --fps $target_fps \
  --output $highres_vid_dir

else:
  %cd '/content/Real-ESRGAN/'
  !python3 /content/Real-ESRGAN/inference_realesrgan_video.py \
  --model_name $model_name \
  --outscale $outscale \
  --face_enhance \
  -dn $denoising_strength \
  --fp32 \
  --input $lowres_vid \
  --fps $target_fps \
  --output $highres_vid_dir

%cd '/content/'
print("\033[0;34m...Upscaling Phase Has Completed, You May Proceed...")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
import os
import cv2
import time
print("\033[0;34m")
start_time = time.time()
#@title $\color{salmon} {\large \textsf { 📹 !BATCH! RUN REAL-ESRGAN VIDEO UPSCALE PHASE (from video) 📹}}$ 
model_name = 'realesr-general-x4v3' #@param ['realesr-general-x4v3', 'realesr-general-wdn-x4v3', 'RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus'] {type:"string"}
batch_viddir = "/content/drive/MyDrive/LINUM/lowres/lowres_input_video/" #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input folder path with videos to upscale **WILL UPSCALE ALL VIDEOS IN THE FOLDER**}}$
highres_vid_dir = '/content/drive/MyDrive/RIFE_Batch' #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input your high res video output path here}}$
if not os.path.exists(highres_vid_dir):
  os.makedirs(highres_vid_dir)
outscale = "4" #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
denoising_strength = 1 #@param{type:'slider', step:0.01, min:0.01, max:1}
target_fps = 30 #@param{type:'number'}
fp32 = 'fp32'
gpu_id = 0 #@param ['0', '1', '2', '3', '4']
updir = os.path.split(highres_vid_dir)[0] + "/"

for vid in os.listdir(batch_viddir):
  new_vid = os.path.join(batch_viddir, vid)
  print(new_vid)
  %cd '/content/Real-ESRGAN'
  !python3 /content/Real-ESRGAN/inference_realesrgan_video.py --model_name $model_name --outscale $outscale -dn $denoising_strength --fp32 --input $new_vid --fps $target_fps --output $highres_vid_dir
  !mv $new_vid $updir --verbose
  print(f"\033[93mMoving {new_vid} to {updir}.")
%cd '/content/'
print("\033[0;34m...Upscaling Phase Has Completed, You May Proceed...")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

# $\color{orange} {\large \textsf{RIFE 4.6 Interpolation!}}$

Please Be Sure to Run The Enviornment Setup Cell!

Added RIFE 4.5 and 4.4 Model. Standard Model is 4.6.

These are just options, you can use the 4.6 model as it is the latest version.

In [ ]:
#@title $\color{orange} {\large \textsf{Define Params and Run RIFE!}}$
import time
print("\033[0;34m")
start_time = time.time()
model_dir = "RIFE46" #@param['RIFE46', 'RIFE45', 'RIFE44']
# model_dir = "/content/Practical-RIFE/" + model_dir
input_RIFE = '/content/drive/MyDrive/LINUM/4k/image_out.mp4' # @param {type: 'string'}
#@markdown $\color{orange} {\textsf{THIS IS THE FILE NAME FOR YOUR COMPRESSED VID}}$

output_RIFE = '/content/drive/MyDrive/LINUM/4k/image_out_looping_RIFE.mp4' # @param {type: 'string'}
#@markdown $\color{orange} {\textsf{INPUT YOUR OUTPUT NAME FOR THE INTERPOLATED VIDEO}}$
fps_RIFE = 60 # @param {type: 'number'}
target_length_RIFE = 18 # @param {type: 'number'}
target_scale_RIFE = 0.5 #@param ['0.25', '0.5', '1.0', '2.0', '4.0']
_skip_RIFEsmoothing = False
#@markdown $\color{orange} {\textsf{Additional Params}}$
print("\n ------------------------------------\n")
print("\n Beginning RIFE motion smoothing phase... \n")

%cd /content/Practical-RIFE/

class Detection:
  #def __init__(self):
  #  pass
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = float(int(a_string))
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(measured_duration,target_length_RIFE): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = fps_RIFE * target_length_RIFE
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x

#----------------------------
_import_mp4_file = False

if _import_mp4_file:
  measured_fps = Detection.detect_fps(input_RIFE)
  print("\n NOTICE: Detected average FPS of ",input_RIFE," is ",measured_fps)
  measured_duration = Detection.detect_duration(input_RIFE)
else: 
  measured_fps = Detection.detect_fps(input_RIFE)
  print("\n NOTICE: Detected average FPS of ",{input_RIFE}," is ",measured_fps)
  measured_duration = Detection.detect_duration(input_RIFE)

print("\n NOTICE: Detected duration INTEGER (in seconds) is ",measured_duration)

if measured_duration < 1: #failsafe
  print("\n NOTICE: Your input appears to be less than one second... \n")
  measured_duration = 1

exp_value = Detection.exp_calc(measured_duration,target_length_RIFE)


print("\n NOTICE: Target duration currently rounds to the closest --exp RIFE can handle. \n")

if (exp_value < 0.5):
  _skip_RIFEsmoothing = False
  print("\n NOTICE: Your fps_RIFE doesn't necessitate RIFE motion smoothing. Skipping RIFE...\n")

if _skip_RIFEsmoothing:
  print("\n NOTICE: Skipping RIFE motion smoothing...\n")
else:
  #---RUN RIFE------------------------------------
  print("\n ------------------------------------\n")
  print("\n NOTICE: Running RIFE... \n")
  %cd /content/Practical-RIFE/
  !python3 /content/Practical-RIFE/inference_video.py \
  --model=$model_dir \
  --fps=$fps_RIFE \
  --exp=$exp_value \
  --video=$input_RIFE \
  --scale=$target_scale_RIFE \
  --output=$output_RIFE
  #--exp=$exp_value
  #---END--------------------------------------
#!ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $_constant_quality -qmin $_constant_quality -qmax $_constant_quality -b:v 0 $output_fullpathname
#END OF MOTION SMOOTHING PHASE
print("\n \033[0;34mEnd of RIFE interpolation phase.\n")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $\color{orange} {\large \textsf{!BATCH! Define Params and Run RIFE!}}$
import time
start_time = time.time()
print("\033[0;34m")
%cd /content/Practical-RIFE/

class Detection:
  #def __init__(self):
  #  pass
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = float(int(a_string))
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(measured_duration,target_length_RIFE): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = fps_RIFE * target_length_RIFE
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x

#----------------------------
model_dir = "RIFE46" #@param['RIFE46', 'RIFE45', 'RIFE44']
rife_viddir = "/content/drive/MyDrive/RIFE_Batch" #@param{type:'string'}
#@markdown $\color{orange} {\textsf{THIS IS THE PATH TO THE FOLDER WITH YOUR VIDEOS TO BE INTERPOLATED}}$
for rife_vid in os.listdir(rife_viddir):
    new_vid = os.path.join(rife_viddir, rife_vid)
    print("\033[35mNOTICE: Interpolation will be performed on: \n\033[0;34m", new_vid)
fps_RIFE = 60 # @param {type: 'number'}
# target_length_RIFE = 35 # @param {type: 'number'}
target_scale_RIFE = 0.5 #@param ['0.25', '0.5', '1.0', '2.0', '4.0']
_skip_RIFEsmoothing = False
#@markdown $\color{orange} {\textsf{Additional Params}}$
print("\n ------------------------------------\n")
print("\n \033[0;34mBeginning RIFE motion smoothing phase... \n")

#----------------------------

if _skip_RIFEsmoothing:
  print("\n \033[33mNOTICE: Skipping RIFE motion smoothing...\n")
else:
  #---RUN RIFE------------------------------------
  print("\n ------------------------------------\n")
  print("\n \033[33mNOTICE: Running RIFE... \n")
  for rife_vid in os.listdir(rife_viddir):
    new_vid = os.path.join(rife_viddir, rife_vid)
    output_RIFE = f"{os.path.splitext(new_vid)[0]}_RIFE{os.path.splitext(new_vid)[1]}"
    measured_fps = Detection.detect_fps(new_vid)
    print("\n \033[33mNOTICE: Detected average FPS of ",new_vid," is ",measured_fps)
    measured_duration = Detection.detect_duration(new_vid)

    print("\n \033[33mNOTICE: Detected duration INTEGER (in seconds) is ",measured_duration)

    if measured_duration < 1: #failsafe
      print("\n \033[33mNOTICE: Your input appears to be less than one second... \n")
      measured_duration = 1
    target_length_RIFE = measured_duration
    exp_value = Detection.exp_calc(measured_duration,target_length_RIFE)

    print("\n \033[33mNOTICE: Target duration currently rounds to the closest --exp RIFE can handle. \n")

    if (exp_value < 0.5):
      _skip_RIFEsmoothing = False
      print("\n \033[33mNOTICE: Your fps_RIFE doesn't necessitate RIFE motion smoothing. Skipping RIFE...\n")

    if _skip_RIFEsmoothing:
      print("\n \033[33mNOTICE: Skipping RIFE motion smoothing...\n")
    print(f"\033[0;34mSaving Video to: ", output_RIFE)
    print("\n ------------------------------------\n")
    %cd /content/Practical-RIFE/
    !python3 /content/Practical-RIFE/inference_video.py \
    --model=$model_dir \
    --fps=$fps_RIFE \
    --exp=$exp_value \
    --video=$new_vid \
    --scale=$target_scale_RIFE \
    --output=$output_RIFE
  
%cd "/content/"
  #---END--------------------------------------
#!ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $_constant_quality -qmin $_constant_quality -qmax $_constant_quality -b:v 0 $output_fullpathname
#END OF MOTION SMOOTHING PHASE
print("\n \033[0;34mEnd of RIFE interpolation phase.\n")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $\color{orange} {\large \textsf {Analyze Lowres Vid Dimensions and FPS}}$ 
print("\033[0;34m")
vcap = cv2.VideoCapture(output_RIFE)

width = vcap.get(cv2.CAP_PROP_FRAME_WIDTH )
height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT )
fps =  vcap.get(cv2.CAP_PROP_FPS)
num_frames = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vcap.get(cv2.CAP_PROP_FPS)

# calculate the video length in seconds
video_length = num_frames / fps
# get the size of the file in bytes
file_size_bytes = os.path.getsize(output_RIFE)

# convert the file size to megabytes
file_size_mb = file_size_bytes / (1024 * 1024)

# print the file size in megabytes
print(f"File size: {file_size_mb:.2f} MB")
# print the video length
print(f"Video length: {video_length} seconds")
print(f"Width is: {width}.")
print(f"Height is: {height}.")
print(f"FPS is: {fps}.")

# $ \color{violet} {\large \textsf{REAL-ESRGAN Single Image or Frame Folder Upscaling!}}$


In [ ]:
#@title $ {\large \textsf { 🚀 RUN REAL-ESRGAN UPSCALE PHASE (from frames) 🚀}}$
#@markdown $ {\large \textsf {Input_dir is either a single image or a folder of images}}$
model_name = 'realesr-general-x4v3' #@param ['realesr-general-x4v3', 'realesr-general-wdn-x4v3', 'RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus'] {type:"string"}
input_dir = '/content/drive/MyDrive/AI/StableDiffusion/2023-02/grudge/20230225075915_13902.png' #@param{type:'string'}
output_dir = '/content/drive/MyDrive/' #@param{type:'string'}
outscale = "5" #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
denoising_strength = 1 #@param{type:'slider', step:0.01, min:0.01, max:1}
fp32 = 'fp32'
gpu_id = 0 #@param ['0', '1', '2', '3', '4']
%cd '/content/'
%cd '/content/Real-ESRGAN/'
!python3 /content/Real-ESRGAN/inference_realesrgan.py --model_name $model_name --outscale $outscale -dn $denoising_strength --fp32 --gpu-id $gpu_id --input $input_dir --output $output_dir
%cd '/content/'
print("\033[0;34m...Upscaling Phase Has Completed, You May Proceed...")

In [ ]:
from PIL import Image
from IPython import display
display_the_image = True #@param{type:'boolean'}
display_image = "/content/drive/MyDrive/Looping/00001.jpg" #@param{type:'string'}
#@markdown Path to the Image you want to Display
if display_the_image:
    im = Image.open(display_image)
    display.display(im)

def analyze_image_dimensions(file_path):
    with Image.open(file_path) as img:
        width, height = img.size
        print(f"Image Width: \033[0;34m{width}, \033[0mImage Height: \033[0;34m{height}")

analyze_image_dimensions(display_image)

In [ ]:
#@title display images in batches
import os
from IPython.display import display
import ipywidgets as widgets

# Set the directory path and initialize variables
dir_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-02/grudge" #@param{type:"string"}

batch_size = 100 #@param{type:"number"}
batch_count = 0
file_list = sorted(os.listdir(dir_path))
file_count = len(file_list)

# Define the function to display the next batch of images
def display_next_batch(b):
    global batch_count
    batch_count += 1
    start_index = (batch_count - 1) * batch_size
    end_index = batch_count * batch_size
    if end_index > file_count:
        end_index = file_count
    for i in range(start_index, end_index):
        file_path = os.path.join(dir_path, file_list[i])
        display(widgets.Image(value=open(file_path, "rb").read()))

# Display the first batch of images
display_next_batch(None)

# Create and display the button to load the next batch of images
button = widgets.Button(description="Load next batch")
button.on_click(display_next_batch)
display(button)


# $ \color{darkblue} {\large \textsf{ESRGAN Upscaling Interpolation}}$

(Use Multiple Models with Trained ESRGAN Models)

This Implementation offers the ability to use different

ESRGAN models to achieve the upscaling result you may desire

and otherwise not get using a single upscaler.

In [ ]:
#@title $ {\large \textsf { 🚀 RUN ESRGAN UPSCALE INTERPOLATION (from frames), Using One or More ESRGAN Models! 🚀}}$
#@markdown **`esrgan_input` may be a single image or a folder path to your images:**

#@markdown **This is a rather advanced implementation, Follow this structure:**

#@markdown $ {\color{yellow} {\textbf{model_path@amount_to_interpolate_by|model_path@amount_to_interpolate_by}}}$

#@markdown $ {\color{yellow} { \textbf {Example: 4x_FatalPixels_340000_G.pth@50|4x_fatal_Anime_500000_G.pth@50}}}$

#@markdown **Interpolation may not work with each model due to their structure(SRVGGNET, RRDB, ETC). Mix and match to your preference. Additionally, you can choose from the massive selection here at [UpscaleWiki](https://upscale.wiki/wiki/Model_Database)**
!pip install -qq rich
print("\033[0;34m")
use_upscale_interpolation = True #@param {type:"boolean"}
esrgan_input = "/content/drive/MyDrive/AI/StableDiffusion/2023-02/grudge/20230225075915_13902.png" #@param {type:"string"}
if not os.path.exists(esrgan_input):
  os.makedirs(esrgan_input)
esrgan_output = "/content/drive/MyDrive" #@param {type:"string"}
if not os.path.exists(esrgan_output):
  os.makedirs(esrgan_output)

esrgan_model_1 = "4x_fatal_Anime_500000_G.pth" #@param ['4x_Ultrasharp.pth', 'RRDB_ESRGAN_x4.pth', '4x_FatalPixels_340000_G.pth', '1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth', '4x_fatal_Anime_500000_G.pth', '4x-UltraSharp.pth', '4x_NMKD-YandereNeo-Lite_320k.pth', 'RRDB_PSNR_x4.pth']
esrgan_model_1_visibility = 80 #@param {type:"slider", min:5, max:100, step:5}
esrgan_model_2 = "4x-UltraSharp.pth" #@param ['4x_Ultrasharp.pth', 'RRDB_ESRGAN_x4.pth', '4x_FatalPixels_340000_G.pth', '1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth', '4x_fatal_Anime_500000_G.pth', '4x-UltraSharp.pth', '4x_NMKD-YandereNeo-Lite_320k.pth', 'RRDB_PSNR_x4.pth']
esrgan_model_2_visibility = 20 #@param {type:"slider", min:5, max:100, step:5}

if esrgan_model_1_visibility + esrgan_model_2_visibility != 100:
    print("\033[0;31mERROR: The sum of model visibility percentages must be 100.\033[0m")

if use_upscale_interpolation:
  %cd "/content/ESRGAN"
  !python upscale.py \
  "$esrgan_model_1@$esrgan_model_1_visibility|$esrgan_model_2@$esrgan_model_2_visibility" \
  --input $esrgan_input \
  --output $esrgan_output

else:
  %cd "/content/ESRGAN"
  !python upscale.py \
  "$esrgan_model_1" \
  --input $esrgan_input \
  --output $esrgan_output

%cd "/content/"

# $ {\large \textsf{Disconnect Runtime}}$

In [ ]:
skip_disconnect_for_run_all = False #@param {type: 'boolean'}

if skip_disconnect_for_run_all == True:
    print('Skipping disconnect, uncheck skip_disconnect_for_run_all if you want to run it')
else:
    from google.colab import runtime
    runtime.unassign()